___

In [67]:
#########################
mostrecentdate = 20230824
currentdate =    20230825
#########################

import calendar
calendar.setfirstweekday(calendar.SUNDAY)
print('• mostrecentdate: ', mostrecentdate)
print('• currentdate:    ', currentdate)
print('\n', calendar.month(int(str(currentdate)[:4]), int(str(currentdate)[4:6])))

# code cell warpping
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Input Date"></form>
''')

___

Robin_stocks

In [68]:
"""Contains all functions for placing orders for stocks, options, and crypto."""
from uuid import uuid4

from robin_stocks.robinhood.crypto import *
from robin_stocks.robinhood.helper import *
from robin_stocks.robinhood.profiles import *
from robin_stocks.robinhood.stocks import *
from robin_stocks.robinhood.urls import *

@login_required
def get_all_stock_orders(info=None):
    """Returns a list of all the orders that have been processed for the account.

    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: Returns a list of dictionaries of key/value pairs for each order. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.

    """
    url = orders_url()
    data = request_get(url, 'pagination')
    return(filter_data(data, info))


@login_required
def get_all_option_orders(info=None):
    """Returns a list of all the option orders that have been processed for the account.

    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: Returns a list of dictionaries of key/value pairs for each option order. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.

    """
    url = option_orders_url()
    data = request_get(url, 'pagination')
    return(filter_data(data, info))


@login_required
def get_all_crypto_orders(info=None):
    """Returns a list of all the crypto orders that have been processed for the account.

    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: Returns a list of dictionaries of key/value pairs for each option order. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.

    """
    url = crypto_orders_url()
    data = request_get(url, 'pagination')
    return(filter_data(data, info))


@login_required
def get_all_open_stock_orders(info=None):
    """Returns a list of all the orders that are currently open.

    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: Returns a list of dictionaries of key/value pairs for each order. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.

    """
    url = orders_url()
    data = request_get(url, 'pagination')

    data = [item for item in data if item['cancel'] is not None]

    return(filter_data(data, info))


@login_required
def get_all_open_option_orders(info=None, account_number=None):
    """Returns a list of all the orders that are currently open.

    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: Returns a list of dictionaries of key/value pairs for each order. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.

    """
    url = option_orders_url(account_number=account_number)
    data = request_get(url, 'pagination')

    data = [item for item in data if item['cancel_url'] is not None]

    return(filter_data(data, info))


@login_required
def get_all_open_crypto_orders(info=None):
    """Returns a list of all the crypto orders that have been processed for the account.

    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: Returns a list of dictionaries of key/value pairs for each option order. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.

    """
    url = crypto_orders_url()
    data = request_get(url, 'pagination')

    data = [item for item in data if item['cancel_url'] is not None]

    return(filter_data(data, info))


@login_required
def get_stock_order_info(orderID):
    """Returns the information for a single order.

    :param orderID: The ID associated with the order. Can be found using get_all_orders(info=None) or get_all_orders(info=None).
    :type orderID: str
    :returns: Returns a list of dictionaries of key/value pairs for the order.

    """
    url = orders_url(orderID)
    data = request_get(url)
    return(data)


@login_required
def get_option_order_info(order_id):
    """Returns the information for a single option order.

    :param order_id: The ID associated with the option order.
    :type order_id: str
    :returns: Returns a list of dictionaries of key/value pairs for the order.

    """
    url = option_orders_url(order_id)
    data = request_get(url)
    return data


@login_required
def get_crypto_order_info(order_id):
    """Returns the information for a single crypto order.

    :param order_id: The ID associated with the option order.
    :type order_id: str
    :returns: Returns a list of dictionaries of key/value pairs for the order.

    """
    url = crypto_orders_url(order_id)
    data = request_get(url)
    return data


@login_required
def find_stock_orders(**arguments):
    """Returns a list of orders that match the keyword parameters.

    :param arguments: Variable length of keyword arguments. EX. find_orders(symbol='FB',cancel=None,quantity=1)
    :type arguments: str
    :returns: Returns a list of orders.

    """ 
    url = orders_url()
    data = request_get(url, 'pagination')

    if (len(arguments) == 0):
        return(data)

    for item in data:
        item['quantity'] = str(float(item['quantity']))
        item['cumulative_quantity'] = str(float(item['cumulative_quantity']))

    if 'symbol' in arguments.keys():
        arguments['instrument'] = get_instruments_by_symbols(
            arguments['symbol'], info='url')[0]
        del arguments['symbol']

    if 'quantity' in arguments.keys():
        arguments['quantity'] = str(arguments['quantity'])

    stop = len(arguments.keys())-1
    list_of_orders = []
    for item in data:
        for i, (key, value) in enumerate(arguments.items()):
            if key not in item:
                print(error_argument_not_key_in_dictionary(key), file=get_output())
                return([None])
            if value != item[key]:
                break
            if i == stop:
                list_of_orders.append(item)

    return(list_of_orders)


@login_required
def cancel_stock_order(orderID):
    """Cancels a specific order.

    :param orderID: The ID associated with the order. Can be found using get_all_stock_orders(info=None).
    :type orderID: str
    :returns: Returns the order information for the order that was cancelled.

    """ 
    url = cancel_url(orderID)
    data = request_post(url)

    if data:
        print('Order '+str(orderID)+' cancelled', file=get_output())
    return(data)


@login_required
def cancel_option_order(orderID):
    """Cancels a specific option order.

    :param orderID: The ID associated with the order. Can be found using get_all_option_orders(info=None).
    :type orderID: str
    :returns: Returns the order information for the order that was cancelled.

    """ 
    url = option_cancel_url(orderID)
    data = request_post(url)

    if data:
        print('Order '+str(orderID)+' cancelled', file=get_output())
    return(data)


@login_required
def cancel_crypto_order(orderID):
    """Cancels a specific crypto order.

    :param orderID: The ID associated with the order. Can be found using get_all_crypto_orders(info=None).
    :type orderID: str
    :returns: Returns the order information for the order that was cancelled.

    """ 
    url = crypto_cancel_url(orderID)
    data = request_post(url)

    if data:
        print('Order '+str(orderID)+' cancelled', file=get_output())
    return(data)


@login_required
def cancel_all_stock_orders():
    """Cancels all stock orders.

    :returns: The list of orders that were cancelled.

    """ 
    url = orders_url()
    data = request_get(url, 'pagination')

    data = [item for item in data if item['cancel'] is not None]

    for item in data:
        request_post(item['cancel'])

    print('All Stock Orders Cancelled', file=get_output())
    return(data)


@login_required
def cancel_all_option_orders():
    """Cancels all option orders.

    :returns: Returns the order information for the orders that were cancelled.

    """ 
    url = option_orders_url()
    data = request_get(url, 'pagination')

    data = [item for item in data if item['cancel_url'] is not None]

    for item in data:
        request_post(item['cancel_url'])

    print('All Option Orders Cancelled', file=get_output())
    return(data)


@login_required
def cancel_all_crypto_orders():
    """Cancels all crypto orders.

    :returns: Returns the order information for the orders that were cancelled.

    """ 
    url = crypto_orders_url()
    data = request_get(url, 'pagination')

    data = [item for item in data if item['cancel_url'] is not None]

    for item in data:
        request_post(item['cancel_url'])

    print('All Crypto Orders Cancelled', file=get_output())
    return(data)


@login_required
def order_buy_market(symbol, quantity, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a market order to be executed immediately.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param quantity: The number of stocks to buy.
    :type quantity: int
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "buy", None, None, account_number, timeInForce, extendedHours, jsonify)


@login_required
def order_buy_fractional_by_quantity(symbol, quantity, account_number=None, timeInForce='gfd', extendedHours=False, jsonify=True):
    """Submits a market order to be executed immediately for fractional shares by specifying the amount that you want to trade.
    Good for share fractions up to 6 decimal places. Robinhood does not currently support placing limit, stop, or stop loss orders
    for fractional trades.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param quantity: The amount of the fractional shares you want to buy.
    :type quantity: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "buy", account_number, None, None, timeInForce, extendedHours, jsonify)


@login_required
def order_buy_fractional_by_price(symbol, amountInDollars, account_number=None, timeInForce='gfd', extendedHours=False, jsonify=True):
    """Submits a market order to be executed immediately for fractional shares by specifying the amount in dollars that you want to trade.
    Good for share fractions up to 6 decimal places. Robinhood does not currently support placing limit, stop, or stop loss orders
    for fractional trades.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param amountInDollars: The amount in dollars of the fractional shares you want to buy.
    :type amountInDollars: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    if amountInDollars < 1:
        print("ERROR: Fractional share price should meet minimum 1.00.", file=get_output())
        return None

    # turn the money amount into decimal number of shares
    price = next(iter(get_latest_price(symbol, 'ask_price', extendedHours)), 0.00)
    fractional_shares = 0 if (price == 0.00) else round_price(amountInDollars/float(price))
    
    return order(symbol, fractional_shares, "buy", account_number, None, None, timeInForce, extendedHours, jsonify)


@login_required
def order_buy_limit(symbol, quantity, limitPrice, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a limit order to be executed once a certain price is reached.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param quantity: The number of stocks to buy.
    :type quantity: int
    :param limitPrice: The price to trigger the buy order.
    :type limitPrice: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "buy", account_number, limitPrice, None, timeInForce, extendedHours, jsonify)


@login_required
def order_buy_stop_loss(symbol, quantity, stopPrice, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a stop order to be turned into a market order once a certain stop price is reached.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param quantity: The number of stocks to buy.
    :type quantity: int
    :param stopPrice: The price to trigger the market order.
    :type stopPrice: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "buy", account_number, None, stopPrice, timeInForce, extendedHours, jsonify)


@login_required
def order_buy_stop_limit(symbol, quantity, limitPrice, stopPrice, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a stop order to be turned into a limit order once a certain stop price is reached.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param quantity: The number of stocks to buy.
    :type quantity: int
    :param limitPrice: The price to trigger the market order.
    :type limitPrice: float
    :param stopPrice: The price to trigger the limit order.
    :type stopPrice: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "buy", account_number, limitPrice, stopPrice, timeInForce, extendedHours, jsonify)


@login_required
def order_buy_trailing_stop(symbol, quantity, trailAmount, trailType='percentage', timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a trailing stop buy order to be turned into a market order when traling stop price reached.

    :param symbol: The stock ticker of the stock to buy.
    :type symbol: str
    :param quantity: The number of stocks to buy.
    :type quantity: int
    :param trailAmount: how much to trail by; could be percentage or dollar value depending on trailType
    :type trailAmount: float
    :param trailType: could be "amount" or "percentage"
    :type trailType: str
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.
    """
    return order_trailing_stop(symbol, quantity, "buy", trailAmount, trailType, timeInForce, extendedHours, jsonify)


@login_required
def order_sell_market(symbol, quantity, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a market order to be executed immediately.

    :param symbol: The stock ticker of the stock to sell.
    :type symbol: str
    :param quantity: The number of stocks to sell.
    :type quantity: int
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "sell", None, None, account_number, timeInForce, extendedHours, jsonify)


@login_required
def order_sell_fractional_by_quantity(symbol, quantity, account_number=None, timeInForce='gfd', priceType='bid_price', extendedHours=False, jsonify=True):
    """Submits a market order to be executed immediately for fractional shares by specifying the amount that you want to trade.
    Good for share fractions up to 6 decimal places. Robinhood does not currently support placing limit, stop, or stop loss orders
    for fractional trades.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param quantity: The amount of the fractional shares you want to buy.
    :type quantity: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "sell", account_number, None, None, timeInForce, extendedHours, jsonify)


@login_required
def order_sell_fractional_by_price(symbol, amountInDollars, account_number=None, timeInForce='gfd', extendedHours=False, jsonify=True):
    """Submits a market order to be executed immediately for fractional shares by specifying the amount in dollars that you want to trade.
    Good for share fractions up to 6 decimal places. Robinhood does not currently support placing limit, stop, or stop loss orders
    for fractional trades.

    :param symbol: The stock ticker of the stock to purchase.
    :type symbol: str
    :param amountInDollars: The amount in dollars of the fractional shares you want to buy.
    :type amountInDollars: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    if amountInDollars < 1:
        print("ERROR: Fractional share price should meet minimum 1.00.", file=get_output())
        return None
    # turn the money amount into decimal number of shares
    price = next(iter(get_latest_price(symbol, 'bid_price', extendedHours)), 0.00)
    fractional_shares = 0 if (price == 0.00) else round_price(amountInDollars/float(price))

    return order(symbol, fractional_shares, "sell", account_number, None, None, timeInForce, extendedHours, jsonify)


@login_required
def order_sell_limit(symbol, quantity, limitPrice, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a limit order to be executed once a certain price is reached.

    :param symbol: The stock ticker of the stock to sell.
    :type symbol: str
    :param quantity: The number of stocks to sell.
    :type quantity: int
    :param limitPrice: The price to trigger the sell order.
    :type limitPrice: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "sell", account_number, limitPrice, None, timeInForce, extendedHours, jsonify)


@login_required
def order_sell_stop_loss(symbol, quantity, stopPrice, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a stop order to be turned into a market order once a certain stop price is reached.

    :param symbol: The stock ticker of the stock to sell.
    :type symbol: str
    :param quantity: The number of stocks to sell.
    :type quantity: int
    :param stopPrice: The price to trigger the market order.
    :type stopPrice: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "sell", account_number, None, stopPrice, timeInForce, extendedHours, jsonify)


@login_required
def order_sell_stop_limit(symbol, quantity, limitPrice, stopPrice, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a stop order to be turned into a limit order once a certain stop price is reached.

    :param symbol: The stock ticker of the stock to sell.
    :type symbol: str
    :param quantity: The number of stocks to sell.
    :type quantity: int
    :param limitPrice: The price to trigger the market order.
    :type limitPrice: float
    :param stopPrice: The price to trigger the limit order.
    :type stopPrice: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order(symbol, quantity, "sell", account_number, limitPrice, stopPrice, timeInForce, extendedHours, jsonify)


@login_required
def order_sell_trailing_stop(symbol, quantity, trailAmount, trailType='percentage', timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a trailing stop sell order to be turned into a market order when traling stop price reached.

    :param symbol: The stock ticker of the stock to sell.
    :type symbol: str
    :param quantity: The number of stocks to sell.
    :type quantity: int
    :param trailAmount: how much to trail by; could be percentage or dollar value depending on trailType
    :type trailAmount: float
    :param trailType: could be "amount" or "percentage"
    :type trailType: str
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.
    """
    return order_trailing_stop(symbol, quantity, "sell", trailAmount, trailType, timeInForce, extendedHours, jsonify)


@login_required
def order_trailing_stop(symbol, quantity, side, trailAmount, trailType='percentage', account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """Submits a trailing stop order to be turned into a market order when traling stop price reached.

    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of stocks to trade.
    :type quantity: int
    :param side: buy or sell
    :type side: str
    :param trailAmount: how much to trail by; could be percentage or dollar value depending on trailType
    :type trailAmount: float
    :param trailType: could be "amount" or "percentage"
    :type trailType: str
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: Optional[str]
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """
    try:
        symbol = symbol.upper().strip()
        trailAmount = float(trailAmount)
    except AttributeError as message:
        print(message)
        return None

    stock_price = round_price(get_latest_price(symbol, extendedHours)[0])

    # find stop price based on whether trailType is "amount" or "percentage" and whether its buy or sell
    percentage = 0
    try:
        if trailType == 'amount':
            margin = trailAmount
        else:
            margin = stock_price * trailAmount * 0.01
            percentage = trailAmount
    except Exception as e:
        print('ERROR: {}'.format(e))
        return None

    stopPrice = stock_price + margin if side == "buy" else stock_price - margin
    stopPrice = round_price(stopPrice)

    payload = {
        'account': load_account_profile(account_number=account_number, info='url'),
        'instrument': get_instruments_by_symbols(symbol, info='url')[0],
        'symbol': symbol,
        'quantity': quantity,
        'ref_id': str(uuid4()),
        'type': 'market',
        'stop_price': stopPrice,
        'time_in_force': timeInForce,
        'trigger': 'stop',
        'side': side,
        'extended_hours': extendedHours
    }

    if side == "buy":
        # price should be greater than stopPrice, adding a 5% threshold
        payload['price'] = round_price(stopPrice * 1.05)

    if trailType == 'amount':
        payload['trailing_peg'] = {'type': 'price', 'price': {'amount': trailAmount, 'currency_code': 'USD'}}
    else:
        payload['trailing_peg'] = {'type': 'percentage', 'percentage': str(percentage)}

    url = orders_url()
    data = request_post(url, payload, json=True, jsonify_data=jsonify)

    return (data)


@login_required
def order(symbol, quantity, side, limitPrice=None, stopPrice=None, account_number=None, timeInForce='gtc', extendedHours=False, jsonify=True):
    """A generic order function.

    :param symbol: The stock ticker of the stock to sell.
    :type symbol: str
    :param quantity: The number of stocks to sell.
    :type quantity: int
    :param side: Either 'buy' or 'sell'
    :type side: str
    :param limitPrice: The price to trigger the market order.
    :type limitPrice: float
    :param stopPrice: The price to trigger the limit or market order.
    :type stopPrice: float
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day.
    :type timeInForce: str
    :param extendedHours: Premium users only. Allows trading during extended hours. Should be true or false.
    :type extendedHours: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the purchase or selling of stocks, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    try:
        symbol = symbol.upper().strip()
    except AttributeError as message:
        print(message, file=get_output())
        return None

    orderType = "market"
    trigger = "immediate"

    if side == "buy":
        priceType = "ask_price"
    else:
        priceType = "bid_price"

    if limitPrice and stopPrice:
        price = round_price(limitPrice)
        stopPrice = round_price(stopPrice)
        orderType = "limit"
        trigger = "stop"
    elif limitPrice:
        price = round_price(limitPrice)
        orderType = "limit"
    elif stopPrice:
        stopPrice = round_price(stopPrice)
        if side == "buy":
            price = stopPrice
        else:
            price = None
        trigger = "stop"
    else:
        price = round_price(next(iter(get_latest_price(symbol, priceType, extendedHours)), 0.00))
    payload = {
        'account': load_account_profile(account_number=account_number, info='url'),
        'instrument': get_instruments_by_symbols(symbol, info='url')[0],
        'symbol': symbol,
        'price': price,
        'quantity': quantity,
        'ref_id': str(uuid4()),
        'type': orderType,
        'stop_price': stopPrice,
        'time_in_force': timeInForce,
        'trigger': trigger,
        'side': side,
        'extended_hours': extendedHours
    }
    # BEGIN PATCH FOR NEW ROBINHOOD BUY FORM (GuitarGuyChrisB 5/26/2023)
    if side == "buy":
        payload['order_form_version'] = "4"
        payload['preset_percent_limit'] = "0.05"
    # END PATCH FOR NEW ROBINHOOD BUY FORM (GuitarGuyChrisB 5/26/2023)

     
    ##################################################################
    # FIX MARKEY SELL AND BUY ORDER
    if orderType == 'market' and side == 'buy':
        payload['type'] = 'limit' 
    if orderType == 'market' and side == 'sell':
        payload['type'] = 'limit' 
    ##################################################################
    
    url = orders_url()


    data = request_post(url, payload, jsonify_data=jsonify)

    return(data)


@login_required
def order_option_credit_spread(price, symbol, quantity, spread, timeInForce='gtc', account_number=None, jsonify=True):
    """Submits a limit order for an option credit spread.

    :param price: The limit price to trigger a sell of the option.
    :type price: float
    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of options to sell.
    :type quantity: int
    :param spread: A dictionary of spread options with the following keys: \n
        - expirationDate: The expiration date of the option in 'YYYY-MM-DD' format.\n
        - strike: The strike price of the option.\n
        - optionType: This should be 'call' or 'put'.\n
        - effect: This should be 'open' or 'close'.\n
        - action: This should be 'buy' or 'sell'.
    :type spread: dict
    :param timeInForce: Changes how long the order will be in effect for. \
     'gtc' = good until cancelled. \
     'gfd' = good for the day. 'ioc' = immediate or cancel. 'opg' = execute at opening.
    :type timeInForce: Optional[str]
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the trading of options, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.
    """
    return(order_option_spread("credit", price, symbol, quantity, spread, timeInForce, account_number, jsonify))


@login_required
def order_option_debit_spread(price, symbol, quantity, spread, timeInForce='gtc', account_number=None, jsonify=True):
    """Submits a limit order for an option debit spread.

    :param price: The limit price to trigger a sell of the option.
    :type price: float
    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of options to sell.
    :type quantity: int
    :param spread: A dictionary of spread options with the following keys: \n
        - expirationDate: The expiration date of the option in 'YYYY-MM-DD' format.\n
        - strike: The strike price of the option.\n
        - optionType: This should be 'call' or 'put'.\n
        - effect: This should be 'open' or 'close'.\n
        - action: This should be 'buy' or 'sell'.
    :type spread: dict
    :param timeInForce: Changes how long the order will be in effect for.
     'gtc' = good until cancelled. \
     'gfd' = good for the day. 'ioc' = immediate or cancel. 'opg' execute at opening.
    :type timeInForce: Optional[str]
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the trading of options, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.
    """
    return(order_option_spread("debit", price, symbol, quantity, spread, timeInForce, account_number, jsonify))


@login_required
def order_option_spread(direction, price, symbol, quantity, spread, account_number=None, timeInForce='gtc', jsonify=True):
    """Submits a limit order for an option spread. i.e. place a debit / credit spread

    :param direction: Can be "credit" or "debit".
    :type direction: str
    :param price: The limit price to trigger a trade of the option.
    :type price: float
    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of options to trade.
    :type quantity: int
    :param spread: A dictionary of spread options with the following keys: \n
        - expirationDate: The expiration date of the option in 'YYYY-MM-DD' format.\n
        - strike: The strike price of the option.\n
        - optionType: This should be 'call' or 'put'.\n
        - effect: This should be 'open' or 'close'.\n
        - action: This should be 'buy' or 'sell'.
    :type spread: dict
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for.
     'gtc' = good until cancelled. \
     'gfd' = good for the day. 'ioc' = immediate or cancel. 'opg' execute at opening.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the trading of options, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.
    """ 
    try:
        symbol = symbol.upper().strip()
    except AttributeError as message:
        print(message, file=get_output())
        return None
    legs = []
    for each in spread:
        optionID = id_for_option(symbol,
                                        each['expirationDate'],
                                        each['strike'],
                                        each['optionType'])
        legs.append({'position_effect': each['effect'],
                     'side': each['action'],
                     'ratio_quantity': 1,
                     'option': option_instruments_url(optionID)})

    payload = {
        'account': load_account_profile(account_number=account_number, info='url'),
        'direction': direction,
        'time_in_force': timeInForce,
        'legs': legs,
        'type': 'limit',
        'trigger': 'immediate',
        'price': price,
        'quantity': quantity,
        'override_day_trade_checks': False,
        'override_dtbp_checks': False,
        'ref_id': str(uuid4()),
    }

    url = option_orders_url()
    data = request_post(url, payload, json=True, jsonify_data=jsonify)

    return(data)


@login_required
def order_buy_option_limit(positionEffect, creditOrDebit, price, symbol, quantity, expirationDate, strike, optionType='both', account_number=None, timeInForce='gtc', jsonify=True):
    """Submits a limit order for an option. i.e. place a long call or a long put.

    :param positionEffect: Either 'open' for a buy to open effect or 'close' for a buy to close effect.
    :type positionEffect: str
    :param creditOrDebit: Either 'debit' or 'credit'.
    :type creditOrDebit: str
    :param price: The limit price to trigger a buy of the option.
    :type price: float
    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of options to buy.
    :type quantity: int
    :param expirationDate: The expiration date of the option in 'YYYY-MM-DD' format.
    :type expirationDate: str
    :param strike: The strike price of the option.
    :type strike: float
    :param optionType: This should be 'call' or 'put'
    :type optionType: str
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day. 'ioc' = immediate or cancel. 'opg' execute at opening.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of options, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    try:
        symbol = symbol.upper().strip()
    except AttributeError as message:
        print(message, file=get_output())
        return None

    optionID = id_for_option(symbol, expirationDate, strike, optionType)

    payload = {
        'account': load_account_profile(account_number=account_number, info='url'),
        'direction': creditOrDebit,
        'time_in_force': timeInForce,
        'legs': [
            {'position_effect': positionEffect, 'side': 'buy',
                'ratio_quantity': 1, 'option': option_instruments_url(optionID)},
        ],
        'type': 'limit',
        'trigger': 'immediate',
        'price': price,
        'quantity': quantity,
        'override_day_trade_checks': False,
        'override_dtbp_checks': False,
        'ref_id': str(uuid4()),
    }

    url = option_orders_url()
    data = request_post(url, payload, json=True, jsonify_data=jsonify)

    return(data)


@login_required
def order_buy_option_stop_limit(positionEffect, creditOrDebit, limitPrice, stopPrice, symbol, quantity, expirationDate, strike, optionType='both', account_number=None, timeInForce='gtc', jsonify=True):
    """Submits a stop order to be turned into a limit order once a certain stop price is reached.

    :param positionEffect: Either 'open' for a buy to open effect or 'close' for a buy to close effect.
    :type positionEffect: str
    :param creditOrDebit: Either 'debit' or 'credit'.
    :type creditOrDebit: str
    :param limitPrice: The limit price to trigger a buy of the option.
    :type limitPrice: float
    :param stopPrice: The price to trigger the limit order.
    :type stopPrice: float
    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of options to buy.
    :type quantity: int
    :param expirationDate: The expiration date of the option in 'YYYY-MM-DD' format.
    :type expirationDate: str
    :param strike: The strike price of the option.
    :type strike: float
    :param optionType: This should be 'call' or 'put'
    :type optionType: str
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day. 'ioc' = immediate or cancel. 'opg' execute at opening.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of options, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    try:
        symbol = symbol.upper().strip()
    except AttributeError as message:
        print(message, file=get_output())
        return None

    optionID = id_for_option(symbol, expirationDate, strike, optionType)

    payload = {
        'account': load_account_profile(account_number=account_number, info='url'),
        'direction': creditOrDebit,
        'time_in_force': timeInForce,
        'legs': [
            {'position_effect': positionEffect, 'side': 'buy',
                'ratio_quantity': 1, 'option': option_instruments_url(optionID)},
        ],
        'type': 'limit',
        'trigger': 'stop',
        'price': limitPrice,
        'stop_price': stopPrice,
        'quantity': quantity,
        'override_day_trade_checks': False,
        'override_dtbp_checks': False,
        'ref_id': str(uuid4()),
    }

    url = option_orders_url()
    data = request_post(url, payload, json=True, jsonify_data=jsonify)

    return(data)


def order_sell_option_stop_limit(positionEffect, creditOrDebit, limitPrice, stopPrice, symbol, quantity, expirationDate, strike, optionType='both', account_number=None, timeInForce='gtc', jsonify=True):
    """Submits a stop order to be turned into a limit order once a certain stop price is reached.

    :param positionEffect: Either 'open' for a buy to open effect or 'close' for a buy to close effect.
    :type positionEffect: str
    :param creditOrDebit: Either 'debit' or 'credit'.
    :type creditOrDebit: str
    :param limitPrice: The limit price to trigger a buy of the option.
    :type limitPrice: float
    :param stopPrice: The price to trigger the limit order.
    :type stopPrice: float
    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of options to buy.
    :type quantity: int
    :param expirationDate: The expiration date of the option in 'YYYY-MM-DD' format.
    :type expirationDate: str
    :param strike: The strike price of the option.
    :type strike: float
    :param optionType: This should be 'call' or 'put'
    :type optionType: str
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day. 'ioc' = immediate or cancel. 'opg' execute at opening.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of options, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    try:
        symbol = symbol.upper().strip()
    except AttributeError as message:
        print(message, file=get_output())
        return None

    optionID = id_for_option(symbol, expirationDate, strike, optionType)

    payload = {
        'account': load_account_profile(account_number=account_number, info='url'),
        'direction': creditOrDebit,
        'time_in_force': timeInForce,
        'legs': [
            {'position_effect': positionEffect, 'side': 'sell',
                'ratio_quantity': 1, 'option': option_instruments_url(optionID)},
        ],
        'type': 'limit',
        'trigger': 'stop',
        'price': limitPrice,
        'stop_price': stopPrice,
        'quantity': quantity,
        'override_day_trade_checks': False,
        'override_dtbp_checks': False,
        'ref_id': str(uuid4()),
    }

    url = option_orders_url()
    data = request_post(url, payload, json=True, jsonify_data=jsonify)

    return(data)


@login_required
def order_sell_option_limit(positionEffect, creditOrDebit, price, symbol, quantity, expirationDate, strike, optionType='both', account_number=None, timeInForce='gtc', jsonify=True):
    """Submits a limit order for an option. i.e. place a short call or a short put.

    :param positionEffect: Either 'open' for a sell to open effect or 'close' for a sell to close effect.
    :type positionEffect: str
    :param creditOrDebit: Either 'debit' or 'credit'.
    :type creditOrDebit: str
    :param price: The limit price to trigger a sell of the option.
    :type price: float
    :param symbol: The stock ticker of the stock to trade.
    :type symbol: str
    :param quantity: The number of options to sell.
    :type quantity: int
    :param expirationDate: The expiration date of the option in 'YYYY-MM-DD' format.
    :type expirationDate: str
    :param strike: The strike price of the option.
    :type strike: float
    :param optionType: This should be 'call' or 'put'
    :type optionType: str
    :param account_number: the robinhood account number.
    :type account_number: Optional[str]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled. \
    'gfd' = good for the day. 'ioc' = immediate or cancel. 'opg' execute at opening.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of options, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """
    try:
        symbol = symbol.upper().strip()
    except AttributeError as message:
        print(message, file=get_output())
        return None

    optionID = id_for_option(symbol, expirationDate, strike, optionType)

    payload = {
        'account': load_account_profile(account_number=account_number, info='url'),
        'direction': creditOrDebit,
        'time_in_force': timeInForce,
        'legs': [
            {'position_effect': positionEffect, 'side': 'sell',
                'ratio_quantity': 1, 'option': option_instruments_url(optionID)},
        ],
        'type': 'limit',
        'trigger': 'immediate',
        'price': price,
        'quantity': quantity,
        'override_day_trade_checks': False,
        'override_dtbp_checks': False,
        'ref_id': str(uuid4()),
    }

    url = option_orders_url()
    data = request_post(url, payload, json=True, jsonify_data=jsonify)

    return(data)


@login_required
def order_buy_crypto_by_price(symbol, amountInDollars, timeInForce='gtc', jsonify=True):
    """Submits a market order for a crypto by specifying the amount in dollars that you want to trade.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param amountInDollars: The amount in dollars of the crypto you want to buy.
    :type amountInDollars: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order_crypto(symbol, "buy", amountInDollars, "price", None, timeInForce, jsonify)


@login_required
def order_buy_crypto_by_quantity(symbol, quantity, timeInForce='gtc', jsonify=True):
    """Submits a market order for a crypto by specifying the decimal amount of shares to buy.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param quantity: The decimal amount of shares to buy.
    :type quantity: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order_crypto(symbol, "buy", quantity, "quantity", None, timeInForce, jsonify)


@login_required
def order_buy_crypto_limit(symbol, quantity, limitPrice, timeInForce='gtc', jsonify=True):
    """Submits a limit order for a crypto by specifying the decimal amount of shares to buy.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param quantity: The decimal amount of shares to buy.
    :type quantity: float
    :param limitPrice: The limit price to set for the crypto.
    :type limitPrice: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order_crypto(symbol, "buy", quantity, "quantity", limitPrice, timeInForce, jsonify)


@login_required
def order_buy_crypto_limit_by_price(symbol, amountInDollars, limitPrice, timeInForce='gtc', jsonify=True):
    """Submits a limit order for a crypto by specifying the decimal price to buy.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param amountInDollars: The amount in dollars of the crypto you want to buy.
    :type amountInDollars: float
    :param limitPrice: The limit price to set for the crypto.
    :type limitPrice: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """
    return order_crypto(symbol, "buy", amountInDollars, "price", limitPrice, timeInForce, jsonify)


@login_required
def order_sell_crypto_by_price(symbol, amountInDollars, timeInForce='gtc', jsonify=True):
    """Submits a market order for a crypto by specifying the amount in dollars that you want to trade.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param amountInDollars: The amount in dollars of the crypto you want to sell.
    :type amountInDollars: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order_crypto(symbol, "sell", amountInDollars, "price", None, timeInForce, jsonify)


@login_required
def order_sell_crypto_by_quantity(symbol, quantity, timeInForce='gtc', jsonify=True):
    """Submits a market order for a crypto by specifying the decimal amount of shares to buy.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param quantity: The decimal amount of shares to sell.
    :type quantity: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """ 
    return order_crypto(symbol, "sell", quantity, "quantity", None, timeInForce, jsonify)


@login_required
def order_sell_crypto_limit(symbol, quantity, limitPrice, timeInForce='gtc', jsonify=True):
    """Submits a limit order for a crypto by specifying the decimal amount of shares to sell.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param quantity: The decimal amount of shares to sell.
    :type quantity: float
    :param limitPrice: The limit price to set for the crypto.
    :type limitPrice: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """
    return order_crypto(symbol, "sell", quantity, "quantity", limitPrice, timeInForce, jsonify)


@login_required
def order_sell_crypto_limit_by_price(symbol, amountInDollars, limitPrice, timeInForce='gtc', jsonify=True):
    """Submits a limit order for a crypto by specifying the decimal price to sell.
    Good for share fractions up to 8 decimal places.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param amountInDollars: The amount in dollars of the crypto you want to sell.
    :type amountInDollars: float
    :param limitPrice: The limit price to set for the crypto.
    :type limitPrice: float
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the buying of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """
    return order_crypto(symbol, "sell", amountInDollars, "price", limitPrice, timeInForce, jsonify)


@login_required
def order_crypto(symbol, side, quantityOrPrice, amountIn="quantity", limitPrice=None, timeInForce="gtc", jsonify=True):
    """Submits an order for a crypto.

    :param symbol: The crypto ticker of the crypto to trade.
    :type symbol: str
    :param side: Either 'buy' or 'sell'
    :type side: str
    :param quantityOrPrice: Either the decimal price of shares to trade or the decimal quantity of shares.
    :type quantityOrPrice: float
    :param amountIn: If left default value of 'quantity', order will attempt to trade cryptos by the amount of crypto \
        you want to trade. If changed to 'price', order will attempt to trade cryptos by the price you want to buy or sell.
    :type amountIn: Optional[str]
    :param limitPrice: The price to trigger the market order.
    :type limitPrice: Optional[float]
    :param timeInForce: Changes how long the order will be in effect for. 'gtc' = good until cancelled.
    :type timeInForce: Optional[str]
    :param jsonify: If set to False, function will return the request object which contains status code and headers.
    :type jsonify: Optional[str]
    :returns: Dictionary that contains information regarding the selling of crypto, \
    such as the order id, the state of order (queued, confired, filled, failed, canceled, etc.), \
    the price, and the quantity.

    """
    try:
        symbol = symbol.upper().strip()
    except AttributeError as message:
        print(message, file=get_output())
        return None

    crypto_id = get_crypto_id(symbol)
    orderType = "market"

    if side == "buy":
        priceType = "ask_price"
    else:
        priceType = "bid_price"

    if limitPrice:
        price = limitPrice
        orderType = "limit"
    else:
        price = round_price(get_crypto_quote_from_id(crypto_id, info=priceType))

    if amountIn == "quantity":
        quantity = quantityOrPrice
    else:
        quantity = round_price(quantityOrPrice/price)

    payload = {
        'account_id': load_crypto_profile(info="id"),
        'currency_pair_id': crypto_id,
        'price': price,
        'quantity': quantity,
        'ref_id': str(uuid4()),
        'side': side,
        'time_in_force': timeInForce,
        'type': orderType
    }

    url = order_crypto_url()

    # This is safe because 'ref_id' guards us from duplicate orders
    attempts = 3
    while attempts > 0:
        data = request_post(url, payload, json=True, jsonify_data=jsonify)
        if data is not None:
            break

        attempts -= 1

    return(data)

if __name__ == '__main__':
    print('> Robin-Stocks imported')

Analysis Modules

In [69]:
# Supervised learning
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from skopt import BayesSearchCV
from sklearn.pipeline import Pipeline
from lazypredict.Supervised import LazyClassifier  # import regression if needed 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, PolynomialFeatures
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score, recall_score, f1_score, fbeta_score, mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
# Deep learning
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

# check missing tickers to sell
import sean
import robin_stocks.robinhood as rs

import pandas_ta as ta
from pandas_datareader import data as pdr
import FinanceDataReader as fdr
from forex_python.converter import CurrencyRates
import yfinance as yf
yf.pdr_override()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")  # ignore runtime warning

# NLTK VADER for sentiment analysis
import nltk
# nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pyfinviz.news import News 
# https://pypi.org/project/pyfinviz/

import math
import random
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# plt.style.use('dark_background')
# plt.style.use('classic')

# article_ml
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score

# email with attachments
import email, smtplib, ssl
port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "aicpasean@gmail.com"  # Enter your address
receiver_email = "aicpasean@gmail.com"  # Enter receiver address
password = sean.gmail_api()

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pytz
# rsi
import datetime as dt

# other required libraries
import requests
import json
from bs4 import BeautifulSoup
import io
import os
import webbrowser
import urllib.request, urllib.error, urllib.parse
from urllib.request import urlopen, Request
from os.path import exists
from tqdm.notebook import tqdm

import time
from time import sleep
from gtts import gTTS 
import jupyter_beeper
from datetime import date, datetime, timedelta
currentdate_verify = date.today()
currentdate_verify = int(currentdate_verify.strftime("%Y%m%d"))

import pandas_datareader as data
data_source = 'yahoo'
start_date = '1970-01-01'

day_plus = datetime.today() + timedelta(days = 1)
day_plus = str(day_plus)[:10]

index_ticker = ['XLE', 'XLF', 'XLRE', 'XLB', 'XLK', 'XLP', 'XLV', 'XLI', 'XLY', 'XLU', 'XLC',
                '^GSPC', 'BTC-USD', 'GLD', 'USO', '^TNX', '^VIX', 'GC=F', 'CL=F', 'DX-Y.NYB']

index_candle_tickers = ['^GSPC']

etf_index_dic = pd.read_csv('etf_index_df.csv')
etf_index_dic = pd.melt(etf_index_dic, var_name = 'sector', value_name = 'ticker')
etf_index_dic = etf_index_dic.sort_values('ticker').reset_index(drop = True)
etf_ticker = etf_index_dic.ticker.tolist()

review_plot_lookbackdays = 121

# reset the counter checking the number of request to yfinance
yf_counter = 0

def yfinance_df_setting(ticker):
    global yf_counter
    
    try:
        # start date is since 1970, which is default value
        start_date = '1970-01-01'
        df = yf.download(ticker, start = start_date, end = day_plus, prepost = True, progress=False).reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        df['Date'] = df.Date.astype('datetime64[ns]')
        return df
    except:
        yf_counter += 1
        if yf_counter <= 50:
            return yfinance_df_setting(ticker)
        else:
            yf_counter = 0
            message = f'Ticker [{ticker}] may be delisted. Return an empty DataFrame.'
            print('Except >>> ' + message)
            alarm()
            voice_message(message)
            return pd.DataFrame()

        
def yfinance_df_rsi(ticker):
    global yf_counter
    
    try:
        start_date = str(datetime.today() - timedelta(days = 300))[:10]   
        df = yf.download(ticker, start = start_date, end = day_plus, prepost = True, progress=False).reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        df['Date'] = df.Date.astype('datetime64[ns]')        
        return df
    except:
        yf_counter += 1
        if yf_counter <= 50:
            return yfinance_df_rsi(ticker)
        else:
            yf_counter = 0
            message = f'Ticker [{ticker}] may be delisted. Return an empty DataFrame.'
            print('Except >>> ' + message)
            alarm()
            voice_message(message)
            return pd.DataFrame()
  

def yfinance_df(ticker):
    global yf_counter
  
    # start date is since 1970, which is default value
    start_date = '1970-01-01'
    non_timeout_tickers = ['XLE', 'XLF', 'XLRE', 'XLB', 'XLK', 'XLP', 'XLV', 'XLI', 'XLY', 'XLU', 'XLC',
                       '^GSPC', 'BTC-USD', 'GLD', 'USO', '^TNX', '^VIX', 'GC=F', 'CL=F', 'DX-Y.NYB']
    if ticker in non_timeout_tickers:
        try:
            time.sleep(0.5)
            df = yf.download(ticker, start = start_date, end = day_plus, prepost = True, progress=False).reset_index()
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
            df['Date'] = df.Date.astype('datetime64[ns]') 
            return df
        except:   
            yf_counter += 1
            if yf_counter <= 50:
                return yfinance_df(ticker)
            else:
                yf_counter = 0
                message = f'Ticker [{ticker}] may be delisted. Return an empty DataFrame.'
                print('Except >>> ' + message)
                alarm()
                voice_message(message)
                return pd.DataFrame()
          
    else:
        try:
            df = yf.download(ticker, start = start_date, end = day_plus, prepost = True, progress=False).reset_index()
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
            df['Date'] = df.Date.astype('datetime64[ns]') 
            return df
        except:       
            yf_counter += 1
            if yf_counter <= 50:
                return yfinance_df(ticker)
            else:
                yf_counter = 0
                message = f'Ticker [{ticker}] may be delisted. Return an empty DataFrame.'
                print('Except >>> ' + message)
                alarm()
                voice_message(message)
                return pd.DataFrame()
   

def yfinance_df_min(ticker): # only for valuation
    global yf_counter
    
    start_date = str(datetime.today() - timedelta(days = 10))[:10]
    non_timeout_tickers = ['XLE', 'XLF', 'XLRE', 'XLB', 'XLK', 'XLP', 'XLV', 'XLI', 'XLY', 'XLU', 'XLC',
                       '^GSPC', 'BTC-USD', 'GLD', 'USO', '^TNX', '^VIX', 'GC=F', 'CL=F', 'DX-Y.NYB']
        
    if ticker in non_timeout_tickers:
        try:
            time.sleep(0.5)
            df = yf.download(ticker, start = start_date, end = day_plus, prepost = True, progress=False).reset_index().tail(1)
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
            df['Date'] = df.Date.astype('datetime64[ns]') 
            return df
        except:    
            yf_counter += 1
            if yf_counter <= 50:
                return yfinance_df_min(ticker)
            else:
                yf_counter = 0
                message = f'Ticker [{ticker}] may be delisted. Return an empty DataFrame.'
                print('Except >>> ' + message)
                alarm()
                voice_message(message)
                return pd.DataFrame()
          
    else:
        try:
            df = yf.download(ticker, start = start_date, end = day_plus, prepost = True, progress=False).reset_index().tail(1)
            df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
            df['Date'] = df.Date.astype('datetime64[ns]') 
            return df
        except: 
            yf_counter += 1
            if yf_counter <= 50:
                return yfinance_df_min(ticker)
            else:
                yf_counter = 0
                message = f'Ticker [{ticker}] may be delisted. Return an empty DataFrame.'
                print('Except >>> ' + message)
                alarm()
                voice_message(message)
                return pd.DataFrame()
        
############################################################################################################################################
def current_time():
    time = f'Time Check | {datetime.now().strftime("%H:%M")}'
    return time    
    

def timechecknow():
    # time
    ttoday = date.today()
    tz_NY = pytz.timezone('America/New_York') 
    datetime_NY = datetime.now(tz_NY) 
    speed_hour = int(datetime_NY.strftime("%H"))
    speed_minute = int(datetime_NY.strftime("%M"))
    speed_second = int(datetime_NY.strftime("%S"))
    minutepassed = np.round((speed_hour-9)*60 + (speed_minute-30) + speed_second/60,30)
    return minutepassed


def day_check():
    checkday = datetime.today().strftime('%A')
    return checkday


def exchange_check():
    import requests
    from bs4 import BeautifulSoup
    try:
        # exchange rate
        krw = requests.get('https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=KRW')
        soup = BeautifulSoup(krw.content, 'html.parser')
        ext = soup.body.get_text()   #get String
        ext = pd.Series(ext)         #change to Series because 'str.extract' does not work with String
        krwdollar = ext.str.extract(r'((US Dollar =)\d{1}[,]?\d{3}[.]?\d{2})')[0].values[0]  #'US Dollar' after parsing
        krw = float(krwdollar[-8:].replace(',',''))
        return krw
    except:
        print("\nExcept >>> exchange_check() Error\n")
        alarm()
        pass

    
def alarm():
    b = jupyter_beeper.Beeper()
    b.beep(frequency=150, secs=0.5, blocking=True)
    b.beep(frequency=200, secs=0.7, blocking=True)
    
    
def alarm_open(times):
    b = jupyter_beeper.Beeper()
    for c in range(1,times+1):
        time.sleep(0.3)
        for i, j in zip([150,200,250], [0.5, 0.5, 0.7]):
            b.beep(frequency=i, secs=j, blocking=True)
    
    
def emailsend_to_server(message):
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)

    # screen message
    print('\n')
    print(f">>> Email sent: {message}")
    print('\n\n')
    
    # text to voice 
    voice_message(message)
    time.sleep(5)
    
    
def voice_message(message):
    text = message
    speech = gTTS(text = message, lang = 'en', slow = False)
    speech.save("text.mp3")   
    webbrowser.open("text.mp3")        

    
def voice_text():
    r = sr.Recognizer()
    
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        
        print('Please say...')        
        audio = r.listen(source)       
        print('Recognizing...')
#         try:
#             print('Audio Recognized: \n ' + r.recognize_google(audio))

#         except Exception as e:
#             print('Error: ' + str(e))
        
        with open('recoded_audio.wav', 'wb') as f:
            f.write(audio.get_wav_data())
            
        return r.recognize_google(audio)

    
def voice_text_num():
    voice_message('Please tell me the threshold')
    voice_text_converted = voice_text()
    voice_message(voice_text_converted)
    return int(voice_text_converted)

# threshold_value = voice_text_num()
# threshold_value
    
    
def exrate():
    print('\n\n' + '\033[1m' + format(int(sum(krwex))) + '\033[0m' + ' Won Monitoring', '\n')

    krw = exchange_check()
    print(np.round(krw,2), '\n')
    
    import pytz
    ttoday = date.today()
    tz_NY = pytz.timezone('America/New_York') 
    datetime_NY = datetime.now(tz_NY) 
    print(datetime_NY.strftime("%H:%M:%S"), ttoday, '\n\n\n') #to put the number, date, hour
    #####################
    if krw < sum(krwex):
    #####################  
        krwex.append(-10)    

        # alarm
#         webbrowser.open("Nuclear Missile Ready.mp3")
        
        # email
        message = """\
Subject: [{}] {}""".format('Korean Won', '%.f' %krw)
        emailsend_to_server(message) 
        
    else:
        pass  
        
    
def slistemail(portfolio):
    print('\n'+current_time()+'\n')
    
    date = currentdate 
    toplist = portfolio   
    rank = pd.DataFrame()

    cal1 = []
    cal2 = []
    for i in range(len(toplist)):
        cal1.append(i+1)
        cal2.append(toplist[i])
        
    rank[date] = cal1
    rank[''] = cal2
    
    toplist = rank.set_index(date)
   
    df = pd.read_csv('sean_emailgroup.csv')
    sender_email = "aicpasean@gmail.com"
    password = sean.gmail_api()

    sentlist = []

    for i in range(len(df.name)):

        try:
            c=i+1
            print(df.name[i],": email sent : #%d" %c)

            subject = "[{}] Blue-chip".format(date)
            body = """\
Hi {}, 

Here are top stocks.
{}


Refer to attachment for more details. 

Yours Sincerely, 
Sean Yeon, CPA, MBA""".format(df.name[i], toplist)   

            # Create a multipart message and set headers
            message = MIMEMultipart()
            message["From"] = sender_email
            message["To"] = df.name[i]
            message["Subject"] = subject
            message["Bcc"] = df.name[i]  # Recommended for mass emails

            # Add body to email
            message.attach(MIMEText(body, "plain"))

            filename = "srank_portfolio.csv"  # In same directory as script

            # Open PDF file in binary mode
            with open(filename, "rb") as attachment:
                # Add file as application/octet-stream
                # Email client can usually download this automatically as attachment
                part = MIMEBase("application", "octet-stream")
                part.set_payload(attachment.read())

            # Encode file in ASCII characters to send by email    
            encoders.encode_base64(part)

            # Add header as key/value pair to attachment part
            part.add_header(
                "Content-Disposition",
                f"attachment; filename= {filename}",
            )

            # Add attachment to message and convert message to string
            message.attach(part)
            text = message.as_string()

            # Log in to server using secure context and send email
            context = ssl.create_default_context()
            with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
                server.login(sender_email, password)
                server.sendmail(sender_email, df.email[i], text)

    #         # record time sent
    #         today = date.today()
    #         tz_NY = pytz.timezone('America/New_York') 
    #         datetime_NY = datetime.now(tz_NY)

    #         # append records
    #         sentlist.append([df.name[i], df.email[i], datetime_NY.strftime("%H:%M:%S")])

        except:
            ('\n%s email error' %df.name[i])

    # sentlist = pd.DataFrame(sentlist, columns = [['sent', 'email', 'time']])

    # record date sent
    # now = today.strftime("%Y,%m,%d")
    # y = now[0:4]
    # m = now[5:7]
    # d = now[8:10]
    # date = y+m+d    
    # sentlist['date'] = date

    # save file as csv

    # sentlist.to_csv('email_sent.csv', index = False)
    # # sentlist.to_csv('email_sent %s.csv' %date, index = False)
    # sentlist = pd.read_csv('email_sent %s.csv'%date)
     
        
def tarot():
    f = open('tarot.json')
    data = json.load(f)
    f.close()
    df = pd.DataFrame(data['cards'])
    ramdom_pick = random.choice(df.loc[random.choice(np.arange(len(df['fortune_telling'])))]['meanings'][random.choice(['light', 'shadow'])])
    ramdom_pick = random.choice(df.loc[random.choice(np.arange(len(df['fortune_telling'])))]['meanings'][random.choice(['light', 'shadow'])])
    return ramdom_pick


def robin_login():   
    login = rs.login('aicpasean@gmail.com', sean.robin_api())
    my_stocks = rs.build_holdings()
    rs.logout()
    robin = pd.DataFrame(my_stocks)
    robin = robin.T.reset_index()
    return robin


def perf_cal(robin):
    stock_value = sum(robin.price.astype(float) * robin.quantity.astype(float))
    stock_cost = sum(robin.average_buy_price.astype(float) * robin.quantity.astype(float))
    stock_return = (stock_value-stock_cost)/stock_cost
    return int(stock_return*100)


def real_performance_selma():
    
    sss_perf = pd.read_csv('sss_perf.csv')

    # get realized gain/loss and holding period
    realized_equity_change_sum = int(sss_perf.equity_change.sum())
    realized_percent_change_avg = np.round(sss_perf.percent_change.mean(), 2)
    realized_holding_period = int(sss_perf.holding_period.mean())

    # get unrealized gain/loss and holding period
    df = robin_login()
    df = df.loc[-df['index'].isin(dead_stock)]
    unrealized_equity_change_sum = int(np.array([float(i) for i in df.equity_change.tolist()]).sum())
    unrealized_percent_change_avg = np.round(np.array([float(i) for i in df.percent_change.tolist()]).mean(), 2)
    df = pd.read_csv('holding_period.csv')
    df = df.loc[-df['index'].isin(dead_stock)]
    if math.isnan(df.days.mean()) == False:
        unrealized_holding_period = int(df.days.mean())
    else:
        unrealized_holding_period = 0

    # combine
    agg_equity_change_sum = realized_equity_change_sum + unrealized_equity_change_sum
    agg_percent_change_avg = np.round((realized_percent_change_avg + unrealized_percent_change_avg)*0.5, 2)
    agg_holding_period = int((realized_holding_period + unrealized_holding_period)/2)

    # group by date
    df = sss_perf.groupby('date').mean().reset_index()
    df['equity_change'] = sss_perf.groupby('date').sum().reset_index()['equity_change']

    # plot   
    fig, ax = plt.subplots()
    fig.set_size_inches(20, 9)
    df.date = df.date.astype(str)
    ax.plot(df.date, df['percent_change'], color = 'blue')
    ax.set_xlabel('\nDate')
    ax.set_ylabel('\nRealized Gain/Loss (%)', color = 'blue')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
    ax.grid(axis = 'x')
    ax1 = ax.twinx()
    ax1.bar(df.date, df['equity_change'], color = 'green', alpha=0.3)
    ax1.set_ylabel('Realized Value ($)', color = 'green')
    ax1.grid(axis = 'y')
    ax.set_xticklabels(df.date, rotation = 90)
    ax.plot(df.date, [agg_percent_change_avg]*df.shape[0], color = 'blue', linestyle = '-', alpha=0.3, linewidth = 10, label ='Avg Gain (Unrealized + Realized)')
    ax.legend()

    # ###############################################################
    # # annotate - version update details 
    # annotate_dict = {}
    # annotate_dict['2023-01-06'] = 'Ver.3.7 - New ML Model' 
    # annotate_dict['2023-03-01'] = 'Loss Cut by DMA' 
    # annotate_dict['2023-04-03'] = 'Ver.3.8 - Strategy def'  
    # annotate_dict['2023-04-10'] = 'pandas_ta - RSI, ADX' 
    # ###############################################################

    # for i, v in annotate_dict.items():
    #     i = df[df.date <= i].tail(1).date.values[0]
    #     plt.annotate(xy = [i, df[df.date == i]['percent_change'].values], text = v)

    plt.title(f'{current_time()} - Realtime Performance')
    plt.show()

    print('\n\n\n', '\033[1m' + "_____________________________________________________" + '\033[0m')
    print('\n', f" • Realized Equity Change (aggregrated):    " + '\033[1m' + f"{realized_equity_change_sum} USD" + '\033[0m')
    print('\n', f" • Realized Percentage Change (avg):        " + '\033[1m' + f"{realized_percent_change_avg} %" + '\033[0m')
    print('\n', f" • Realized Holding Period (avg):           " + '\033[1m' + f"{realized_holding_period} days" + '\033[0m')
    print('\n', '\033[1m' + "-----------------[Excl. dead stocks]-----------------" + '\033[0m')
    print('\n', f" • Unrealized Equity Change (aggregrated):  " + '\033[1m' + f"{unrealized_equity_change_sum} USD" + '\033[0m')
    print('\n', f" • Unrealized Percentage Change (avg):      " + '\033[1m' + f"{unrealized_percent_change_avg} %" + '\033[0m')
    print('\n', f" • Unrealized Holding Period (avg):         " + '\033[1m' + f"{unrealized_holding_period} days" + '\033[0m')
    print('\n', '\033[1m' + "-----------------[Excl. dead stocks]-----------------" + '\033[0m')
    print('\n', f" • Aggregrated Equity Change (aggregrated): " + '\033[1m' + f"{agg_equity_change_sum} USD" + '\033[0m')
    print('\n', f" • Aggregrated Percentage Change (avg):     " + '\033[1m' + f"{agg_percent_change_avg} %" + '\033[0m')
    print('\n', f" • Aggregrated Holding Period (avg):        " + '\033[1m' + f"{agg_holding_period} days" + '\033[0m')
    print('\033[1m' + "______________________________________________________" + '\033[0m')
    print('\033[1m' + '\n\n\n• Recent' + '\033[0m' +  ' - Top 5')
    display(sss_perf.sort_values('date', ascending = False).reset_index(drop = True).head())
    print('\033[1m' + '\n• Gain' + '\033[0m' +  ' - Top 5')
    display(sss_perf.nlargest(5,'percent_change').reset_index(drop = True))
    print('\033[1m' + '\n• Loss' + '\033[0m' +  ' - Top 5')
    display(sss_perf.nsmallest(5,'percent_change').reset_index(drop = True))
    print('\n\n\n')

    voice_message(f"""\
    Here is our real performance update
    Aggregrated Equity Change is {agg_equity_change_sum} dollars
    Aggregrated Percentage Change is {agg_percent_change_avg} %
    Aggregrated Holding Period is {agg_holding_period} days
    """)   

    time.sleep(7)  
    
    
def dmv_plot(ticker):
    # generate a moving average plot per each day
    
    df = yfinance_df_rsi(ticker)[['Date', 'Adj Close']]
    df = df.rename(columns = {'Adj Close':'Current Price'})
    focus_days = [5, 10, 20, 30, 60, 120]

    for i in focus_days:
        df[f"{i}_EMA"] = df['Current Price'].ewm(span= i, adjust=False).mean()

    df.dropna(inplace = True)

    plt.figure(figsize = (16,9))
    plt.title(f'Exponential Moving Average - {ticker}')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    for column in df.columns[1:]:
        plt.errorbar(data = df, x = 'Date', y = column, linewidth = 1.5, label = column)
        plt.annotate(xy = [df[-1:].Date.values, df[-1:][column].values], text = column)

    # plt.legend()
    plt.show()

    # buy-condition test
    analysis = analysis_price(ticker)
    print('• Loss-Cut Condition: ' + '\033[1m'+ f'{analysis[8]}\n' + '\033[0m')
    
    
def analysis_price(ticker):
    with open("averageline.txt", 'r') as f:
        averageline = [line.rstrip('\n') for line in f]
        averageline = int(averageline[0])

    df = yfinance_df(ticker)
    
    # fill ZERO value with the previous value
    for i in df.all().reset_index().values:
        if i[1] == False:
            df[i[0]] = df[i[0]].replace(to_replace=0, method='ffill')
    
    df['VC'] = df['Volume'].diff()
    df['PC'] = df['Adj Close'].diff()
    df['PriceC_by_VolumeC'] = df['PC']/df['VC']
  
    # price check
    ticker_price_mean = df.tail(averageline)['Adj Close'].mean()

    ticker_price_yesterday = df.tail(2).head(1)['Adj Close'].mean()
    ticker_price_current = df.tail(1)['Adj Close'].mean()
    ticker_price_drop = ((ticker_price_yesterday - ticker_price_current) / ticker_price_yesterday)*100
    ticker_price_surge = ((ticker_price_current - ticker_price_yesterday) / ticker_price_yesterday)*100
    
    # ema
    ticker_price_longest_mean = df['Adj Close'].ewm(span= 120, adjust=False).mean().values[-1]
    ticker_price_longer_mean = df['Adj Close'].ewm(span= 60, adjust=False).mean().values[-1]
    ticker_price_midest_mean = df['Adj Close'].ewm(span= 30, adjust=False).mean().values[-1]
    ticker_price_mider_mean = df['Adj Close'].ewm(span= 20, adjust=False).mean().values[-1]
    ticker_price_mid_mean = df['Adj Close'].ewm(span= 10, adjust=False).mean().values[-1]
    ticker_price_short_mean = df['Adj Close'].ewm(span= 5, adjust=False).mean().values[-1]
    
    ticker_price_trend_surge_1 = ticker_price_longest_mean < ticker_price_longer_mean < ticker_price_current
    ticker_price_trend_surge_2 = ticker_price_longest_mean < ticker_price_longer_mean < ticker_price_midest_mean < ticker_price_current
    ticker_price_trend_surge_3 = ticker_price_longest_mean < ticker_price_longer_mean < ticker_price_mider_mean < ticker_price_current
    ticker_price_trend_surge_4 = ticker_price_longest_mean < ticker_price_longer_mean < ticker_price_mid_mean < ticker_price_current
    ticker_price_trend_surge_5 = ticker_price_longest_mean < ticker_price_longer_mean < ticker_price_short_mean < ticker_price_current

    ticker_price_trend_drop_1 = ticker_price_longest_mean < ticker_price_current < ticker_price_longer_mean
    ticker_price_trend_drop_2 = ticker_price_longer_mean < ticker_price_current < ticker_price_longest_mean
    ticker_price_trend_drop_3 = ticker_price_current < ticker_price_longer_mean < ticker_price_longest_mean
    ticker_price_trend_drop_4 = ticker_price_current < ticker_price_longest_mean < ticker_price_longer_mean 
    ticker_price_trend_drop = ticker_price_trend_drop_1 + ticker_price_trend_drop_2 + ticker_price_trend_drop_3 + ticker_price_trend_drop_4
    ticker_price_trend_drop = (ticker_price_trend_drop > 0)    

    ticker_price_trend = (df[-1:]['Adj Close'].mean()-df[-20:-19]['Adj Close'].mean())/df[-20:-19]['Adj Close'].mean()*100
    
    ticker_price_long_hill = df[-60:-20]['Adj Close'].max()
    ticker_price_short_hill = df[-20:]['Adj Close'].max()
    ticker_price_hill = ticker_price_long_hill < ticker_price_short_hill
    
    df = df[-30:] # not fixed
    Price_Volume_botton_day = df.nsmallest(1, 'PriceC_by_VolumeC').Date.astype(str).values[0]
    day_minus = datetime.today() - timedelta(days = 5) # not fixed
    day_minus = str(day_minus)[:10] # fixed
    ticker_vol_price = day_minus > Price_Volume_botton_day

    return(ticker_price_drop, 
           ticker_price_trend_surge_1, 
           ticker_price_surge, 
           ticker_price_mean, 
           ticker_price_current, 
           ticker_price_trend, 
           ticker_price_hill, 
           ticker_price_trend_surge_2, 
           ticker_price_trend_drop, 
           ticker_vol_price, 
           ticker_price_trend_surge_3,
           ticker_price_trend_surge_4,
           ticker_price_trend_surge_5)

    ## For moving average tuning
    # for i in range(1,30):
    #     mostrecentdate = 20220101 + i
    #     analysis_price('^GSPC', mostrecentdate)[1]
    #     print('\n\n')

    
if __name__ == '__main__':
    
    # start declearing def() 
    currentdate_event = currentdate

    if currentdate != currentdate_verify:
        print(f"Exception >>> Input: {currentdate} vs Actual: {currentdate_verify}")
        mostrecentdate = str(mostrecentdate)[0:4]+'-'+str(mostrecentdate)[4:6]+'-'+str(mostrecentdate)[6:8]
        currentdate = str(currentdate)[0:4]+'-'+str(currentdate)[4:6]+'-'+str(currentdate)[6:8]  
        
#         # code cell break ###############
#         class StopExecution(Exception):
#             def _render_traceback_(self):
#                 pass

#         raise StopExecution
#         # code cell break ###############

    else:
        print("> Date Verified")
        with open("mostrecentdate.txt", 'w') as f:
            f.write(str(mostrecentdate)) 
        with open("currentdate.txt", 'w') as f:
            f.write(str(currentdate)) 

        mostrecentdate = str(mostrecentdate)[0:4]+'-'+str(mostrecentdate)[4:6]+'-'+str(mostrecentdate)[6:8]
        currentdate = str(currentdate)[0:4]+'-'+str(currentdate)[4:6]+'-'+str(currentdate)[6:8]
        
        # remove the previous files
        monitoring_previous_exist = exists(f'qed_monitoring_done_{mostrecentdate}.txt')
        if monitoring_previous_exist == True:
            os.remove(f'qed_monitoring_done_{mostrecentdate}.txt')

        focus_previous_exist = exists(f'focused_{mostrecentdate}.txt')
        if focus_previous_exist == True:
            os.remove(f'focused_{mostrecentdate}.txt')
        
    ydate = str(yfinance_df_setting('^GSPC')[-1:].Date.values[0])[:10]

    if ydate == currentdate:
        print("> yFinance Verified (Direct Call: yf.download(ticker))")

    else:
        print("\nException >>> yFinance\n")
        print(f'yFinance: {ydate} vs Currentdate: {currentdate}')
    #     print(yfinance_df_setting('^GSPC')[-1:])
        alarm()

voice_message("""\
      Hi Sean. I'm Selma.""")
time.sleep(3)


print('\n> Modules #1 imported')

In [70]:
# for sentiment analysis
min_market_sentiment = 0
mega_tickers = [
    'GOOGL', 'META', 'TCEHY', 'SNAP', 'BIDU', 'MTCH',
    'QCOM', 'INTC', 'AMD', 'TXN', 'AVGO', 'MU',
    'AAPL', 'DELL', 'HPQ', 'FUJIY', 'CAJPY', 'STX',
    'MSFT', 'ORCL', 'NOW', 'FTNT', 'PANW', 'VMW',
    'NVDA', 'TSM', 'AVGO', 'INTC', 'AMD', 'QCOM',
    'CRM', 'ADBE', 'SAP', 'INTU', 'TEAM', 'DASTY',
    'TSLA', 'TM', 'GM', 'MBGAF', 'F', 'LCID',
    'AMZN', 'BABA', 'JD', 'MELI', 'PDD', 'DASH',
    'V', 'MA', 'PYPL', 'ADP', 'SQ', 'FISV',
    'JPM', 'BAC', 'WFC', 'RY', 'TD', 'HSBC',
    'JNJ', 'PFE', 'LLY', 'NVO', 'MRK', 'AZN',
    'WMT', 'COST', 'WMMVY', 'CRRFY', 'BJ', 'ASAI',
    'XOM', 'CVX', 'SHEL', 'BP', 'TTE', 'EQNR',
    'AMT', 'CCI', 'EQIX', 'PSA', 'DLR', 'SBAC',
    'DIS', 'NFLX', 'SPOT', 'LYV', 'WMG', 'ROKU']


def treemap(df, column):
    print('\033[1m' + f'{column}' + '\033[0m')
    df = df.drop(index = df[df['Symbol'] == 'GOOG'].index.values[0]).reset_index(drop = True)
    df = df[df[column] > 0]
    fig = px.treemap(df, path=[px.Constant("S&P 500"), 'GICS Sector', 'Symbol'], 
                     values = column,
                     color = 'Security', 
                     hover_data = ['5D Perf'],
                     color_continuous_scale='RdBu',
                     color_continuous_midpoint=np.average(df['P/E TTM'], weights=df['P/E FWD']))
    fig.update_traces(root_color="white")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.show()
    

def focus_record(ticker):
    # create & update focused file if true
    from os.path import exists
    focus_today_exist = exists(f'focused_{currentdate}.txt')
    if focus_today_exist == False:
        with open(f'focused_{currentdate}.txt', 'w') as f:
            f.write(str(ticker))
        voice_message(f"""\
                      The initial focus file has been created with {ticker}""")
    else:
        with open(f'focused_{currentdate}.txt', 'r') as f:
            focused = [line.rstrip('\n') for line in f]
        focused.append(ticker)
        with open(f'focused_{currentdate}.txt', 'w') as f:
            for s in focused:
                f.write(str(s) + '\n')


def volatility_check(ticker):
    df = yfinance_df_rsi(ticker).tail(averageline).reset_index()
    price_change = []
    for i in range(df.shape[0]):
        if i == 0:
            price_change.append(0)
        else:
            price_change.append((df['Adj Close'].values[i] - df['Adj Close'].values[i-1]) / df['Adj Close'].values[i-1])
    df['price_change'] = price_change   
    return np.std(df['price_change']) * np.sqrt(252)


def mdd_cal(tickers):
    '''
    input: ticker in list
    output: average value
    '''
    mdd_avg = []
    for ticker in tickers:
        print(f'{mdd_lookback_days} days MDD calculating - Ticker: {ticker}', end = '                                                  \r')
        df = yfinance_df_rsi(ticker)[-mdd_lookback_days:]
        mdd = (df['Adj Close'].max() - df['Adj Close'].min()) / df['Adj Close'].max()
        mdd_avg.append(mdd)
        print(end = '                                                              \r')
    return np.average(mdd_avg)


def candle_review(ticker):
    # get the price change(%)
    percent_change_plot = np.round(analysis_price(ticker)[2], 2)
    print('\n• Ticker: ' + '\033[1m' + f'{ticker} ==> ' + f'{percent_change_plot} %\n' + '\033[0m') 
    
    df = pd.read_csv(f'candle_analysis_{ticker}_{currentdate}.csv', index_col = 'Unnamed: 0')
    df.Minutepassed = df.Minutepassed.astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(20, 9)

    ax.plot(df.Minutepassed, df['Adj Close'], color = 'blue')
    ax.set_xlabel('Minutepassed')
    ax.set_ylabel('Price', color = 'blue')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
    ax.grid(axis = 'x')

    ax1 = ax.twinx()
    ax1.bar(df.Minutepassed, df['VolumeC_per_PriceC'], color = 'red')
    ax1.set_ylabel('Volume Change per Price Change', color = 'red')
    ax1.grid(axis = 'y')

    # ax.set_xticklabels(df.Date, rotation = 45)
    plt.title(f'Volume Change per Price Change - {VC_per_PC_limit_V} ({ticker})')
    plt.show()
    print('\n\n')
        
    fig, ax = plt.subplots()
    fig.set_size_inches(20, 9)

    ax.plot(df.Minutepassed, df['Adj Close'], color = 'blue')
    ax.set_xlabel('Minutepassed')
    ax.set_ylabel('Price', color = 'blue')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
    ax.grid(axis = 'x')

    ax1 = ax.twinx()
    ax1.bar(df.Minutepassed, df['VolumeSpeed_vs_mean'], color = 'red')
    ax1.set_ylabel('Volume Speed vs Mean', color = 'red')
    ax1.grid(axis = 'y')

    # ax.set_xticklabels(df.Date, rotation = 45)
    plt.title(f'Volume Speed vs Mean - {VS_vs_MEAN_limit} ({ticker})')
    plt.show()
    print('\n\n')
    
    fig, ax = plt.subplots()
    fig.set_size_inches(20, 9)

    ax.plot(df.Minutepassed, df['Adj Close'], color = 'blue')
    ax.set_xlabel('Minutepassed')
    ax.set_ylabel('Price', color = 'blue')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
    ax.grid(axis = 'x')

    ax1 = ax.twinx()
    ax1.bar(df.Minutepassed, df['Volume_speed_acceleration'], color = 'red')
    ax1.set_ylabel('Volume Speed Acceleration', color = 'red')
    ax1.grid(axis = 'y')

    # ax.set_xticklabels(df.Date, rotation = 45)
    plt.title(f'Volume Speed Acceleration [abs] - {VS_Accel_limit} ({ticker})')
    plt.show()
    
    # display df
    df_1 = df[df['VolumeC_per_PriceC'] >= VC_per_PC_limit_V]
    df_2 = df_1[df_1['VolumeSpeed_vs_mean'] >= VS_vs_MEAN_limit]
    df_3 = df_2[np.absolute(df_2['Volume_speed_acceleration']) >= VS_Accel_limit]
    df_3 = df_3.reset_index(drop = True)
    display(df_3[['Minutepassed', 'VolumeC_per_PriceC', 'VolumeSpeed_vs_mean', 'Volume_speed_acceleration']])
    print('\n')
    if len(df_3) != 0:        
        sentiment_decision = (sentiment_analysis(ticker) > 0)
        print('Sentiment Analysis Result: ' + '\033[1m' + f'{sentiment_decision}' + '\033[0m')
        if sentiment_decision == True:
            strategy_analysis_main(ticker, 'on', 'off')
        else:
            pass
    else:
        print('\033[1m' + 'No Candle Sign' + '\033[0m')
    print('\n\n\n')
    

def candle_gathering():
    # candle gahtering starts after the sector valuation has been executed
    print(f'\n\n\n\n\n{current_time()}  Candle Gathering')
    
    if cyclecount%100 == 0:
        with open("portfolio_original.txt", 'r') as f:
            portfolio = [line.rstrip('\n') for line in f]
        with open("buymonitor_blacklist_original.txt", 'r') as f:
            buymonitor_blacklist_original = [line.rstrip('\n') for line in f]
        with open("buymonitor_blacklist.txt", 'w') as f:
            for s in buymonitor_blacklist_original:  
                f.write(str(s) + '\n')

    else:
        with open("portfolio_original.txt", 'r') as f:
            portfolio = [line.rstrip('\n') for line in f]
        with open("portfolio_superpass.txt", 'r') as f:
            portfolio_superpass = [line.rstrip('\n') for line in f]
        with open("etf_portfolio.txt", 'r') as f:
            etf_portfolio = [line.rstrip('\n') for line in f]
        with open("buymonitor_blacklist.txt", 'r') as f:
            buymonitor_blacklist = [line.rstrip('\n') for line in f]
        with open("inv_tickers.txt", 'r') as f:
            inv_tickers = [line.rstrip('\n') for line in f]
        with open("portfolio_sector_true.txt", 'r') as f:
            portfolio_sector_true = [line.rstrip('\n') for line in f]
        with open("portfolio_sector_valuation_true.txt", 'r') as f:
            portfolio_sector_valuation_true = [line.rstrip('\n') for line in f]
            
        # exclude Sector valuation - False tickers. buymonitor() will only use candle-gathering for these tickers
        sector_valuation_ticker = pd.read_csv('sector_valuation.csv')
        sector_valuation_ticker = sector_valuation_ticker[sector_valuation_ticker['Date'] == currentdate]
        sector_valuation_ticker = sector_valuation_ticker.reset_index(drop = True)
        
        if len(sector_valuation_ticker) != 0:
            # Candle tickers    
            purchased = pd.read_csv('robin_all.csv')['index'].values.tolist()
            stock = portfolio_sector_true + portfolio_sector_valuation_true + portfolio_superpass + etf_portfolio + inv_tickers + purchased + index_candle_tickers

            stock = [i for i in stock if i not in buymonitor_blacklist]
            stock = [i for n, i in enumerate(stock) if i not in stock[:n]] #duplicated                

            # shuffle the order considering the loop has a break
            random.shuffle(stock)
            
            from tqdm.notebook import tqdm
            for i in tqdm(range(len(stock))):
                time.sleep(0.3) 
                try:
                    print('Candle gathering: ', stock[i], end = '          \r')
                    candle_sign = volume_vs_sign(stock[i])
                    if candle_sign == 3:
                        print('\nBuy-Sign detected. Break Candle gathering.\n')
                        
                        try:
                            # create bypass token to read in 'buymonitor()'
                            with open(f"candle_token_{stock[i]}.txt", 'w') as f:
                                f.write(str(timechecknow()))
                            print("\n\nFile Created >>> " + '\033[1m' + f"candle_token_{stock[i]}.txt\n\n" + '\033[0m')
                        except:
                            print("\n\nExcept >>> File Creation Failure: " + '\033[1m' + f"candle_token_{stock[i]}.txt\n\n" + '\033[0m')
                            alarm()
                            
                        break

                except KeyError:
                    alarm()
                    print('Except >>> Candle gathering:', stock[i])

                    # append to the purchased ticker to monitoring blacklist; only purchase once a day
                    with open("buymonitor_blacklist.txt", 'r') as f:
                        buymonitor_blacklist = [line.rstrip('\n') for line in f]
                    buymonitor_blacklist.append(stock[i])
                    with open("buymonitor_blacklist.txt", 'w') as f:
                        for s in buymonitor_blacklist:  
                            f.write(str(s) + '\n') 
                    pass
            with open("_Candle Time Checker.txt", 'w') as f:
                f.write(str(current_time()))  

        else:
            print(f'\n\n{current_time()}  The initial sector valuation has not been executed.\n\n')
            time.sleep(180)


def rsi(ticker):
    df = yfinance_df_rsi(ticker)

    ## 14_Day RSI
    df['RSI'] = ta.rsi(df['Adj Close'], length = 14)

    # moving average - 70 days
    date_list = df['Date'][69:].values
    mvp = 70
    mavg = pd.DataFrame()
    for column in df.columns[1:]:
        mv_change = np.array(df[column])
        mv = []
        for i in range(len(mv_change)-mvp+1):
            mv.append(np.average(mv_change[i:mvp+i]))
            i+=1
        mavg[column] = pd.DataFrame(mv)
    df_70 = mavg
    df_70['Date'] = date_list
    df_70 = df_70.rename(columns = {'Adj Close' : 'Adj Close_70D', 'RSI' : 'RSI_70D'})[['Date', 'Adj Close_70D', 'RSI_70D']]

    # moving average - 40 days
    date_list = df['Date'][39:].values
    mvp = 40
    mavg = pd.DataFrame()
    for column in df.columns[1:]:
        mv_change = np.array(df[column])
        mv = []
        for i in range(len(mv_change)-mvp+1):
            mv.append(np.average(mv_change[i:mvp+i]))
            i+=1
        mavg[column] = pd.DataFrame(mv)
    df_40 = mavg
    df_40['Date'] = date_list
    df_40 = df_40.rename(columns = {'Adj Close' : 'Adj Close_40D', 'RSI' : 'RSI_40D'})[['Date', 'Adj Close_40D', 'RSI_40D']]
    
    # moving average - 20 days
    date_list = df['Date'][19:].values
    mvp = 20
    mavg = pd.DataFrame()
    for column in df.columns[1:]:
        mv_change = np.array(df[column])
        mv = []
        for i in range(len(mv_change)-mvp+1):
            mv.append(np.average(mv_change[i:mvp+i]))
            i+=1
        mavg[column] = pd.DataFrame(mv)
    df_20 = mavg
    df_20['Date'] = date_list
    df_20 = df_20.rename(columns = {'Adj Close' : 'Adj Close_20D', 'RSI' : 'RSI_20D'})[['Date', 'Adj Close_20D', 'RSI_20D']]
    
    # merge
    df['Date'] = df['Date'].astype('datetime64[ns]')
    df = pd.merge(df, df_20, how = 'inner', on = 'Date')
    df = pd.merge(df, df_40, how = 'inner', on = 'Date')
    df = pd.merge(df, df_70, how = 'inner', on = 'Date')
    
    # accel Creation for 70D and 40D
    RSI_70D_change = []
    for i in range(len(df['RSI_70D'])):
        if i == 0:
            RSI_70D_change.append(0)
        else:
            change = (df['RSI_70D'][i] - df['RSI_70D'][i-1]) / df['RSI_70D'][i-1] *100
            RSI_70D_change.append(change)
    df['RSI_70D_change'] = RSI_70D_change
    df = df.drop(index = 0).reset_index(drop = True)

    RSI_70D_change_accel = []
    for i in range(len(df['RSI_70D_change'])):
        if i == 0:
            RSI_70D_change_accel.append(0)
        else:
            change = np.abs((df['RSI_70D_change'][i] - df['RSI_70D_change'][i-1]) / df['RSI_70D_change'][i-1] *100)
            RSI_70D_change_accel.append(change)
    df['RSI_70D_change_accel'] = RSI_70D_change_accel
    df = df.drop(index = 0).reset_index(drop = True)
        
    RSI_40D_change = []
    for i in range(len(df['RSI_40D'])):
        if i == 0:
            RSI_40D_change.append(0)
        else:
            change = (df['RSI_40D'][i] - df['RSI_40D'][i-1]) / df['RSI_40D'][i-1] *100
            RSI_40D_change.append(change)
    df['RSI_40D_change'] = RSI_40D_change
    df = df.drop(index = 0).reset_index(drop = True)

    RSI_40D_change_accel = []
    for i in range(len(df['RSI_40D_change'])):
        if i == 0:
            RSI_40D_change_accel.append(0)
        else:
            change = np.abs((df['RSI_40D_change'][i] - df['RSI_40D_change'][i-1]) / df['RSI_40D_change'][i-1] *100)
            RSI_40D_change_accel.append(change)
    df['RSI_40D_change_accel'] = RSI_40D_change_accel
    df = df.drop(index = 0).reset_index(drop = True)
    
    # save
#     df.to_csv(f"./Backup_rsi/rsi_{ticker}_{currentdate}.csv", index = False)

#     if ticker in index_ticker or ticker in etf_ticker:
#         df.to_csv(f"rsi_{ticker}.csv", index = False)
    df.dropna(inplace = True)
    df.to_csv('rsi_report.csv', index = False)

    # buy condition 
    rsi_max_basic = df[-1:]['RSI'].mean() < rsi_max_cut
    rsi_70_min_plus = df.tail(3)['RSI_70D'].mean() >= 50
    rsi_40_min_plus = df.tail(3)['RSI_40D'].mean() >= 40
    price_avg_20_70 = df.tail(3)['Adj Close_20D'].mean() > df.tail(3)['Adj Close_70D'].mean()
    rsi_surge = df.tail(20)['RSI_40D_change'].mean() >= 0 
    rsi_accel = df.tail(20)['RSI_40D_change_accel'].mean() > np.percentile(df['RSI_40D_change_accel'], 75)
    
    # sell condition
    rsi_70_min_minus = df.tail(3)['RSI_70D'].mean() < 50 
    rsi_40_min_minus = df.tail(3)['RSI_40D'].mean() < 40 
    price_avg_70_20 = df.tail(3)['Adj Close_20D'].mean() < df.tail(3)['Adj Close_70D'].mean()
    
    # sell-alarm condition
    rsi_70_min_minus = df.tail(3)['RSI_70D'].mean() < 50
    rsi_40_min_minus = df.tail(3)['RSI_40D'].mean() < 40
    rsi_drop = df.tail(20)['RSI_40D_change'].mean() < 0 
    
    buy = rsi_max_basic and rsi_70_min_plus and rsi_40_min_plus and price_avg_20_70 
    sell = rsi_70_min_minus or rsi_40_min_minus or price_avg_70_20 
    sell_alarm = rsi_70_min_minus and rsi_40_min_minus and price_avg_70_20 and rsi_drop and rsi_accel
    loss_cut = rsi_70_min_minus and rsi_40_min_minus and price_avg_70_20 
    portfolio = rsi_70_min_plus and rsi_40_min_plus

    return [buy, sell, sell_alarm, loss_cut, portfolio]


def rsi_plot(ticker):
    ticker_name = ticker
    ticker_rsi = rsi(ticker)
    ticker_rsi = pd.read_csv('rsi_report.csv')[-review_plot_lookbackdays:]

    fig, ax = plt.subplots()
    fig.set_size_inches(20, 9)

    ax.plot(ticker_rsi.Date, ticker_rsi['Adj Close'], color = 'blue')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price', color = 'blue')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
#     ax.grid(axis = 'x')

    ax1 = ax.twinx()
    ax1.plot(ticker_rsi.Date, ticker_rsi['RSI'], color = 'red')
    ax1.set_ylabel('RSI', color = 'red')
    ax1.grid(axis = 'y')

    ax.set_xticklabels(ticker_rsi.Date, rotation = 90)
    ax1.plot(ticker_rsi.Date, [30]*ticker_rsi.shape[0] ,color = 'black', linestyle = '--')
    ax1.plot(ticker_rsi.Date, [70]*ticker_rsi.shape[0] ,color = 'black', linestyle = '--')

    plt.title(f'Price vs RSI ({ticker_name})')
    plt.show()

    # Regression analysis
    import statsmodels.api as sm
    ticker_rsi['intercept'] = 1
    lm = sm.OLS(ticker_rsi['RSI'], ticker_rsi[['intercept', 'Adj Close']])
    results = lm.fit()

    rsi_coef = np.round(results.params.loc['Adj Close'], 2)
    rsi_rsq = np.round(results.rsquared*100, 2)

    print("• R-Squared: " + '\033[1m' + f"{rsi_rsq} %" + '\033[0m')
    print("• RSI per Price Change: " + '\033[1m' + f"{rsi_coef}" + '\033[0m')
    

def rsi_portfolio():
    ##### RSI Portfolio searching for S&P 500 tickers #####
    print('\n', '\033[1m' + '>>> RSI Portfolio Searching .....' + '\033[0m', '\n')
    voice_message("""\
        RSI Portfolio Searching starts""")        
    snp = pd.read_csv('snp500\snp_sector_index.csv').sort_values('Symbol')['Symbol'].to_list()
    rsi_portfolio = []
    c = 0
    for i in snp:
        c+=1
        print(f"({c}) RSI Checking - Ticker: {i}")
        try:
            if rsi(i)[0] == True:
                print(f"                                      >>> RSI into Portfolio - Ticker: {i}")
                rsi_portfolio.append(i)
            else:
                pass
        except:
            print(f"{i} is delisted.")
            pass

    with open('rsi_portfolio.txt', 'w') as f:
        for s in rsi_portfolio:
            f.write(str(s) + '\n')

    print(f"\n\nPortfolio created: {rsi_portfolio}")
    
    
def sector_stock_plot(stock_ticker):
    title_name = stock_ticker
    sector_ticker = pd.read_csv('./snp500/snp_sector_index.csv')
    sector_ticker = sector_ticker[sector_ticker['Symbol'] == stock_ticker]['Ticker'].values[0]
    
    stock_ticker = yfinance_df(stock_ticker)[-review_plot_lookbackdays:]
    sector_ticker = yfinance_df(sector_ticker)[-review_plot_lookbackdays:]
    stock_ticker.Date = stock_ticker.Date.astype(str)
    sector_ticker.Date = sector_ticker.Date.astype(str)

    # twin plot for sector vs stock
    fig, ax = plt.subplots()
    fig.set_size_inches(20, 9)

    ax.plot(stock_ticker.Date, stock_ticker['Adj Close'], color = 'blue')
    ax.set_xlabel('Date')
    ax.set_ylabel('Stock Price', color = 'blue')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
#     ax.grid(axis = 'x')

    ax1 = ax.twinx()
    ax1.plot(sector_ticker.Date, sector_ticker['Adj Close'], color = 'red')
    ax1.set_ylabel('Sector Price', color = 'red')
    ax1.grid(axis = 'y')

    ax.set_xticklabels(stock_ticker.Date, rotation = 90)

    plt.title(f'Ticker vs Sector ({title_name})')
    plt.show()
    
    # Regression analysis
    import statsmodels.api as sm
    stock_ticker['intercept'] = 1
    lm = sm.OLS(sector_ticker['Adj Close'].values, stock_ticker[['intercept', 'Adj Close']])
    results = lm.fit()

    stock_coef = np.round(results.params.loc['Adj Close'], 2)
    stock_rsq = np.round(results.rsquared*100, 2)

    print("• R-Squared: " + '\033[1m' + f"{stock_rsq} %" + '\033[0m')
    print("• Stock Coef. vs Sector: " + '\033[1m' + f"$ {stock_coef}\n\n\n" + '\033[0m')
    
    
def volume_price_period_plot(ticker):
    # candle chart for period
    
    title_name = ticker
    df = yfinance_df_rsi(ticker)[-review_plot_lookbackdays:]
    df.Date = df.Date.astype(str)

#     sc = MinMaxScaler()
#     vol_scaled = sc.fit_transform(df['Volume'].values.reshape(-1, 1))
#     price_scaled = sc.transform(df['Adj Close'].values.reshape(-1, 1))
#     df['Volume_scaled'] = np.absolute(vol_scaled)
#     df['Price_scaled'] = np.absolute(price_scaled)
#     df['PriceC_by_VolumeC'] = vol_scaled/price_scaled
    df['VC'] = df['Volume'].diff()
    df['PC'] = df['Adj Close'].diff()
    df['PriceC_by_VolumeC'] = df['PC']/df['VC']

    Up_Down = []
    for i in df['Adj Close'].diff():
        if i > 0:
            Up_Down.append(1)
        else:
            Up_Down.append(0)
    df['Up_Down'] = Up_Down

    fig, ax = plt.subplots()
    fig.set_size_inches(20, 9)

    ax.plot(df.Date, df['Adj Close'], color = 'blue')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price', color = 'blue')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
#     ax.grid(axis = 'x')

    ax1 = ax.twinx()
    
#     ax1.bar(df.Date, df['PriceC_by_VolumeC'], color = 'red')
#     ax1.set_ylabel('Volume per Price', color = 'red')
    ax1.bar(df.Date, df['PriceC_by_VolumeC'], color = 'red')
    ax1.set_ylabel('Price Change by Volume Change', color = 'red')
    
    ax1.grid(axis = 'y')

    ax.set_xticklabels(df.Date, rotation = 90)
    plt.title(f'Price Change by Volume Change ({title_name}) - Super Pass Lookback days: {superpass_buysign_lookbackdays}')
    plt.show()
    
    
def value_snp_pick():
    # need to adjust thresholds loosely
    snp = pd.read_csv('./snp500/snp500.csv').sort_values('Symbol')
    slist = snp.Symbol.values 
    
    # filter out Blacklist
    with open('buymonitor_blacklist.txt', 'r') as f:
        buymonitor_blacklist = [line.rstrip('\n') for line in f]
    slist_before_ratioAnalysis = [i for i in slist if i not in buymonitor_blacklist]
    
    snp_value_list_1 = slist_before_ratioAnalysis[:int(len(slist_before_ratioAnalysis)/2)]
    snp_value_list_2 = slist_before_ratioAnalysis[int(len(slist_before_ratioAnalysis)/2):]
    
    # ROE setting
    set_roe = pd.read_csv('snp500 %s.csv' %currentdate).describe().reset_index()['Return on Equity']
    set_roe_median = float(set_roe[5])
    set_roe_quartile = float(set_roe[4])
    
    list1 = ratio_analysis(snp_value_list_1, set_roe_median, set_roe_quartile)
    list2 = ratio_analysis(snp_value_list_2, set_roe_median, set_roe_quartile)
    
    snp_value_tickers = list1 + list2
    
    with open('snp_value_tickers.txt', 'w') as f:
        for s in snp_value_tickers:
            f.write(str(s) + '\n')
            
    
def ratio_analysis(slist_before_ratioAnalysis, set_roe_median, set_roe_quartile):

    # filter out Ratio Analysis
    slist_before_backTest = []
    import time
    print('Ratio Analysis:\n', slist_before_ratioAnalysis)
    print('\n')
    c = 1
    for i in slist_before_ratioAnalysis:
        try:
            print('Analysing: ', c, '/', len(slist_before_ratioAnalysis),'- Ticker: ', i, end = '                     \r')
            c+=1        
            time.sleep(0.3)
            
            df = pd.DataFrame([yf.Ticker(i).info]).fillna(0)
            df_fin = yf.Ticker(i).financials.reset_index()

            eps_growth_rate = (np.absolute((df['forwardEps'].values[0] - df['trailingEps'].values[0])) / df['trailingEps'].values[0]) * 100
            ebit_4yr = df_fin[df_fin['index'] == 'Ebit'].iloc[:, 1:].values
            rev_4yr = df_fin[df_fin['index'] == 'Total Revenue'].iloc[:, 1:].values
            try:
                ebit_ratio_4yr = ebit_4yr / rev_4yr
            except ZeroDivisionError:
                pass
            if type(ebit_ratio_4yr) == float:
                ebit_ratio_4yr = np.flip(ebit_ratio_4yr).tolist()
            else:
                ebit_ratio_4yr = np.flip(ebit_ratio_4yr).tolist()[0]
            avg_ebit_ratio_4yr = np.average(ebit_ratio_4yr) # >10%
            if type(ebit_ratio_4yr) == float:
                ebit_ratio_4yr_diff = np.absolute(ebit_ratio_4yr)
            else:
                ebit_ratio_4yr_diff = np.absolute(np.diff(ebit_ratio_4yr))
            ebit_growth_4yr = []
            for eb in range(len(ebit_ratio_4yr_diff)):
                ebit_growth_4yr.append(ebit_ratio_4yr_diff[eb] / np.absolute(ebit_ratio_4yr[eb]))
            avg_ebit_growth_ratio = np.average(ebit_growth_4yr) # 4yr avg ebit growth% > 10%    

            ni_4yr = df_fin[df_fin['index'] == 'Net Income'].iloc[:, 1:].values
            ni_4yr = np.flip(ni_4yr).tolist()[0]
            ni_4yr_diff = np.absolute(np.diff(ni_4yr))
            ni_4yr_growth = []
            for ni in range(len(ni_4yr_diff)):
                ni_4yr_growth.append(ni_4yr_diff[ni] / np.absolute(ni_4yr[ni]))
            avg_ni_4yr_growth = np.average(ni_4yr_growth) # 20% < 4 yr avg net income growth < 50%

            if df['sector'].values[0] != 'Financial Services':

                #yf.Ticker("googl").recommendations.reset_index()
                try:    
                    if (eps_growth_rate + df['trailingAnnualDividendRate'].values[0]) / df['trailingPE'].values[0] > 1.02 and avg_ebit_ratio_4yr > 0.1 and avg_ebit_growth_ratio > 0.1 and 0.3 <= avg_ni_4yr_growth <= 3 and df['recommendationKey'].values[0] == 'buy' and df['operatingCashflow'].values[0] > 0 and df['freeCashflow'].values[0] > 0 and df['ebitdaMargins'].values[0] > 0.3 and df['currentRatio'].values[0] > 1.2 and df['debtToEquity'].values[0] < 175 and df['returnOnEquity'].values[0] > set_roe_median and df['beta'].values[0] < 1.5 and df['targetMeanPrice'].values[0] > df['currentPrice'].values[0] and df['forwardPE'].values[0] < df['trailingPE'].values[0] and df['pegRatio'].values[0] < 1:
                        slist_before_backTest.append(i)
                        print("______________________________\n")                                                    
                        print(f"Ticker all passed: {i}")
                        print("______________________________\n\n") 
                    else:
                        pass
                except:
                    print('Net Income Minus: ', i, end = '                 \r')
                    pass
            else:
                # for bank stocks
                try:
                    if (eps_growth_rate + df['trailingAnnualDividendRate'].values[0]) / df['trailingPE'].values[0] > 1.02 and 0.3 <= avg_ni_4yr_growth <= 3 and df['recommendationKey'].values[0] == 'buy' and df['operatingCashflow'].values[0] > 0 and df['currentRatio'].values[0] > 1 and df['debtToEquity'].values[0] < 175 and df['returnOnEquity'].values[0] > set_roe_median and df['beta'].values[0] < 1.5 and df['targetMeanPrice'].values[0] > df['currentPrice'].values[0] and df['forwardPE'].values[0] < df['trailingPE'].values[0] and df['pegRatio'].values[0] < 1:
                        slist_before_backTest.append(i)
                        print("______________________________\n")                                                    
                        print(f"Ticker all passed: {i}")
                        print("______________________________\n\n") 
                    else:
                        pass
                except:
                    print('Net Income Minus: ', i, end = '                \r')
                    pass
        except:
            print('Except from ratio analysis out of Sector: ', i, end = '                  \r')
            pass

    # Back test: average price change within 10 years 
    slist_after_backtest = []
    print('\n\nBack test:\n', slist_before_backTest)
    for ticker in slist_before_backTest:
        try:
            df = yfinance_df(ticker)
            # check all time record
            price_odd = []
            price_even = []
            for i in range(len(df['Adj Close'])):
                if i%2 == 0:
                    price_odd.append(df['Adj Close'][i])
                else:
                    price_even.append(df['Adj Close'][i])
            price_change = []
            for i in range(len(price_even)):
                price_change.append((price_even[i] - price_odd[i]) / price_odd[i]*100)
            backtest_yr_change = pd.DataFrame(np.absolute(price_change)).rename(columns = {0:'PriceChange'})['PriceChange'].mean()
###############################################################################################
            if backtest_yr_change < 2.5:  # filter out per average price change more than < xx
###############################################################################################
                slist_after_backtest.append(ticker)
            else:
                pass
        except:
            alarm()
            
            print('except from back test: ', ticker)
    print('\nFinal tickers:\n', slist_after_backtest)
    print('\n\n\n')
    
    return slist_after_backtest    


def sentiment_analysis(ticker):
    df = pd.read_csv('./snp500/snp_sector_index.csv')
    sector_ticker = df[df['GICS Sector'] == df[df['Symbol'] == ticker]['GICS Sector'].values[0]].Ticker.values[0]
    mega_tickers = df[df['GICS Sector'] == df[df['Symbol'] == ticker]['GICS Sector'].values[0]].Symbol.tolist()
    print(f'\n================[ Sentiment Analysis: {ticker} / {sector_ticker} ]================')
    
    return sentiment_analysis_cal(mega_tickers)


def sentiment_analysis_cal(mega_tickers):
    analysis_result = []
    for i in tqdm(range(len(mega_tickers))):
        try:
            sentiment_analysis = news_sentiment_analysis(mega_tickers[i])
            print(f'Ticker: {mega_tickers[i]} | {np.round(sentiment_analysis, 3)}', end = '                            \r')
            if sentiment_analysis != 0:
                analysis_result.append(sentiment_analysis)
            time.sleep(0.1)
        except:
            pass
    print(end = '                                                                                   \r')
    avg_sentiment = np.average(analysis_result)  
    if avg_sentiment < 0:
        avg_sentiment = 0        
    return avg_sentiment


def sentiment_portfolio(tickers, fold=7):
    
    print('\033[1m' + f'\n\n\n================[ Sentiment Portfolio Analysis ]=====================' + '\033[0m')

    # set the target threshold to filter out
    comparison_value = sentiment_analysis_cal(mega_tickers)*fold

    sentiment_portfolio_list = []
    for i in tqdm(range(len(tickers))):
        try:
            analysis_result = news_sentiment_analysis(tickers[i])
            print(f'Ticker: {tickers[i]} | {np.round(analysis_result, 3)}', end='                            \r')
            if analysis_result > comparison_value:
                sentiment_portfolio_list.append(tickers[i])
            time.sleep(0.1)
        except:
            pass
    print(end='                                                                                   \r')
    print('\033[1m' + f'=====================================================================' + '\033[0m')
    with open('sentiment_portfolio.txt', 'w') as f:
        for s in sentiment_portfolio_list:
            f.write(str(s) + '\n')
    return sentiment_portfolio_list


def news_sentiment_analysis(ticker, display = 'off'):
    
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    url = finwiz_url + ticker
    req = Request(url=url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')

    parsed_news = []
    # Iterate through the news
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        headline = x.get_text() 
        # get url
        a_tag = x.find('a')
        url = a_tag['href'] if a_tag else ''
        # split text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element
        if len(date_scrape) == 1:
            time = date_scrape[0]
            date = np.nan
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, headline, url])

    # Instantiate the sentiment intensity analyzer
    vader = SentimentIntensityAnalyzer()
    # Set column names
    columns = ['ticker', 'date', 'time', 'headline', 'url']
    # Convert the parsed_news list into a DataFrame called 'df'
    df = pd.DataFrame(parsed_news, columns=columns)

    # Iterate through the headlines and get the polarity scores using vader
    scores = df['headline'].apply(vader.polarity_scores).tolist()

    # Convert the 'scores' list of dicts into a DataFrame
    scores_df = pd.DataFrame(scores)
    
    # Join the DataFrames of the news and the list of dicts
    df = df.join(scores_df, rsuffix='_right')
    
    # Convert the date column from string to datetime
    df['date'] = pd.to_datetime(df.date).dt.date
    df.date = df.date.fillna(method = 'ffill')
    df_accu_check = df.groupby(['ticker','date']).mean().reset_index()
    
    # reverse roder
    df = df[::-1].reset_index(drop = True)
    
    # get tema for compound
    df["compound_TEMA"] = ta.tema(df['compound'], 3)

    if display == 'on':
        # display
        plt.figure(figsize = (16, 9))
        plt.title(f'{current_time()} Sentiment Analysis - Ticker: {ticker}')
        plt.errorbar(df.index, df.compound, color = 'black', alpha = 0.3, linewidth = 3, label = 'Compound')
        plt.errorbar(df.index, df.compound_TEMA, color = 'black', alpha = 0.8, linewidth = 3, label = 'Compound TEMA')
        plt.bar(df.index, df.pos, color = 'blue', alpha = 0.3, label = 'Positive')
        plt.bar(df.index, -df.neg, color = 'red', alpha = 0.3, label = 'Negative')
        plt.legend()
        plt.xlabel('\nNumber of Articles')
        plt.ylabel('\nSentiment')
        plt.grid()
        plt.show()        
    
    # adjust the number of the most recent articles to refer
    num_of_articles = int(len(df) * 0.2)
    if num_of_articles < 10:
        num_of_articles = 10
    return_value = df[-num_of_articles:].compound_TEMA.mean() #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    
    # date check
    article_first_date = str(df[:1].date.values[0])+ ' ' + str(df[:1].time.values[0])
    article_effect_date = str(df[-num_of_articles:][:1].date.values[0])+ ' ' + str(df[:1].time.values[0])
    article_last_date = str(df[-1:].date.values[0])+ ' ' + str(df[-1:].time.values[0])

    if display == 'on':
        print('\n\n=================[Result]==================')
        print(' • Sentiment Compound:  ' + '\033[1m' + f'{np.round(return_value, 2)}' + '\033[0m')
        print(' • Positive Sign (avg): ' + f'{np.round(df[-num_of_articles:].pos.mean(), 2)}')
        print(' • Negative Sign (avg): ' + f'{np.round(df[-num_of_articles:].neg.mean(), 2)}')
        print(' • Total Articles:      ' + f'{len(df)}')
        print(' • Target Articles:     ' + f'{num_of_articles}')
        print(' • Article First Date:  ' + f'{article_first_date}')
        print(' • Article Effect Date: ' + f'{article_effect_date}')
        print(' • Article Last Date:   ' + f'{article_last_date}')
        print('===========================================')
    
    return return_value


def daily_news_sentiment_all(display = 'off'):

    news = News()
    # available variables:
    # print(news.main_url)  # scraped URL
    # print(news.soup)  # beautiful soup object
    # print(news.news_df)  # NEWS table information in a pd.DataFrame object
    # print(news.blogs_df)  # BLOGS table information in a pd.DataFrame object

    df = news.news_df
    vader = SentimentIntensityAnalyzer()
    scores = df['Headline'].apply(vader.polarity_scores).tolist()
    scores_df = pd.DataFrame(scores)
    df = df.join(scores_df, rsuffix='_right')
    df = df.iloc[::-1].reset_index(drop = True)
    df["compound_TEMA"] = ta.tema(df['compound'], 10)

    if display == 'on':
        plt.figure(figsize = (16, 9))
        plt.title(f'{current_time()} Market Sentiment Analysis')
        plt.errorbar(df.Time, df.compound, color = 'black', alpha = 0.3, linewidth = 3, label = 'Compound')
        plt.errorbar(df.Time, df.compound_TEMA, color = 'black', alpha = 0.8, linewidth = 3, label = 'Compound TEMA')
        plt.bar(df.Time, df.pos, color = 'blue', alpha = 0.3, label = 'Positive')
        plt.bar(df.Time, -df.neg, color = 'red', alpha = 0.3, label = 'Negative')
        plt.xticks(rotation = 90)
        plt.xlabel('\nNumber of Articles')
        plt.ylabel('\nSentiment')
        plt.legend()
        plt.grid()
        plt.show()
        
    # adjust the number of the most recent articles to refer
    num_of_articles = int(len(df) * 0.5)
    if num_of_articles < 10:
        num_of_articles = 10
    return_value = df[-num_of_articles:].compound_TEMA.mean() #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    
    if display == 'on':
        print('\n===========[Result]===========')
        print(' • Sentiment Compound:  ' + '\033[1m' + f'{np.round(return_value, 2)}' + '\033[0m')
        print(' • Positive Sign (avg): ' + f'{np.round(df[-num_of_articles:].pos.mean(), 2)}')
        print(' • Negative Sign (avg): ' + f'{np.round(df[-num_of_articles:].neg.mean(), 2)}')
        print(' • Total Articles:      ' + f'{len(df)}')
        print(' • Target Articles:     ' + f'{num_of_articles}')    
        print('==============================')
        
        if return_value < min_market_sentiment:
            print('\033[1m' + f'\n\n\nMarket Sentiment is less than {min_market_sentiment}: {np.round(return_value, 2)}\n\n\n' + '\033[0m')
        else:
            print('\033[1m' + f'\n\n\nMarket Sentiment is higher than {min_market_sentiment}: {np.round(return_value, 2)}\n\n\n' + '\033[0m')
        
    return return_value


def top_volume_portfolio(tickers):
    print('\033[1m' + f'\n\n\n===============[ Top Volume Portfolio Analysis ]=====================' + '\033[0m')

    volume_portfolio = {}
    ticker_list = []
    volume_diff_list = []
    for i in tqdm(range(len(tickers))):
        try:
            volume_diff = yfinance_df_rsi(tickers[i]).Volume.diff().values[-1]
            print(f'Ticker: {tickers[i]} | {volume_diff}', end = '                            \r')            
            ticker_list.append(tickers[i])
            volume_diff_list.append(volume_diff)

        except:
            print(f'Except >>> Volume Check Error - Ticker: {tickers[i]}')        

    volume_portfolio['ticker'] = ticker_list
    volume_portfolio['volume_diff'] = volume_diff_list        

    print(end = '                                                                                   \r')
    print('\033[1m' + f'=====================================================================' + '\033[0m')

    top_tickers = pd.DataFrame(volume_portfolio).sort_values('volume_diff', ascending = False).reset_index(drop = True).ticker[:10]
    return top_tickers


def buy_sign_day(ticker, lookback_days):
    '''
    input: ticker name
    Volume - 80 percentile
    Volume change by Price change - 95 percentile
    lookback - 252 days (default)
    output: True if sign-date exist within look back    
    '''
    try:
        df = yfinance_df(ticker)[-252:]
        df['Volume_Change'] = df['Volume'].diff()
        df['Price_Change'] = df['Adj Close'].diff()
        df['VC_per_PC'] = df['Volume_Change'] / df['Price_Change']
        Up_Down = []
        for i in df['Adj Close'].diff():
            if i > 0:
                Up_Down.append(1)
            else:
                Up_Down.append(0)
        df['Up_Down'] = Up_Down
        df.dropna(inplace = True)

        filter_mask = (df.Volume > np.percentile(df.Volume, 80)) & (df.VC_per_PC > np.percentile(df.VC_per_PC, 95)) & (df.Up_Down == 0)
        sign_days = df[filter_mask].sort_values('Date', ascending = False)
        sign_date_volume = sign_days[:1].Date.values
        sign_date_volume = str(sign_date_volume)[2:12]

        day_plus = datetime.today() - timedelta(days = lookback_days) #<<< lookback
        day_plus = str(day_plus)[:10]

        return day_plus < sign_date_volume
    
    except:
        print('Except >>> DataFrame is empty.')
        return False


def trade_list(trade_type, stock):    
    # remove previous file
    
    previous_exist = exists(f'{trade_type}_{mostrecentdate}.txt')
    if previous_exist == True:
        os.remove(f'{trade_type}_{mostrecentdate}.txt')
    else:
        pass

    # create or update file
    today_exist = exists(f'{trade_type}_{currentdate}.txt')
    if today_exist == False:
        with open(f'{trade_type}_{currentdate}.txt', 'w') as f:
            f.write(str(stock))
        voice_message(f"""\
                      The initial {trade_type} file has been created with {stock}""")
        time.sleep(10)
        
    else:
        with open(f'{trade_type}_{currentdate}.txt', 'r') as f:
            trade_list = [line.rstrip('\n') for line in f]
        trade_list.append(stock)
        trade_list = [i for n, i in enumerate(trade_list) if i not in trade_list[:n]] 
        with open(f'{trade_type}_{currentdate}.txt', 'w') as f:
            for s in trade_list:
                f.write(str(s) + '\n')

    # read and return
    with open(f'{trade_type}_{currentdate}.txt', 'r') as f:
        updated_trade_list = [line.rstrip('\n') for line in f]
        
    return updated_trade_list


def portfolio_backtest():
    # back test for the performance between 20 days and 5 days on average 

    with open("portfolio_original.txt", 'r') as f:
        portfolio = [line.rstrip('\n') for line in f]

    if len(portfolio) < 5 and timechecknow() < 100:
        print('\n\nExcept >>> The number of portfolio is less than the minimum condition. Re-generating Portfolio.\n\n')
        voice_message("""\
            The number of portfolio is less than the minimum condition. Re-generating Portfolio.""")  
        alarm()
        main_creation()    
        sectorcheck_main(1)
        
    back_test_perf = []
    for ticker in portfolio:
        back_test_perf.append(analysis_price(ticker)[5])

    portfolio_backtest_result = np.round(np.average(back_test_perf), 2)
    snp_backtest_result = np.round(analysis_price('^GSPC')[5], 2)
    print(f'\n\n{current_time()}')
    print('_____________________________________________\n')
    print('Back Test Result: (Avg.5 days vs Avg.20 days)\n\n• S&P500: ' + f'   {snp_backtest_result}%\n')
    print('• Portfolio: ' + '\033[1m' + f'{portfolio_backtest_result}%' + '\033[0m')
    print('_____________________________________________\n')
    
    if portfolio_backtest_result - snp_backtest_result < 0:
        print('\n\nExcept >>> Back Test Failed: Portfolio performance is less than S&P500. Re-generating Portfolio.\n\n')
        voice_message("""\
            Back Test Failed. Re-generating Portfolio.""")  
        alarm()
        main_creation()   
        sectorcheck_main(1)
        
        
def proba_value_reset():   
    global predict_proba_value
    global modelchoice_minimum_fbeta
    
    # proba value adjusting by market trend
    predict_proba_value = proba_initial_value 
    modelchoice_minimum_fbeta = initial_modelchoice_minimum_fbeta
    market_trend_index = (sentiment_analysis_cal(mega_tickers)*0.1) + (sum(valuecheck('^GSPC'))*0.01)

    if market_trend_index > 0.02:
        market_trend_index = 0.02
    if market_trend_index < -0.02:
        market_trend_index = -0.02
    
    predict_proba_value = predict_proba_value - market_trend_index
    
    # proba value adjusting by real performance
    snp_price_analysis = analysis_price('^GSPC')
    snp_price_analysis_trend = snp_price_analysis[1] + snp_price_analysis[7] + snp_price_analysis[10] + snp_price_analysis[11] + snp_price_analysis[12] 
    dmv_plot('^GSPC')
    
    if snp_price_analysis_trend == 0:
        predict_proba_value = predict_proba_value + 0.1
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta + 0.1
        
    elif snp_price_analysis_trend == 1:
        predict_proba_value = predict_proba_value + 0.05
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta + 0.05
        
    elif snp_price_analysis_trend == 2:
        predict_proba_value = predict_proba_value + 0.01
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta + 0.01
        
    elif snp_price_analysis_trend == 3:
        predict_proba_value = predict_proba_value + 0.005
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta + 0.005
        
    elif snp_price_analysis_trend == 4:
        predict_proba_value = predict_proba_value + 0.0025
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta + 0.0025
        
    elif snp_price_analysis_trend == 5:
        predict_proba_value = predict_proba_value
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta
        
    else:
        alarm()
        print('\n\n\nExcept >>> snp_price_analysis_trend has incorrect value.\n\n\n')
        
    print('\n\n\n===> Min. number of positive sets at ' + '\033[1m' + f'{min_num_positive_models} <===' + '\033[0m')
    
    # apply min/max thresholds: predict_proba_value
    print(f'\n\n\n• predict_proba_value before adjusting: {np.round(predict_proba_value, 2)}')
    if predict_proba_value > predict_proba_value_max:
        predict_proba_value = predict_proba_value_max
        print(f'• predict_proba_value exceeds the max limit. Reset at {predict_proba_value_max}.')
        
    if predict_proba_value < predict_proba_value_min:
        predict_proba_value = predict_proba_value_min
        print(f'• predict_proba_value less than the min limit. Reset at {predict_proba_value_min}.')
        
    print(f'• predict_proba_value after adjusting: {np.round(predict_proba_value, 2)}\n\n\n')   
    
    
    # apply min/max thresholds: modelchoice_minimum_fbeta
    print(f'• modelchoice_minimum_fbeta before adjusting: {np.round(modelchoice_minimum_fbeta, 2)}')
    if modelchoice_minimum_fbeta > modelchoice_minimum_fbeta_max:
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta_max
        print(f'• modelchoice_minimum_fbeta exceeds the max limit. Reset at {modelchoice_minimum_fbeta_max}.')
        
    if modelchoice_minimum_fbeta < modelchoice_minimum_fbeta_min:
        modelchoice_minimum_fbeta = modelchoice_minimum_fbeta_min
        print(f'• modelchoice_minimum_fbeta less than the min limit. Reset at {modelchoice_minimum_fbeta_min}.')
        
    print(f'• modelchoice_minimum_fbeta after adjusting: {np.round(modelchoice_minimum_fbeta, 2)}\n\n\n')   

  
    # announce the final change  
    message_proba = f'Proba value sets at {int(predict_proba_value*100)} %'
    message_fbeta = f'Minimum Model Choice fbeta sets at {int(modelchoice_minimum_fbeta*100)} %'
    if timechecknow() < 60:
        voice_message(message_proba)
        time.sleep(10)
        voice_message(message_fbeta)
        time.sleep(10)
    else:
        print("===> Proba value sets at " + '\033[1m' + f"{np.round(predict_proba_value, 2)} <===\n\n\n" + '\033[0m')
        print("===> Minimum Model Choice fbeta sets at " + '\033[1m' + f"{np.round(modelchoice_minimum_fbeta, 2)} <===\n\n\n" + '\033[0m')
        

print('> Modules #2 imported')

In [71]:
##########################################################
# min. holding period before loss-cut executed
min_holding_period = 20  # days

# Strategy settings
strategy_days = 1200
days_plot = 200

min_RealizedCount = 10

parameter_search_times = 50    

# Strategy verification
min_AvgRealizedGain = 0                # more than
min_Worstperf = -10                    # more than
max_HoldingDays = min_holding_period   # less than or equal
max_PerfRange = 30                     # less than 
min_SuccessRate = 70                   # higher than
min_SuccessRate_original = min_SuccessRate
min_SuccessRate_fixed = 55             # higher than
##########################################################

# parameter random search range
##########################################################
freqLine_list = np.arange(8, 11, 1) 
freqLine_gap_list = np.arange(1.00, 1.06, 0.01) 
evma_sum_list = np.arange(3, 8, 1) 
macd_gap_list = np.arange(0.02, 0.07, 0.01)
realize_gain_percent_list = np.arange(0.01, 0.04, 0.005)     
realize_loss_percent_list = np.arange(-0.06, -0.03, 0.01) 
##########################################################


def stg_volatility_check(df):
    price_change = []
    for i in range(df.shape[0]):
        if i == 0:
            price_change.append(0)
        else:
            price_change.append((df['Current Price'].values[i] - df['Current Price'].values[i-1]) / df['Current Price'].values[i-1])
    df['price_change'] = price_change   
    return np.round(np.std(df['price_change']) * np.sqrt(252), 2)


def twin_plot(x1_axis, y1_axis, x1_color, x1_label, y1_label, y1_label_color,
              x2_axis, y2_axis, x2_color, y2_label, y2_label_color,
              title_name, 
              line1_x, line1_y, line1_color, style1,
              line2_x, line2_y, line2_color, style2):

    fig, ax = plt.subplots()
    fig.set_size_inches(16, 9)
    ax.plot(x1_axis, y1_axis, color = x1_color)
    ax.set_xlabel(x1_label)
    ax.set_ylabel(y1_label, color = y1_label_color)
    ax1 = ax.twinx()
    ax1.plot(x2_axis, y2_axis, color = x2_color)
    ax1.set_ylabel(y2_label, color = y2_label_color)
    ax.grid(axis = 'x')
    ax1.grid(axis = 'y')
    plt.title(title_name)
    ax1.plot(line1_x, line1_y, color = line1_color, linestyle = style1)
    ax1.plot(line2_x, line2_y, color = line2_color, linestyle = style2)

    
def twin_bar(x1_axis, y1_axis, x1_color, x1_label, y1_label, y1_label_color,
              x2_axis, y2_axis, x2_color, y2_label, y2_label_color,
              title_name, 
              line1_x, line1_y, line1_color, style1,
              line2_x, line2_y, line2_color, style2):

    fig, ax = plt.subplots()
    fig.set_size_inches(16, 9)
    ax.plot(x1_axis, y1_axis, color = x1_color)
    ax.set_xlabel(x1_label)
    ax.set_ylabel(y1_label, color = y1_label_color)
    ax1 = ax.twinx()
    ax1.bar(x2_axis, y2_axis, color = x2_color, width = 20, alpha = 0.2)
    ax1.set_ylabel(y2_label, color = y2_label_color)
    ax.grid(axis = 'x')
    ax1.grid(axis = 'y')
    plt.title(title_name)
    ax.plot(line1_x, line1_y, color = line1_color, linestyle = style1)
    ax.plot(line2_x, line2_y, color = line2_color, linestyle = style2)
    

def strategy_main_plot(ticker, df_plot, freq_mid_line_1):
    # Main plot
    for i in range(1,4):
        df_plot = df_plot[-int(len(df_plot)/i):]
        plt.figure(figsize = (16, 9))
        plt.title(f'Exponential Moving Average {(i)} | {ticker}')
        plt.grid()
        errorbar_columns = ['Current Price', freq_mid_line_1, '60_EMA', '120_EMA']
        for column in errorbar_columns:
            plt.errorbar(data = df_plot, x = 'Date', y = column, linewidth = 1.5, label = column)
            plt.annotate(xy = [df_plot[-1:].Date.values, df_plot[-1:][column].values], text = column)
        
        annotate = df_plot[df_plot['Long_Sign'] >= 1][['Date', 'Current Price']].reset_index(drop = True)
        for i in range(len(annotate)):
            plt.annotate(xy = [annotate.Date.values[i], annotate['Current Price'].values[i]], text = 'LT', color = 'Blue', alpha=0.8)
        
        annotate = df_plot[df_plot['Short_Sign'] >= 1][['Date', 'Current Price']].reset_index(drop = True)
        for i in range(len(annotate)):
            plt.annotate(xy = [annotate.Date.values[i], annotate['Current Price'].values[i]], text = 'ST', color = 'Red', alpha=0.8)
        plt.show()
        
        
def ichimoku_chart(ticker, df_plot):
    for i in range(1,4):
        # Ichimoku
        df_plot = df_plot[-int(len(df_plot)/i):]
        plt.figure(figsize = (16,9))
        plt.title(f'Ichimoku Chart {(i)} | {ticker}')
        plt.errorbar(df_plot.Date, df_plot['Current Price'], label = 'Current Price', linewidth = 18, color = 'black', alpha = 0.05)
        plt.errorbar(df_plot.Date, df_plot.AdvSpan_1, label = 'AdvSpan #1', linewidth = 5, color = 'red', alpha = 0.6)
        plt.errorbar(df_plot.Date, df_plot.AdvSpan_2, label = 'AdvSpan #2', linewidth = 5, color = 'green', alpha = 0.4)
        plt.errorbar(df_plot.Date, df_plot.Conversion, label = 'Conversion', linewidth = 2, color = 'black', alpha = 1)
        plt.errorbar(df_plot.Date, df_plot.Standard, label = 'Standard', linewidth = 2, color = 'blue', alpha = 1)
        plt.errorbar(df_plot.Date, df_plot.FlwSpan, label = 'FlwSpan', linewidth = 2, color = 'black', alpha = 0.4)
        ichimoku_columns = ['Current Price', 'AdvSpan_1', 'AdvSpan_2','Conversion','Standard','FlwSpan']
        for column in ichimoku_columns:
            plt.annotate(xy = [df_plot[-1:].Date.values, df_plot[-1:][column].values], text = column)
        plt.grid()
        plt.legend()
        plt.show()
        

def bollinger_plot(ticker, df_plot, squeeze_ceiling_buy, squeeze_floor_sell, loose_ceiling_sell, loose_floor_buy):
    # bollinger band
    for i in range(1,4):
        # bollinger band
        df_plot = df_plot[-int(len(df_plot)/i):]
        plt.figure(figsize = (16, 9))
        plt.title(f'Bollinger Band {(i)} | {ticker}')
        plt.errorbar(df_plot.Date, df_plot['High'], label = 'High', color = 'green', alpha = 0.3)
        plt.errorbar(df_plot.Date, df_plot['Current Price'], label = 'Current Price', color = 'blue', alpha = 0.3, linewidth = 5)
        plt.errorbar(df_plot.Date, df_plot['Low'], label = 'Low', color = 'green', alpha = 0.3)
        plt.errorbar(df_plot.Date, df_plot['BBU_20_2.0'], label = 'BB_Upper', color = 'black', alpha = 0.5)
        # plt.errorbar(df_plot.Date, df_plot['BBM_20_2.0'], label = '5_SMA', alpha = 0.3)
        plt.errorbar(df_plot.Date, df_plot['BBL_20_2.0'], label = 'BB_Low', color = 'black', alpha = 0.5)
        plt.errorbar(df_plot.Date, df_plot['200_EMA'], label = '200 EMA', color = 'blue', alpha = 0.6)
        plt.xticks(rotation = 90)
        squeeze_ceiling_buy.reset_index(drop = True, inplace = True)
        for i in range(len(squeeze_ceiling_buy)):
            try:
                plt.annotate(xy = [squeeze_ceiling_buy.Date.values[i], squeeze_ceiling_buy[squeeze_ceiling_buy['Date'] == squeeze_ceiling_buy.Date.values[i]]['BBU_20_2.0'].values[0]], text = 'Sqz Buy', color = 'black', fontsize = 12)
            except:
                pass

        squeeze_floor_sell.reset_index(drop = True, inplace = True)
        for i in range(len(squeeze_floor_sell)):    
            try:
                plt.annotate(xy = [squeeze_floor_sell.Date.values[i], squeeze_floor_sell[squeeze_floor_sell['Date'] == squeeze_floor_sell.Date.values[i]]['BBL_20_2.0'].values[0]], text = 'Sqz Sell', color = 'red', fontsize = 12)
            except:
                pass
            
        loose_ceiling_sell.reset_index(drop = True, inplace = True)
        for i in range(len(loose_ceiling_sell)):
            try:
                plt.annotate(xy = [loose_ceiling_sell.Date.values[i], loose_ceiling_sell[loose_ceiling_sell['Date'] == loose_ceiling_sell.Date.values[i]]['BBU_20_2.0'].values[0]], text = 'Loo Sell', color = 'red', fontsize = 12)
            except:
                pass

        loose_floor_buy.reset_index(drop = True, inplace = True)
        for i in range(len(loose_floor_buy)):    
            try:
                plt.annotate(xy = [loose_floor_buy.Date.values[i], loose_floor_buy[loose_floor_buy['Date'] == loose_floor_buy.Date.values[i]]['BBL_20_2.0'].values[0]], text = 'Loo Buy', color = 'black', fontsize = 12)
            except:
                pass

        plt.grid()
        plt.legend()
        plt.show()
        print(f'\n• Squeeze Ceiling Buy: {len(squeeze_ceiling_buy)}')
        print(f'• Squeeze Floor Sell:  {len(squeeze_floor_sell)}\n\n')
    
    
def bollinger_sign_gen(ticker):
    '''
    input: ticker name
    output: bollinger sign - 20 days moving average with std. at 5
    '''
    print('\n\n\n===============[ Bollinger Sign Analysis: ' + '\033[1m' + f'{ticker}' + '\033[0m' + ' ]=====================')
    global min_SuccessRate
    min_SuccessRate_original = min_SuccessRate
    min_SuccessRate = int(min_SuccessRate * 0.5)
        
    if strategy_analysis_main(ticker) != -10:
        df = pd.read_csv(f'strategy_anaysis_{ticker }_{currentdate}.csv')
        df['bollinger_sum'] = df['squeeze_ceiling_buy'] + df['loose_floor_buy'] + df['squeeze_floor_sell'] + df['loose_ceiling_sell']
        df = df[df['bollinger_sum'] != 0][-1:]

        squeeze_ceiling_buy = df.squeeze_ceiling_buy.values[0]
        loose_floor_buy = df.loose_floor_buy.values[0]
        buy_sign = squeeze_ceiling_buy + loose_floor_buy

        squeeze_floor_sell = df.squeeze_floor_sell.values[0]
        loose_ceiling_sell = df.loose_ceiling_sell.values[0]
        sell_sign = squeeze_floor_sell + loose_ceiling_sell

        if sell_sign > 0:
            print('Bollinger - ' + '\033[1m' + 'Sell' + '\033[0m')
        elif buy_sign > 0:
            print('Bollinger - ' + '\033[1m' +'Buy' + '\033[0m')
        else:
            print('Except >>> Bollinger Sum Error')
        min_SuccessRate = min_SuccessRate_original
        print('\n\n====================================================================')
        return [buy_sign, sell_sign]
    else:
        print('Except >>> Strategy Analysis Failed')
        return [-10, -10]

    
def BBand_Buy_Sign_Count(tickers):
    passed = []
    c = 1
    print('\n')
    for ticker in tickers:
        print(f'Bollinger Sign Checking: {c} / {len(tickers)}', end = '                      \r')
        if bollinger_sign_gen(ticker)[0] == 1:
            passed.append(ticker)
        c+=1
    success_rate = len(passed) / len(tickers)
    print('\n\nSuccess Rate: ' + '\033[1m' + f'{np.round(success_rate*100, 2)} %' + '\033[0m')
    return success_rate   


def macro_strategy(ticker, 
                   period, 
                   opt_freqLine, 
                   opt_freqLine_gap,
                   opt_evma_sum,
                   opt_macd_gap,
                   realize_gain_percent, 
                   realize_loss_percent,
                   plot_display_1, 
                   plot_display_2):
    
    global SuccessRate
    global timesRealized
    # https://github.com/twopirllc/pandas-ta
    
    df = yfinance_df(ticker)[-period:]
    df = df.rename(columns = {'Adj Close':'Current Price'})
    df['Current Price'] = (df['Current Price'] + df['High'] + df['Low'])/3
    
    ## 14_Day RSI
    df['RSI'] = ta.rsi(df['Current Price'], length = 14)
    focus_days = [40, 70]
    for i in focus_days:
        df[f"{i}_RSI"] = df['RSI'].ewm(span= i, adjust=False).mean()
    df.dropna(inplace = True)
    df = df.reset_index(drop = True)
    
    ## 14_Day ADX
    df = df.join(ta.adx(df.High, df.Low, df.Close, length = 14))
    df.dropna(inplace = True)
    df = df.reset_index(drop = True)
    
    # ema
    focus_days = [opt_freqLine, opt_freqLine+2, opt_freqLine+4, opt_freqLine+6, opt_freqLine+8, 12, 27, 60, 120, 200]
    for i in focus_days:
        df[f"{i}_EMA"] = df['Current Price'].ewm(span= i, adjust=False).mean()
    df.dropna(inplace = True)

    # find the s/t ema line having most crossing with the current price
    sc = StandardScaler()
    freq_mid_line = {}
    for column_num in range(13, 18):
        close_sc = sc.fit_transform(np.array(df['Current Price']).reshape(-1, 1))
        compare_sc = sc.transform(np.array(df[df.columns[column_num]]).reshape(-1, 1))
        df[f'vs_{df.columns[column_num]}'] = np.absolute(close_sc - compare_sc)
        freq_mid_line[f'vs_{df.columns[column_num]}'] = len(df[df[f'vs_{df.columns[column_num]}'] < 0.1])     
    freq_mid_line_1 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-1][0][3:]
    freq_mid_line_2 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-2][0][3:]
    freq_mid_line_3 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-3][0][3:]
    
    # Cross_date: when the current price is located as 'freq_mid_line' > 'Current Price' > '60_EMA'    
    df['Current_freq_1'] = df['Current Price'] - df[freq_mid_line_1]
    df['Current_freq_2'] = df['Current Price'] - df[freq_mid_line_2]
    df['Current_freq_3'] = df['Current Price'] - df[freq_mid_line_3]
    df['Current_60_EMA'] = df['Current Price'] - df['60_EMA']
    
    df.loc[(df['Current_freq_1'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_1'] = 1
    df['Cross_zone_1'] = df['Cross_zone_1'].fillna(0)
    
    df.loc[(df['Current_freq_2'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_2'] = 1
    df['Cross_zone_2'] = df['Cross_zone_2'].fillna(0)
    
    df.loc[(df['Current_freq_3'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_3'] = 1
    df['Cross_zone_3'] = df['Cross_zone_3'].fillna(0)
    
# strategy 1: EMA lines should be arranged as One of Short-Terms > Mid-Term > Long-Term
    ###############################################################################    
    # EMA Plus Condition  
    df.loc[df['Current Price'] >= (df[freq_mid_line_1] * opt_freqLine_gap), 'EMA_Plus_1'] = 1 
    df.fillna(0, inplace = True)
    
    df.loc[df['Current Price'] >= (df[freq_mid_line_2] * opt_freqLine_gap), 'EMA_Plus_2'] = 1 
    df.fillna(0, inplace = True)
    
    df.loc[df['Current Price'] >= (df[freq_mid_line_3] * opt_freqLine_gap), 'EMA_Plus_3'] = 1 
    df.fillna(0, inplace = True)
    
    ### modify ###
    df['EMA_Plus_Sum'] = df['EMA_Plus_1'] + df['EMA_Plus_2'] + df['EMA_Plus_3']  
    df.drop(columns = {'EMA_Plus_1', 'EMA_Plus_2', 'EMA_Plus_3'}, inplace = True)
    
        
    # EMA Minus Condition    
    df.loc[(df['Current Price'] * opt_freqLine_gap) < df[freq_mid_line_1], 'EMA_Minus'] = -1
    df.fillna(0, inplace = True)
    
    ### modify ###
    df['EMA_Minus_Sum'] = df['EMA_Minus'] 
    df.drop(columns = {'EMA_Minus'}, inplace = True)    
    ###############################################################################
    
    # tema
    focus_days = [opt_freqLine, opt_freqLine+2, opt_freqLine+4, opt_freqLine+6, opt_freqLine+8]
    for i in focus_days:
        df[f"{i}_TEMA"] = ta.tema(df['Current Price'], i)
    df.dropna(inplace = True)

    # find the s/t ema line having most crossing with the current price
    freq_mid_line = {}
    for column_num in range(36, 41):
        close_sc = sc.fit_transform(np.array(df['Current Price']).reshape(-1, 1))
        compare_sc = sc.transform(np.array(df[df.columns[column_num]]).reshape(-1, 1))
        df[f'vs_{df.columns[column_num]}'] = np.absolute(close_sc - compare_sc)
        freq_mid_line[f'vs_{df.columns[column_num]}'] = len(df[df[f'vs_{df.columns[column_num]}'] < 0.1])     
    freq_mid_line_11 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-1][0][3:]
    freq_mid_line_22 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-2][0][3:]
    freq_mid_line_33 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-3][0][3:]
    
    # Cross_date: when the current price is located as 'freq_mid_line' > 'Current Price' > '60_EMA'    
    df['Current_freq_11'] = df['Current Price'] - df[freq_mid_line_11]
    df['Current_freq_22'] = df['Current Price'] - df[freq_mid_line_22]
    df['Current_freq_33'] = df['Current Price'] - df[freq_mid_line_33]
    
    df.loc[(df['Current_freq_11'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_11'] = 1
    df['Cross_zone_11'] = df['Cross_zone_11'].fillna(0)
    
    df.loc[(df['Current_freq_22'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_22'] = 1
    df['Cross_zone_22'] = df['Cross_zone_22'].fillna(0)
    
    df.loc[(df['Current_freq_33'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_33'] = 1
    df['Cross_zone_33'] = df['Cross_zone_33'].fillna(0)
    
# strategy 2: TEMA lines should be arranged as One of Short-Terms > Mid-Term > Long-Term
    ###############################################################################    
    # TEMA Plus Condition  
    df.loc[df['Current Price'] >= (df[freq_mid_line_11] * opt_freqLine_gap), 'TEMA_Plus_1'] = 1 
    df.fillna(0, inplace = True)
    
    df.loc[df['Current Price'] >= (df[freq_mid_line_22] * opt_freqLine_gap), 'TEMA_Plus_2'] = 1 
    df.fillna(0, inplace = True)
    
    df.loc[df['Current Price'] >= (df[freq_mid_line_33] * opt_freqLine_gap), 'TEMA_Plus_3'] = 1 
    df.fillna(0, inplace = True)
    
    ### modify ###
    df['TEMA_Plus_Sum'] = df['TEMA_Plus_1'] + df['TEMA_Plus_2'] + df['TEMA_Plus_3']  
    df.drop(columns = {'TEMA_Plus_1', 'TEMA_Plus_2', 'TEMA_Plus_3'}, inplace = True)
    
        
    # TEMA Minus Condition    
    df.loc[(df['Current Price'] * opt_freqLine_gap) < df[freq_mid_line_11], 'TEMA_Minus'] = -1
    df.fillna(0, inplace = True)
    
    ### modify ###
    df['TEMA_Minus_Sum'] = df['TEMA_Minus'] 
    df.drop(columns = {'TEMA_Minus'}, inplace = True)    
    ############################################################################### 
    
    # vwap_ema
    df['VWAP'] = (df['Volume'] * df['Current Price']) / df['Volume']
    focus_days = [opt_freqLine, opt_freqLine+2, opt_freqLine+4, opt_freqLine+6, opt_freqLine+8]
    for i in focus_days:
        df[f"{i}_VWAP_EMA"] = ta.ema(df['VWAP'], i)
    df.dropna(inplace = True)

    # find the s/t ema line having most crossing with the current price
    freq_mid_line = {}
    for column_num in range(56, 60):
        close_sc = sc.fit_transform(np.array(df['VWAP']).reshape(-1, 1))
        compare_sc = sc.transform(np.array(df[df.columns[column_num]]).reshape(-1, 1))
        df[f'vs_{df.columns[column_num]}'] = np.absolute(close_sc - compare_sc)
        freq_mid_line[f'vs_{df.columns[column_num]}'] = len(df[df[f'vs_{df.columns[column_num]}'] < 0.1])     
    freq_mid_line_111 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-1][0][3:]
    freq_mid_line_222 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-2][0][3:]
    freq_mid_line_333 = sorted(freq_mid_line.items(), key=lambda x:x[1])[-3][0][3:]
    
    # Cross_date: when the current price is located as 'freq_mid_line' > 'Current Price' > '60_EMA'    
    df['Current_freq_111'] = df['VWAP'] - df[freq_mid_line_111]
    df['Current_freq_222'] = df['VWAP'] - df[freq_mid_line_222]
    df['Current_freq_333'] = df['VWAP'] - df[freq_mid_line_333]
    
    df.loc[(df['Current_freq_111'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_111'] = 1
    df['Cross_zone_111'] = df['Cross_zone_111'].fillna(0)
    
    df.loc[(df['Current_freq_222'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_222'] = 1
    df['Cross_zone_222'] = df['Cross_zone_222'].fillna(0)
    
    df.loc[(df['Current_freq_333'] < 0) & (df['Current_60_EMA'] > 0), 'Cross_zone_333'] = 1
    df['Cross_zone_333'] = df['Cross_zone_333'].fillna(0)
    
# strategy 2: TEMA lines should be arranged as One of Short-Terms > Mid-Term > Long-Term
    ###############################################################################    
    # TEMA Plus Condition  
    df.loc[df['VWAP'] >= (df[freq_mid_line_111] * opt_freqLine_gap), 'VWAP_EMA_Plus_1'] = 1 
    df.fillna(0, inplace = True)
    
    df.loc[df['VWAP'] >= (df[freq_mid_line_222] * opt_freqLine_gap), 'VWAP_EMA_Plus_2'] = 1 
    df.fillna(0, inplace = True)
    
    df.loc[df['VWAP'] >= (df[freq_mid_line_333] * opt_freqLine_gap), 'VWAP_EMA_Plus_3'] = 1 
    df.fillna(0, inplace = True)
    
    ### modify ###
    df['VWAP_EMA_Plus_Sum'] = df['VWAP_EMA_Plus_1'] + df['VWAP_EMA_Plus_2'] + df['VWAP_EMA_Plus_3']  
    df.drop(columns = {'VWAP_EMA_Plus_1', 'VWAP_EMA_Plus_2', 'VWAP_EMA_Plus_3'}, inplace = True)
    
        
    # TEMA Minus Condition    
    df.loc[(df['VWAP'] * opt_freqLine_gap) < df[freq_mid_line_111], 'VWAP_EMA_Minus'] = -1
    df.fillna(0, inplace = True)
    
    ### modify ###
    df['VWAP_EMA_Minus_Sum'] = df['VWAP_EMA_Minus'] 
    df.drop(columns = {'VWAP_EMA_Minus'}, inplace = True)    
    ############################################################################### 
    
## strategy 4: EMA Volume diff. should be positive 
    ###############################################################################
    evma_focus_days = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    for i in evma_focus_days:
        df[f"{i}_EVMA"] = df['Volume'].ewm(span= i, adjust=False).mean()
        df[f"{i}_EVMA_diff"] = df[f"{i}_EVMA"].diff() > 0
        df[f"{i}_EVMA_diff"] = df[f"{i}_EVMA_diff"].map({True:1, False:0})
    df.dropna(inplace = True)    
   
    ### modify ###     
    df['EVMA_SUM'] = df['1_EVMA_diff'] + df['2_EVMA_diff'] + df['3_EVMA_diff'] + df['4_EVMA_diff'] + df['5_EVMA_diff'] + df['6_EVMA_diff'] + df['7_EVMA_diff'] + df['8_EVMA_diff'] + df['9_EVMA_diff'] + df['10_EVMA_diff'] #<<<
    df.drop(columns = {'1_EVMA_diff', '2_EVMA_diff', '3_EVMA_diff', '4_EVMA_diff', '5_EVMA_diff', '6_EVMA_diff', '7_EVMA_diff', '8_EVMA_diff', '9_EVMA_diff', '10_EVMA_diff'}, inplace = True) #<<<
    
    df.fillna(0, inplace = True)
    df.reset_index(drop = True, inplace = True)
    ###############################################################################
    
    # MACD line is calculated by subtracting the 26-period EMA from the 12-period EMA
    # A nine-day EMA of the MACD line is called the signal line, which is then plotted on top of the MACD line, 
    # which can function as a trigger for buy or sell signals. 
    # Traders may buy the security when the MACD line crosses above the signal line 
    df['MACD'] = df['12_EMA'] - df['27_EMA']    
    df['SignalLine'] = df['MACD'].ewm(span= 9, adjust=False).mean()
    
## strategy 5 (0~1): MACD line should crosses above the signal line
    df['MACD_SignalLine'] = df['MACD'] - df['SignalLine']
    df['SignalLine_Gap'] = df['SignalLine'] * opt_macd_gap
     
    df.loc[df['MACD'] >= df['SignalLine_Gap'], 'MACD_Sign'] = 1
    df.loc[df['MACD'] < df['SignalLine_Gap'], 'MACD_Sign'] = 0   
    
## strategy 6 bollinger band
    df = df.join(ta.bbands(df.Close, length=20, std = 2))
    df.dropna(inplace=True)
    sc = StandardScaler()
    
    close_sc = sc.fit_transform(np.array(df['Current Price']).reshape(-1, 1))
    high_sc = sc.transform(np.array(df['High']).reshape(-1, 1))
    low_sc = sc.transform(np.array(df['Low']).reshape(-1, 1))
    BBU_sc = sc.transform(np.array(df['BBU_20_2.0']).reshape(-1, 1))
    BBL_sc = sc.transform(np.array(df['BBL_20_2.0']).reshape(-1, 1))

    df['BB_Gap'] = BBU_sc - BBL_sc
    df['price_diff_sc'] = np.absolute(df['Current Price'].diff())
    df['bb_gap_diff_sc'] = np.absolute(df['BB_Gap'].diff())
    
    ###################################################
    bb_gap_threshold = df['bb_gap_diff_sc'].max()/2
    ###################################################
    
    df['Ceiling'] = high_sc - BBU_sc  
    df['Floor'] = low_sc - BBL_sc     
    
    df.loc[(df['Ceiling'] > 0) & 
           (df['bb_gap_diff_sc'] < bb_gap_threshold), 
           'squeeze_ceiling_buy'] = 1   
    df.loc[(df['Floor'] < 0) & 
           (df['bb_gap_diff_sc'] < bb_gap_threshold), 
           'squeeze_floor_sell'] = 1  
    df.loc[(df['Ceiling'] > 0) & 
           (df['bb_gap_diff_sc'] > bb_gap_threshold), 
           'loose_ceiling_sell'] = 1  
    df.loc[(df['Floor'] < 0) & 
           (df['bb_gap_diff_sc'] > bb_gap_threshold), 
           'loose_floor_buy'] = 1  
    df.fillna(0, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(drop = True, inplace = True)
    
    squeeze_ceiling_buy = df[df['squeeze_ceiling_buy'] == 1]  # buy sign
    squeeze_floor_sell = df[df['squeeze_floor_sell'] == 1]    # sell sign
    loose_ceiling_sell = df[df['loose_ceiling_sell'] == 1]    # sell sign
    loose_floor_buy = df[df['loose_floor_buy'] == 1]          # buy sign

## strategy 7 ichimoku  
    df = df.join(ta.ichimoku(df.High, df.Low, df.Close)[0])
    df.reset_index(drop = True, inplace = True)
    df = df.rename(columns = {'ISA_9':'AdvSpan_1', 'ISB_26':'AdvSpan_2', 'ITS_9':'Conversion', 'IKS_26':'Standard', 'ICS_26':'FlwSpan'})
    df = df.fillna(method = 'ffill')
    
## strategy 8 CCI      
    df = df.join(ta.cci(df.High, df.Low, df.Close))
    df = df.rename(columns = {'CCI_14_0.015':'CCI'}) 

    #
    # Create new columns from here '^'   
    df = df.dropna()
    df = df.reset_index(drop = True)
 
## strategy creation 
    # Long ##########################################################################################################################################################
    df.loc[(df['MACD_SignalLine'] > opt_macd_gap) &
           (df['Conversion'] > df['Standard']) &
           (df['CCI'] > 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['EMA_Plus_Sum'] >= 1) &
           (df['EVMA_SUM'] >= opt_evma_sum) &
           (df['RSI'] < 70),
           'Long_Sign_1'] = 1 
    
# ema crossover
    df.loc[(df['MACD_SignalLine'] > opt_macd_gap) &
           (df['Conversion'] > df['Standard']) &
           (df['CCI'] > 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Cross_zone_1'] == 1) &
           (df['EMA_Plus_Sum'] >= 1) &
           (df['DMP_14'] > 20), 
           'Long_Sign_2'] = 1  
    
    df.loc[(df['MACD_SignalLine'] > opt_macd_gap) &
           (df['Conversion'] > df['Standard']) &
           (df['CCI'] > 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Cross_zone_2'] == 1) &
           (df['EMA_Plus_Sum'] >= 1) &           
           (df['DMP_14'] > 20), 
           'Long_Sign_3'] = 1  
    
    df.loc[(df['MACD_SignalLine'] > opt_macd_gap) &
           (df['Conversion'] > df['Standard']) &
           (df['CCI'] > 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Cross_zone_3'] == 1) &
           (df['EMA_Plus_Sum'] >= 1) &
           (df['DMP_14'] > 20), 
           'Long_Sign_4'] = 1 
    
# vwap ema crossover 
    df.loc[(df['MACD_SignalLine'] > opt_macd_gap) &
           (df['Conversion'] > df['Standard']) &
           (df['CCI'] > 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Cross_zone_222'] == 1) &
           (df['VWAP_EMA_Plus_Sum'] >= 1) &           
           (df['DMP_14'] > 20), 
           'Long_Sign_5'] = 1  
    
    df.loc[(df['MACD_SignalLine'] > opt_macd_gap) &
           (df['Conversion'] > df['Standard']) &
           (df['CCI'] > 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Cross_zone_333'] == 1) &
           (df['VWAP_EMA_Plus_Sum'] >= 1) &
           (df['DMP_14'] > 20), 
           'Long_Sign_6'] = 1  
    
# vwap ema price
    df.loc[(df['MACD_SignalLine'] > opt_macd_gap) &
           (df['Conversion'] > df['Standard']) &
           (df['VWAP_EMA_Plus_Sum'] >= 1) &
           (df['CCI'] > 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['RSI'] < 40) &
           (df['DMP_14'] > 20), 
           'Long_Sign_7'] = 1   

# bband long term
    df.loc[(df['ADX_14'] > 20) &
           (df['CCI'] > 0) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Ceiling'] > 0) &
           (df['EMA_Plus_Sum'] >= 1) &
           (df['BBL_20_2.0'] > df['200_EMA']), 
           'Long_Sign_8'] = 1  
    
    df.loc[(df['ADX_14'] > 20) &
           (df['CCI'] > 0) &
           (df['bb_gap_diff_sc'] > bb_gap_threshold) &
           (df['Floor'] < 0) &
           (df['EMA_Plus_Sum'] >= 1) &
           (df['BBL_20_2.0'] > df['200_EMA']), 
           'Long_Sign_9'] = 1  
    
    df.fillna(0, inplace = True)

    # Short #########################################################################################################################################################
    df.loc[(df['MACD_SignalLine'] < opt_macd_gap) &
           (df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['DMN_14'] > 25),
           'Short_Sign_1'] = 1   
    
    df.loc[(df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['EMA_Minus_Sum'] == -1) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['DMN_14'] > 25),
           'Short_Sign_2'] = 1   
              
    df.loc[(df['MACD_SignalLine'] < opt_macd_gap) &
           (df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['RSI'] > 70),
           'Short_Sign_3'] = 1   
    
    df.loc[(df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Floor'] < 0) &
           (df['TEMA_Plus_Sum'] == 0),
           'Short_Sign_4'] = 1 
    
    df.loc[(df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] > bb_gap_threshold) &
           (df['Ceiling'] > 0) &
           (df['TEMA_Plus_Sum'] == 0),
           'Short_Sign_5'] = 1 
    
    df.loc[(df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['Floor'] < 0) &
           (df['VWAP_EMA_Plus_Sum'] == 0),
           'Short_Sign_6'] = 1 
    
    df.loc[(df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] > bb_gap_threshold) &
           (df['Ceiling'] > 0) &
           (df['VWAP_EMA_Plus_Sum'] == 0),
           'Short_Sign_7'] = 1 
    
    df.loc[(df['Conversion'] < df['Standard']) &
           (df['CCI'] < 0) &
           (df['TEMA_Minus_Sum'] == -1) &
           (df['ADX_14'] > 20) &
           (df['bb_gap_diff_sc'] < bb_gap_threshold) &
           (df['DMN_14'] > 25),
           'Short_Sign_8'] = 1 
    
    df.fillna(0, inplace = True)
    
    # select or read the best lottery file
    lottery_list_exist = exists(f"lottery_list_{ticker}_{currentdate}.txt")
    if lottery_list_exist == True:
        with open(f"lottery_list_{ticker}_{currentdate}.txt", 'r') as f:
            lottery_list = [line.rstrip('\n') for line in f]
        long_lottery = eval(lottery_list[0])
        short_lottery = eval(lottery_list[1])    
        os.remove(f"lottery_list_{ticker}_{currentdate}.txt")
    else:
        long_lottery = random.sample(range(1, 10), 5)
        short_lottery = random.sample(range(1, 9), 4)
    
    # apply the lottery
    df['Long_Sign'] = df[f'Long_Sign_{long_lottery[0]}'] + df[f'Long_Sign_{long_lottery[1]}'] + df[f'Long_Sign_{long_lottery[2]}'] + df[f'Long_Sign_{long_lottery[3]}'] + df[f'Long_Sign_{long_lottery[4]}']               
    df['Short_Sign'] = df[f'Short_Sign_{short_lottery[0]}'] + df[f'Short_Sign_{short_lottery[1]}'] + df[f'Short_Sign_{short_lottery[2]}'] + df[f'Short_Sign_{short_lottery[3]}']         
    
    #################################################################################################################################################################

    # backtest
    df['Shares'] = 0
    df['Unit Cost'] = 0
    df['Avg Unit Cost'] = 0
    
    df['Target Percent Margin_Gain'] = realize_gain_percent
    df['Target Percent Margin_Loss'] = realize_loss_percent
    df['Target Gain Price'] = 0
    df['Target Loss Price'] = 0
    
    df['Unrealized Value'] = 0
    df['Unrealized Gain'] = 0
    df['Realized Value'] = 0
    df['Realized Gain'] = 0
    df['LongPosition Days'] = 0

    for i in range(1, len(df)):
        
        # Target setting
        if df['Avg Unit Cost'][i-1] != 0:
            df['Target Gain Price'][i] = df['Avg Unit Cost'][i-1] + (df['Avg Unit Cost'][i-1] * df['Target Percent Margin_Gain'][i])
            df['Target Loss Price'][i] = df['Avg Unit Cost'][i-1] + (df['Avg Unit Cost'][i-1] * df['Target Percent Margin_Loss'][i])    
        else:
            df['Target Gain Price'][i] = df['Current Price'][i] + (df['Current Price'][i] * df['Target Percent Margin_Gain'][i])
            df['Target Loss Price'][i] = df['Current Price'][i] + (df['Current Price'][i] * df['Target Percent Margin_Loss'][i])    
        
        # Purchase
        if df['Long_Sign'][i] >= 1 and df['Short_Sign'][i] == 0:

            if df['Shares'][i-1] != 0:
                df['Shares'][i] = df['Shares'][i-1]+1
            else:
                df['Shares'][i] = 1
                
            df['Unit Cost'][i] = df['Current Price'][i]
            
            if df['Avg Unit Cost'][i-1] != 0:
                df['Avg Unit Cost'][i] = ((df['Avg Unit Cost'][i-1] * df['Shares'][i-1]) + df['Unit Cost'][i]) / df['Shares'][i]
            else:
                df['Avg Unit Cost'][i] = df['Unit Cost'][i]            

            df['Unrealized Value'][i] = df['Avg Unit Cost'][i] * df['Shares'][i]
            df['Unrealized Gain'][i] = (df['Current Price'][i] - df['Avg Unit Cost'][i]) / df['Avg Unit Cost'][i]
            
            if df['LongPosition Days'][i-1] != 0:
                df['LongPosition Days'][i] = df['LongPosition Days'][i-1] + 1
            else:
                df['LongPosition Days'][i] = 1
    
        # Sell
        elif (df['Long_Sign'][i] == 0 and df['Short_Sign'][i] >= 1) or df['Current Price'][i] >= df['Target Gain Price'][i] or df['Current Price'][i] < df['Target Loss Price'][i]:
#         elif df['Current Price'][i] >= df['Target Gain Price'][i] or df['Current Price'][i] < df['Target Loss Price'][i]:
                          
            if df['Shares'][i-1] != 0:
                df['Shares'][i] = 0
            else:
                pass
            
            if df['Avg Unit Cost'][i-1] != 0:
                df['Avg Unit Cost'][i] = 0
            else:
                pass
            
            df['Unrealized Value'][i] = 0
            df['Unrealized Gain'][i] = 0
            
            if df['Shares'][i-1] != 0:
                df['Realized Value'][i] = df['Current Price'][i] * df['Shares'][i-1]
                df['Realized Gain'][i] = (df['Current Price'][i] - df['Avg Unit Cost'][i-1]) / df['Avg Unit Cost'][i-1]
            else:
                pass
        
            if df['Realized Gain'][i] != 0:
                df['LongPosition Days'][i] = df['LongPosition Days'][i-1]+1
            else:
                pass
        
        # hold 
        else:
            
            if df['Shares'][i-1] != 0:
                df['Shares'][i] = df['Shares'][i-1]
            else:
                pass
            
            if df['Avg Unit Cost'][i-1] != 0:
                df['Avg Unit Cost'][i] = df['Avg Unit Cost'][i-1]
            else:
                pass
            
            if df['Avg Unit Cost'][i] != 0:
                df['Unrealized Value'][i] = df['Avg Unit Cost'][i] * df['Shares'][i]
            else:
                pass
            
            if df['Avg Unit Cost'][i] != 0:
                df['Unrealized Gain'][i] = (df['Current Price'][i] - df['Avg Unit Cost'][i]) / df['Avg Unit Cost'][i]
            else:
                pass
            
            if df['Avg Unit Cost'][i] == 0:
                df['LongPosition Days'][i] = 0
            else:
                df['LongPosition Days'][i] = df['LongPosition Days'][i-1] + 1
        
    # save
    df.to_csv(f'strategy_anaysis_{ticker}_{currentdate}.csv', index = False)

## Display
    df_plot = df[-days_plot:]

    if plot_display_1 == 'off' and plot_display_2 == 'off':
        pass
    
    elif plot_display_1 == 'on' and plot_display_2 == 'off':
        # main plot
        strategy_main_plot(ticker, df_plot, freq_mid_line_1)
        
        # backtest
        twin_bar(x1_axis = df.Date, y1_axis = df['Unrealized Gain']*100, x1_color = 'red', x1_label = '\nDate', y1_label = '\nUnrealized Gain (%)', y1_label_color = 'red',
                 x2_axis = df.Date, y2_axis = df['Realized Gain']*100, x2_color = 'blue', y2_label = '\nRealized Gain (%)', y2_label_color = 'blue', 
                 title_name = f'{current_time()} - Performance | {ticker}', 
                 line1_x = df.Date, line1_y = [df['Unrealized Gain'].min()*100]*df.shape[0], line1_color = 'red', style1 = '--',
                 line2_x = df.Date, line2_y = [df['Unrealized Gain'].max()*100]*df.shape[0], line2_color = 'red', style2 = '--')
        plt.show()
        
    elif plot_display_1 == 'on' and plot_display_2 == 'on':
        # main plot
        strategy_main_plot(ticker, df_plot, freq_mid_line_1)
        
        # Volume
        twin_bar(x1_axis = df_plot.Date, y1_axis = df_plot['6_EVMA'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\n6_EVMA', y1_label_color = 'blue',
                 x2_axis = df_plot.Date, y2_axis = df_plot['EVMA_SUM'], x2_color = 'red', y2_label = '\nEVMA_SUM', y2_label_color = 'red',
                 title_name = f'{current_time()} - Volume | {ticker}', 
                 line1_x = df_plot.Date, line1_y = [7]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                 line2_x = df_plot.Date, line2_y = [7]*df_plot.shape[0], line2_color = 'black', style2 = '--')

        # ema price
        twin_bar(x1_axis = df_plot.Date, y1_axis = df_plot['Current Price'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nCurrent Price', y1_label_color = 'blue',
                 x2_axis = df_plot.Date, y2_axis = df_plot['EMA_Plus_Sum'], x2_color = 'red', y2_label = '\nEMA_Plus_Sum', y2_label_color = 'red',
                 title_name = f'{current_time()} - EMA Price | {ticker}', 
                 line1_x = df_plot.Date, line1_y = [1]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                 line2_x = df_plot.Date, line2_y = [1]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        
        # tema price
        twin_bar(x1_axis = df_plot.Date, y1_axis = df_plot['Current Price'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nCurrent Price', y1_label_color = 'blue',
                 x2_axis = df_plot.Date, y2_axis = df_plot['TEMA_Plus_Sum'], x2_color = 'red', y2_label = '\nTEMA_Plus_Sum', y2_label_color = 'red',
                 title_name = f'{current_time()} - TEMA Price | {ticker}', 
                 line1_x = df_plot.Date, line1_y = [1]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                 line2_x = df_plot.Date, line2_y = [1]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        
        # vwma ema price
        twin_bar(x1_axis = df_plot.Date, y1_axis = df_plot['VWAP'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nVWAP', y1_label_color = 'blue',
                 x2_axis = df_plot.Date, y2_axis = df_plot['VWAP_EMA_Plus_Sum'], x2_color = 'red', y2_label = '\nVWAP_EMA_Plus_Sum', y2_label_color = 'red',
                 title_name = f'{current_time()} - VWAP EMA Price | {ticker}', 
                 line1_x = df_plot.Date, line1_y = [1]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                 line2_x = df_plot.Date, line2_y = [1]*df_plot.shape[0], line2_color = 'black', style2 = '--')

        # MACD_1
        twin_plot(x1_axis = df_plot.Date, y1_axis = df_plot['MACD'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nMACD', y1_label_color = 'blue',
                  x2_axis = df_plot.Date, y2_axis = df_plot['SignalLine'], x2_color = 'red', y2_label = '\nSignalLine', y2_label_color = 'red',
                  title_name = f'{current_time()} - MACD > SignalLine | {ticker}', 
                  line1_x = df_plot.Date, line1_y = [opt_macd_gap]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                  line2_x = df_plot.Date, line2_y = [opt_macd_gap]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        
        # MACD_2
        twin_bar(x1_axis = df_plot.Date, y1_axis = df_plot['MACD_SignalLine'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nMACD ~ SignalLine', y1_label_color = 'blue',
                 x2_axis = df_plot.Date, y2_axis = df_plot['MACD_SignalLine'], x2_color = 'green', y2_label = '\nMACD ~ SignalLine', y2_label_color = 'red',
                 title_name = f'{current_time()} - MACD ~ SignalLine | {ticker}', 
                 line1_x = df_plot.Date, line1_y = [opt_macd_gap]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                 line2_x = df_plot.Date, line2_y = [opt_macd_gap]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        
        # RSI
        twin_plot(x1_axis = df_plot.Date, y1_axis = df_plot['Current Price'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nCurrent Price', y1_label_color = 'blue',
                  x2_axis = df_plot.Date, y2_axis = df_plot['RSI'], x2_color = 'red', y2_label = '\nRSI', y2_label_color = 'red',
                  title_name = f'{current_time()} - RSI (Long: < 70 || Short: > 70) | {ticker}', 
                  line1_x = df_plot.Date, line1_y = [70]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                  line2_x = df_plot.Date, line2_y = [90]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        # 40 RSI
        twin_plot(x1_axis = df_plot.Date, y1_axis = df_plot['Current Price'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nCurrent Price', y1_label_color = 'blue',
                  x2_axis = df_plot.Date, y2_axis = df_plot['40_RSI'], x2_color = 'red', y2_label = '\n40_RSI', y2_label_color = 'red',
                  title_name = f'{current_time()} - 40 RSI (Long: > 40 || Short: <= 40) | {ticker}', 
                  line1_x = df_plot.Date, line1_y = [40]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                  line2_x = df_plot.Date, line2_y = [40]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        # 70 RSI
        twin_plot(x1_axis = df_plot.Date, y1_axis = df_plot['Current Price'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nCurrent Price', y1_label_color = 'blue',
                  x2_axis = df_plot.Date, y2_axis = df_plot['70_RSI'], x2_color = 'red', y2_label = '\n70_RSI', y2_label_color = 'red',
                  title_name = f'{current_time()} - 70 RSI (Long: > 45 || Short: <= 45) | {ticker}', 
                  line1_x = df_plot.Date, line1_y = [45]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                  line2_x = df_plot.Date, line2_y = [45]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        
        # ADX
        plt.figure(figsize = (16,9))
        plt.title(f'Average Directional Index | {ticker}')
        plt.grid()
        errorbar_columns = errorbar_columns = ['ADX_14', 'DMP_14', 'DMN_14']
        for column in errorbar_columns:
            plt.errorbar(data = df_plot, x = 'Date', y = column, linewidth = 1.5, label = column)
            plt.annotate(xy = [df_plot[-1:].Date.values, df_plot[-1:][column].values], text = column)
        plt.plot(df_plot.Date, [20]*df_plot.shape[0], color = 'black', linestyle = '--')
        plt.plot(df_plot.Date, [25]*df_plot.shape[0], color = 'black', linestyle = '--')
        
        # CCI
        twin_plot(x1_axis = df_plot.Date, y1_axis = df_plot['Close'], x1_color = 'blue', x1_label = '\nDate', y1_label = '\nCurrent Price', y1_label_color = 'blue',
                 x2_axis = df_plot.Date, y2_axis = df_plot['CCI'], x2_color = 'red', y2_label = '\nCCI', y2_label_color = 'red',
                 title_name = f'{current_time()} - Commodity Channel Index | {ticker}', 
                 line1_x = df_plot.Date, line1_y = [100]*df_plot.shape[0], line1_color = 'black', style1 = '--',
                 line2_x = df_plot.Date, line2_y = [-100]*df_plot.shape[0], line2_color = 'black', style2 = '--')
        
        # Ichimoku
        ichimoku_chart(ticker, df_plot)
        
        # bollinger band
        bollinger_plot(ticker, df_plot, squeeze_ceiling_buy, squeeze_floor_sell, loose_ceiling_sell, loose_floor_buy)
    
        # backtest
        twin_bar(x1_axis = df.Date, y1_axis = df['Unrealized Gain']*100, x1_color = 'red', x1_label = '\nDate', y1_label = '\nUnrealized Gain (%)', y1_label_color = 'red',
                 x2_axis = df.Date, y2_axis = df['Realized Gain']*100, x2_color = 'blue', y2_label = '\nRealized Gain (%)', y2_label_color = 'blue', 
                 title_name = f'{current_time()} - Performance | {ticker}', 
                 line1_x = df.Date, line1_y = [df['Unrealized Gain'].min()*100]*df.shape[0], line1_color = 'red', style1 = '--',
                 line2_x = df.Date, line2_y = [df['Unrealized Gain'].max()*100]*df.shape[0], line2_color = 'red', style2 = '--')
        plt.show()
    else:
        pass
    
    # performance
    duration = len(df)
    exposure_days = len(df[df['Unrealized Value'] != 0])
    exposure_percentage = np.round(exposure_days / duration * 100, 2)
    equity_peak = df[df['Unrealized Value'] == df['Unrealized Value'].max()]
    equity_peak_value = np.round(equity_peak['Unrealized Value'].values[0], 2)
    equity_peak_positionDays = equity_peak['LongPosition Days'].values[0]
    AvgUnrealizedGain = np.round(df['Unrealized Gain'].mean() *100, 2)
    
    pdf = df[df['Realized Gain'] != 0][['Date', 'Realized Gain', 'LongPosition Days']].reset_index(drop = True)
    if len(pdf) == 0:
        AvgRealizedGain = 0
        AvgHoldingDays = int(df[-1:]['LongPosition Days'].mean())
        stg_volatility = 0
        BestPerf = df['Unrealized Gain'].max() *100
        BestPerf_date = str(df[df['Unrealized Gain'] == df['Unrealized Gain'].max()].Date.values[0])[:10]
        Worstperf = df['Unrealized Gain'].min() *100        
        Worstperf_date = str(df[df['Unrealized Gain'] == df['Unrealized Gain'].min()].Date.values[0])[:10]
        PerfRange = BestPerf - Worstperf
        SuccessRate = 0
        timesRealized = 0
    else:
        AvgRealizedGain = np.round(pdf['Realized Gain'].mean() *100, 2)
        AvgHoldingDays = int(pdf['LongPosition Days'].mean())
        stg_volatility = stg_volatility_check(df[df['Unrealized Gain'] != 0])
        BestPerf = df['Unrealized Gain'].max() *100
        BestPerf_date = str(df[df['Unrealized Gain'] == df['Unrealized Gain'].max()].Date.values[0])[:10]
        Worstperf = df['Unrealized Gain'].min() *100        
        Worstperf_date = str(df[df['Unrealized Gain'] == df['Unrealized Gain'].min()].Date.values[0])[:10]
        PerfRange = BestPerf - Worstperf
        pdf['Result'] = (pdf['Realized Gain'] > 0)
        SuccessRate = np.round(pdf.Result.mean()*100, 2)    
        timesRealized = len(pdf['Realized Gain'])      
    if plot_display_1 == 'off':
        pass
    else:
        pd.set_option('display.max_rows', None)
        if plot_display_1 == 'off':
            pass
        else:
            pdf['Realized Gain'] = np.round(pdf['Realized Gain']*100, 2)
            display(pdf)
            
        firstDate = str(df['Date'].values[0])
        long_sign_sum = df['Long_Sign'].sum()
        short_sign_sum = df['Short_Sign'].sum()
        long_to_short = long_sign_sum / short_sign_sum
        
        print('<<< Backtest Result: ' + '\033[1m' + f'{ticker}' + '\033[0m' + ' >>>')
        print('--------------------------------------------------------------------------')
        print('• Start Date: ' + '\033[1m' + f'{firstDate[:10]}' + '\033[0m')
        print('• Duration: ' + '\033[1m' + f'{duration}' + '\033[0m' + ' days')
        print('• Exposure Days: ' + '\033[1m' + f'{exposure_days}' + '\033[0m' + ' days')
        print('• Exposure [%]: ' + '\033[1m' + f'{exposure_percentage}' + '\033[0m' + ' %')
        print('• Equity Peak [$]: ' + '\033[1m' + f'{equity_peak_value}' + '\033[0m')
        print('• Equity Peak Position Days: ' + '\033[1m' + f'{equity_peak_positionDays}' + '\033[0m' + ' days')
        
        print( '\033[1m' + f'• Realized(>={min_RealizedCount}): ' + f'{timesRealized}' + ' times' + '\033[0m')
        print('• Long Sign:  ' + '\033[1m' + f'{int(long_sign_sum)}' + '\033[0m' + ' signs')
        print('• Short Sign: ' + '\033[1m' + f'{int(short_sign_sum)}' + '\033[0m' + ' signs')
        print('--------------------------------------------------------------------------')
        print('==> Long to Short: ' + '\033[1m' + f'{np.round(long_to_short, 2)}' + '\033[0m')
        
        print('\n\n<<< Strategy Verification: ' + '\033[1m' + f'{ticker}' + '\033[0m' + ' >>>')
        print('--------------------------------------------------------------------------')
        print(f'• Total Search Times: ' + '\033[1m' + f'{parameter_search_times} cycles' + '\033[0m')
        print('\033[1m' + f'• Success Rate (>{min_SuccessRate}): ' + f'{SuccessRate} %' + '\033[0m')
        print('\033[1m' + f'• Average Realized Gain (>{min_AvgRealizedGain}): ' + f'{AvgRealizedGain} %' + '\033[0m')
        print('\033[1m' + f'• Realized Gain Volatility: ' + f'{stg_volatility}' + '\033[0m')
        print(f'• Average Unrealized Gain : ' + '\033[1m' + f'{AvgUnrealizedGain} %' + '\033[0m')
        print(f'• Highest Unrealized Gain : ' + '\033[1m' + f'{np.round(BestPerf, 2)} %' + '\033[0m')
        print(f'• Highest Unrealized Gain Date: ' + '\033[1m' + f'{BestPerf_date}' + '\033[0m')
        print(f'• Lowest Unrealized Gain (>{min_Worstperf}): ' + '\033[1m' + f'{np.round(Worstperf, 2)} %' + '\033[0m')
        print(f'• Lowest Unrealized Gain Date: ' + '\033[1m' + f'{Worstperf_date}' + '\033[0m')
        print(f'• Unrealized Gain Range (<{max_PerfRange}): ' + '\033[1m' + f'{np.round(PerfRange, 2)} %' + '\033[0m')
        print(f'• Average LongPosition Days (<={max_HoldingDays}): ' + '\033[1m' + f'{AvgHoldingDays} days' + '\033[0m')
        print('--------------------------------------------------------------------------')
        
    return [AvgRealizedGain, AvgUnrealizedGain, AvgHoldingDays, BestPerf, Worstperf, PerfRange, SuccessRate, timesRealized, long_lottery, short_lottery]


def stg_para_search(ticker, parameter_search_times):
    
    stg_df = pd.DataFrame(columns = ["Ticker",
                                     "Period",
                                     "Searching Count",
                                     "Freq Line",
                                     "Freq Line Gap",
                                     "EVMA Sum",
                                     "MACD Gap",
                                     "Target Gain Percent",
                                     "Target Loss Percent",
                                     "Avg Realized Gain",
                                     "Avg Unrealized Gain",
                                     "Avg Holding Days",
                                     "Best Perf",
                                     "Worst Perf",
                                     "Perf Range", 
                                     "Success Rate Percent", 
                                     "Realized Count"])

    print(f'\n\n\n{current_time()} - Strategy Parameter Searching '  + '\033[1m' + f'{min_SuccessRate}'  + '\033[0m' + ' - Ticker: ' + '\033[1m' + f'{ticker}' + '\033[0m')
    for i in tqdm(range(parameter_search_times)):
        # random research on parameters
        freqLine_choice = random.choice(freqLine_list)
        freqLine_gap_choice = random.choice(freqLine_gap_list)
        evma_sum_choice = random.choice(evma_sum_list)
        macd_gap_choice = random.choice(macd_gap_list)
        gain_choice = random.choice(realize_gain_percent_list)
        loss_choice = random.choice(realize_loss_percent_list)  
        
        stg = macro_strategy(ticker, 
                             period = strategy_days, 
                             opt_freqLine = freqLine_choice,
                             opt_freqLine_gap = freqLine_gap_choice,
                             opt_evma_sum = evma_sum_choice,
                             opt_macd_gap = macd_gap_choice,
                             realize_gain_percent = gain_choice,
                             realize_loss_percent = loss_choice,
                             plot_display_1 = 'off', 
                             plot_display_2 = 'off')

        info_dict = {}
        try:
            info_dict["Ticker"] = ticker
            info_dict["Period"] = strategy_days
            info_dict["Searching Count"] = i
            info_dict["Freq Line"] = freqLine_choice
            info_dict["Freq Line Gap"] = freqLine_gap_choice
            info_dict["EVMA Sum"] = evma_sum_choice
            info_dict["MACD Gap"] = macd_gap_choice
            info_dict["Target Gain Percent"] = gain_choice
            info_dict["Target Loss Percent"] = loss_choice
            info_dict["Avg Realized Gain"] = stg[0]
            info_dict["Avg Unrealized Gain"] = stg[1]
            info_dict["Avg Holding Days"] = stg[2]
            info_dict["Best Perf"] = stg[3]
            info_dict["Worst Perf"] = stg[4]
            info_dict["Perf Range"] = stg[5]
            info_dict["Success Rate Percent"] = stg[6]
            info_dict["Realized Count"] = stg[7]
        except:
            alarm()
            print(f"unable to read stg: {i} th")

        stg_df = stg_df.append(info_dict, ignore_index=True)
        
#         print(f'Debug - {i} Realized Counts: {stg[7]} > {min_RealizedCount}')

        # break loop 
        if stg[0] > min_AvgRealizedGain and stg[2] <= max_HoldingDays and stg[4] > min_Worstperf and stg[5] < max_PerfRange and stg[6] > min_SuccessRate and stg[7] > min_RealizedCount:
#             print('\n>>> Break Condition has been met. Save the condition <<<\n')
            
            # save lottery list
            lottery_list = [stg[8], stg[9]]            
            with open(f"lottery_list_{ticker}_{currentdate}.txt", 'w') as f:
                for s in lottery_list:
                    f.write(str(s) + '\n')
            break

    # save
    stg_df = stg_df[(stg_df['Avg Realized Gain'] > min_AvgRealizedGain) & 
            (stg_df['Avg Holding Days'] <= max_HoldingDays) &        
            (stg_df['Worst Perf'] > min_Worstperf) & 
            (stg_df['Perf Range'] < max_PerfRange) &        
            (stg_df['Success Rate Percent'] > min_SuccessRate) & 
            (stg_df['Realized Count'] > min_RealizedCount)]
    stg_df.to_csv(f'stg_para_search_{ticker}_{currentdate}.csv', index = False)

    
def strategy_analysis_main(ticker, display_1 = 'off', display_2 = 'off'):
    
    '''
    input: ticker name, display_1, display_2 (e.g. strategy_analysis_main('^GSPC', 'on', 'off'))
    output: signs
    
    '''

    # display settings 
    # on + off: summary | on + on: all

    # generate parameter search table
    stg_para_search(ticker, parameter_search_times)

    # get the highest parameter
    df = pd.read_csv(f'stg_para_search_{ticker}_{currentdate}.csv').head(1)
    
#     display(df)
    
    if len(df) != 0:
        best_searching_count = df['Searching Count'].values[0]
        best_freqLine = df['Freq Line'].values[0] 
        best_freqLine_gap = df['Freq Line Gap'].values[0]
        best_evma_sum = df['EVMA Sum'].values[0]
        best_macd_gap = df['MACD Gap'].values[0]
        best_gain = df['Target Gain Percent'].values[0]
        best_loss = df['Target Loss Percent'].values[0]


        # make a strategy with the parameter 
        # analysis_result[AvgRealizedGain, AvgHoldingDays, BestPerf, Worstperf, PerfRange]
        analysis_result = macro_strategy(ticker, 
                                         period = strategy_days, 
                                         opt_freqLine = best_freqLine,
                                         opt_freqLine_gap = best_freqLine_gap,
                                         opt_evma_sum = best_evma_sum,
                                         opt_macd_gap = best_macd_gap,
                                         realize_gain_percent = best_gain,
                                         realize_loss_percent = best_loss,
                                         plot_display_1 = display_1, 
                                         plot_display_2 = display_2)

        if display_1 == 'off':
            pass
        else:
            print('\n\n<<< Optimized Parameters: ' + '\033[1m' + f'{ticker}' + '\033[0m' + ' >>>')
            print('--------------------------------------------------------------------------')
            print('\033[1m' + f'• Best Searching Count - {int(best_searching_count)}/{parameter_search_times}' + '\033[0m')
            print(f'• Best Freq Line - {freqLine_list}: ' + '\033[1m' + f'{np.round(best_freqLine, 3)}' + '\033[0m')
            print(f'• Best Freq Line Gap - {freqLine_gap_list}: ' + '\033[1m' + f'{np.round(best_freqLine_gap, 3)}' + '\033[0m')
            print(f'• Best EVMA Sum - {evma_sum_list}: ' + '\033[1m' + f'{np.round(best_evma_sum, 3)}' + '\033[0m')
            print(f'• Best MACD Gap - {macd_gap_list}: ' + '\033[1m' + f'{np.round(best_macd_gap, 3)}' + '\033[0m')
            print(f'• Best Gain [%] - {realize_gain_percent_list}: ' + '\033[1m' + f'{np.round(best_gain*100, 2)}' + '\033[0m')
            print(f'• Best Loss [%] - {realize_loss_percent_list}: ' + '\033[1m' + f'{np.round(best_loss*100, 2)}' + '\033[0m')
            print('--------------------------------------------------------------------------')

        df = pd.read_csv(f'strategy_anaysis_{ticker}_{currentdate}.csv')

        if analysis_result[0] > min_AvgRealizedGain and analysis_result[2] <= max_HoldingDays and analysis_result[4] > min_Worstperf and analysis_result[5] < max_PerfRange and analysis_result[6] >= min_SuccessRate and analysis_result[7] >= min_RealizedCount:
            print('==> Strategy Verification Result: ' + '\033[1m' + 'Passed' + '\033[0m')

            # trading condition
            trading_df = df.tail(analysis_result[2])
            long_sign_sum = trading_df.Long_Sign.values.sum()
            short_sign_sum = trading_df.Short_Sign.values.sum()

            # recent Long sign date
            if len(trading_df[trading_df.Long_Sign > 0]) > 0:
                recent_long_date = trading_df[trading_df.Long_Sign > 0].Date.values[-1]
            else:
                recent_long_date = 0

            # recent Short sign date
            if len(trading_df[trading_df.Short_Sign > 0]) > 0:
                recent_short_date = trading_df[trading_df.Short_Sign > 0].Date.values[-1]
            else:
                recent_short_date = 0

            # sign decision
            if display_1 == 'off':
                pass
            else:
                print(f'\n\n<<< Sign Check: {analysis_result[2]} days >>>')
                print('--------------------------------------------------------------------------')
                print(f'{analysis_result[2]} days total (Buy | Sell): ' + f'{int(long_sign_sum)} | {int(short_sign_sum)}')
                print('--------------------------------------------------------------------------')

            if df['Current Price'].tail(1).values.mean() >= df['Target Gain Price'].tail(1).values.mean() or df['Current Price'].tail(1).values.mean() < df['Target Loss Price'].tail(1).values.mean():
                print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'Realize Sign\n' + '\033[0m')
                return [-1, best_gain, best_loss]

            else:
                # Buy condition
                if long_sign_sum > 0 and short_sign_sum == 0:
                    print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'Long Sign\n' + '\033[0m')
                    return [1, best_gain, best_loss]

                # Sell condition
                elif long_sign_sum == 0 and short_sign_sum > 0 and analysis_price(ticker)[8] == True:
                    print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'Short Sign\n' + '\033[0m')
                    return [-1, best_gain, best_loss]

                # Dual condition
                elif long_sign_sum > 0 and short_sign_sum > 0:
                    if recent_long_date > recent_short_date:
                        print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'Long Sign\n' + '\033[0m')
                        return [1, best_gain, best_loss]

                    elif recent_long_date + recent_short_date == 0:
                        print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'No Sign\n' + '\033[0m')
                        return [0, best_gain, best_loss]          

                    else:
                        if analysis_price(ticker)[8] == True:
                            print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'Short Sign\n' + '\033[0m')
                            return [-1, best_gain, best_loss]
                        else:
                            print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'No Sign\n' + '\033[0m')
                            return [0, best_gain, best_loss]  
                            
                # no sign condition
                else:
                    print(f'==> {current_time()} - Sign Check Result: ' + '\033[1m' + 'No Sign\n' + '\033[0m')
                    return [0, best_gain, best_loss]

        else:
            print(f'==> {current_time()} - Strategy Verification Result: ' + '\033[1m' + 'Failed\n' + '\033[0m')
            print('\033[1m' + '!!! Disregard the random result !!!\n' + '\033[0m')
            return [-10, best_gain, best_loss]
        
    else:
        print(f'==> {current_time()} - Strategy Verification Result: ' + '\033[1m' + 'Failed\n' + '\033[0m')
        return [-10, 0, 0]
    
    
if __name__ == '__main__':
    print('> Modules #3 imported')

In [72]:
tfds.disable_progress_bar()
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

tf.keras.utils.set_random_seed(0)

def tensorflow_model_processing(days, h, o, l, tf_X_train, tf_y_train, tf_X_test, tf_y_test, tf_X_val, tf_y_val, pred_features):
    # initiate
    tf.keras.backend.clear_session() 

    # build model
    layer_neurons = [256, 128, 64, 32, 16, 8]
    input_layers_features = tf_X_train.shape[1]
    output_layers_features = 1

    # model design
    tf_model = tf.keras.Sequential()
    tf_model.add(tf.keras.layers.Flatten(input_shape=(input_layers_features, 1)))
    for neurons in layer_neurons:
        tf_model.add(tf.keras.layers.Dense(neurons, activation = h))
        tf_model.add(tf.keras.layers.Dropout(0.2))
    tf_model.add(tf.keras.layers.Dense(output_layers_features, activation = o))
    
    # compile
    tf_model.compile(optimizer='adam', 
                     loss = l, 
                     metrics=[tfa.metrics.FBetaScore(num_classes=1, beta=0.5, threshold=0.5)])

    # Stop training when there is no improvement in the validation loss for n consecutive epochs
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_fbeta_score', patience = 10)

    # Save the Model with the lowest validation loss
    save_best = tf.keras.callbacks.ModelCheckpoint('./best_model.h5',
                                                   monitor = 'val_fbeta_score',
                                                   save_best_only=True)

#         # evaluate loss and fbeta before tuning
#         loss, fbeta = tf_model.evaluate(tf_X_test, tf_y_test, verbose = 0)
#         print(f'\n\nTensorFlow Model Evalution before training\n• Loss: {loss}\n• fbeta: {fbeta}\n\n')

    # train the model
    EPOCHS = 500    
    history = tf_model.fit(tf_X_train, tf_y_train, 
                           epochs = EPOCHS, 
                           validation_data = (tf_X_val, tf_y_val), 
                           batch_size = 8, 
                           verbose = 0, 
                           callbacks = [early_stopping, save_best])
    
    # history = tf_model.fit(tf_X_train, tf_y_train, epochs = EPOCHS, validation_data = (tf_X_val, tf_y_val), batch_size = 8, verbose = 1, callbacks = [early_stopping])
    # history = tf_model.fit(tf_X_train, tf_y_train, epochs = EPOCHS, validation_data = (tf_X_val, tf_y_val), batch_size = 8, verbose = 1)

    # evaluate loss and fbeta after tuning
    loss, fbeta = tf_model.evaluate(tf_X_test, tf_y_test, verbose = 0)
    print(f'\nTensorFlow Model Evalution after training\n• Loss: {loss}\n• fbeta: {fbeta}\n\n\n\n\n')

    # predict up or down
    prediction = tf_model.predict(pred_features)
    print(prediction)
    print(f'{days} days moving average applied\n\n\n')
    return [days, h, o, l, loss, fbeta, prediction[0][0]]


def opt_data_processing(df):
    # features to predict
    pred_features = df[-1:]

    # get the outcome from the tomorrow price
    df['Tmr_price'] = df['Stock_price'].shift(-1)
    df['classifier_result'] = (df['Tmr_price'] > df['Stock_price']).astype(int)
    df.drop(columns = 'Tmr_price', inplace = True)

    # features to train &test
    df = df[:-1]

    # outcome 
    outcomes = df.pop('classifier_result').values
    # filter out unwanted columns
    features = df.values

    # for non-tf
    X_train, X_test, y_train, y_test = train_test_split(features, outcomes, test_size=0.2, random_state = 0, stratify = outcomes)
    sc = MinMaxScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  
    non_tf_pred_features = sc.transform(pred_features)

    # for tf
    tf_X_train, tf_X_test, tf_y_train, tf_y_test = train_test_split(features, outcomes, test_size=0.2, random_state = 0, stratify = outcomes)

    def tf_normalize(set):  
        return tf.keras.utils.normalize(set)
    tf_X_train = tf_normalize(tf_X_train)
    tf_X_test = tf_normalize(X_test[:int(len(X_test)/2)])
    tf_X_val = tf_normalize(X_test[int(len(X_test)/2):])
    tf_y_test = y_test[:int(len(y_test)/2)]
    tf_y_val = y_test[int(len(y_test)/2):] 
    tf_pred_features = tf_normalize(pred_features)
    
    return tf_X_train, tf_y_train, tf_X_test, tf_y_test, tf_X_val, tf_y_val, tf_pred_features


def param_optimizer(df):
    with open("moving_avg_value.txt", 'r') as f:
        moving_avg_value = [line.rstrip('\n') for line in f]
        moving_avg_value = int(moving_avg_value[0]) 
    
    # for hidden layer model
#     hidden_activation_f = ['relu', 'tanh']
#     output_activation_f = ['softmax', 'sigmoid']
#     loss_f = ['BinaryCrossentropy', 'Hinge', 'MeanSquaredError']
    
    # for LSTM model
    hidden_activation_f = ['relu'] # n/a
    output_activation_f = ['sigmoid']    
    loss_f = ['BinaryCrossentropy', 'Hinge', 'MeanSquaredError']

    param_tune = []
    for days in range(moving_avg_value, moving_avg_value+1):
        print(f'\n\n########## Initial setting: {days} days moving average ##########\n\n')
        # ML-Classifier
        # Get moving average
        mvp = days
        mavg = pd.DataFrame()
        for column in df.columns[1:]:
            mv_change = np.array(df[column])
            mv = []
            for i in range(len(mv_change)-mvp+1):
                mv.append(np.average(mv_change[i:mvp+i]))
                i+=1
            mavg[column] = pd.DataFrame(mv)
        tf_X_train, tf_y_train, tf_X_test, tf_y_test, tf_X_val, tf_y_val, pred_features = opt_data_processing(mavg)
        
        for h in hidden_activation_f:
            print(f'>>> Hidden Function: {h}')
            for o in output_activation_f:
                print(f'>> Output Function: {o}')
                for l in loss_f:
                    print(f'> Loss Function: {l}')
                    param_tune.append(tensorflow_model_processing(days, h, o, l, tf_X_train, tf_y_train, tf_X_test, tf_y_test, tf_X_val, tf_y_val, pred_features))
                    print('----------------------------------------------------')
    return param_tune

def mvg_optimizer(df, hidden_activation, output_activation, loss):
    with open("moving_avg_value.txt", 'r') as f:
        moving_avg_value = [line.rstrip('\n') for line in f]
        moving_avg_value = int(moving_avg_value[0])    
        
        if moving_avg_value == 3:
            moving_avg_value = 4
    
    mvg_tune = []
    for days in range(moving_avg_value-3, moving_avg_value+5):
        if days < 1:
            days = 1
        try:
            print(f'\n\n\n##### {days} days moving average #####')
            # ML-Classifier
            # Get moving average
            mvp = days
            mavg = pd.DataFrame()
            for column in df.columns[1:]:
                mv_change = np.array(df[column])
                mv = []
                for i in range(len(mv_change)-mvp+1):
                    mv.append(np.average(mv_change[i:mvp+i]))
                    i+=1
                mavg[column] = pd.DataFrame(mv)
            tf_X_train, tf_y_train, tf_X_test, tf_y_test, tf_X_val, tf_y_val, pred_features = opt_data_processing(mavg)
            tensorflow_model_processing_result = tensorflow_model_processing(days, hidden_activation, output_activation, loss, tf_X_train, tf_y_train, tf_X_test, tf_y_test, tf_X_val, tf_y_val, pred_features)
            mvg_tune.append(tensorflow_model_processing_result)
            print('====================================================')   
            print('Loop breaker:', tensorflow_model_processing_result[5])            
            if tensorflow_model_processing_result[5] > moving_avg_searchbreak:
                print('\n\n\n\n\n########## Moving average searching: Break ##########\n\n\n\n\n')
                break                
        except:
            print(f'Except >>> {days} moving average')
            pass

    df = pd.DataFrame(mvg_tune)
    df.columns = ['moving_avg', 'hidden_activation', 'output_activation', 'loss_compile', 'loss', 'fbeta', 'result']
    
    df = df.sort_values('loss').reset_index(drop = True)
    df.to_csv('para_mvg_tuned.csv', index = False)
    return df

def main_param():
    voice_message("""\
    TensorFlow optimizing starts""")
    from os.path import exists
    snp_source_exists = exists('predict_aim_sourcedata_monitoring_^GSPC_%s.csv' %currentdate)
    if snp_source_exists == True:
        df_source = pd.read_csv('predict_aim_sourcedata_monitoring_^GSPC_%s.csv' %currentdate)
    else:
        df_source = pd.read_csv('predict_aim_sourcedata_snp.csv')
        
    # parameter optimize
    df = df_source.copy()
    df = pd.DataFrame(param_optimizer(df))
    df.columns = ['moving_avg', 'hidden_activation', 'output_activation', 'loss_compile', 'loss', 'fbeta', 'result']
    df = df.sort_values('loss').reset_index(drop = True)

    param_top_ten = df[df['loss'] == df.head(1).loss.values[0]].sort_values('loss').reset_index(drop = True).head(10)
    param_top_ten.to_csv('param_top_ten.csv', index = False)
    print(param_top_ten)
    
    hidden_activation = param_top_ten.hidden_activation[0] # hidden_activation
    output_activation = param_top_ten.output_activation[0] # output_activation
    loss = param_top_ten.loss_compile[0]      # loss_compile
    
    print('\n\n\n##################################################')
    print(f'• Hidden Layer Activator ==> {hidden_activation}')
    print(f'• Output Layer Activator ==> {output_activation}')
    print(f'• Loss Compiler          ==> {loss}')
    print('##################################################\n\n\n\n\n')
    
    # moving average optimize
    final_opt_df = mvg_optimizer(df_source, hidden_activation, output_activation, loss)  
    df = final_opt_df.copy()
    print(df.sort_values('loss').reset_index(drop = True))
    
    # plot
    df = df.sort_values('moving_avg', ascending = False)
    fig, ax = plt.subplots()
    fig.set_size_inches(16, 9)

    ax.plot(df.moving_avg, df.fbeta, color = 'blue', marker = 'X')
    ax.set_xlabel('Moving Average (days)')
    ax.set_ylabel('fbeta', color = 'blue')
    # ax.axvline(x = 5, color = 'black', linestyle = '--')
    # ax.grid(axis = 'x')

    ax1 = ax.twinx()
    ax1.plot(df.moving_avg, df.loss, color = 'red', marker = 'X')
    ax1.set_ylabel('Loss', color = 'red')
    ax1.grid(axis = 'y')

    # ax.set_xticklabels(df.Date, rotation = 90)
    plt.show()
        
        
if __name__ == '__main__':
    print('> Modules #4 imported')

In [73]:
def pred_tensorflow(tf_X_train, tf_y_train, tf_X_val, tf_y_val, tf_X_test, tf_y_test, pred_features):
    with tf.device('/GPU:0'):
        
        # parameter input
        opt_param_verified_result = opt_param_verified()
        hidden_activation = opt_param_verified_result[0]
        output_activation = opt_param_verified_result[1]
        loss_compile = opt_param_verified_result[2]        
#         print(f'• Hidden Layer Activator           ==> {hidden_activation}')
#         print(f'• Output Layer Activator           ==> {output_activation}')
#         print(f'• Loss Compiler                    ==> {loss_compile}\n')
        
        # initiate
        tf.keras.backend.clear_session() 
        
        # build model
        layer_neurons = [256, 128, 64, 32, 16, 8]
        input_layers_features = tf_X_train.shape[1]
        output_layers_features = 1
        
        # model design
        tf_model = tf.keras.Sequential()
        tf_model.add(tf.keras.layers.Flatten(input_shape=(input_layers_features, 1)))
        for neurons in layer_neurons:
            tf_model.add(tf.keras.layers.Dense(neurons, activation = hidden_activation))
            tf_model.add(tf.keras.layers.Dropout(0.2))
        tf_model.add(tf.keras.layers.Dense(output_layers_features, activation = output_activation))
        
        # compile
        tf_model.compile(optimizer='adam', 
                         loss = loss_compile, 
                         metrics=[tfa.metrics.FBetaScore(num_classes=1, beta=0.5, threshold=0.5)])

        # Stop training when there is no improvement in the validation loss for n consecutive epochs
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_fbeta_score', patience = 10)

        # Save the Model with the lowest validation loss
        save_best = tf.keras.callbacks.ModelCheckpoint('./best_model.h5',
                                                       monitor = 'val_fbeta_score',
                                                       save_best_only=True)

#         # evaluate loss and fbeta before tuning
#         loss, fbeta = tf_model.evaluate(tf_X_test, tf_y_test, verbose = 0)
#         print(f'\n\nTensorFlow Model Evalution before training\n• Loss: {loss}\n• fbeta: {fbeta}\n\n')

        # train the model
        EPOCHS = 500    
        history = tf_model.fit(tf_X_train, tf_y_train, 
                               epochs = EPOCHS, 
                               validation_data = (tf_X_val, tf_y_val), 
                               batch_size = 8, 
                               verbose = 0, 
                               callbacks = [early_stopping, save_best])
        
        # history = tf_model.fit(tf_X_train, tf_y_train, epochs = EPOCHS, validation_data = (tf_X_val, tf_y_val), batch_size = 8, verbose = 1, callbacks = [early_stopping])
        # history = tf_model.fit(tf_X_train, tf_y_train, epochs = EPOCHS, validation_data = (tf_X_val, tf_y_val), batch_size = 8, verbose = 1)

        # evaluate loss and fbeta after tuning
        loss, fbeta = tf_model.evaluate(tf_X_test, tf_y_test, verbose = 0)
#         print(f'TensorFlow Model Evalution after training\n• Loss: {loss}\n• fbeta: {fbeta}\n\n')
#         print(f'• Applied_Hidden Layer Activator           ==> {hidden_activation}')
#         print(f'• Applied_Output Layer Activator           ==> {output_activation}')
#         print(f'• Applied_Loss Compiler                    ==> {loss_compile}\n')
#         print(f'• Applied_Moving Average Value (optimized) ==> {moving_avg_value} days\n')

        # predict up or down
        prediction = tf_model.predict(pred_features)
    
    return [fbeta[0], prediction[0][0]]


def pred_tensorflow_LSTM(tf_X_train, tf_y_train, tf_X_val, tf_y_val, tf_X_test, tf_y_test, pred_features):
    with tf.device('/GPU:0'):
        
        # parameter input
        opt_param_verified_result = opt_param_verified()
        hidden_activation = opt_param_verified_result[0]
        output_activation = opt_param_verified_result[1]
        loss_compile = opt_param_verified_result[2]        
#         print(f'• Hidden Layer Activator           ==> {hidden_activation}')
#         print(f'• Output Layer Activator           ==> {output_activation}')
#         print(f'• Loss Compiler                    ==> {loss_compile}\n')
        
        # initiate
        tf.keras.backend.clear_session() 
        
        # build model
        layer_neurons = [256, 128, 64, 32, 16, 8]
        input_layers_features = tf_X_train.shape[1]
        output_layers_features = 1
        
#         # model design
#         tf_model = tf.keras.Sequential()
#         tf_model.add(tf.keras.layers.Flatten(input_shape=(input_layers_features, 1)))
#         for neurons in layer_neurons:
#             tf_model.add(tf.keras.layers.Dense(neurons, activation = hidden_activation))
#             tf_model.add(tf.keras.layers.Dropout(0.2))
#         tf_model.add(tf.keras.layers.Dense(output_layers_features, activation = output_activation))

        # LSTM model design
        tf_model = tf.keras.Sequential()    
        tf_model.add(LSTM(100, return_sequences = True, input_shape = (input_layers_features, 1)))
        tf_model.add(LSTM(100, return_sequences = False))
        tf_model.add(Dense(25))
        tf_model.add(Dense(output_layers_features, activation = output_activation))
        
        # compile
        tf_model.compile(optimizer='adam', 
                         loss = loss_compile, 
                         metrics=[tfa.metrics.FBetaScore(num_classes=1, beta=0.5, threshold=0.5)])

        # Stop training when there is no improvement in the validation loss for n consecutive epochs
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_fbeta_score', patience = 10)

        # Save the Model with the lowest validation loss
        save_best = tf.keras.callbacks.ModelCheckpoint('./best_model.h5',
                                                       monitor = 'val_fbeta_score',
                                                       save_best_only=True)

#         # evaluate loss and fbeta before tuning
#         loss, fbeta = tf_model.evaluate(tf_X_test, tf_y_test, verbose = 0)
#         print(f'\n\nTensorFlow Model Evalution before training\n• Loss: {loss}\n• fbeta: {fbeta}\n\n')

        # train the model
        EPOCHS = 500    
        history = tf_model.fit(tf_X_train, tf_y_train, 
                               epochs = EPOCHS, 
                               validation_data = (tf_X_val, tf_y_val), 
                               batch_size = 8, 
                               verbose = 0, 
                               callbacks = [early_stopping, save_best])
        
        # history = tf_model.fit(tf_X_train, tf_y_train, epochs = EPOCHS, validation_data = (tf_X_val, tf_y_val), batch_size = 8, verbose = 1, callbacks = [early_stopping])
        # history = tf_model.fit(tf_X_train, tf_y_train, epochs = EPOCHS, validation_data = (tf_X_val, tf_y_val), batch_size = 8, verbose = 1)

        # evaluate loss and fbeta after tuning
        loss, fbeta = tf_model.evaluate(tf_X_test, tf_y_test, verbose = 0)
#         print(f'TensorFlow Model Evalution after training\n• Loss: {loss}\n• fbeta: {fbeta}\n\n')
#         print(f'• Applied_Hidden Layer Activator           ==> {hidden_activation}')
#         print(f'• Applied_Output Layer Activator           ==> {output_activation}')
#         print(f'• Applied_Loss Compiler                    ==> {loss_compile}\n')
#         print(f'• Applied_Moving Average Value (optimized) ==> {moving_avg_value} days\n')

        # predict up or down
        prediction = tf_model.predict(pred_features)
    
    return [fbeta[0], prediction[0][0]]


def pred_logi(X_train, X_test, y_train, y_test, pred_features):
    pipe = Pipeline(steps = [('classifier', LogisticRegression(random_state = 0))])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict_proba(X_test)[:,1]
    y_pred[y_pred > predict_proba_value] = 1
    y_pred[y_pred <= predict_proba_value] = 0
    y_pred = pd.Series(y_pred) 

    fbeta = fbeta_score(y_test, y_pred, beta=0.5)
    prediction = pipe.predict_proba(pred_features)[:,1][0]
    
    return [fbeta, prediction]


def pred_svc(X_train, X_test, y_train, y_test, pred_features):
    clf_svc = SVC(random_state = 0, probability = True)

    # Set up the hyperparameter search
    param_dist = {"C": [0.1, 0.5, 1, 3, 5],
                  "kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
                  "degree": [1, 4]}
    
    scorer = make_scorer(fbeta_score, beta = 0.5)
    
    # Run a randomized search over the hyperparameters
    random_search = RandomizedSearchCV(estimator = clf_svc, 
                                       param_distributions = param_dist,
                                       scoring = scorer,
                                       cv = 2, 
                                       n_iter = 10, 
                                       n_jobs = -1)

    # Fit the model on the training data
    random_search.fit(X_train, y_train)
    
    # reflect Proba Conversion Rate
    y_pred = random_search.best_estimator_.predict_proba(X_test)[:,1]
    y_pred[y_pred > predict_proba_value] = 1
    y_pred[y_pred <= predict_proba_value] = 0
    y_pred = pd.Series(y_pred)   

    fbeta = fbeta_score(y_test, y_pred, beta=0.5)
    prediction = random_search.best_estimator_.predict_proba(pred_features)[:,1][0]
     
    return [fbeta, prediction]


def pred_rf(X_train, X_test, y_train, y_test, pred_features):
    clf_rf = RandomForestClassifier(random_state = 0)

    # Set up the hyperparameter search
    param_dist = {"max_depth": [3, None],
                  "n_estimators": list(range(10, 200)),
                  "max_features": list(range(1, X_test.shape[1]+1)),
                  "min_samples_split": list(range(2, 11)),
                  "min_samples_leaf": list(range(1, 11)),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    
    scorer = make_scorer(fbeta_score, beta = 0.5)
    
    # Run a randomized search over the hyperparameters
    random_search = RandomizedSearchCV(estimator = clf_rf, 
                                       param_distributions = param_dist,
                                       scoring = scorer,
                                       cv = 2, 
                                       n_iter = 10, 
                                       n_jobs = -1)

    # Fit the model on the training data
    random_search.fit(X_train, y_train)
    
    # reflect Proba Conversion Rate
    y_pred = random_search.best_estimator_.predict_proba(X_test)[:,1]
    y_pred[y_pred > predict_proba_value] = 1
    y_pred[y_pred <= predict_proba_value] = 0
    y_pred = pd.Series(y_pred)   

    fbeta = fbeta_score(y_test, y_pred, beta=0.5)
    prediction = random_search.best_estimator_.predict_proba(pred_features)[:,1][0]
     
    return [fbeta, prediction]


def pred_ada(X_train, X_test, y_train, y_test, pred_features):
    clf_ada = AdaBoostClassifier(random_state = 0)

    # Set up the hyperparameter search
    # look at  setting up your search for n_estimators, learning_rate
    # http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
    param_dist = {"n_estimators": [10, 100, 200, 400],
                  "learning_rate": [0.001, 0.005, .01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 10, 20]}
    
    scorer = make_scorer(fbeta_score, beta = 0.5)
    
    # Run a randomized search over the hyperparameters
    random_search = RandomizedSearchCV(estimator = clf_ada, 
                                       param_distributions = param_dist,
                                       scoring = scorer,
                                       cv = 2, 
                                       n_iter = 10, 
                                       n_jobs = -1)

    # Fit the model on the training data
    random_search.fit(X_train, y_train)
    
    # reflect Proba Conversion Rate
    y_pred = random_search.best_estimator_.predict_proba(X_test)[:,1]
    y_pred[y_pred > predict_proba_value] = 1
    y_pred[y_pred <= predict_proba_value] = 0
    y_pred = pd.Series(y_pred)       

    fbeta = fbeta_score(y_test, y_pred, beta=0.5)
    prediction = random_search.best_estimator_.predict_proba(pred_features)[:,1][0]
    
    return [fbeta, prediction]


def pred_grd(X_train, X_test, y_train, y_test, pred_features):
    clf_grd = GradientBoostingClassifier(random_state = 0)

    # Set up the hyperparameter search
    param_dist = {'learning_rate': [0.01, 0.02, 0.03],
                  'subsample'    : [0.9, 0.5, 0.2],
                  'n_estimators' : [100, 500, 1000], 
                  'max_depth'    : [4, 6, 8]}
    
    scorer = make_scorer(fbeta_score, beta = 0.5)

    # Run a randomized search over the hyperparameters
    random_search = RandomizedSearchCV(estimator = clf_grd, 
                                       param_distributions = param_dist,
                                       scoring = scorer,
                                       cv = 2, 
                                       n_iter = 10, 
                                       n_jobs = -1)

    # Fit the model on the training data
    random_search.fit(X_train, y_train)

    # reflect Proba Conversion Rate
    y_pred = random_search.best_estimator_.predict_proba(X_test)[:,1]
    y_pred[y_pred > predict_proba_value] = 1
    y_pred[y_pred <= predict_proba_value] = 0
    y_pred = pd.Series(y_pred)       

    fbeta = fbeta_score(y_test, y_pred, beta=0.5)
    prediction = random_search.best_estimator_.predict_proba(pred_features)[:,1][0]

    return [fbeta, prediction]


def pred_xgb(X_train, X_test, y_train, y_test, pred_features):
    clf_xg = xgb.XGBClassifier(random_state = 0)

    # Set up the hyperparameter search
    param_dist = {'learning_rate'   : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
                  'max_depth'       : [3, 4, 5, 6, 8, 10, 12, 15], 
                  'min_child_weight': [1, 3, 5, 7],
                  'gamma'           : [0.0, 0.1, 0.2, 0.3, 0.4],
                  'colsample_bytree': [0.3, 0.4, 0.5, 0.7]}
    
    scorer = make_scorer(fbeta_score, beta = 0.5)

    # Run a randomized search over the hyperparameters
    random_search = RandomizedSearchCV(estimator = clf_xg, 
                                       param_distributions = param_dist,
                                       scoring = scorer,
                                       cv = 2, 
                                       n_iter = 10, 
                                       n_jobs = -1)

    # Fit the model on the training data
    random_search.fit(X_train, y_train)

    # reflect Proba Conversion Rate
    y_pred = random_search.best_estimator_.predict_proba(X_test)[:,1]
    y_pred[y_pred > predict_proba_value] = 1
    y_pred[y_pred <= predict_proba_value] = 0
    y_pred = pd.Series(y_pred)       

    fbeta = fbeta_score(y_test, y_pred, beta=0.5)
    prediction = random_search.best_estimator_.predict_proba(pred_features)[:,1][0]

    return [fbeta, prediction]


def pred_lr_opt(df):
    df = df.drop(columns = ['Date'])

    # features to predict
    pred_features = df[-1:].drop(columns = ['Stock_price'])

    # features to train &test
    df = df[:-1]         

    # outcome 
    outcomes = df.pop('Stock_price').values
    # filter out unwanted columns
    features = df.values

    X_train, X_test, y_train, y_test = train_test_split(features, outcomes, test_size=0.2, random_state = 0)

    mse_opt = {}
    for degree_value in range(1,4):    
        # find the poly degree having the lowest MSE
        poly_feat = PolynomialFeatures(degree = degree_value)
        X_train_poly = poly_feat.fit_transform(X_train)
        X_test_poly = poly_feat.transform(X_test)
        poly_model = LinearRegression(fit_intercept = False).fit(X_train_poly, y_train)
        y_pred = poly_model.predict(X_test_poly)
        mse = mean_squared_error(y_test, y_pred)
        mse_opt[degree_value] = mse

    mse = pd.DataFrame([mse_opt]).T
    mse.rename(columns = {0:'mse'}, inplace = True)
    mse = mse.reset_index()
    degree_opt = mse[mse['mse'] == mse.mse.min()]['index'].values[0]

    poly_feat = PolynomialFeatures(degree = degree_opt)
    X_train_poly = poly_feat.fit_transform(X_train)
    X_test_poly = poly_feat.transform(X_test)
    poly_model = LinearRegression(fit_intercept = False).fit(X_train_poly, y_train)
    y_pred = poly_model.predict(X_test_poly)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    pred_features_poly = poly_feat.fit_transform(pred_features)
    prediction = float(poly_model.predict(pred_features_poly)[0])
    return [int(features.shape[0]-1), degree_opt, r2, mse, prediction]


def pred_lr(df):
    opt_tail_value = {}
    for i in range(df.shape[0]-30, df.shape[0]):
        pred_lr_opt_result = pred_lr_opt(df.tail(i))
        opt_tail_value[i] = pred_lr_opt_result[3]
#         if pred_lr_opt_result[3] < pred_lr_opt_result[4]*0.03:
#             break

    tail_value = pd.DataFrame([opt_tail_value]).T
    tail_value.rename(columns = {0:'mse'}, inplace = True)
    tail_value = tail_value.reset_index()
    tail_value = tail_value[tail_value['mse'] == tail_value.mse.min()]['index'].values[0]
        
    lr_result = pred_lr_opt(df.tail(tail_value))
    print('\n\n#################################')
    print(f'• No. Observations: {lr_result[0]} records')
    print(f'• Most recent {tail_value} days')
    print(f'• Poly degree: {lr_result[1]}')
    print(f'• R squared: {lr_result[2]}')
    print(f'• MSE: {lr_result[3]}')
    print(f'• Prediction: {lr_result[4]}')    
    print('#################################')
        
    return lr_result[4]


def pred_lstm(df):
    df = df.drop(columns=['Date'])
    df = df[['Stock_price']]  # We'll only use the 'Stock_price' column for simplicity

    # Normalize data using Min-Max scaling
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = scaler.fit_transform(df)

    # Prepare sequences and labels
    sequence_length = 30  # Number of past days to consider for prediction
    X, y = [], []
    for i in range(len(df_scaled) - sequence_length):
        X.append(df_scaled[i:i + sequence_length])
        y.append(df_scaled[i + sequence_length])

    X = np.array(X)
    y = np.array(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Evaluate the model
    mse = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    # Make a prediction for the next day
    last_sequence = df_scaled[-sequence_length:]
    last_sequence = last_sequence.reshape((1, sequence_length, 1))
    prediction = model.predict(last_sequence)
    prediction = scaler.inverse_transform(prediction)[0][0]

    print('\n\n#################################')
    print(f'• No. Observations: {int(df.shape[0]-1)} records')
    print(f'• R squared: {r2}')
    print(f'• MSE: {mse}')
    print(f'• Prediction: {prediction}')    
    print('#################################')
    
    return prediction


def classifer_data_input(df, moving_avg_value):

    with open("moving_avg_value.txt", 'r') as f:
        moving_avg_value = [line.rstrip('\n') for line in f]
        moving_avg_value = int(moving_avg_value[0]) 

    # ML-Classifier
    # Get moving average
    mvp = moving_avg_value
    mavg = pd.DataFrame()
    for column in df.columns[1:]:
        mv_change = np.array(df[column])
        mv = []
        for i in range(len(mv_change)-mvp+1):
            mv.append(np.average(mv_change[i:mvp+i]))
            i+=1
        mavg[column] = pd.DataFrame(mv)
    df = mavg

    # features to predict
    pred_features = df[-1:]

    # get the outcome from the tomorrow price
    df['Tmr_price'] = df['Stock_price'].shift(-1)
    df['classifier_result'] = (df['Tmr_price'] > df['Stock_price']).astype(int)
    df.drop(columns = 'Tmr_price', inplace = True)

    # features to train &test
    df = df[:-1]

    # outcome 
    outcomes = df.pop('classifier_result').values
    # filter out unwanted columns
    features = df.values

    # for non-tf
    X_train, X_test, y_train, y_test = train_test_split(features, outcomes, test_size=0.2, random_state = 42, stratify = outcomes)
    sc = MinMaxScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  
    non_tf_pred_features = sc.transform(pred_features)

    # for tf
    tf_X_train, tf_X_test, tf_y_train, tf_y_test = train_test_split(features, outcomes, test_size=0.2, random_state= 42, stratify = outcomes)

    tf_X_train = sc.transform(tf_X_train)
    tf_X_test = sc.transform(X_test[:int(len(X_test)/2)])
    tf_X_val = sc.transform(X_test[int(len(X_test)/2):])
    tf_y_test = y_test[:int(len(y_test)/2)]
    tf_y_val = y_test[int(len(y_test)/2):] 
    tf_pred_features = sc.transform(pred_features)

    print('\n')
    
    print('tf DNN', end = '                               \r')
    tensorflow = pred_tensorflow(tf_X_train, tf_y_train, tf_X_val, tf_y_val, tf_X_test, tf_y_test, tf_pred_features)
    print(end = '                                     \r')    
    
    print('tf LSTM', end = '                               \r')
    tensorflow_LSTM = pred_tensorflow_LSTM(tf_X_train, tf_y_train, tf_X_val, tf_y_val, tf_X_test, tf_y_test, tf_pred_features)
    print(end = '                                     \r')
    
    print('logi', end = '                               \r')
    logi = pred_logi(X_train, X_test, y_train, y_test, non_tf_pred_features)
    print(end = '                                     \r')
    
    print('svc', end = '                               \r')
    svc = pred_svc(X_train, X_test, y_train, y_test, non_tf_pred_features)
    print(end = '                                     \r')
    
    print('rf', end = '                               \r')
    rf = pred_rf(X_train, X_test, y_train, y_test, non_tf_pred_features)
    print(end = '                                     \r')
    
    print('ada', end = '                               \r')
    ada = pred_ada(X_train, X_test, y_train, y_test, non_tf_pred_features)
    print(end = '                                     \r')
    
    print('grd', end = '                               \r')
    grd = pred_grd(X_train, X_test, y_train, y_test, non_tf_pred_features)
    print(end = '                                     \r')    

    print('xgb', end = '                               \r')
    grd = pred_xgb(X_train, X_test, y_train, y_test, non_tf_pred_features)
    print(end = '                                     \r')        

    result = {}
    result['tensorflow_DNN'] = tensorflow[1]
    result['tensorflow_LSTM'] = tensorflow_LSTM[1]
    result['logi'] = logi[1]
    result['svc'] = svc[1]
    result['rf'] = rf[1]
    result['ada'] = ada[1]
    result['grd'] = grd[1]
    result['xgb'] = grd[1]

    fbeta = {}
    fbeta['tensorflow_DNN'] = tensorflow[0]
    fbeta['tensorflow_LSTM'] = tensorflow_LSTM[0]
    fbeta['logi'] = logi[0]
    fbeta['svc'] = svc[0]
    fbeta['rf'] = rf[0]
    fbeta['ada'] = ada[0]
    fbeta['grd'] = grd[0]    
    fbeta['xgb'] = grd[0]   

    rank = pd.DataFrame([fbeta, result]).T.rename(columns = {0:'fbeta' , 1:'result'})
    rank.sort_values('fbeta', ascending = False, inplace = True)
    print(f'\n\n• Moving Average Days: {mvp}')
    print('====================================\n', rank)
    print('====================================')
    
    print('\n========[Bullish Conditions]========')
    print(f'• Min. fbeta value:  {np.round(modelchoice_minimum_fbeta, 2)}')
    print(f'• Min. Result value: {np.round(predict_proba_value, 2)}')
    print('------------------------------------')
    print(f'• Average fbeta: {min_average_fbeta}')
    print(f'• Number of (+) Models: {min_num_positive_models}')
    print(f'• Pred. (+) Percent: {min_positive_percent}')    
    print('====================================')
    rank = rank[rank['fbeta'] >= modelchoice_minimum_fbeta]
    
    # Proba conversion
    rank_prediction = []
    for result_prediction in rank['result'].values:
        if result_prediction >= np.round(predict_proba_value, 2):
            rank_prediction.append(1)
        else:
            rank_prediction.append(0)
    rank['sign'] = rank_prediction
    
    print(rank)
    
    average_fbeta = np.round(rank.fbeta.mean(), 2)
    print('\n==============[Result]==============')
    print(f'• Average fbeta: {average_fbeta}')
    
    positive_modes = rank['sign'].sum()
    print(f'• Number of (+) Models: {positive_modes}')
    
    average_result = np.round(rank['sign'].mean(), 2)
    print(f'• Pred. (+) Percent: {average_result}')
    print('====================================')
    
    if average_result >= min_positive_percent and positive_modes >= min_num_positive_models and average_fbeta > min_average_fbeta:
        print("• Result: " + '\033[1m' + '(+)' + '\033[0m')
        print('====================================\n\n\n')
        return [average_fbeta, 1]
    else:
        print("• Result: " + '\033[1m' + '(-)' + '\033[0m')
        print('====================================\n\n\n')
        return [average_fbeta, 0]   

    

def opt_param_verified(): # decide parameters and moving average value
    global moving_avg_value
    
    df = pd.read_csv('para_mvg_tuned.csv')
    df.sort_values('loss', inplace = True)
    df.reset_index(drop = True, inplace = True)
    hidden_activation = df.hidden_activation[0]
    output_activation = df.output_activation[0]
    loss_compile = df.loss_compile[0]

#     # moving average value based on 'loss'
#     df.sort_values('loss', inplace = True)
#     df.reset_index(drop = True, inplace = True)
#     moving_avg_value = df[df['loss'] == df.head(1).loss.values[0]].sort_values('moving_avg').head(1).moving_avg.values[0]
#     with open("moving_avg_value.txt", 'w') as f:
#         f.write(str(moving_avg_value))
        
    # moving average value based on 'fbeta'   
    df.sort_values('fbeta', ascending = False, inplace = True)
    df.reset_index(drop = True, inplace = True)
    moving_avg_value = df[df['fbeta'] == df.head(1).fbeta.values[0]].sort_values('moving_avg').head(1).moving_avg.values[0]
    if moving_avg_value < 0:
        moving_avg_value = 1
        
    if moving_avg_value > moving_avg_value_max:
        moving_avg_value = moving_avg_value_max 
    if moving_avg_value < moving_avg_value_min:
        moving_avg_value = moving_avg_value_min 
    else:
        pass        
    with open("moving_avg_value.txt", 'w') as f:
        f.write(str(moving_avg_value))
            
    return [hidden_activation, output_activation, loss_compile, moving_avg_value]
            
 
def main_engine():    
    print('\u2022 TensorFlow version:', tf.__version__)
    print('\u2022 tf.keras version:', tf.keras.__version__)
    print('\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

    print('\n\nAlgorithm Test with S&P500 ...')
    moving_avg_value = opt_param_verified()[3]

    from os.path import exists
    snp_source_exists = exists('predict_aim_sourcedata_monitoring_^GSPC_%s.csv' %currentdate)
    if snp_source_exists == True:
        df_source = pd.read_csv('predict_aim_sourcedata_monitoring_^GSPC_%s.csv' %currentdate)
    else:
        df_source = pd.read_csv('predict_aim_sourcedata_snp.csv')

    al_test_results = classifer_data_input(df_source, moving_avg_value)
    if al_test_results[0] < minimum_fbeta:
        print(f'\n\nExcept >>> The fbeta average is less than the minimum value:\n{al_test_results[0]} ==(Improve)==> {minimum_fbeta}')
        voice_message("""\
        The highest prediction fbeta is less than the mininum value. Code cell break """)
        # code cell break ###############
        class StopExecution(Exception):
            def _render_traceback_(self):
                pass

        raise StopExecution
        # code cell break ###############

    else:
        print('\n\nPrediction Result: Passed\n\n')
        voice_message("""\
        TensorFlow has been optimized""")
        pass

    param_result = opt_param_verified()
    print(f'• Hidden_activation:   {param_result[0]}')
    print(f'• Output_activation:   {param_result[1]}')
    print(f'• Loss_compile:        {param_result[2]}')
    print(f'• Moving Average Days: {param_result[3]}')

    
if __name__ == '__main__':
    print('> Modules #5 imported')

___

# Dashboard

In [ ]:
def event_check():  
   
    event_schedule = {}
    ############# Update Event #########################################
    event_schedule['2022 FED December Projections 1st day'] = 20221213
    event_schedule['2022 FED December Projections 2nd day'] = 20221214
    event_schedule['2023 FED January'] = 20230131
    event_schedule['2023 FED Feburary'] = 20230201
    event_schedule['2023 FED March Projections 1st day'] = 20230321
    event_schedule['2023 FED March Projections 2nd day'] = 20230322
    event_schedule['2023 FED May 1st'] = 20230502
    event_schedule['2023 FED May 2nd'] = 20230503
    event_schedule['2023 FED June Projections 1st day'] = 20230613
    event_schedule['2023 FED June Projections 2nd day'] = 20230614
    event_schedule['2023 FED July 1st'] = 20230725
    event_schedule['2023 FED July 2nd'] = 20230726
    event_schedule['2023 FED September Projections 1st day'] = 20230919
    event_schedule['2023 FED September Projections 2nd day'] = 20230920
    event_schedule['2023 FED October'] = 20231031
    event_schedule['2023 FED November'] = 20231101
    event_schedule['2023 FED December Projections 1st day'] = 20231212
    event_schedule['2023 FED December Projections 2nd day'] = 20231213
    ####################################################################

    event_schedule = pd.DataFrame([event_schedule]).T.rename(columns = {0:'Date'})
    event_schedule.reset_index(inplace = True)
    event_schedule.rename(columns = {'index':'Event'}, inplace = True)
    event_schedule = event_schedule[event_schedule['Date'] >= currentdate_event]
    event_day = str(event_schedule.Date.values[0])[0:4]+'-'+str(event_schedule.Date.values[0])[4:6]+'-'+str(event_schedule.Date.values[0])[6:8]  
    event_day = datetime.strptime(event_day, '%Y-%m-%d').date()
    currentdate_event_datetime = str(currentdate_event)[0:4]+'-'+str(currentdate_event)[4:6]+'-'+str(currentdate_event)[6:8]  
    currentdate_event_datetime = datetime.strptime(currentdate_event_datetime, '%Y-%m-%d').date()
    upcoming_remain = event_day - currentdate_event_datetime
    try:
        upcoming_remain = int(str(upcoming_remain)[:2])
    except:
        upcoming_remain = 0
        
    if timechecknow() < 10:
        
        voice_message(f"""\
        Today is {date.today()}

        We have {num_of_purchased} stocks on hold.
        The total unrealized performance average is at {stock_return_percent}%. 

        The upcoming event is {event_schedule[:1].Event.values[0]} 
        scheduled on {event_day} 
        which is in {upcoming_remain} days.
        We are going to narrow down the mid-term gain percentage by {int(event_risk_ratio*100)}% during the day. 

        Sean, please stay positive and enjoy your progress. Here is a fortune message: {tarot()}
        """)
        time.sleep(5)
        
    else:
        voice_message(f"""\
        We have {num_of_purchased} stocks on hold.
        The total unrealized performance average is at {stock_return_percent}%. 
        """)
        
    time.sleep(5)
        
    return [upcoming_remain, event_schedule[:1].Event.values[0], event_day]


def sean_index_plot():
    
    from os.path import exists
    snp_source_exists = exists('predict_aim_sourcedata_monitoring_^GSPC_%s.csv' %currentdate)
    if snp_source_exists == True:
        df = pd.read_csv('predict_aim_sourcedata_monitoring_^GSPC_%s.csv' %currentdate)
    else:
        df = pd.read_csv('predict_aim_sourcedata_snp.csv')
    df_predict = df.copy()
    df.drop(columns = ['High', 'Low' , 'Open' , 'Volume'], inplace = True)

    df_s = df.drop(['Date'], axis = 1)
    col_names = df_s.columns

    # Feature Scaling
    sc = StandardScaler()
    df_s = sc.fit_transform(df_s)
    df_s = pd.concat([df['Date'], pd.DataFrame(df_s, columns = col_names)], axis = 1)

    df_stats = df_s.copy()
    df_stats['intercept'] = 1

    import statsmodels.api as sm
    lm = sm.OLS(df_stats['Stock_price'], df_stats[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
    results = lm.fit()
    # print(results.summary())

    df_s = df_s.drop(['Stock_price'], axis = 1)
    coef = results.params[1:].reset_index().rename(columns = {0:'coef'})
    for column in df_s.columns.tolist()[1:]:
        df_s[column] = df_s[column] * coef[coef['index'] == column].values.tolist()[0][1]

    Average_index = df_s.drop(columns = 'Date').sum(axis = 1) / 10
    sean_index = pd.DataFrame()
    sean_index['Date'] = df_s.Date
    sean_index['Average_index'] = Average_index
    sean_index = sean_index.reset_index(drop = True)
    sean_index.to_csv('sean_index.csv', index = False)
    snp500_today = pred_lstm(df_predict)

    sean_index_plot = sean_index.tail(averageline)                 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    price_record = yfinance_df_setting('^GSPC').tail(averageline)  #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    fig, ax = plt.subplots()
    fig.set_size_inches(16, 9)
    #     sean_index_plot['Date'] = sean_index_plot['Date'].astype('datetime64[ns]')

    ax.plot(sean_index_plot['Date'], sean_index_plot['Average_index'], color = 'red')
    ax.set_xlabel('\nDate')
    ax.set_ylabel('\nPredicted S&P 500', color = 'red')
    # ax.axvline(x = 1, color = 'black', linestyle = '--')
    ax.grid()

    ax1 = ax.twinx()
    ax1.plot(sean_index_plot['Date'], price_record['Adj Close'], color = 'blue')
    ax1.set_ylabel('\nActual S&P 500', color = 'blue')

    ax.set_xticklabels(sean_index_plot['Date'], rotation = 90)

    plt.title('Sean Index')
    plt.show()

    # prediction accuracy
    sean_index['Date'] = sean_index['Date'].astype('datetime64[ns]')
    price_record = yfinance_df_setting('^GSPC')
    df = pd.merge(sean_index, price_record, on = 'Date', how = 'left')
    df.loc[df['Average_index'].shift(-1).diff() > 0, 'sean_diff'] = 1
    df.loc[df['Average_index'].shift(-1).diff() <= 0, 'sean_diff'] = 0
    df.loc[df['Adj Close'].shift(-1).diff() > 0, 'snp_diff'] = 1
    df.loc[df['Adj Close'].shift(-1).diff() <= 0, 'snp_diff'] = 0
    df.loc[df['sean_diff'] == df['snp_diff'], 'Accuracy'] = 1
    df.loc[df['sean_diff'] != df['snp_diff'], 'Accuracy'] = 0
    prediction_accuracy = df.Accuracy.mean()*100

    if df[-2:].Average_index.diff().values[-1] > 0:
        prediction_result = 'Up'
    else:
        prediction_result = 'Down'

    print('==============[Result]==============')
    # print(' • S&P 500 Actual:          ' + '%.2f' %analysis_price('^GSPC')[4])
    # print(' • S&P 500 Prediction:      ' + '%.2f' %snp500_today)
    print(' • Prediction Accuracy[%]:  ' + '%.2f' %prediction_accuracy)
    print(' • Prediction Result:       ' + '\033[1m' + prediction_result + '\033[0m')
    print('====================================')


if __name__ == '__main__':
    # adjust below when selling
    dead_stock = ['QCOM']

    robin = robin_login() 
    num_of_purchased = len(robin['index'])
    sellmonitor_freq_limit = 10     ###### the max number of stocks on hold
    sellmonitor_freq = int((sellmonitor_freq_limit) - (num_of_purchased - len(dead_stock))) ### adjust 

    stock_return_percent = perf_cal(robin)
    performance_updated_df = robin[['index', 'equity', 'equity_change', 'percent_change']]

    Upcoming_Announce_Date = pd.read_csv('srank.csv')[['Symbol', 'Upcoming Announce Date']]
    Upcoming_Announce_Date.rename(columns = {'Symbol':'index'}, inplace = True)
    performance_updated_df = pd.merge(performance_updated_df, Upcoming_Announce_Date, on = 'index', how = 'left')
    
    performance_updated_df.sort_values('percent_change', ascending = False, inplace = True)
    performance_updated_df = performance_updated_df.reset_index(drop = True)
    equity_change = np.round(performance_updated_df['equity_change'].astype(float).sum(), 2)
    performance_updated_df.to_csv('performance_updated_df.csv', index = False)
    
    vars = ['price', 'quantity', 'average_buy_price', 'equity', 'percent_change', 
            'intraday_percent_change', 'equity_change', 'pe_ratio', 'percentage']
    for var in vars:
        robin[var] = pd.to_numeric(robin[var])
    robin['quantity'] = robin['quantity'].astype(int)
    robin.to_csv('robin_all.csv', index = False)
    portfolio_list = robin['index'].values.tolist()


    
#___< Buy condition >________________________________________________________________________________________

    # order limit$
    #########################################################################################################
    dollar_limit_order_dollar = 100 # must be less than to equal 500 per day trading
    #########################################################################################################
    
    # split condition dollar amount
    #########################################################################################################
    split_condition_dollar = 100
    #########################################################################################################
    
    # Current Exchange
    #########################################################################################################
    krwex = [exchange_check()*0.995] # Currency exchange alarm
    #########################################################################################################
    

    # averageline for volatility, candle, and etc.
    #########################################################################################################
    averageline = 70 #average price to guess trend  
    with open("averageline.txt", 'w') as f:
        f.write(str(averageline))
    #########################################################################################################

    
    # event
    #########################################################################################################
    # fed event gain cut
    event_risk_ratio = 0.7
    
    event_check_result = event_check()
    upcoming_remain = event_check_result[0]
    upcoming_event = event_check_result[1]
    eventdate = event_check_result[2]
    #########################################################################################################
    
    
    # Financial Statement Filtering 
    #########################################################################################################
    # 'snp500_xxxx-xx-xx.csv' comes from SeekingAlpha
    # 'srank' after regression analysis; SeanScore
    # 'slist_xxxx-xx-xx.csv' is filtered by YahooFinance
    
    bluechips = 400 # initial setting
    
    buy_sign_lookback_days = 252
    
    superpass_buysign_lookbackdays = 3
    
    mdd_lookback_days = 120
    
    corr = 0 # slist corr. (default: 1st)
        
    fs_cut = 93 # Financial Statement Filtering
    fs_cut_plus = 3 # weight on focal indexes
    
    vola_cut_rate_value = 2
    
    portfolio_sector_ticker = 1.02
    
    rsi_max_cut = 65
    
    volume_ticker_percent = 0.1
    #########################################################################################################
    
    # Buy condition
    #########################################################################################################
    buymonitor_blacklist = ['GOOG']# not for perf_reveiw** srank.csv
                                    
    inv_tickers = ['GLD', 'USO', 'BTC-USD']
    with open("inv_tickers.txt", 'w') as f:
        for s in inv_tickers:  
            f.write(str(s) + '\n')  

    with open("buymonitor_blacklist.txt", 'w') as f:
        for s in buymonitor_blacklist:  
            f.write(str(s) + '\n')  

    with open("buymonitor_blacklist_original.txt", 'w') as f:
        for s in buymonitor_blacklist:  
            f.write(str(s) + '\n')   

    df = pd.read_csv('srank.csv') # Top 5 market cap tickers
    buymonitor_whitelist = df.sort_values('1M Perf', ascending = False
                                         ).head()['Symbol'].tolist() 
    #########################################################################################################

    

#___< Sell condition >_______________________________________________________________________________________

    # holding period sets in Strategy Def()
    #########################################################################################################
    # filter out stocks within the min. holding period
    current_index = sorted(robin['index'].tolist())
    holding_period = pd.read_csv('holding_period.csv')
    holding_period['days_left'] = holding_period['days'] - min_holding_period
    holding_period_index = sorted(holding_period[holding_period['days'] >= min_holding_period]['index'].tolist())
    holding_period_index = [i for i in holding_period_index if i in current_index]
    #########################################################################################################
    
  
    # realize 
    ######################################################################################################### 
    # limit% for gain range and buying range
    snp_yfinance = yfinance_df_setting('^GSPC')
    snp_20 = snp_yfinance.tail(20)['Adj Close'].mean()
    snp_5 = snp_yfinance.tail(5)['Adj Close'].mean()
    snptrend = np.round(((snp_5 - snp_20)/snp_20),2)

    mid_gain_adjust_percent = 0.2

    mid_gain_market_reflect_percent_by_etf = 0.1

    realize_gain_min = 1
    
    realize_gain_max = 3
  
    buy_limit_rate = 5+snptrend*10

    buy_snp_t_y = [np.round(buy_limit_rate/2, 2)] 

    # Gold Buy price change limit
    buy_gold_t_y = [np.round(buy_limit_rate/2.2, 2)] 

    # Oil Buy price change limit
    buy_oil_t_y = [np.round(buy_limit_rate/2.2, 2)] 

    # Bitcoin Buy price change limit
    buy_bit_t_y = [np.round(buy_limit_rate/2.2, 2)] 
    #########################################################################################################


    # loss-cut split sales 
    #########################################################################################################
    sale_split = 3
    #########################################################################################################
   

    
#___< ML Model >_____________________________________________________________________________________________

    ######################################################################################################### 
    # TF optimizing 
    with open("moving_avg_value.txt", 'r') as f:
        moving_avg_value = [line.rstrip('\n') for line in f]
        moving_avg_value = int(moving_avg_value[0]) 
    
    # fbeta - loop break threshold 
    moving_avg_searchbreak = 0.62
    
    moving_avg_value_min = 3
    
    moving_avg_value_max = 5
    
    if moving_avg_value > moving_avg_value_max:
        print(f'Except >>> Moving Average Value exceeds the limit of {moving_avg_value_max} days')
        voice_message(f"""\
                      Moving Average Value exceeds the limit of {moving_avg_value_max} days""")  
    if moving_avg_value < moving_avg_value_min:
        print(f'Except >>> Moving Average Value less then the limit of {moving_avg_value_min} days')
        voice_message(f"""\
                      Moving Average Value less then the limit of {moving_avg_value_min} days""")  
    
    # min. fbeta score when optimizing combined model score only (fixed var.)
    minimum_fbeta = 0.60

    if upcoming_remain >= 0 and upcoming_remain <= 5:
        
        # candle
        VC_per_PC_limit_V = 300000
        PC_limit_V = 0.000001
        VS_vs_MEAN_limit = 0.8
        VS_Accel_limit = 25

        # proba (+) conversion rate for y_pred
        proba_initial_value = 0.57 #<=== Important
        
        # min. fbeta score when choosing model to predict
        initial_modelchoice_minimum_fbeta = 0.57 #<=== Important
        
        # aggregrated min. percentage to convert (+) sign    
        min_positive_percent = 0.57 #<=== Important 
        
        # min. aggregated average fbeta
        min_average_fbeta = 0.62 #<=== Important (*if the actual fbeta is low, increase)      
        
    else:
        # candle
        VC_per_PC_limit_V = 1000
        PC_limit_V = 0.000001
        VS_vs_MEAN_limit = 0.5
        VS_Accel_limit = 25
        
        proba_initial_value = 0.55 #<=== Important    
        initial_modelchoice_minimum_fbeta = 0.55 #<=== Important
        min_positive_percent = 0.55 #<=== Important 
        min_average_fbeta = 0.60 #<=== Important (*if the actual fbeta is low, increase)   
        
    # fixed variable: 1
    min_num_positive_models = 3
    
    # not fixed variables: 4
    modelchoice_minimum_fbeta = initial_modelchoice_minimum_fbeta
    proba_initial_value_original = proba_initial_value
    predict_proba_value = proba_initial_value 
    # Min / Max thresholds
    modelchoice_minimum_fbeta_min = 0.50
    modelchoice_minimum_fbeta_max = 0.65
    predict_proba_value_min = 0.50
    predict_proba_value_max = 0.65
    
    # S&P 500 Check alarm drop% 
    check_snp_t_y_original = [-1.0] 
    check_snp_t_y = check_snp_t_y_original # proba value is adjusted by snp drop%
    
    # buy valuecheck >= x (valuecheck[0] + valuecheck[1]*3)    
    valuation_conversion_value_buy = 2.6
    
    # buy valuecheck >= x (valuecheck[0] + valuecheck[1]*3)    
    valuation_conversion_value_etf_buy = 3

    # sell valuecheck < x (valuecheck[0] + valuecheck[1]*3)  
    valuation_conversion_value_sell = 3

    # losscut valuecheck < x (valuecheck[0] + valuecheck[1]*3)  for tickers_to_sell_small
    valuation_conversion_value_losscut = 2    
    
    # losscut valuecheck == x (valuecheck[0] + valuecheck[1]*3)  for all tickers
    valuation_conversion_value_losscut_all = 1  
    #########################################################################################################
    
    for stock in holding_period['index'].tolist():
        if stock in current_index:
            pass
        else:
            alarm()
            print('Except >>>' + f'{stock} does not exist in Robinhood.')
    
    
#___< Display >_____________________________________________________________________________________________
    #########################################################################################################
    print('\n Date: ', '\033[1m' + f"{currentdate}" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', "• Exchange Rate - Dollar to KRW: " + '\033[1m' + f" ₩{int(krwex[0])} <== ₩{int(exchange_check())} " + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', "• Dollar amount limit to buy: " + '\033[1m' + f" ${dollar_limit_order_dollar} " + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', "• Split condition dollar min. amount: " + '\033[1m' + f" ${split_condition_dollar} " + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', "• Split sales on Loss-cut: " + '\033[1m' + f" divide by {sale_split} " + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', "• Sell Monitor Freq. Limit(Max. num of stocks): " + '\033[1m' + f"Every {sellmonitor_freq} times" + '\033[0m')
    print('_____________________________________________________________________________________________________')    
    print('\n', f"• Mid-Gain Adjust Percentage - Portfolio:   " + '\033[1m' + f"{mid_gain_adjust_percent} %" + '\033[0m')
    print('\n', f"• Mid-Gain Adjust Percentage - Market(ETF): " + '\033[1m' + f"{mid_gain_market_reflect_percent_by_etf} %" + '\033[0m')
    print('\n', f"• Min. Realize Gain(Mid)%: " + '\033[1m' + f"  {realize_gain_min} %" + '\033[0m')
    print('\n', f"• Max. Realize Gain(Mid)%: " + '\033[1m' + f"  {realize_gain_max} %" + '\033[0m')
    print('\n', f"• Realize Gain Adjustment%: " + '\033[1m' + f" {event_risk_ratio} %" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', f"• Buy Limit Rate%: " + '\033[1m' + f"{np.round(buy_limit_rate, 2)} %" + '\033[0m')
    print('\n', f"  > S&P500 Buy price change initial limit: " + '\033[1m' + f"{buy_snp_t_y} %" + '\033[0m')
    print('\n', f"  > Gold Buy price change limit: " + '\033[1m' + f"{buy_gold_t_y} %" + '\033[0m')
    print('\n', f"  > Oil Buy price change limit: " + '\033[1m' + f"{buy_oil_t_y} %" + '\033[0m')
    print('\n', f"  > Bitcoin Buy price change limit: " + '\033[1m' + f"{buy_bit_t_y} %" + '\033[0m')
    print('\n', f"  > ETF Buy price change initial limit: " + '\033[1m' + f"{buy_snp_t_y} %" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', f"• Valuation Conversion Value - Buy (S&P): " + '\033[1m' + f"{valuation_conversion_value_buy}" + '\033[0m')   
    print('\n', f"• Valuation Conversion Value - Buy (Else): " + '\033[1m' + f"{valuation_conversion_value_etf_buy}" + '\033[0m')  
    print('\n', f"• Valuation Conversion Value - Sell: " + '\033[1m' + f"{valuation_conversion_value_sell}" + '\033[0m') 
    print('\n', f"• Valuation Conversion Value - LossCut(for small): " + '\033[1m' + f"{valuation_conversion_value_losscut}" + '\033[0m')
    print('\n', f"• Valuation Conversion Value - LossCut(for all): " + '\033[1m' + f"{valuation_conversion_value_losscut_all}" + '\033[0m') 
    print('_____________________________________________________________________________________________________')
    print('\n', f"• Volume Sign Candle Limit- Change of Volume per Change of Price: " + '\033[1m' + f"{VC_per_PC_limit_V}" + '\033[0m')
    print('\n', f"• Volume Sign Candle Limit- Change of Price: " + '\033[1m' + f"{PC_limit_V}" + '\033[0m')
    print('\n', f"• Volume Speed vs Mean Limit: " + '\033[1m' + f"{VS_vs_MEAN_limit}" + '\033[0m')
    print('\n', f"• Volume Speed Acceleration Limit: " + '\033[1m' + f"{VS_Accel_limit}" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', f"• S&P 500 Check alarm drop%: " + '\033[1m' + f"{check_snp_t_y[0]} %" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', f"• Bluechips# for Slist: " + '\033[1m' + f"{bluechips} EA" + '\033[0m')
    print('\n', f"• Buy Sign Lookback Days: " + '\033[1m' + f"{buy_sign_lookback_days} days (*inactive)" + '\033[0m')
    print('\n', f"• Super Pass - Buy Sign Lookback Days: " + '\033[1m' + f"{superpass_buysign_lookbackdays} days" + '\033[0m')
    print('\n', f"• MDD Lookback Days: " + '\033[1m' + f"{mdd_lookback_days} days" + '\033[0m')
    print('\n', f"• Financial Statement Filtering: " + '\033[1m' + f"{fs_cut} %" + '\033[0m')
    print('\n', f"• Financial Statement Filtering Plus: " + '\033[1m' + f"{fs_cut_plus} %" + '\033[0m')
    print('\n', f"• Volatility Cut Value (vs S&P500): " + '\033[1m' + f"{vola_cut_rate_value} times" + '\033[0m')
    print('\n', f"• RSI Cut Max: " + '\033[1m' + f"{rsi_max_cut}" + '\033[0m')
    print('\n', f"• 1M Perf. vs Sector - (ETF Ticker x Value): " + '\033[1m' + f"{portfolio_sector_ticker} times" + '\033[0m')
    print('\n', f"• Top ticker sorting by [Volume x Price]: " + '\033[1m' + f"{volume_ticker_percent} times" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', f"• Average Line: " + '\033[1m' + f"{averageline} days" + '\033[0m')
    print('\n', f"• Moving Average Search Break: " + '\033[1m' + f"{moving_avg_searchbreak} <<<" + '\033[0m')
    print('\n', f"• Moving Average Value: " + '\033[1m' + f"{int(moving_avg_value)} days (Min: {moving_avg_value_min} | Max: {moving_avg_value_max}) <<< Do NOT change" + '\033[0m')
    print('\n', f"• Min.fbeta Score verifying optimization: " + '\033[1m' + f"{int(minimum_fbeta*100)} %" + '\033[0m')
    print('\n', f"• Initial Proba Conversion Rate: " + '\033[1m' + f"{int(proba_initial_value*100)} % <<<" + '\033[0m')
    print('\n', f"• Proba Conversion Rate (min): " + '\033[1m' + f"{int(predict_proba_value_min*100)} % <<<" + '\033[0m')
    print('\n', f"• Proba Conversion Rate (max): " + '\033[1m' + f"{int(predict_proba_value_max*100)} % <<<" + '\033[0m')
    print('\n', f"• Model Choice Min. fbeta (min): " + '\033[1m' + f"{int(modelchoice_minimum_fbeta_min*100)} % <<<" + '\033[0m')
    print('\n', f"• Model Choice Min. fbeta (max): " + '\033[1m' + f"{int(modelchoice_minimum_fbeta_max*100)} % <<<" + '\033[0m')
    print('\n', f"• Initial Model Choosing Min. fbeta Score: " + '\033[1m' + f"{int(initial_modelchoice_minimum_fbeta*100)} % <<<" + '\033[0m')
    print('\n', f"• Min. number of Positive-models: " + '\033[1m' + f"{int(min_num_positive_models)} EA <<<" + '\033[0m')
    print('\n', f"• Aggregrated Min. Positive Percent: " + '\033[1m' + f"{int(min_positive_percent*100)} % <<<" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', f"• Buymonitor Whitelist:\n" + '\033[1m' + f"   {sorted(buymonitor_whitelist)}" + '\033[0m')
    print('_____________________________________________________________________________________________________')
    print('\n', f"• Mid-Term Gain Cut(%) on Event Date: " + '\033[1m' + f"{int(event_risk_ratio*100)} %" + '\033[0m')
    print('\n', f"  > The upcoming event: " + '\033[1m' + f"{upcoming_event}" + '\033[0m')
    print('\n', f"  > Event date: " + '\033[1m' + f"{str(eventdate)}" + '\033[0m')
    print('\n', f"  > Days remaining until the upcoming event: " + '\033[1m' + f"{int(upcoming_remain)} days" + '\033[0m')
    print('_____________________________________________________________________________________________________\n\n')
    print('===[ Portfolio Summary ]=============================================================================\n')
    print(performance_updated_df) 
    print("\n• Equity change: " + '\033[1m' + f"             $ {equity_change}" + '\033[0m')
    print("\n• Performance average: " + '\033[1m' + f"         {stock_return_percent} %" + '\033[0m')
    print("\n• Number of stocks on hand: " + '\033[1m' + f"    {int(num_of_purchased)} stocks" + '\033[0m')
    print("\n• Sell monitoring cycle: " + '\033[1m' + f"       {sellmonitor_freq} cycles" + '\033[0m')
    print("\n• Loss-cut Min. Holding Period: " + '\033[1m' + f"{min_holding_period} days" + '\033[0m')
    print("\n• Stocks not working: " + '\033[1m' + f"   {sorted(dead_stock)}" + '\033[0m')
    print("\n• Stocks on hold: " + '\033[1m' + f"       {sorted(current_index)}" + '\033[0m')
    print("\n• Stocks aged the limit: " + '\033[1m' + f"{sorted(holding_period_index)}" + '\033[0m')
    print('\n• Hoding Period:\n\n' + '\033[1m' + f'{holding_period}' + '\033[0m')

    # sell monitor cycle check
    if sellmonitor_freq == 1:
        print('\033[1m' + 'The number of holding stocks has reached the limit. No buy monitor executes.\n\n' + '\033[0m')
    elif sellmonitor_freq < 1:
        message = 'Sell Monitor frequency is less than 1. Code cell breaks.'
        print('\n\n' + '\033[1m' + 'Except >>> ' + message + '\n\n' + '\033[0m')
        alarm()    
        # code cell break ###############
        class StopExecution(Exception):
            def _render_traceback_(self):
                pass

        raise StopExecution
        # code cell break ###############
    else:
        pass
    
    # performance review
    if timechecknow() < 10:
#         time.sleep(30)
        time.sleep(5)
    else:
        time.sleep(5)
    real_performance_selma()
    
#     print('\n=============================================================================[ Portfolio Summary ]===\n')
#     if timechecknow() < 0:
#         # price - moving average plot for holding tickers
#         for ticker in sorted(robin['index'].tolist()):
#             try:
#                 sector_stock_plot(ticker)
#             except:
#                 pass
#             dmv_plot(ticker)
#             rsi_plot(ticker)
#             strategy_analysis = strategy_analysis_main(ticker, 'on', 'off')[0]
#             news_sentiment_analysis(ticker, 'on') 
#             if strategy_analysis == -1:
#                 message = f"{ticker} has a sell sign"
#                 voice_message(message)
#                 print(message)
#             print('\n\n\n\n\n')
#         strategy_analysis_main('005930.KS', 'on', 'off')
#         print('\n\n\n\n\n')
        
#     if timechecknow() < 0:
#         print('\n\n\n\n\n===[ Top & Bottom Performer ]========================================================================\n')
#         # strategy analyiss 
#         ticker = df.sort_values('1M Perf', ascending = False).head(1)['Symbol'].values[0]
#         strategy_analysis_main(ticker, 'on', 'off')
#         news_sentiment_analysis(ticker, 'on') 
#         print('\n\n\n\n\n')
#         ticker = df['Symbol'].tolist()[int(len(df)/2)]
#         strategy_analysis_main(ticker, 'on', 'off')
#         news_sentiment_analysis(ticker, 'on') 
        
#         # treemap for srank
#         print('\n\n\n')
#         srank = pd.read_csv('srank.csv')
#         for index_value in ['Market Cap', 'Volume', '5D Perf', '1M Perf', '6M Perf', 'YTD Perf', 'EBITDA Margin', 'Return on Equity', 'Quick Ratio']:
#             treemap(srank, index_value)
        
    # sean index prediction
    sean_index_plot()
    rsi_plot('^GSPC')
    daily_news_sentiment_all('on')
    
    print('\n' + current_time() + '\n\n' + '\033[1m' + tarot() + '\033[0m' + '\n')
    
    # # if yfinance is not on service
    # https://financedata.github.io/posts/finance-data-reader-users-guide.html

    # import FinanceDataReader as fdr
    # print(fdr.__version__)
    # fdr.DataReader('GOOGL')['Close'].plot();

    # df_NASDAQ = fdr.StockListing('NASDAQ')
    # df_NYSE = fdr.StockListing('NYSE')
    # df_snp = fdr.StockListing('SP500')
    # df_amex = fdr.StockListing('AMEX')

___

# Indexes

## Macroeconomic Indices

In [9]:
def main_macro():
    
    # jobless claim
    url = 'https://fred.stlouisfed.org/data/ICSA.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    joblessclaim = webContent[710:]
    joblessclaim = joblessclaim.split()
    date = []
    number = []
    for i in range(len(joblessclaim)):
        if i%2 == 0:
            date.append(joblessclaim[i])
        else:
            number.append(joblessclaim[i])
    jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
    # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
    jls.Jclaim.replace('.', np.NaN, inplace = True)
    jls.fillna(method = 'ffill', inplace = True)
    jls = jls[jls['Date'] <= currentdate]

    jls['Date'] = jls['Date'].astype('datetime64[ns]')
    jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
    jls = jls.tail(averageline)

    # cpi index
    url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    cpi = webContent[4075:]
    cpi = cpi.split()
    date = []
    number = []
    for i in range(len(cpi)):
        if i%2 == 0:
            date.append(cpi[i])
        else:
            number.append(cpi[i])
    cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
    cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
    cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])
    cpidx = cpidx.tail(20)

    # inflation index
    url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    inf = webContent[1222:]
    inf = inf.split()
    date = []
    number = []
    for i in range(len(inf)):
        if i%2 == 0:
            date.append(inf[i])
        else:
            number.append(inf[i])
    inf = pd.DataFrame({'Date': date, 'Inf': number})
    drp = inf[inf.Inf == '.'].index.values
    inf = inf.drop(index = drp).reset_index(drop=True)
    inf['Date'] = inf['Date'].astype('datetime64[ns]')
    inf['Inf'] = pd.to_numeric(inf['Inf'])
    inf = inf.tail(averageline)

    # 30-Year Fixed Rate Mortgage Average & cleaning
    url = 'https://fred.stlouisfed.org/data/MORTGAGE30US.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    mortg = webContent[1895:]
    mortg = mortg.split()
    date = []
    number = []
    for i in range(len(mortg)):
        if i%2 == 0:
            date.append(mortg[i])
        else:
            number.append(mortg[i])
    mortg = pd.DataFrame({'Date': date, 'Mortg': number})
    mortg['Date'] = mortg['Date'].astype('datetime64[ns]')
    mortg['Mortg'] = pd.to_numeric(mortg['Mortg'])
    mortg = mortg.tail(averageline)
    
    # University of Michigan: Consumer Sentiment 
    url = 'https://fred.stlouisfed.org/data/Umcsent.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    Umcsent = webContent[6897:]
    Umcsent = Umcsent.split()
    date = []
    number = []
    for i in range(len(Umcsent)):
        if i%2 == 0:
            date.append(Umcsent[i])
        else:
            number.append(Umcsent[i])
    Umcsent = pd.DataFrame({'Date': date, 'Umcsent': number})
    Umcsent['Date'] = Umcsent['Date'].astype('datetime64[ns]')
    Umcsent['Umcsent'] = pd.to_numeric(Umcsent['Umcsent'])
    Umcsent = Umcsent.tail(20)

    # University of Michigan: Inflation Expectation
    url = 'https://fred.stlouisfed.org/data/mich.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    umcinf = webContent[1109:]
    umcinf = umcinf.split()
    date = []
    number = []
    for i in range(len(umcinf)):
        if i%2 == 0:
            date.append(umcinf[i])
        else:
            number.append(umcinf[i])
    umcinf = pd.DataFrame({'Date': date, 'umcinf': number})
    umcinf['Date'] = umcinf['Date'].astype('datetime64[ns]')
    umcinf['umcinf'] = pd.to_numeric(umcinf['umcinf'])
    umcinf = umcinf.tail(20)
    
    # Real GDP
    url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    Rgdp = webContent[1005:]
    Rgdp = Rgdp.split()
    date = []
    number = []
    for i in range(len(Rgdp)):
        if i%2 == 0:
            date.append(Rgdp[i])
        else:
            number.append(Rgdp[i])
    Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
    Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
    Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])
    Rgdp = Rgdp.tail(20)
    
  
    # 10 year treasury note
    rate = yfinance_df_setting('^TNX').tail(averageline)   

    # CBOE Volatility Index
    cboe = yfinance_df_setting('^VIX').tail(averageline)   

    # Crude Oil
    mar = yfinance_df_setting('CL=F').tail(averageline)   

    # Bitcoin (*Last)
    bit = yfinance_df_setting('BTC-USD').tail(averageline)   

    # bond ETF 10 yaer
    ief = yfinance_df_setting('IEF').tail(averageline)

    # Gold
    gold = yfinance_df_setting('GC=F').tail(averageline)     

    # Gold ETF
    gld = yfinance_df_setting('GLD').tail(averageline)   

    # Dollar Index
    dollar = yfinance_df_setting('DX-Y.NYB').tail(averageline) 

    # Semi-Con
    sox = yfinance_df_setting('^SOX').tail(averageline)   


    # plot
    plt.figure(figsize = (16, 100))

    plt.subplot(16,1,1)
    plt.errorbar(x = jls.tail(52)['Date'], y = jls.tail(52)['Jclaim'], label = 'Jobless Claim', color = sb.color_palette()[0])
    plt.title('Jobless Claim')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    print(jls.tail())
    
    plt.subplot(16,1,2)
    plt.errorbar(x = cboe['Date'], y = cboe['Adj Close'], label = 'Cboe', color = sb.color_palette()[1])
    plt.title('Cboe Valitality')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,3)
    plt.errorbar(x = cpidx['Date'], y = cpidx['Cpi'], label = 'CPI',  color = sb.color_palette()[2])
    plt.title('CPI')
#     plt.yscale('log')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    
    plt.subplot(16,1,4)
    plt.errorbar(x = Rgdp['Date'], y = Rgdp['Rgdp'], label = 'Real GDP',  color = sb.color_palette()[3])
    plt.title('Real GDP')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,5)
    plt.errorbar(x = Umcsent['Date'], y = Umcsent['Umcsent'], label = 'UMC Consumer Sentiment ', color = sb.color_palette()[4])
    plt.title('UMC Consumer Sentiment ')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    
    plt.subplot(16,1,6)
    plt.errorbar(x = inf['Date'], y = inf['Inf'], label = 'Inflation rate', color = sb.color_palette()[5])
    plt.title('Inflation')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,7)
    plt.errorbar(x = umcinf['Date'], y = umcinf['umcinf'], label = 'UMC Inflation Expectation', color = sb.color_palette()[6])
    plt.title('UMC Inflation Expectation')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    
    plt.subplot(16,1,8)
    plt.errorbar(x = rate['Date'], y = rate['Adj Close'], label = 'Treasury Yield 10 Years', color = sb.color_palette()[7])
    plt.title('Treasury Yield 10 Years')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,9)
    plt.errorbar(x = ief['Date'], y = ief['Adj Close'], label = 'iShares 7-10 Year Treasury Bond ETF', color = sb.color_palette()[8])
    plt.title('iShares 7-10 Year Treasury Bond ETF: IEF')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    
    plt.subplot(16,1,10)
    plt.errorbar(x = mar['Date'], y = mar['Adj Close'], label = 'Crude Oil', color = sb.color_palette()[9])
    plt.title('Crude Oil')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,11)
    plt.errorbar(x = bit['Date'], y = bit['Adj Close'], label = 'BitCoin', color = sb.color_palette()[0])
    plt.title('BitCoin')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,12)
    plt.errorbar(x = mortg['Date'], y = mortg['Mortg'], label = '30-Year Fixed Rate Mortgage Average', color = sb.color_palette()[1])
    plt.title('30-Year Fixed Rate Mortgage Average')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,13)
    plt.errorbar(x = gold['Date'], y = gold['Adj Close'], label = 'Gold', color = sb.color_palette()[2])
    plt.title('Gold')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,14)
    plt.errorbar(x = gld['Date'], y = gld['Adj Close'], label = 'GLD', color = sb.color_palette()[3])
    plt.title('GLD')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,15)
    plt.errorbar(x = dollar['Date'], y = dollar['Adj Close'], label = 'Dollar', color = sb.color_palette()[4])
    plt.title('Dollar')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(16,1,16)
    plt.errorbar(x = sox['Date'], y = sox['Adj Close'], label = 'SOX', color = sb.color_palette()[5])
    plt.title('SOX')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    
    
if __name__ == '__main__':
    minutepassed = timechecknow()
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        main_macro()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Microeconomic Indices

In [10]:
import urllib.request, urllib.error, urllib.parse
import pandas as pd
import yfinance as yf
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

def main_micro():

    sector_tickers = {'S&P 500' : '^GSPC',
                      'Energy' : 'XLE',
                      'Financials' : 'XLF', 
                      'Real Estate' : 'XLRE', 
                      'Materials' : 'XLB',
                      'Information Technology' : 'XLK', 
                      'Consumer Staples' : 'XLP', 
                      'Health Care' : 'XLV', 
                      'Industrials' :'XLI', 
                      'Consumer Discretionary' : 'XLY', 
                      'Utilities' : 'XLU', 
                      'Communication Services' : 'XLC',
                      'QQQ' : 'QQQ',
                      'SPY' : 'SPY', 
                      'SPYD' : 'SPYD', 
                      'SPYG' : 'SPYG', 
                      'SPYV' : 'SPYV'}

    plt.figure(figsize = (16, 80))
    c = 1
    for k, v in sector_tickers.items():
        price_record = yfinance_df_setting(v).tail(averageline) 
        plt.subplot(len(sector_tickers),1,c)
        c+=1
        plt.errorbar(x = price_record['Date'], y = price_record['Adj Close'], label = k, color = sb.color_palette()[int(np.random.randint(10, size = 11)[0])])
#         plt.yscale('log')
        plt.title(k)
        plt.grid(axis = 'x')
        plt.grid(axis = 'y')
    
if __name__ == '__main__':
    minutepassed = timechecknow()
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        main_micro()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Buffet Index

In [11]:
import urllib.request, urllib.error, urllib.parse
import pandas as pd
import yfinance as yf
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

def main_Buffet():
    
    # Real GDP
    url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    Rgdp = webContent[1005:]
    Rgdp = Rgdp.split()
    date = []
    number = []
    for i in range(len(Rgdp)):
        if i%2 == 0:
            date.append(Rgdp[i])
        else:
            number.append(Rgdp[i])
    Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
    Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
    Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

    df = yfinance_df_setting('^GSPC')

    df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
    df.Rgdp.replace(np.NaN, 0, inplace = True)
    for i in range(len(df.Rgdp)):
        if df.Rgdp[i] == 0:
            df.Rgdp[i] = df.Rgdp[i-1]
    df.dropna(inplace = True)

    df['Buffett'] = df['Adj Close'] / df['Rgdp']*10
    df = df.reset_index(drop = True)

    ######################################## Plot ##
    plt.figure(figsize = (16,18))

    plt.subplot(3,1,1)
    df = df[-5000:]
    plt.errorbar(x = df['Date'], y = df['Buffett'], label = 'Buffett (log)', color = sb.color_palette()[0])
    # plt.errorbar(x = df['Date'], y = df['Adj Close'], label = 'S&P 500', color = sb.color_palette()[1])
    plt.title('Buffett Index (LT)')
#     plt.yscale('log')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    
    plt.subplot(3,1,2)
    df = df[-1000:]
    plt.errorbar(x = df['Date'], y = df['Buffett'], label = 'Buffett (log)', color = sb.color_palette()[0])
    # plt.errorbar(x = df['Date'], y = df['Adj Close'], label = 'S&P 500', color = sb.color_palette()[1])
    plt.title('Buffett Index (ST)')
#     plt.yscale('log')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    
    plt.subplot(3,1,3)
    df = df[-500:]
    plt.errorbar(x = df['Date'], y = df['Buffett'], label = 'Buffett (log)', color = sb.color_palette()[0])
    # plt.errorbar(x = df['Date'], y = df['Adj Close'], label = 'S&P 500', color = sb.color_palette()[1])
    plt.title('Buffett Index (SST)')
#     plt.yscale('log')
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')


    # #define colors to use
    # col1 = 'steelblue'
    # col2 = 'red'

    # #define subplots
    # fig,ax = plt.subplots()
    # fig.set_size_inches(16,12)
    # fig.set_dpi(100)

    # #add first line to plot
    # ax.plot(df.Date, df['Adj Close'], color=col1)

    # #add x-axis label
    # ax.set_xlabel('Date')

    # #add y-axis label
    # ax.set_ylabel('S&P 500', color=col1)

    # #define second y-axis that shares x-axis with current plot
    # ax2 = ax.twinx()

    # #add second line to plot
    # ax2.plot(df.Date, df.Buffett, color=col2)

    # #add second y-axis label
    # ax2.set_ylabel('Buffett Index', color=col2)

    # plt.grid(axis = 'x')
    # plt.grid(axis = 'y')

    # plt.show()

    
if __name__ == '__main__':
    minutepassed = timechecknow()
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        main_Buffet()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Sector RSI Plot

In [12]:
if __name__ == '__main__':
    minutepassed = timechecknow()
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        for ticker in ['BTC-USD', 'GLD', 'USO']:
            rsi_plot(ticker) 
        for ticker in etf_index_dic.ticker.tolist():
            ticker_name = etf_index_dic[etf_index_dic['ticker'] == ticker]['sector'].values[0]
            rsi_plot(ticker)  
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

# Prediction

## Prediction - Aim: S&P 500

In [13]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sb
import urllib.request, urllib.error, urllib.parse
import matplotlib.pyplot as plt
%matplotlib inline

# find the highest rsquared date
def rsquare(dfc):
    
    # Regression analysis
    import statsmodels.api as sm
    dfc['intercept'] = 1
    lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']])
    results = lm.fit()
    return results.rsquared, results.pvalues

def main_snp():
    
    # find the highest Rsquared of S&P 500 
    ticker = '^GSPC'
    tkr = yfinance_df(ticker)   
    sfive = tkr

    # 10 year treasery bond yield
    rate = yfinance_df('^TNX')   
    irate = rate[['Date', 'Adj Close']]

    # CBOE Volatility Index
    cboe = yfinance_df('^VIX')   
    cboe = cboe[['Date', 'Adj Close']]

    # Gold Feb 22 (GC=F)
    gold = yfinance_df('GC=F')   
    gold = gold[['Date', 'Adj Close']]

    # Crude Oil
    mar = yfinance_df('CL=F')   
    mar = mar[['Date', 'Adj Close']]

    # Dollar Index
    dollar = yfinance_df('DX-Y.NYB')
    dollar = dollar[['Date', 'Adj Close']]

    # Bitcoin (*Last)
    bit = yfinance_df('BTC-USD')   
    bit = bit[['Date', 'Adj Close']]

    # cpi index
    url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    cpi = webContent[4075:]
    cpi = cpi.split()
    date = []
    number = []
    for i in range(len(cpi)):
        if i%2 == 0:
            date.append(cpi[i])
        else:
            number.append(cpi[i])
    cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
    cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
    cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

    # jobless claim index
    url = 'https://fred.stlouisfed.org/data/ICSA.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    joblessclaim = webContent[710:]
    joblessclaim = joblessclaim.split()
    date = []
    number = []
    for i in range(len(joblessclaim)):
        if i%2 == 0:
            date.append(joblessclaim[i])
        else:
            number.append(joblessclaim[i])
    jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
    # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
    jls.Jclaim.replace('.', np.NaN, inplace = True)
    jls.fillna(method = 'ffill', inplace = True)
    jls = jls[jls['Date'] <= currentdate]    
    
    jls['Date'] = jls['Date'].astype('datetime64[ns]')
    jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
    jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
    cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

    # inflation index
    url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    inf = webContent[1222:]
    inf = inf.split()
    date = []
    number = []
    for i in range(len(inf)):
        if i%2 == 0:
            date.append(inf[i])
        else:
            number.append(inf[i])
    inf = pd.DataFrame({'Date': date, 'Inf': number})
    drp = inf[inf.Inf == '.'].index.values
    inf = inf.drop(index = drp).reset_index(drop=True)
    inf['Date'] = inf['Date'].astype('datetime64[ns]')
    inf['Inf'] = pd.to_numeric(inf['Inf'])

    # merge yfinance tickers & cleaning
    df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(sfive, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
    df = df.fillna(0)
    df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

    # cleaning - cpi and Jclaim
    for i in range(len(df.Cpi)):
        if df.Cpi[i] == 0.0:
            df.Cpi[i] = df.Cpi[i-1]    
    df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
    for i in range(len(df.Jclaim)):
        if df.Jclaim[i] == 0.0:
            df.Jclaim[i] = df.Jclaim[i-1]
    drp = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=drp)

    # cleaning - inflation rate since 1990
    df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df[df['Date'] > '2003-01-01']
    df = df.fillna(0)
    if df.tail(1).Inf.values[0] == 0:
        df.tail(1).Inf = df.tail(2).Inf.values[0]


#     ## added but low P-value, also distort Int-rate    

#     # 30-Year Fixed Rate Mortgage Average & cleaning
#     url = 'https://fred.stlouisfed.org/data/MORTGAGE30US.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     mortg = webContent[32820:]
#     mortg = mortg.split()
#     date = []
#     number = []
#     for i in range(len(mortg)):
#         if i%2 == 0:
#             date.append(mortg[i])
#         else:
#             number.append(mortg[i])
#     mortg = pd.DataFrame({'Date': date, 'Mortg': number})
#     mortg['Date'] = mortg['Date'].astype('datetime64[ns]')
#     mortg['Mortg'] = pd.to_numeric(mortg['Mortg'])

#     df = df.merge(mortg, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df[df['Date'] >= '2003-01-03']
#     df = df.fillna(0).reset_index(drop=True)
#     for i in range(len(df.Mortg)):
#         if df.Mortg[i] == 0.0:
#             df.Mortg[i] = df.Mortg[i-1]
#     drp = df[df['Mortg'] == 0].index.values
#     df = df.drop(index=drp)
    
    
#     # University of Michigan: Consumer Sentiment 
#     url = 'https://fred.stlouisfed.org/data/Umcsent.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     Umcsent = webContent[6897:]
#     Umcsent = Umcsent.split()
#     date = []
#     number = []
#     for i in range(len(Umcsent)):
#         if i%2 == 0:
#             date.append(Umcsent[i])
#         else:
#             number.append(Umcsent[i])
#     Umcsent = pd.DataFrame({'Date': date, 'Umcsent': number})
#     Umcsent['Date'] = Umcsent['Date'].astype('datetime64[ns]')
#     Umcsent['Umcsent'] = pd.to_numeric(Umcsent['Umcsent'])
    
#     # cleaning
#     df = df.merge(Umcsent, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.fillna(0)
#     if df.tail(1).Umcsent.values[0] == 0:
#         df.tail(1).Umcsent = df.tail(2).Umcsent.values[0]

    # merge - cboe     
    df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - gold
    df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - Crude Oil Mar 22 
    df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

    # merge - Dollar Index 
    df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

    # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
    df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)

    # dropna and save the source date
    df = df.dropna()

    # fillavg = df[df['Inf'] == 0].index.values
    # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

    dfdrop = df[df['Stock_price'] == 0].index.values
    df = df.drop(index = dfdrop)   

    # dfdrop = df[df['Inf'] == 0].index.values
    # df = df.drop(index = dfdrop)

    fillavg = df[df['Inf'] == 0].index.values
    df['Inf'].loc[fillavg] = df['Inf'].tail(5).max() 


    dfdrop = df[df['Gold'] == 0].index.values
    df = df.drop(index=dfdrop)

    dfdrop = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=dfdrop)

    # Fillna Cboe with the highest value for the past 20 working days. 
    cboe_fillna = df[df['Cboe'] == 0].index.values
    df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 

    # Real GDP
    url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    Rgdp = webContent[1005:]
    Rgdp = Rgdp.split()
    date = []
    number = []
    for i in range(len(Rgdp)):
        if i%2 == 0:
            date.append(Rgdp[i])
        else:
            number.append(Rgdp[i])
    Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
    Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
    Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

    df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
    df.Rgdp.replace(np.NaN, 0, inplace = True)
    for i in range(len(df.Rgdp)):
        if df.Rgdp[i] == 0:
            df.Rgdp[i] = df.Rgdp[i-1]
    df.dropna(inplace = True)
    df = df.drop(columns = ['Close'])
    ################################################################################## add from below

    if df.all().sum() != len(df.columns):
        df.to_csv('errorlog_prediction_snp.csv', index = False)
        print('\nZero value checked: ', ticker, '\n')
        print(df.all())
        alarm() # if any zero value, alarm
    else:
        df.to_csv('predict_aim_sourcedata_snp.csv', index = False)
        
    if str(df[-1:].Date.values[0])[:10] == currentdate:
        print('> The last line == currentdate\n')
    else:
        print('>>> The last line != currentdate\n')
#         alarm()
        
    ##################################### source data save

    # range selection - optimized since [2014-09-17]

    highest = 0

    date = []
    ml = []
    for i in range(len(df.Date.values)-100):
        dfc = df[df['Date'] > df.Date.values[i]]
        result = rsquare(dfc)[0]

        date.append(df.Date.values[i])
        ml.append(result)    

        if highest < result:
            highest = result
        else:
            pass

    dff = pd.DataFrame({'Date': date, 'Result': ml})    
    plt.figure(figsize = (16,9))
    plt.errorbar(dff.Date, dff.Result)
    plt.title('S&P500 R-squared')

    print('__________________________________\n')
    print('The highest R-squared: \n\n', dff[dff['Result']==dff['Result'].max()])
    print('__________________________________')

    # rday: most optimized day cut
    # numpy.datetime64 to string
    ts = pd.to_datetime(str(dff[dff['Result']==dff['Result'].max()].Date.values[0])) 
    rday_snp = ts.strftime('%Y-%m-%d')
    with open("rday_snp.txt", 'w') as f:
        f.write(str(rday_snp)) 

    # Hypothese: 
    # R-squared represents market uncertainties 
    # S&P 500 having its highest R-squared represents the genuine market price
    # The null hypothesis is that S&P 500 follows the best-optimized market, which is most genuine because we do long-term investing
    # originated from S&P 500 stocks which follow long-term price trend. 
    # therefore, we apply the highest rsquared day; Rday. 

    dfr = df[df['Date'] > rday_snp]
    result_optimized = rsquare(dfr)

    # linear regression
    prediction = pred_lstm(dfr)    
    current = dfr.Stock_price.tail(1).values
    current_to_predict = current/prediction

    print('\n• Current S&P500:    ', '%.5f' %current)
    print('\n• Predicted S&P500:  ', '%.5f' %prediction)
    print('\n• Current to Predicted: ', '%.5f' %current_to_predict)
   
    if current < prediction:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'UP' + '\033[0m')
    else:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'DOWN' + '\033[0m')
    print('__________________________________\n')

    # classifier
    prediction = classifer_data_input(dfr, moving_avg_value) # [0]: precision / [1]: prediction result
    if prediction[1] == 1:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'UP' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    else:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'DOWN' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    print('__________________________________\n\n')

    # print out stats summary for multi-linear regression
    import statsmodels.api as sm
    dfr['intercept'] = 1
    lm = sm.OLS(dfr['Stock_price'], dfr[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
    results = lm.fit()
    print(results.summary())
    
    
if __name__ == '__main__':
    minutepassed = timechecknow()
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        main_snp()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Prediction - Aim: Oil (USO)

In [14]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sb
import urllib.request, urllib.error, urllib.parse
import matplotlib.pyplot as plt
%matplotlib inline    


# find the highest rsquared date
def rsquare(dfc):
    
    # Regression analysis
    import statsmodels.api as sm
    dfc['intercept'] = 1
    lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Bitcoin', 'Dollar', 'Rgdp']])
    results = lm.fit()
    return results.rsquared, results.pvalues

def machinelearning(dfr):
    # Machine learning
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Lasso
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.preprocessing import StandardScaler

    x = dfr.drop(columns = ['Date', 'Stock_price', 'intercept', 'Oil'])
    x = x.to_numpy()
    y = dfr.Stock_price.to_numpy()
    
    scaler = StandardScaler()   ####
    x = scaler.fit_transform(x) ####

    poly_feat = PolynomialFeatures(degree = 5)
    x = poly_feat.fit_transform(x)
#     poly_model = LinearRegression(fit_intercept = False).fit(x, y)
    
    sample_house = [x[x.shape[0]-1]]
    model = LinearRegression()  
    model.fit(x, y) 
    
    prediction = model.predict(sample_house)
    return prediction

def main_oil():

    # find the highest Rsquared of GOLD 
    ticker = 'USO'
    tkr = yfinance_df(ticker)   
    sfive = tkr

    # 10 year treasery bond yield
    rate = yfinance_df('^TNX')   
    irate = rate[['Date', 'Adj Close']]

    # CBOE Volatility Index
    cboe = yfinance_df('^VIX')   
    cboe = cboe[['Date', 'Adj Close']]

    # S&P 500
    snp = yfinance_df('^GSPC')   
    snp = snp[['Date', 'Adj Close']]

    # Crude Oil
    mar = yfinance_df('CL=F')   
    mar = mar[['Date', 'Adj Close']]

    # Dollar Index
    dollar = yfinance_df('DX-Y.NYB')
    dollar = dollar[['Date', 'Adj Close']]

    # Bitcoin (*Last)
    bit = yfinance_df('BTC-USD')   
    bit = bit[['Date', 'Adj Close']]

    ################################# End of gathering => Clearning starts  ####################################

    # cleaning starts
    # cpi index
    url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    cpi = webContent[4075:]
    cpi = cpi.split()
    date = []
    number = []
    for i in range(len(cpi)):
        if i%2 == 0:
            date.append(cpi[i])
        else:
            number.append(cpi[i])
    cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
    cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
    cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

    # jobless claim index
    url = 'https://fred.stlouisfed.org/data/ICSA.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    joblessclaim = webContent[710:]
    joblessclaim = joblessclaim.split()
    date = []
    number = []
    for i in range(len(joblessclaim)):
        if i%2 == 0:
            date.append(joblessclaim[i])
        else:
            number.append(joblessclaim[i])
    jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
    # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
    jls.Jclaim.replace('.', np.NaN, inplace = True)
    jls.fillna(method = 'ffill', inplace = True)
    jls = jls[jls['Date'] <= currentdate]
    
    jls['Date'] = jls['Date'].astype('datetime64[ns]')
    jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
    jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
    cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

    # inflation index
    url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    inf = webContent[1222:]
    inf = inf.split()
    date = []
    number = []
    for i in range(len(inf)):
        if i%2 == 0:
            date.append(inf[i])
        else:
            number.append(inf[i])
    inf = pd.DataFrame({'Date': date, 'Inf': number})
    drp = inf[inf.Inf == '.'].index.values
    inf = inf.drop(index = drp).reset_index(drop=True)
    inf['Date'] = inf['Date'].astype('datetime64[ns]')
    inf['Inf'] = pd.to_numeric(inf['Inf'])

    # merge yfinance tickers & cleaning
    df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(sfive, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
    df = df.fillna(0)
    df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

    # cleaning - cpi and Jclaim
    for i in range(len(df.Cpi)):
        if df.Cpi[i] == 0.0:
            df.Cpi[i] = df.Cpi[i-1]    
    df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
    for i in range(len(df.Jclaim)):
        if df.Jclaim[i] == 0.0:
            df.Jclaim[i] = df.Jclaim[i-1]
    drp = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=drp)

    # cleaning - inflation rate since 1990
    df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df[df['Date'] > '2003-01-01']
    df = df.fillna(0)
    if df.tail(1).Inf.values[0] == 0:
        df.tail(1).Inf = df.tail(2).Inf.values[0]

    # merge - cboe     
    df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - snp
    df = df.merge(snp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'snp'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - Crude Oil Mar 22 
    df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

    # merge - Dollar Index 
    df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

    # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
    df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)

    ######################## Do Not change above, starts under the last line #######################
    # dropna and save the source date
    df = df.dropna()

    # fillavg = df[df['Inf'] == 0].index.values
    # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

    dfdrop = df[df['Stock_price'] == 0].index.values
    df = df.drop(index = dfdrop)   

    # dfdrop = df[df['Inf'] == 0].index.values
    # df = df.drop(index = dfdrop)

    fillavg = df[df['Inf'] == 0].index.values
    df['Inf'].loc[fillavg] = df['Inf'].tail(5).max() 

    dfdrop = df[df['snp'] == 0].index.values
    df = df.drop(index=dfdrop)

    dfdrop = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=dfdrop)

    # Fillna Cboe with the highest value for the past 20 working days. 
    cboe_fillna = df[df['Cboe'] == 0].index.values
    df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 
    
    # Real GDP
    url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    Rgdp = webContent[1005:]
    Rgdp = Rgdp.split()
    date = []
    number = []
    for i in range(len(Rgdp)):
        if i%2 == 0:
            date.append(Rgdp[i])
        else:
            number.append(Rgdp[i])
    Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
    Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
    Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

    df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
    df.Rgdp.replace(np.NaN, 0, inplace = True)
    for i in range(len(df.Rgdp)):
        if df.Rgdp[i] == 0:
            df.Rgdp[i] = df.Rgdp[i-1]
    df.dropna(inplace = True)
    df = df.drop(columns = ['Close'])
    
    if df.all().sum() != len(df.columns):
        df.to_csv('errorlog_prediction_oil.csv', index = False)
        print('\nZero value checked: ', ticker, '\n')
        print(df.all())
        alarm() # if any zero value, alarm
    else:
        df.to_csv('predict_aim_sourcedata_oil.csv', index = False)
        
    if str(df[-1:].Date.values[0])[:10] == currentdate:
        print('> The last line == currentdate\n')
    else:
        print('>>> The last line != currentdate\n')
#         alarm()        
        
    ##################################### source data save

    # range selection - optimized since [2014-09-17]

    highest = 0

    date = []
    ml = []
    for i in range(len(df.Date.values)-100):
        dfc = df[df['Date'] > df.Date.values[i]]
        result = rsquare(dfc)[0]

        date.append(df.Date.values[i])
        ml.append(result)    

        if highest < result:
            highest = result
        else:
            pass

    dff = pd.DataFrame({'Date': date, 'Result': ml})    
    plt.figure(figsize = (16,9))
    plt.errorbar(dff.Date, dff.Result)
    plt.title('Oil R-squared')

    print('__________________________________\n')
    print('The highest R-squared: \n\n', dff[dff['Result']==dff['Result'].max()])
    print('__________________________________')

    # rday: most optimized day cut
    # numpy.datetime64 to string
    ts = pd.to_datetime(str(dff[dff['Result']==dff['Result'].max()].Date.values[0])) 
    rday_oil = ts.strftime('%Y-%m-%d')
    with open("rday_oil.txt", 'w') as f:
        f.write(str(rday_oil)) 

    # Hypothese: 
    # R-squared represents market uncertainties 
    # GOLD having its highest R-squared represents the genuine market price
    # The null hypothesis is that GOLD follows the best-optimized market, which is most genuine because we do long-term investing
    # originated from GOLD stocks which follow long-term price trend. 
    # therefore, we apply the highest rsquared day; Rday. 

    dfr = df[df['Date'] > rday_oil]
    result_optimized = rsquare(dfr)

    # linear regression
    prediction = pred_lstm(dfr)    
    current = dfr.Stock_price.tail(1).values
    current_to_predict = current/prediction

    print('\n• Current Oil:    ', '%.5f' %current)
    print('\n• Predicted Oil:  ', '%.5f' %prediction)
    print('\n• Current to Predicted: ', '%.5f' %current_to_predict)
   
    if current < prediction:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'UP' + '\033[0m')
    else:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'DOWN' + '\033[0m')
    print('__________________________________\n')
    
  # classifier
    prediction = classifer_data_input(dfr, moving_avg_value) # [0]: precision / [1]: prediction result

    if prediction[1] == 1:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'UP' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    else:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'DOWN' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    print('__________________________________\n\n')
    
    # print out stats summary for multi-linear regression
    import statsmodels.api as sm
    dfr['intercept'] = 1
    lm = sm.OLS(dfr['Stock_price'], dfr[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Bitcoin', 'Dollar', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
    results = lm.fit()
    print(results.summary())
    
    
if __name__ == '__main__':
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        main_oil()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Prediction - Aim: Gold (GLD)

In [15]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sb
import urllib.request, urllib.error, urllib.parse
import matplotlib.pyplot as plt
%matplotlib inline    


# find the highest rsquared date
def rsquare(dfc):
    
    # Regression analysis
    import statsmodels.api as sm
    dfc['intercept'] = 1
    lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'snp', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']])
    results = lm.fit()
    return results.rsquared, results.pvalues

def machinelearning(dfr):
    # Machine learning
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Lasso
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.preprocessing import StandardScaler

    x = dfr.drop(columns = ['Date', 'Stock_price', 'intercept', 'Cboe'])
    x = x.to_numpy()
    y = dfr.Stock_price.to_numpy()
    
    scaler = StandardScaler()   ####
    x = scaler.fit_transform(x) ####

    poly_feat = PolynomialFeatures(degree = 5)
    x = poly_feat.fit_transform(x)
#     poly_model = LinearRegression(fit_intercept = False).fit(x, y)
    
    sample_house = [x[x.shape[0]-1]]
    model = LinearRegression()  
    model.fit(x, y) 
    
    prediction = model.predict(sample_house)
    return prediction

def main_gold():

    # find the highest Rsquared of GOLD 
    ticker = 'GLD'
    tkr = yfinance_df(ticker)   
    sfive = tkr

    # 10 year treasery bond yield
    rate = yfinance_df('^TNX')   
    irate = rate[['Date', 'Adj Close']]

    # CBOE Volatility Index
    cboe = yfinance_df('^VIX')   
    cboe = cboe[['Date', 'Adj Close']]

    # S&P 500
    snp = yfinance_df('^GSPC')   
    snp = snp[['Date', 'Adj Close']]

    # Crude Oil
    mar = yfinance_df('CL=F')   
    mar = mar[['Date', 'Adj Close']]

    # Dollar Index
    dollar = yfinance_df('DX-Y.NYB')
    dollar = dollar[['Date', 'Adj Close']]

    # Bitcoin (*Last)
    bit = yfinance_df('BTC-USD')   
    bit = bit[['Date', 'Adj Close']]

    ################################# End of gathering => Clearning starts  ####################################

    # cleaning starts
    # cpi index
    url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    cpi = webContent[4075:]
    cpi = cpi.split()
    date = []
    number = []
    for i in range(len(cpi)):
        if i%2 == 0:
            date.append(cpi[i])
        else:
            number.append(cpi[i])
    cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
    cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
    cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

    # jobless claim index
    url = 'https://fred.stlouisfed.org/data/ICSA.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    joblessclaim = webContent[710:]
    joblessclaim = joblessclaim.split()
    date = []
    number = []
    for i in range(len(joblessclaim)):
        if i%2 == 0:
            date.append(joblessclaim[i])
        else:
            number.append(joblessclaim[i])
    jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
    # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
    jls.Jclaim.replace('.', np.NaN, inplace = True)
    jls.fillna(method = 'ffill', inplace = True)
    jls = jls[jls['Date'] <= currentdate]
    
    jls['Date'] = jls['Date'].astype('datetime64[ns]')
    jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
    jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
    cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

    # inflation index
    url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    inf = webContent[1222:]
    inf = inf.split()
    date = []
    number = []
    for i in range(len(inf)):
        if i%2 == 0:
            date.append(inf[i])
        else:
            number.append(inf[i])
    inf = pd.DataFrame({'Date': date, 'Inf': number})
    drp = inf[inf.Inf == '.'].index.values
    inf = inf.drop(index = drp).reset_index(drop=True)
    inf['Date'] = inf['Date'].astype('datetime64[ns]')
    inf['Inf'] = pd.to_numeric(inf['Inf'])

    # merge yfinance tickers & cleaning
    df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(sfive, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
    df = df.fillna(0)
    df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

    # cleaning - cpi and Jclaim
    for i in range(len(df.Cpi)):
        if df.Cpi[i] == 0.0:
            df.Cpi[i] = df.Cpi[i-1]    
    df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
    for i in range(len(df.Jclaim)):
        if df.Jclaim[i] == 0.0:
            df.Jclaim[i] = df.Jclaim[i-1]
    drp = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=drp)

    # cleaning - inflation rate since 1990
    df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df[df['Date'] > '2003-01-01']
    df = df.fillna(0)
    if df.tail(1).Inf.values[0] == 0:
        df.tail(1).Inf = df.tail(2).Inf.values[0]

    # merge - cboe     
    df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - snp
    df = df.merge(snp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'snp'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - Crude Oil Mar 22 
    df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

    # merge - Dollar Index 
    df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

    # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
    df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)

    ######################## Do Not change above, starts under the last line #######################
    # dropna and save the source date
    df = df.dropna()

    # fillavg = df[df['Inf'] == 0].index.values
    # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

    dfdrop = df[df['Stock_price'] == 0].index.values
    df = df.drop(index = dfdrop)   

    # dfdrop = df[df['Inf'] == 0].index.values
    # df = df.drop(index = dfdrop)

    fillavg = df[df['Inf'] == 0].index.values
    df['Inf'].loc[fillavg] = df['Inf'].tail(5).max() 

    dfdrop = df[df['snp'] == 0].index.values
    df = df.drop(index=dfdrop)

    dfdrop = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=dfdrop)

    # Fillna Cboe with the highest value for the past 20 working days. 
    cboe_fillna = df[df['Cboe'] == 0].index.values
    df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 
    
    # Real GDP
    url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    Rgdp = webContent[1005:]
    Rgdp = Rgdp.split()
    date = []
    number = []
    for i in range(len(Rgdp)):
        if i%2 == 0:
            date.append(Rgdp[i])
        else:
            number.append(Rgdp[i])
    Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
    Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
    Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

    df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
    df.Rgdp.replace(np.NaN, 0, inplace = True)
    for i in range(len(df.Rgdp)):
        if df.Rgdp[i] == 0:
            df.Rgdp[i] = df.Rgdp[i-1]
    df.dropna(inplace = True)
    df = df.drop(columns = ['Close'])

    if df.all().sum() != len(df.columns):
        df.to_csv('errorlog_prediction_gold.csv', index = False)
        print('\nZero value checked: ', ticker, '\n')
        print(df.all())
        alarm() # if any zero value, alarm
    else:
        df.to_csv('predict_aim_sourcedata_gold.csv', index = False)
        
    if str(df[-1:].Date.values[0])[:10] == currentdate:
        print('> The last line == currentdate\n')
    else:
        print('>>> The last line != currentdate\n')
#         alarm()
        
    ##################################### source data save

    # range selection - optimized since [2014-09-17]

    highest = 0

    date = []
    ml = []
    for i in range(len(df.Date.values)-100):
        dfc = df[df['Date'] > df.Date.values[i]]
        result = rsquare(dfc)[0]

        date.append(df.Date.values[i])
        ml.append(result)    

        if highest < result:
            highest = result
        else:
            pass

    dff = pd.DataFrame({'Date': date, 'Result': ml})    
    plt.figure(figsize = (16,9))
    plt.errorbar(dff.Date, dff.Result)
    plt.title('GOLD R-squared')

    print('__________________________________\n')
    print('The highest R-squared: \n\n', dff[dff['Result']==dff['Result'].max()])
    print('__________________________________')

    # rday: most optimized day cut
    # numpy.datetime64 to string
    ts = pd.to_datetime(str(dff[dff['Result']==dff['Result'].max()].Date.values[0])) 
    rday_gold = ts.strftime('%Y-%m-%d')
    with open("rday_gold.txt", 'w') as f:
        f.write(str(rday_gold)) 

    # Hypothese: 
    # R-squared represents market uncertainties 
    # GOLD having its highest R-squared represents the genuine market price
    # The null hypothesis is that GOLD follows the best-optimized market, which is most genuine because we do long-term investing
    # originated from GOLD stocks which follow long-term price trend. 
    # therefore, we apply the highest rsquared day; Rday. 

    dfr = df[df['Date'] > rday_gold]
    result_optimized = rsquare(dfr)

    # linear regression
    prediction = pred_lstm(dfr)    
    current = dfr.Stock_price.tail(1).values
    current_to_predict = current/prediction

    print('\n• Current Gold:    ', '%.5f' %current)
    print('\n• Predicted Gold:  ', '%.5f' %prediction)
    print('\n• Current to Predicted: ', '%.5f' %current_to_predict)
   
    if current < prediction:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'UP' + '\033[0m')
    else:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'DOWN' + '\033[0m')
    print('__________________________________\n')

  # classifier
    prediction = classifer_data_input(dfr, moving_avg_value) # [0]: precision / [1]: prediction result

    if prediction[1] == 1:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'UP' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    else:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'DOWN' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    print('__________________________________\n\n')

    # print out stats summary for multi-linear regression
    import statsmodels.api as sm
    dfr['intercept'] = 1
    lm = sm.OLS(dfr['Stock_price'], dfr[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'snp', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
    results = lm.fit()
    print(results.summary())


if __name__ == '__main__':
    minutepassed = timechecknow()
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        main_gold()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Prediction - Aim: BitCoin

In [16]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sb
import urllib.request, urllib.error, urllib.parse
import matplotlib.pyplot as plt
%matplotlib inline    

    
# find the highest rsquared date
def rsquare(dfc):

    # Regression analysis
    import statsmodels.api as sm
    dfc['intercept'] = 1
    lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Oil', 'Dollar', 'Gold', 'Rgdp']])
    results = lm.fit()
    return results.rsquared, results.pvalues

def machinelearning(dfr):
    # Machine learning
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Lasso
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.preprocessing import StandardScaler

    x = dfr.drop(columns = ['Date', 'intercept', 'Stock_price'])
    x = x.to_numpy()
    y = dfr.Stock_price.to_numpy()

    scaler = StandardScaler()   ####
    x = scaler.fit_transform(x) ####

    poly_feat = PolynomialFeatures(degree = 5)
    x = poly_feat.fit_transform(x)
#     poly_model = LinearRegression(fit_intercept = False).fit(x, y)

    sample_house = [x[x.shape[0]-1]]
    model = LinearRegression()  
    model.fit(x, y) 
    
    prediction = model.predict(sample_house)
    return prediction

def main_bit():
    
    # find the highest Rsquared of bitcoin 
    ticker = 'BTC-USD'
    tkr = yfinance_df(ticker)   
    sfive = tkr

    # 10 year treasery bond yield
    rate = yfinance_df('^TNX')   
    irate = rate[['Date', 'Adj Close']]

    # CBOE Volatility Index
    cboe = yfinance_df('^VIX')   
    cboe = cboe[['Date', 'Adj Close']]

    # S&P 500
    snp = yfinance_df('^GSPC')   
    snp = snp[['Date', 'Adj Close']]

    # Crude Oil
    mar = yfinance_df('CL=F')   
    mar = mar[['Date', 'Adj Close']]

    # Dollar Index
    dollar = yfinance_df('DX-Y.NYB')
    dollar = dollar[['Date', 'Adj Close']]

    # Gold
    gold = yfinance_df('GC=F')   
    gold = gold[['Date', 'Adj Close']]

    ################################# End of gathering => Clearning starts  ####################################

    # cleaning starts
    # cpi index
    url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    cpi = webContent[4075:]
    cpi = cpi.split()
    date = []
    number = []
    for i in range(len(cpi)):
        if i%2 == 0:
            date.append(cpi[i])
        else:
            number.append(cpi[i])
    cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
    cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
    cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

    # jobless claim index
    url = 'https://fred.stlouisfed.org/data/ICSA.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    joblessclaim = webContent[710:]
    joblessclaim = joblessclaim.split()
    date = []
    number = []
    for i in range(len(joblessclaim)):
        if i%2 == 0:
            date.append(joblessclaim[i])
        else:
            number.append(joblessclaim[i])
    jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
    # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
    jls.Jclaim.replace('.', np.NaN, inplace = True)
    jls.fillna(method = 'ffill', inplace = True)
    jls = jls[jls['Date'] <= currentdate]
    
    jls['Date'] = jls['Date'].astype('datetime64[ns]')
    jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
    jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
    cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

    # inflation index
    url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    inf = webContent[1222:]
    inf = inf.split()
    date = []
    number = []
    for i in range(len(inf)):
        if i%2 == 0:
            date.append(inf[i])
        else:
            number.append(inf[i])
    inf = pd.DataFrame({'Date': date, 'Inf': number})
    drp = inf[inf.Inf == '.'].index.values
    inf = inf.drop(index = drp).reset_index(drop=True)
    inf['Date'] = inf['Date'].astype('datetime64[ns]')
    inf['Inf'] = pd.to_numeric(inf['Inf'])

    # merge yfinance tickers & cleaning
    df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.merge(sfive, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
    df = df.fillna(0)
    df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

    # cleaning - cpi and Jclaim
    for i in range(len(df.Cpi)):
        if df.Cpi[i] == 0.0:
            df.Cpi[i] = df.Cpi[i-1]    
    df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
    for i in range(len(df.Jclaim)):
        if df.Jclaim[i] == 0.0:
            df.Jclaim[i] = df.Jclaim[i-1]
    drp = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=drp)

    # cleaning - inflation rate since 1990
    df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df[df['Date'] > '2003-01-01']
    df = df.fillna(0)
    if df.tail(1).Inf.values[0] == 0:
        df.tail(1).Inf = df.tail(2).Inf.values[0]

    # merge - cboe     
    df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - snp
    df = df.merge(snp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'snp'}).reset_index(drop=True)
    df = df.fillna(0)

    # merge - Crude Oil Mar 22 
    df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

    # merge - Dollar Index 
    df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

    # merge - gold
    df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
    df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)

    ######################## Do Not change above, starts under the last line #######################
    # dropna and save the source date
    df = df.dropna()

    # fillavg = df[df['Inf'] == 0].index.values
    # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

    dfdrop = df[df['Stock_price'] == 0].index.values
    df = df.drop(index = dfdrop)   

    # dfdrop = df[df['Inf'] == 0].index.values
    # df = df.drop(index = dfdrop)

    fillavg = df[df['Inf'] == 0].index.values
    df['Inf'].loc[fillavg] = df['Inf'].tail(5).max() 

    dfdrop = df[df['snp'] == 0].index.values
    df = df.drop(index=dfdrop)

    dfdrop = df[df['Int_rate'] == 0].index.values
    df = df.drop(index=dfdrop)

    # Fillna Cboe with the highest value for the past 20 working days. 
    cboe_fillna = df[df['Cboe'] == 0].index.values
    df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 

    # Real GDP
    url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
    response = urllib.request.urlopen(url)
    webContent = response.read().decode('UTF-8')
    Rgdp = webContent[1005:]
    Rgdp = Rgdp.split()
    date = []
    number = []
    for i in range(len(Rgdp)):
        if i%2 == 0:
            date.append(Rgdp[i])
        else:
            number.append(Rgdp[i])
    Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
    Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
    Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

    df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
    df.Rgdp.replace(np.NaN, 0, inplace = True)
    for i in range(len(df.Rgdp)):
        if df.Rgdp[i] == 0:
            df.Rgdp[i] = df.Rgdp[i-1]
    df.dropna(inplace = True)
    df = df.drop(columns = ['Close'])
    
    if df.all().sum() != len(df.columns):
        df.to_csv('errorlog_prediction_bitcoin.csv', index = False)
        print('\nZero value checked: ', ticker, '\n')
        print(df.all())
        alarm() # if any zero value, alarm
    else:
        df.to_csv('predict_aim_sourcedata_bitcoin.csv', index = False)
        
    if str(df[-1:].Date.values[0])[:10] == currentdate:
        print('> The last line == currentdate\n')
    else:
        print('>>> The last line != currentdate\n')
#         alarm()
        
    ##################################### source data save

    # range selection - optimized since [2014-09-17]

    highest = 0

    date = []
    ml = []
    for i in range(len(df.Date.values)-100):
        dfc = df[df['Date'] > df.Date.values[i]]
        result = rsquare(dfc)[0]

        date.append(df.Date.values[i])
        ml.append(result)    

        if highest < result:
            highest = result
        else:
            pass

    dff = pd.DataFrame({'Date': date, 'Result': ml})    
    plt.figure(figsize = (16,9))
    plt.errorbar(dff.Date, dff.Result)
    plt.title('Bitcoin R-squared')

    print('__________________________________\n')
    print('The highest R-squared: \n\n', dff[dff['Result']==dff['Result'].max()])
    print('__________________________________')

    # rday: most optimized day cut
    # numpy.datetime64 to string
    ts = pd.to_datetime(str(dff[dff['Result']==dff['Result'].max()].Date.values[0])) 
    rday_bit = ts.strftime('%Y-%m-%d')
    with open("rday_bit.txt", 'w') as f:
        f.write(str(rday_bit)) 

    # Hypothese: 
    # R-squared represents market uncertainties 
    # bitcoin having its highest R-squared represents the genuine market price
    # The null hypothesis is that bitcoin follows the best-optimized market, which is most genuine because we do long-term investing
    # originated from bitcoin stocks which follow long-term price trend. 
    # therefore, we apply the highest rsquared day; Rday. 

    dfr = df[df['Date'] > rday_bit]
    result_optimized = rsquare(dfr)

    # linear regression
    prediction = pred_lstm(dfr)    
    current = dfr.Stock_price.tail(1).values
    current_to_predict = current/prediction

    print('\n• Current Bitcoin:    ', '%.5f' %current)
    print('\n• Predicted Bitcoin:  ', '%.5f' %prediction)
    print('\n• Current to Predicted: ', '%.5f' %current_to_predict)
   
    if current < prediction:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'UP' + '\033[0m')
    else:
        print(f"\n• Linear Regression Result: ", '\033[1m' + 'DOWN' + '\033[0m')
    print('__________________________________\n')

    # classifier
    prediction = classifer_data_input(dfr, moving_avg_value) # [0]: precision / [1]: prediction result

    if prediction[1] == 1:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'UP' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    else:
        print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'DOWN' + '\033[0m')
        print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
    print('__________________________________\n\n')

    # print out stats summary for multi-linear regression
    import statsmodels.api as sm
    dfr['intercept'] = 1
    lm = sm.OLS(dfr['Stock_price'], dfr[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Oil', 'Dollar', 'Gold', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
    results = lm.fit()
    print(results.summary())
    
    
if __name__ == '__main__':
    minutepassed = timechecknow()
    if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -20:
        main_bit()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Prediction - Inflation

In [18]:
# import warnings
# warnings.filterwarnings("ignore")
# import yfinance as yf
# import numpy as np
# import pandas as pd
# import seaborn as sb
# import urllib.request, urllib.error, urllib.parse
# import matplotlib.pyplot as plt
# %matplotlib inline

# # find the highest rsquared date
# def rsquare(dfc):
    
#     # Regression analysis
#     import statsmodels.api as sm
#     dfc['intercept'] = 1
#     lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'snp', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']])
#     results = lm.fit()
#     return results.rsquared, results.pvalues

# def machinelearning(dfr):
#     # Machine learning
#     from sklearn.linear_model import LinearRegression
#     from sklearn.linear_model import Lasso
#     from sklearn.preprocessing import PolynomialFeatures
#     from sklearn.preprocessing import StandardScaler

#     x = dfr.drop(columns = ['Date', 'Stock_price', 'intercept'])
#     x = x.to_numpy()
#     y = dfr.Stock_price.to_numpy()
    
#     scaler = StandardScaler()   ####
#     x = scaler.fit_transform(x) ####

#     poly_feat = PolynomialFeatures(degree = 5)
#     x = poly_feat.fit_transform(x)
# #     poly_model = LinearRegression(fit_intercept = False).fit(x, y)
    
#     sample_house = [x[x.shape[0]-1]]
#     model = LinearRegression()  
#     model.fit(x, y) 
    
#     prediction = model.predict(sample_house)
#     return prediction

# def main_inf():
    
#     # find the highest Rsquared of S&P 500 
#     ticker = '^GSPC'
#     tkr = yfinance_df(ticker)   
#     sfive = tkr

#     # 10 year treasery bond yield
#     rate = yfinance_df('^TNX')   
#     irate = rate[['Date', 'Adj Close']]

#     # CBOE Volatility Index
#     cboe = yfinance_df('^VIX')   
#     cboe = cboe[['Date', 'Adj Close']]

#     # Gold Feb 22 (GC=F)
#     gold = yfinance_df('GC=F')   
#     gold = gold[['Date', 'Adj Close']]

#     # Crude Oil
#     mar = yfinance_df('CL=F')   
#     mar = mar[['Date', 'Adj Close']]

#     # Dollar Index
#     dollar = yfinance_df('DX-Y.NYB')
#     dollar = dollar[['Date', 'Adj Close']]

#     # Bitcoin (*Last)
#     bit = yfinance_df('BTC-USD')   
#     bit = bit[['Date', 'Adj Close']]

#     # cpi index
#     url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     cpi = webContent[4075:]
#     cpi = cpi.split()
#     date = []
#     number = []
#     for i in range(len(cpi)):
#         if i%2 == 0:
#             date.append(cpi[i])
#         else:
#             number.append(cpi[i])
#     cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
#     cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
#     cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

#     # jobless claim index
#     url = 'https://fred.stlouisfed.org/data/ICSA.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     joblessclaim = webContent[710:]
#     joblessclaim = joblessclaim.split()
#     date = []
#     number = []
#     for i in range(len(joblessclaim)):
#         if i%2 == 0:
#             date.append(joblessclaim[i])
#         else:
#             number.append(joblessclaim[i])
#     jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
#     # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
#     jls.Jclaim.replace('.', np.NaN, inplace = True)
#     jls.fillna(method = 'ffill', inplace = True)
#     jls = jls[jls['Date'] <= currentdate]
    
#     jls['Date'] = jls['Date'].astype('datetime64[ns]')
#     jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
#     jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
#     cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

#     # inflation index
#     url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     inf = webContent[1222:]
#     inf = inf.split()
#     date = []
#     number = []
#     for i in range(len(inf)):
#         if i%2 == 0:
#             date.append(inf[i])
#         else:
#             number.append(inf[i])
#     inf = pd.DataFrame({'Date': date, 'Inf': number})
#     drp = inf[inf.Inf == '.'].index.values
#     inf = inf.drop(index = drp).reset_index(drop=True)
#     inf['Date'] = inf['Date'].astype('datetime64[ns]')
#     inf['Inf'] = pd.to_numeric(inf['Inf'])

#     # merge yfinance tickers & cleaning
#     df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.merge(sfive, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
#     df = df.fillna(0)
#     df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

#     # cleaning - cpi and Jclaim
#     for i in range(len(df.Cpi)):
#         if df.Cpi[i] == 0.0:
#             df.Cpi[i] = df.Cpi[i-1]    
#     df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
#     for i in range(len(df.Jclaim)):
#         if df.Jclaim[i] == 0.0:
#             df.Jclaim[i] = df.Jclaim[i-1]
#     drp = df[df['Int_rate'] == 0].index.values
#     df = df.drop(index=drp)

#     # cleaning - inflation rate since 1990
#     df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df[df['Date'] > '2003-01-01']
#     df = df.fillna(0)
#     if df.tail(1).Inf.values[0] == 0:
#         df.tail(1).Inf = df.tail(2).Inf.values[0]


# #     ## added but low P-value, also distort Int-rate    

# #     # 30-Year Fixed Rate Mortgage Average & cleaning
# #     url = 'https://fred.stlouisfed.org/data/MORTGAGE30US.txt'
# #     response = urllib.request.urlopen(url)
# #     webContent = response.read().decode('UTF-8')
# #     mortg = webContent[32820:]
# #     mortg = mortg.split()
# #     date = []
# #     number = []
# #     for i in range(len(mortg)):
# #         if i%2 == 0:
# #             date.append(mortg[i])
# #         else:
# #             number.append(mortg[i])
# #     mortg = pd.DataFrame({'Date': date, 'Mortg': number})
# #     mortg['Date'] = mortg['Date'].astype('datetime64[ns]')
# #     mortg['Mortg'] = pd.to_numeric(mortg['Mortg'])

# #     df = df.merge(mortg, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
# #     df = df[df['Date'] >= '2003-01-03']
# #     df = df.fillna(0).reset_index(drop=True)
# #     for i in range(len(df.Mortg)):
# #         if df.Mortg[i] == 0.0:
# #             df.Mortg[i] = df.Mortg[i-1]
# #     drp = df[df['Mortg'] == 0].index.values
# #     df = df.drop(index=drp)
    
    
# #     # University of Michigan: Consumer Sentiment 
# #     url = 'https://fred.stlouisfed.org/data/Umcsent.txt'
# #     response = urllib.request.urlopen(url)
# #     webContent = response.read().decode('UTF-8')
# #     Umcsent = webContent[6897:]
# #     Umcsent = Umcsent.split()
# #     date = []
# #     number = []
# #     for i in range(len(Umcsent)):
# #         if i%2 == 0:
# #             date.append(Umcsent[i])
# #         else:
# #             number.append(Umcsent[i])
# #     Umcsent = pd.DataFrame({'Date': date, 'Umcsent': number})
# #     Umcsent['Date'] = Umcsent['Date'].astype('datetime64[ns]')
# #     Umcsent['Umcsent'] = pd.to_numeric(Umcsent['Umcsent'])
    
# #     # cleaning
# #     df = df.merge(Umcsent, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
# #     df = df.fillna(0)
# #     if df.tail(1).Umcsent.values[0] == 0:
# #         df.tail(1).Umcsent = df.tail(2).Umcsent.values[0]

#     # merge - cboe     
#     df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
#     df = df.fillna(0)

#     # merge - gold
#     df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
#     df = df.fillna(0)

#     # merge - Crude Oil Mar 22 
#     df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

#     # merge - Dollar Index 
#     df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

#     # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
#     df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)

#     # dropna and save the source date
#     df = df.dropna()

#     # fillavg = df[df['Inf'] == 0].index.values
#     # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

#     dfdrop = df[df['Stock_price'] == 0].index.values
#     df = df.drop(index = dfdrop)   

#     # dfdrop = df[df['Inf'] == 0].index.values
#     # df = df.drop(index = dfdrop)

#     fillavg = df[df['Inf'] == 0].index.values
#     df['Inf'].loc[fillavg] = df['Inf'].tail(5).max() 


#     dfdrop = df[df['Gold'] == 0].index.values
#     df = df.drop(index=dfdrop)

#     dfdrop = df[df['Int_rate'] == 0].index.values
#     df = df.drop(index=dfdrop)

#     # Fillna Cboe with the highest value for the past 20 working days. 
#     cboe_fillna = df[df['Cboe'] == 0].index.values
#     df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 

#     # Real GDP
#     url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     Rgdp = webContent[1005:]
#     Rgdp = Rgdp.split()
#     date = []
#     number = []
#     for i in range(len(Rgdp)):
#         if i%2 == 0:
#             date.append(Rgdp[i])
#         else:
#             number.append(Rgdp[i])
#     Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
#     Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
#     Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

#     df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
#     df.Rgdp.replace(np.NaN, 0, inplace = True)
#     for i in range(len(df.Rgdp)):
#         if df.Rgdp[i] == 0:
#             df.Rgdp[i] = df.Rgdp[i-1]
#     df.dropna(inplace = True)
#     df = df.drop(columns = ['Close'])
    
#     ################################################################################## add from below
    
#     # outcome change ##############################################
#     df.rename(columns = {'Stock_price':'change'}, inplace = True)
#     df.rename(columns = {'Inf':'Stock_price'}, inplace = True)
#     df.rename(columns = {'change':'snp'}, inplace = True)  
#     ###############################################################

#     if df.all().sum() != len(df.columns):
#         df.to_csv('errorlog_prediction_inf.csv', index = False)
#         print('\nZero value checked: ', ticker, '\n')
#         print(df.all())
#         alarm() # if any zero value, alarm
#     else:
#         df.to_csv('predict_aim_sourcedata_inf.csv', index = False)
        
#     if str(df[-1:].Date.values[0])[:10] == currentdate:
#         print('> The last line == currentdate\n')
#     else:
#         print('>>> The last line != currentdate\n')
# #         alarm()
        
#     ##################################### source data save

#     # range selection - optimized since [2014-09-17]

#     highest = 0

#     date = []
#     ml = []
#     for i in range(len(df.Date.values)-100):
#         dfc = df[df['Date'] > df.Date.values[i]]
#         result = rsquare(dfc)[0]

#         date.append(df.Date.values[i])
#         ml.append(result)    

#         if highest < result:
#             highest = result
#         else:
#             pass

#     dff = pd.DataFrame({'Date': date, 'Result': ml})    
#     plt.figure(figsize = (16,9))
#     plt.errorbar(dff.Date, dff.Result)
#     plt.title('Inf R-squared')

#     print('__________________________________\n')
#     print('The highest R-squared: \n\n', dff[dff['Result']==dff['Result'].max()])
#     print('__________________________________')

#     # rday: most optimized day cut
#     # numpy.datetime64 to string
#     ts = pd.to_datetime(str(dff[dff['Result']==dff['Result'].max()].Date.values[0])) 
#     rday_snp = ts.strftime('%Y-%m-%d')
# #     with open("rday_snp.txt", 'w') as f:
# #         f.write(str(rday_snp)) 

#     # Hypothese: 
#     # R-squared represents market uncertainties 
#     # S&P 500 having its highest R-squared represents the genuine market price
#     # The null hypothesis is that S&P 500 follows the best-optimized market, which is most genuine because we do long-term investing
#     # originated from S&P 500 stocks which follow long-term price trend. 
#     # therefore, we apply the highest rsquared day; Rday. 

#     dfr = df[df['Date'] > rday_snp]
#     result_optimized = rsquare(dfr)

#     # linear regression
#     prediction = pred_lstm(dfr)    
#     current = dfr.Stock_price.tail(1).values
#     current_to_predict = current/prediction

#     print('\n• Current Inflation:    ', '%.5f' %current)
#     print('\n• Predicted Inflation:  ', '%.5f' %prediction)
#     print('\n• Current to Predicted: ', '%.5f' %current_to_predict)
   
#     if current < prediction:
#         print(f"\n• Linear Regression Result: ", '\033[1m' + 'UP' + '\033[0m')
#     else:
#         print(f"\n• Linear Regression Result: ", '\033[1m' + 'DOWN' + '\033[0m')
#     print('__________________________________\n')

#     # classifier
#     prediction = classifer_data_input(dfr, moving_avg_value) # [0]: precision / [1]: prediction result

#     if prediction[1] == 1:
#         print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'UP' + '\033[0m')
#         print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
#     else:
#         print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'DOWN' + '\033[0m')
#         print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
#     print('__________________________________\n\n')

#     # print out stats summary for multi-linear regression
#     import statsmodels.api as sm
#     dfr['intercept'] = 1
#     lm = sm.OLS(dfr['Stock_price'], dfr[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'snp', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
#     results = lm.fit()
#     print(results.summary())
    
    
# if __name__ == '__main__':
#     minutepassed = timechecknow()
#     if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -30:
#         main_inf()
#     else:
#         print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Prediction - Interest

In [19]:
# import warnings
# warnings.filterwarnings("ignore")
# import yfinance as yf
# import numpy as np
# import pandas as pd
# import seaborn as sb
# import urllib.request, urllib.error, urllib.parse
# import matplotlib.pyplot as plt
# %matplotlib inline

# # find the highest rsquared date
# def rsquare(dfc):
    
#     # Regression analysis
#     import statsmodels.api as sm
#     dfc['intercept'] = 1
#     lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Inf', 'snp', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']])
#     results = lm.fit()
#     return results.rsquared, results.pvalues

# def machinelearning(dfr):
#     # Machine learning
#     from sklearn.linear_model import LinearRegression
#     from sklearn.linear_model import Lasso
#     from sklearn.preprocessing import PolynomialFeatures
#     from sklearn.preprocessing import StandardScaler

#     x = dfr.drop(columns = ['Date', 'Stock_price', 'intercept'])
#     x = x.to_numpy()
#     y = dfr.Stock_price.to_numpy()
    
#     scaler = StandardScaler()   ####
#     x = scaler.fit_transform(x) ####

#     poly_feat = PolynomialFeatures(degree = 5)
#     x = poly_feat.fit_transform(x)
# #     poly_model = LinearRegression(fit_intercept = False).fit(x, y)
    
#     sample_house = [x[x.shape[0]-1]]
#     model = LinearRegression()  
#     model.fit(x, y) 
    
#     prediction = model.predict(sample_house)
#     return prediction

# def main_int():
    
#     # find the highest Rsquared of S&P 500 
#     ticker = '^GSPC'
#     tkr = yfinance_df(ticker)   
#     sfive = tkr

#     # 10 year treasery bond yield
#     rate = yfinance_df('^TNX')   
#     irate = rate[['Date', 'Adj Close']]

#     # CBOE Volatility Index
#     cboe = yfinance_df('^VIX')   
#     cboe = cboe[['Date', 'Adj Close']]

#     # Gold Feb 22 (GC=F)
#     gold = yfinance_df('GC=F')   
#     gold = gold[['Date', 'Adj Close']]

#     # Crude Oil
#     mar = yfinance_df('CL=F')   
#     mar = mar[['Date', 'Adj Close']]

#     # Dollar Index
#     dollar = yfinance_df('DX-Y.NYB')
#     dollar = dollar[['Date', 'Adj Close']]

#     # Bitcoin (*Last)
#     bit = yfinance_df('BTC-USD')   
#     bit = bit[['Date', 'Adj Close']]

#     # cpi index
#     url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     cpi = webContent[4075:]
#     cpi = cpi.split()
#     date = []
#     number = []
#     for i in range(len(cpi)):
#         if i%2 == 0:
#             date.append(cpi[i])
#         else:
#             number.append(cpi[i])
#     cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
#     cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
#     cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

#     # jobless claim index
#     url = 'https://fred.stlouisfed.org/data/ICSA.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     joblessclaim = webContent[710:]
#     joblessclaim = joblessclaim.split()
#     date = []
#     number = []
#     for i in range(len(joblessclaim)):
#         if i%2 == 0:
#             date.append(joblessclaim[i])
#         else:
#             number.append(joblessclaim[i])
#     jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
#     # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
#     jls.Jclaim.replace('.', np.NaN, inplace = True)
#     jls.fillna(method = 'ffill', inplace = True)
#     jls = jls[jls['Date'] <= currentdate]
    
#     jls['Date'] = jls['Date'].astype('datetime64[ns]')
#     jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
#     jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
#     cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

#     # inflation index
#     url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     inf = webContent[1222:]
#     inf = inf.split()
#     date = []
#     number = []
#     for i in range(len(inf)):
#         if i%2 == 0:
#             date.append(inf[i])
#         else:
#             number.append(inf[i])
#     inf = pd.DataFrame({'Date': date, 'Inf': number})
#     drp = inf[inf.Inf == '.'].index.values
#     inf = inf.drop(index = drp).reset_index(drop=True)
#     inf['Date'] = inf['Date'].astype('datetime64[ns]')
#     inf['Inf'] = pd.to_numeric(inf['Inf'])

#     # merge yfinance tickers & cleaning
#     df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.merge(sfive, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
#     df = df.fillna(0)
#     df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

#     # cleaning - cpi and Jclaim
#     for i in range(len(df.Cpi)):
#         if df.Cpi[i] == 0.0:
#             df.Cpi[i] = df.Cpi[i-1]    
#     df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
#     for i in range(len(df.Jclaim)):
#         if df.Jclaim[i] == 0.0:
#             df.Jclaim[i] = df.Jclaim[i-1]
#     drp = df[df['Int_rate'] == 0].index.values
#     df = df.drop(index=drp)

#     # cleaning - inflation rate since 1990
#     df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df[df['Date'] > '2003-01-01']
#     df = df.fillna(0)
#     if df.tail(1).Inf.values[0] == 0:
#         df.tail(1).Inf = df.tail(2).Inf.values[0]


# #     ## added but low P-value, also distort Int-rate    

# #     # 30-Year Fixed Rate Mortgage Average & cleaning
# #     url = 'https://fred.stlouisfed.org/data/MORTGAGE30US.txt'
# #     response = urllib.request.urlopen(url)
# #     webContent = response.read().decode('UTF-8')
# #     mortg = webContent[32820:]
# #     mortg = mortg.split()
# #     date = []
# #     number = []
# #     for i in range(len(mortg)):
# #         if i%2 == 0:
# #             date.append(mortg[i])
# #         else:
# #             number.append(mortg[i])
# #     mortg = pd.DataFrame({'Date': date, 'Mortg': number})
# #     mortg['Date'] = mortg['Date'].astype('datetime64[ns]')
# #     mortg['Mortg'] = pd.to_numeric(mortg['Mortg'])

# #     df = df.merge(mortg, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
# #     df = df[df['Date'] >= '2003-01-03']
# #     df = df.fillna(0).reset_index(drop=True)
# #     for i in range(len(df.Mortg)):
# #         if df.Mortg[i] == 0.0:
# #             df.Mortg[i] = df.Mortg[i-1]
# #     drp = df[df['Mortg'] == 0].index.values
# #     df = df.drop(index=drp)
    
    
# #     # University of Michigan: Consumer Sentiment 
# #     url = 'https://fred.stlouisfed.org/data/Umcsent.txt'
# #     response = urllib.request.urlopen(url)
# #     webContent = response.read().decode('UTF-8')
# #     Umcsent = webContent[6897:]
# #     Umcsent = Umcsent.split()
# #     date = []
# #     number = []
# #     for i in range(len(Umcsent)):
# #         if i%2 == 0:
# #             date.append(Umcsent[i])
# #         else:
# #             number.append(Umcsent[i])
# #     Umcsent = pd.DataFrame({'Date': date, 'Umcsent': number})
# #     Umcsent['Date'] = Umcsent['Date'].astype('datetime64[ns]')
# #     Umcsent['Umcsent'] = pd.to_numeric(Umcsent['Umcsent'])
    
# #     # cleaning
# #     df = df.merge(Umcsent, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
# #     df = df.fillna(0)
# #     if df.tail(1).Umcsent.values[0] == 0:
# #         df.tail(1).Umcsent = df.tail(2).Umcsent.values[0]

#     # merge - cboe     
#     df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
#     df = df.fillna(0)

#     # merge - gold
#     df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
#     df = df.fillna(0)

#     # merge - Crude Oil Mar 22 
#     df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

#     # merge - Dollar Index 
#     df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

#     # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
#     df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)

#     # dropna and save the source date
#     df = df.dropna()

#     # fillavg = df[df['Inf'] == 0].index.values
#     # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

#     dfdrop = df[df['Stock_price'] == 0].index.values
#     df = df.drop(index = dfdrop)   

#     # dfdrop = df[df['Inf'] == 0].index.values
#     # df = df.drop(index = dfdrop)

#     fillavg = df[df['Inf'] == 0].index.values
#     df['Inf'].loc[fillavg] = df['Inf'].tail(5).max() 


#     dfdrop = df[df['Gold'] == 0].index.values
#     df = df.drop(index=dfdrop)

#     dfdrop = df[df['Int_rate'] == 0].index.values
#     df = df.drop(index=dfdrop)

#     # Fillna Cboe with the highest value for the past 20 working days. 
#     cboe_fillna = df[df['Cboe'] == 0].index.values
#     df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 

#     # Real GDP
#     url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     Rgdp = webContent[1005:]
#     Rgdp = Rgdp.split()
#     date = []
#     number = []
#     for i in range(len(Rgdp)):
#         if i%2 == 0:
#             date.append(Rgdp[i])
#         else:
#             number.append(Rgdp[i])
#     Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
#     Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
#     Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

#     df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
#     df.Rgdp.replace(np.NaN, 0, inplace = True)
#     for i in range(len(df.Rgdp)):
#         if df.Rgdp[i] == 0:
#             df.Rgdp[i] = df.Rgdp[i-1]
#     df.dropna(inplace = True)
#     df = df.drop(columns = ['Close'])
    
#     ################################################################################## add from below
    
#     # outcome change ##############################################
#     df.rename(columns = {'Stock_price':'change'}, inplace = True)
#     df.rename(columns = {'Int_rate':'Stock_price'}, inplace = True)
#     df.rename(columns = {'change':'snp'}, inplace = True)  
#     ###############################################################

#     if df.all().sum() != len(df.columns):
#         df.to_csv('errorlog_prediction_int.csv', index = False)
#         print('\nZero value checked: ', ticker, '\n')
#         print(df.all())
#         alarm() # if any zero value, alarm
#     else:
#         df.to_csv('predict_aim_sourcedata_int.csv', index = False)
        
#     if str(df[-1:].Date.values[0])[:10] == currentdate:
#         print('> The last line == currentdate\n')
#     else:
#         print('>>> The last line != currentdate\n')
# #         alarm()
        
#     ##################################### source data save

#     # range selection - optimized since [2014-09-17]

#     highest = 0

#     date = []
#     ml = []
#     for i in range(len(df.Date.values)-100):
#         dfc = df[df['Date'] > df.Date.values[i]]
#         result = rsquare(dfc)[0]

#         date.append(df.Date.values[i])
#         ml.append(result)    

#         if highest < result:
#             highest = result
#         else:
#             pass

#     dff = pd.DataFrame({'Date': date, 'Result': ml})    
#     plt.figure(figsize = (16,9))
#     plt.errorbar(dff.Date, dff.Result)
#     plt.title('Int R-squared')

#     print('__________________________________\n')
#     print('The highest R-squared: \n\n', dff[dff['Result']==dff['Result'].max()])
#     print('__________________________________')

#     # rday: most optimized day cut
#     # numpy.datetime64 to string
#     ts = pd.to_datetime(str(dff[dff['Result']==dff['Result'].max()].Date.values[0])) 
#     rday_snp = ts.strftime('%Y-%m-%d')
# #     with open("rday_snp.txt", 'w') as f:
# #         f.write(str(rday_snp)) 

#     # Hypothese: 
#     # R-squared represents market uncertainties 
#     # S&P 500 having its highest R-squared represents the genuine market price
#     # The null hypothesis is that S&P 500 follows the best-optimized market, which is most genuine because we do long-term investing
#     # originated from S&P 500 stocks which follow long-term price trend. 
#     # therefore, we apply the highest rsquared day; Rday. 

#     dfr = df[df['Date'] > rday_snp]
#     result_optimized = rsquare(dfr)

#     # linear regression
#     prediction = pred_lstm(dfr)    
#     current = dfr.Stock_price.tail(1).values
#     current_to_predict = current/prediction

#     print('\n• Current Int. Rate:    ', '%.5f' %current)
#     print('\n• Predicted Int. Rate:  ', '%.5f' %prediction)
#     print('\n• Current to Predicted: ', '%.5f' %current_to_predict)
   
#     if current < prediction:
#         print(f"\n• Linear Regression Result: ", '\033[1m' + 'UP' + '\033[0m')
#     else:
#         print(f"\n• Linear Regression Result: ", '\033[1m' + 'DOWN' + '\033[0m')
#     print('__________________________________\n')

#     # classifier
#     prediction = classifer_data_input(dfr, moving_avg_value) # [0]: precision / [1]: prediction result

#     if prediction[1] == 1:
#         print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'UP' + '\033[0m')
#         print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
#     else:
#         print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'DOWN' + '\033[0m')
#         print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
#     print('__________________________________\n\n')

#     # print out stats summary for multi-linear regression
#     import statsmodels.api as sm
#     dfr['intercept'] = 1
#     lm = sm.OLS(dfr['Stock_price'], dfr[['intercept', 'Cpi', 'Jclaim', 'Inf', 'snp', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
#     results = lm.fit()
#     print(results.summary())
    
    
# if __name__ == '__main__':
#     minutepassed = timechecknow()
#     if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -30:
#         main_int()
#     else:
#         print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Prediction - Dollar

In [20]:
# import warnings
# warnings.filterwarnings("ignore")
# import yfinance as yf
# import numpy as np
# import pandas as pd
# import seaborn as sb
# import urllib.request, urllib.error, urllib.parse
# import matplotlib.pyplot as plt
# %matplotlib inline

# # find the highest rsquared date
# def rsquare(dfc):
    
#     # Regression analysis
#     import statsmodels.api as sm
#     dfc['intercept'] = 1
#     lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Inf', 'snp', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Int_rate', 'Rgdp']])
#     results = lm.fit()
#     return results.rsquared, results.pvalues

# def machinelearning(dfr):
#     # Machine learning
#     from sklearn.linear_model import LinearRegression
#     from sklearn.linear_model import Lasso
#     from sklearn.preprocessing import PolynomialFeatures
#     from sklearn.preprocessing import StandardScaler

#     x = dfr.drop(columns = ['Date', 'Stock_price', 'intercept'])
#     x = x.to_numpy()
#     y = dfr.Stock_price.to_numpy()
    
#     scaler = StandardScaler()   ####
#     x = scaler.fit_transform(x) ####

#     poly_feat = PolynomialFeatures(degree = 5)
#     x = poly_feat.fit_transform(x)
# #     poly_model = LinearRegression(fit_intercept = False).fit(x, y)
    
#     sample_house = [x[x.shape[0]-1]]
#     model = LinearRegression()  
#     model.fit(x, y) 
    
#     prediction = model.predict(sample_house)
#     return prediction

# def main_dollar():
    
#     # find the highest Rsquared of S&P 500 
#     ticker = '^GSPC'
#     tkr = yfinance_df(ticker)   
#     sfive = tkr

#     # 10 year treasery bond yield
#     rate = yfinance_df('^TNX')   
#     irate = rate[['Date', 'Adj Close']]

#     # CBOE Volatility Index
#     cboe = yfinance_df('^VIX')   
#     cboe = cboe[['Date', 'Adj Close']]

#     # Gold Feb 22 (GC=F)
#     gold = yfinance_df('GC=F')   
#     gold = gold[['Date', 'Adj Close']]

#     # Crude Oil
#     mar = yfinance_df('CL=F')   
#     mar = mar[['Date', 'Adj Close']]

#     # Dollar Index
#     dollar = yfinance_df('DX-Y.NYB')
#     dollar = dollar[['Date', 'Adj Close']]

#     # Bitcoin (*Last)
#     bit = yfinance_df('BTC-USD')   
#     bit = bit[['Date', 'Adj Close']]

#     # cpi index
#     url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     cpi = webContent[4075:]
#     cpi = cpi.split()
#     date = []
#     number = []
#     for i in range(len(cpi)):
#         if i%2 == 0:
#             date.append(cpi[i])
#         else:
#             number.append(cpi[i])
#     cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
#     cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
#     cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

#     # jobless claim index
#     url = 'https://fred.stlouisfed.org/data/ICSA.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     joblessclaim = webContent[710:]
#     joblessclaim = joblessclaim.split()
#     date = []
#     number = []
#     for i in range(len(joblessclaim)):
#         if i%2 == 0:
#             date.append(joblessclaim[i])
#         else:
#             number.append(joblessclaim[i])
#     jls = pd.DataFrame({'Date': date, 'Jclaim': number})
    
#     # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
#     jls.Jclaim.replace('.', np.NaN, inplace = True)
#     jls.fillna(method = 'ffill', inplace = True)
#     jls = jls[jls['Date'] <= currentdate]
    
#     jls['Date'] = jls['Date'].astype('datetime64[ns]')
#     jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
#     jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
#     cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

#     # inflation index
#     url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     inf = webContent[1222:]
#     inf = inf.split()
#     date = []
#     number = []
#     for i in range(len(inf)):
#         if i%2 == 0:
#             date.append(inf[i])
#         else:
#             number.append(inf[i])
#     inf = pd.DataFrame({'Date': date, 'Inf': number})
#     drp = inf[inf.Inf == '.'].index.values
#     inf = inf.drop(index = drp).reset_index(drop=True)
#     inf['Date'] = inf['Date'].astype('datetime64[ns]')
#     inf['Inf'] = pd.to_numeric(inf['Inf'])

#     # merge yfinance tickers & cleaning
#     df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.merge(sfive, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
#     df = df.fillna(0)
#     df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

#     # cleaning - cpi and Jclaim
#     for i in range(len(df.Cpi)):
#         if df.Cpi[i] == 0.0:
#             df.Cpi[i] = df.Cpi[i-1]    
#     df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
#     for i in range(len(df.Jclaim)):
#         if df.Jclaim[i] == 0.0:
#             df.Jclaim[i] = df.Jclaim[i-1]
#     drp = df[df['Int_rate'] == 0].index.values
#     df = df.drop(index=drp)

#     # cleaning - inflation rate since 1990
#     df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df[df['Date'] > '2003-01-01']
#     df = df.fillna(0)
#     if df.tail(1).Inf.values[0] == 0:
#         df.tail(1).Inf = df.tail(2).Inf.values[0]


# #     ## added but low P-value, also distort Int-rate    

# #     # 30-Year Fixed Rate Mortgage Average & cleaning
# #     url = 'https://fred.stlouisfed.org/data/MORTGAGE30US.txt'
# #     response = urllib.request.urlopen(url)
# #     webContent = response.read().decode('UTF-8')
# #     mortg = webContent[32820:]
# #     mortg = mortg.split()
# #     date = []
# #     number = []
# #     for i in range(len(mortg)):
# #         if i%2 == 0:
# #             date.append(mortg[i])
# #         else:
# #             number.append(mortg[i])
# #     mortg = pd.DataFrame({'Date': date, 'Mortg': number})
# #     mortg['Date'] = mortg['Date'].astype('datetime64[ns]')
# #     mortg['Mortg'] = pd.to_numeric(mortg['Mortg'])

# #     df = df.merge(mortg, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
# #     df = df[df['Date'] >= '2003-01-03']
# #     df = df.fillna(0).reset_index(drop=True)
# #     for i in range(len(df.Mortg)):
# #         if df.Mortg[i] == 0.0:
# #             df.Mortg[i] = df.Mortg[i-1]
# #     drp = df[df['Mortg'] == 0].index.values
# #     df = df.drop(index=drp)
    
    
# #     # University of Michigan: Consumer Sentiment 
# #     url = 'https://fred.stlouisfed.org/data/Umcsent.txt'
# #     response = urllib.request.urlopen(url)
# #     webContent = response.read().decode('UTF-8')
# #     Umcsent = webContent[6897:]
# #     Umcsent = Umcsent.split()
# #     date = []
# #     number = []
# #     for i in range(len(Umcsent)):
# #         if i%2 == 0:
# #             date.append(Umcsent[i])
# #         else:
# #             number.append(Umcsent[i])
# #     Umcsent = pd.DataFrame({'Date': date, 'Umcsent': number})
# #     Umcsent['Date'] = Umcsent['Date'].astype('datetime64[ns]')
# #     Umcsent['Umcsent'] = pd.to_numeric(Umcsent['Umcsent'])
    
# #     # cleaning
# #     df = df.merge(Umcsent, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
# #     df = df.fillna(0)
# #     if df.tail(1).Umcsent.values[0] == 0:
# #         df.tail(1).Umcsent = df.tail(2).Umcsent.values[0]

#     # merge - cboe     
#     df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
#     df = df.fillna(0)

#     # merge - gold
#     df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
#     df = df.fillna(0)

#     # merge - Crude Oil Mar 22 
#     df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

#     # merge - Dollar Index 
#     df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

#     # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
#     df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
#     df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)

#     # dropna and save the source date
#     df = df.dropna()

#     # fillavg = df[df['Inf'] == 0].index.values
#     # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

#     dfdrop = df[df['Stock_price'] == 0].index.values
#     df = df.drop(index = dfdrop)   

#     # dfdrop = df[df['Inf'] == 0].index.values
#     # df = df.drop(index = dfdrop)

#     fillavg = df[df['Inf'] == 0].index.values
#     df['Inf'].loc[fillavg] = df['Inf'].tail(5).max() 


#     dfdrop = df[df['Gold'] == 0].index.values
#     df = df.drop(index=dfdrop)

#     dfdrop = df[df['Int_rate'] == 0].index.values
#     df = df.drop(index=dfdrop)

#     # Fillna Cboe with the highest value for the past 20 working days. 
#     cboe_fillna = df[df['Cboe'] == 0].index.values
#     df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 

#     # Real GDP
#     url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
#     response = urllib.request.urlopen(url)
#     webContent = response.read().decode('UTF-8')
#     Rgdp = webContent[1005:]
#     Rgdp = Rgdp.split()
#     date = []
#     number = []
#     for i in range(len(Rgdp)):
#         if i%2 == 0:
#             date.append(Rgdp[i])
#         else:
#             number.append(Rgdp[i])
#     Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
#     Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
#     Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

#     df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
#     df.Rgdp.replace(np.NaN, 0, inplace = True)
#     for i in range(len(df.Rgdp)):
#         if df.Rgdp[i] == 0:
#             df.Rgdp[i] = df.Rgdp[i-1]
#     df.dropna(inplace = True)
#     df = df.drop(columns = ['Close'])
    
#     ################################################################################## add from below
    
#     # outcome change ###################################################################
#     df.rename(columns = {'Stock_price':'change'}, inplace = True)
#     df.rename(columns = {'Dollar':'Stock_price'}, inplace = True) #<- Change index here
#     df.rename(columns = {'change':'snp'}, inplace = True)  
#     ####################################################################################

#     if df.all().sum() != len(df.columns):
#         df.to_csv('errorlog_prediction_dollar.csv', index = False)
#         print('\nZero value checked: ', ticker, '\n')
#         print(df.all())
#         alarm() # if any zero value, alarm
#     else:
#         df.to_csv('predict_aim_sourcedata_dollar.csv', index = False)
        
#     if str(df[-1:].Date.values[0])[:10] == currentdate:
#         print('> The last line == currentdate\n')
#     else:
#         print('>>> The last line != currentdate\n')
# #         alarm()
        
#     ##################################### source data save

#     # range selection - optimized since [2014-09-17]

#     highest = 0

#     date = []
#     ml = []
#     for i in range(len(df.Date.values)-100):
#         dfc = df[df['Date'] > df.Date.values[i]]
#         result = rsquare(dfc)[0]

#         date.append(df.Date.values[i])
#         ml.append(result)    

#         if highest < result:
#             highest = result
#         else:
#             pass

#     dff = pd.DataFrame({'Date': date, 'Result': ml})    
#     plt.figure(figsize = (16,9))
#     plt.errorbar(dff.Date, dff.Result)
#     plt.title('Dollar R-squared')

#     print('__________________________________\n')
#     print('The highest R-squared: \n\n', dff[dff['Result']==dff['Result'].max()])
#     print('__________________________________')

#     # rday: most optimized day cut
#     # numpy.datetime64 to string
#     ts = pd.to_datetime(str(dff[dff['Result']==dff['Result'].max()].Date.values[0])) 
#     rday_snp = ts.strftime('%Y-%m-%d')
# #     with open("rday_snp.txt", 'w') as f:
# #         f.write(str(rday_snp)) 

#     # Hypothese: 
#     # R-squared represents market uncertainties 
#     # S&P 500 having its highest R-squared represents the genuine market price
#     # The null hypothesis is that S&P 500 follows the best-optimized market, which is most genuine because we do long-term investing
#     # originated from S&P 500 stocks which follow long-term price trend. 
#     # therefore, we apply the highest rsquared day; Rday. 

#     dfr = df[df['Date'] > rday_snp]
#     result_optimized = rsquare(dfr)

#     # linear regression
#     prediction = pred_lstm(dfr)    
#     current = dfr.Stock_price.tail(1).values
#     current_to_predict = current/prediction

#     print('\n• Current Dollar:    ', '%.5f' %current)
#     print('\n• Predicted Dollar:  ', '%.5f' %prediction)
#     print('\n• Current to Predicted: ', '%.5f' %current_to_predict)
   
#     if current < prediction:
#         print(f"\n• Linear Regression Result: ", '\033[1m' + 'UP' + '\033[0m')
#     else:
#         print(f"\n• Linear Regression Result: ", '\033[1m' + 'DOWN' + '\033[0m')
#     print('__________________________________\n')

#     # classifier
#     prediction = classifer_data_input(dfr, moving_avg_value) # [0]: precision / [1]: prediction result

#     if prediction[1] == 1:
#         print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'UP' + '\033[0m')
#         print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
#     else:
#         print(f"• {moving_avg_value} days MVG Result: ", '\033[1m' + 'DOWN' + '\033[0m')
#         print('\n• Precision Score =', np.round(prediction[0]*100, 2), '%')
#     print('__________________________________\n\n')

#     # print out stats summary for multi-linear regression
#     import statsmodels.api as sm
#     dfr['intercept'] = 1
#     lm = sm.OLS(dfr['Stock_price'], dfr[['intercept', 'Cpi', 'Jclaim', 'Inf', 'snp', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Int_rate', 'Rgdp']]) # exclude 'Bitcoin' due to high pvalue
#     results = lm.fit()
#     print(results.summary())
    
    
# if __name__ == '__main__':
#     minutepassed = timechecknow()
#     if day_check() != 'Saturday' and day_check() != 'Sunday' and minutepassed < -30:
#         main_dollar()
#     else:
#         print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')

## Valuation

In [ ]:
# prediction source data is cut based on the most released date of Inf_rate, 
# which has no update during the weekends.
from os.path import exists
import time
today_to_add = date.today()
import pandas_datareader as data
data_source = 'yahoo'

def compensation_plot(df):
  
    today = pd.read_csv('predict_valuation_today %s.csv' %currentdate)

    # include only list to Buy monitor
    df = pd.read_csv('predict_valuation.csv').sort_values('symbol')
    df = df.loc[df['symbol'].isin(today['symbol'].values)]

    df1 = df.query('symbol == "BTC-USD" or symbol == "USO" or symbol == "GLD"')
    df = df.query('symbol != "BTC-USD" and symbol != "USO" and symbol != "GLD"')

    plt.figure(figsize = (16,15))

    plt.subplot(2,1,1)
    plot1 = pd.DataFrame(df.groupby('symbol')['error'].mean()).sort_values('symbol').reset_index()
    plt.bar(plot1.symbol, plot1.error)
    plt.title('Compensation error: gap rate between Prediction and Price')
    plt.xticks(rotation = 90)
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

    plt.subplot(2,1,2)
    plot2 = pd.DataFrame(df.groupby('symbol')['compensation_corr'].mean()).sort_values('symbol').reset_index()
    plt.bar(plot2.symbol, plot2.compensation_corr, color = 'g')
    plt.title('Correlation coefficient with S&P 500 for compensation(*Must match with above)')
    plt.xticks(rotation = 90)
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    plt.show()
    
#     print(df1.groupby('symbol')[['error','compensation_corr']].mean())

    if len(plot1.symbol) != len(plot2.symbol):
        print('!! plot1 is not equal to plot2 !!')
        alarm()
    else:
        pass    
    
    pd.set_option('display.max_row', 50)
    predict_valuation_today = pd.read_csv('predict_valuation_today %s.csv' %currentdate).sort_values(['up_down', 'valuation', 'error'], ascending = False)
    print('\nAverage Error(%): ', np.average(predict_valuation_today.error), '\n')
    print(predict_valuation_today.reset_index(drop=True)[['symbol', 'up_down', 'valuation']])
    

def predict(target, ticker):
    
    target = None
    
    if ticker == 'GLD':
        target = 'GLD'
    elif ticker == 'USO':
        target = 'USO'       
    elif ticker == 'BTC-USD':
        target = 'BTC-USD'
    else:
        target = '^GSPC'    

    # target to get corr
    target_index = yfinance_df(target)  
    target_index = target_index[['Date', 'Adj Close']]
    # ticker to get corr
    ticker_index = yfinance_df(ticker)   
    # corr
    cor = pd.merge(target_index, ticker_index, how = 'inner', on = 'Date')

    import statsmodels.api as sm
    cor['intercept'] = 1
    lm = sm.OLS(cor['Adj Close_y'], cor[['intercept', 'Adj Close_x']])
    results = lm.fit()
#     compensation_cor = results.rsquared * results.params[1]    
#     compensation_cor = results.params[1] * np.absolute(cor.corr()['Adj Close_x'][1])
    compensation_cor = np.absolute(cor.corr()['Adj Close_x'][1])
    
    # check if exist for today
    # if exists, only update the current rows
    
    # remove past data
    sourcedata_exists = exists(f"predict_aim_sourcedata_monitoring_{ticker}_{mostrecentdate}.csv")
    if sourcedata_exists == True:
        import os
        os.remove(f"predict_aim_sourcedata_monitoring_{ticker}_{mostrecentdate}.csv")
    else:
        pass
    
    sourcedata_exists = exists(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv")
    if target == '^GSPC' and sourcedata_exists == True:
        print(f"File Exists - Target: {target} & Ticker: {ticker} ..... Updating the current values", end = '                                                                                                                                          \r')
        df = pd.read_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv")
        realtime_vars = {'Int_rate':'^TNX', 
                         'Stock_price':ticker, 
                         'Cboe':'^VIX', 
                         'Gold':'GC=F', 
                         'Oil':'CL=F', 
                         'Bitcoin':'BTC-USD', 
                         'Dollar':'DX-Y.NYB'}
        for i, v in realtime_vars.items():
            df.loc[df.shape[0]-1, [i]] = yfinance_df_min(v)['Adj Close'].values[0]
    else:
        print(f"Initial downloading - Target: {target} & Ticker: {ticker}", end = '                                                                                                         \r')
    # if not exists, gather data from the beginning
    # 10 year treasery bold yield
        rate = yfinance_df('^TNX')  
        irate = rate[['Date', 'Adj Close']]

        # CBOE Volatility Index
        cboe = yfinance_df('^VIX')
        cboe = cboe[['Date', 'Adj Close']]

        if target == '^GSPC':
            # Gold Feb 22 (GC=F)
            gold = yfinance_df('GC=F')  
            gold = gold[['Date', 'Adj Close']]    

        elif target == 'GLD' or target == 'USO' or target == 'BTC-USD':
            # S&P 500
            snp = yfinance_df('^GSPC')
            snp = snp[['Date', 'Adj Close']]
        else:
            pass

        # Crude Oil
        mar = yfinance_df('CL=F')  
        mar = mar[['Date', 'Adj Close']]

        if target == '^GSPC' or target == 'GLD' or target == 'USO':
            # Bitcoin (*last)
            bit = yfinance_df('BTC-USD')   
            bit = bit[['Date', 'Adj Close']]
        elif target == 'BTC-USD':
            # Gold
            gold = yfinance_df('GC=F')   
            gold = gold[['Date', 'Adj Close']]

        else:
            pass

        # Dollar Index
        dollar = yfinance_df('DX-Y.NYB')
        dollar = dollar[['Date', 'Adj Close']]

        # cpi index
        url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        cpi = webContent[4075:]
        cpi = cpi.split()
        date = []
        number = []
        for i in range(len(cpi)):
            if i%2 == 0:
                date.append(cpi[i])
            else:
                number.append(cpi[i])
        cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
        cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
        cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

        # jobless claim index
        url = 'https://fred.stlouisfed.org/data/ICSA.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        joblessclaim = webContent[710:]
        joblessclaim = joblessclaim.split()
        date = []
        number = []
        for i in range(len(joblessclaim)):
            if i%2 == 0:
                date.append(joblessclaim[i])
            else:
                number.append(joblessclaim[i])
        jls = pd.DataFrame({'Date': date, 'Jclaim': number})
        
        # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
        jls.Jclaim.replace('.', np.NaN, inplace = True)
        jls.fillna(method = 'ffill', inplace = True)
        jls = jls[jls['Date'] <= currentdate]
        
        jls['Date'] = jls['Date'].astype('datetime64[ns]')
        jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
        jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
        cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

        # inflation index
        url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        inf = webContent[1222:]
        inf = inf.split()
        date = []
        number = []
        for i in range(len(inf)):
            if i%2 == 0:
                date.append(inf[i])
            else:
                number.append(inf[i])
        inf = pd.DataFrame({'Date': date, 'Inf': number})
        drp = inf[inf.Inf == '.'].index.values
        inf = inf.drop(index = drp).reset_index(drop=True)
        inf['Date'] = inf['Date'].astype('datetime64[ns]')
        inf['Inf'] = pd.to_numeric(inf['Inf'])

        time.sleep(0.5)

        # combine & cleaning
        df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.merge(ticker_index, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
        df = df.fillna(0)
        df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

        # cleaning - cpi and Jclaim 
        for i in range(len(df.Cpi)):
            if df.Cpi[i] == 0:
                df.Cpi[i] = df.Cpi[i-1]    
        df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
        for i in range(len(df.Jclaim)):
            if df.Jclaim[i] == 0.0:
                df.Jclaim[i] = df.Jclaim[i-1]
        drp = df[df['Int_rate'] == 0].index.values
        df = df.drop(index=drp)

        # cleaning - inflation rate since 1990
        df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df[df['Date'] > '2003-01-01']
        df = df.fillna(0)
        if df.tail(1).Inf.values[0] == 0:
            df.tail(1).Inf = df.tail(2).Inf.values[0]

        # merge - cboe     
        df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
        df = df.fillna(0)

        if target == '^GSPC':  
            # merge - gold
            df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
            df = df.fillna(0)

        elif target == 'GLD' or target == 'USO' or target == 'BTC-USD':  
            # merge - snp
            df = df.merge(snp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'snp'}).reset_index(drop=True)
            df = df.fillna(0)

        else:
            pass

        # merge - Crude Oil Mar 22 
        df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

        if target == 'BTC-USD':
            # merge - gold
            df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
            df = df.fillna(0)
        elif target == '^GSPC' or target == 'GLD' or target == 'USO':
            # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
            df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)
        else:
            pass

        # merge - Dollar Index 
        df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

        # dropna and save the source date
        df = df.dropna()

        # fillavg = df[df['Inf'] == 0].index.values
        # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

        dfdrop = df[df['Stock_price'] == 0].index.values
        df = df.drop(index = dfdrop)   

        # dfdrop = df[df['Inf'] == 0].index.values
        # df = df.drop(index = dfdrop)

        fillavg = df[df['Inf'] == 0].index.values
        df['Inf'].loc[fillavg] = df['Inf'].tail(5).max()             

        # Fillna Cboe with the highest value for the past 20 working days. 
        cboe_fillna = df[df['Cboe'] == 0].index.values
        df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 
        
        # Real GDP
        url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        Rgdp = webContent[1005:]
        Rgdp = Rgdp.split()
        date = []
        number = []
        for i in range(len(Rgdp)):
            if i%2 == 0:
                date.append(Rgdp[i])
            else:
                number.append(Rgdp[i])
        Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
        Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
        Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

        df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
        df.Rgdp.replace(np.NaN, 0, inplace = True)
        for i in range(len(df.Rgdp)):
            if df.Rgdp[i] == 0:
                df.Rgdp[i] = df.Rgdp[i-1]
        df.dropna(inplace = True)

        if target == '^GSPC':
            dfdrop = df[df['Gold'] == 0].index.values
            df = df.drop(index=dfdrop)

        elif target == 'BTC-USD':
            dfdrop = df[df['Gold'] == 0].index.values
            df = df.drop(index=dfdrop)
            dfdrop = df[df['snp'] == 0].index.values
            df = df.drop(index=dfdrop)

        elif target == 'GLD' or target == 'USO':
            dfdrop = df[df['snp'] == 0].index.values
            df = df.drop(index=dfdrop)

        else:
            pass            

        dfdrop = df[df['Int_rate'] == 0].index.values
        df = df.drop(index=dfdrop)
        df = df.drop(columns = ['Close'])
        ### end of new gathering ####

    if df.all().sum() != len(df.columns):
        df.to_csv('errorlog_prediction_monitoring_%s.csv' %ticker, index = False)
        print('\nZero value checked - Check error log file: ', ticker, '\n')    
        alarm() # if any zero value, alarm
        print(df.all())
        error = yfinance_df(ticker) 
        print('Regeneration by yfinance with the ticker\n', error.all())

        print('\nRe-checking: ', ticker, '\n')
        predict(ticker)
    else:
        if str(df[-1:].Date.values[0])[:10] == currentdate:
            df.to_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv", index = False)
        else:
            print("\n\nThe last line of the dateframe does not match with the currentdate.\n>>> The file is not saved. <<<\n\n")
    ################################################################################### new merge - add above
    
    with open("rday_oil.txt", 'r') as f:
        rday_oil = [line.rstrip('\n') for line in f][0]
    with open("rday_gold.txt", 'r') as f:
        rday_gold = [line.rstrip('\n') for line in f][0]
    with open("rday_bit.txt", 'r') as f:
        rday_bit = [line.rstrip('\n') for line in f][0]
    with open("rday_snp.txt", 'r') as f:
        rday_snp = [line.rstrip('\n') for line in f][0]
        
    
    # range selection - COVID19
    if target == '^GSPC':
        dfc = df[df['Date'] > rday_snp]
    elif target == 'GLD':
        dfc = df[df['Date'] > rday_gold]
    elif target == 'USO':
        dfc = df[df['Date'] > rday_oil]
    elif target == 'BTC-USD':
        dfc = df[df['Date'] > rday_bit]
    else:
        pass    
    df_classifier = dfc.copy()

    # Regression analysis
    import statsmodels.api as sm
    dfc['intercept'] = 1
    
    if target == '^GSPC':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']]) 
    elif target == 'GLD':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'snp', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']])
    elif target == 'BTC-USD':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Oil', 'Dollar', 'Gold', 'Rgdp']])
    elif target == 'USO':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Bitcoin', 'Dollar', 'Rgdp']])
    else:
        pass

    results = lm.fit()

    # linear regression
    if target == '^GSPC':
        dfl = dfc.drop(columns = ['intercept'])
    elif target == 'GLD':
        dfl = dfc.drop(columns = ['intercept', 'Cboe'])      
    elif target == 'USO':
        dfl = dfc.drop(columns = ['intercept', 'Oil']) 
    elif target == 'BTC-USD':
        dfl = dfc.drop(columns = ['intercept'])
    else:
        pass
    
    prediction = pred_lstm(dfl) 
    err = (prediction - dfc.tail(1).Stock_price.values[0])/dfc.tail(1).Stock_price.values[0]
    tprice = yfinance_df(ticker).tail(1)['Adj Close'].values[0]
    
    # classifier
    prediction_classifier = classifer_data_input(df_classifier, moving_avg_value) # [0]: precision / [1]: prediction result > pass df w/t r-day considered

    # check the most recent date if yFinance includes it    
    lastdate = str(df[-1:].Date.values[0])[:10]
    if lastdate == currentdate:
        prediction_up_down = prediction_classifier[1]
    else:
        prediction_up_down = 0
    
    return [today_to_add, ticker, prediction, tprice, results.rsquared, err, compensation_cor, prediction_up_down]


def valuation(ticker):
    portfolio = ticker

    pricepredict = {}
    date = []
    symbol = []
    prediction = []
    price = []
    rsquared = []
    error = []
    compensation = []
    up_down = []
    target = None

    from tqdm.notebook import tqdm

    for i in tqdm(range(len(portfolio))):
        time.sleep(0.3)
        try:
            print('Compensating: ', portfolio[i], end = '                                                                                \r')
            result = predict(target, portfolio[i])
            date.append(result[0])
            symbol.append(result[1])
            prediction.append(result[2])
            price.append(result[3])
            rsquared.append(result[4])
            error.append(result[5])
            compensation.append(result[6])
            up_down.append(result[7])
        except:
            print('Except: Ticker valuation - Check error log file: ', portfolio[i])
            alarm()
            pass

    pricepredict['date'] = date
    pricepredict['time'] = timechecknow()
    pricepredict['symbol'] = symbol
    pricepredict['prediction'] = prediction
    pricepredict['price'] = price
    pricepredict['rsquared'] = rsquared
    pricepredict['error'] = np.absolute(error)
    pricepredict['compensation_corr'] = compensation
    pricepredict['up_down'] = up_down

    # to send today_predict
    df = pd.DataFrame(pricepredict)  

    # apply accmulated gap to compensate 'gap'
    dfacc = pd.read_csv('predict_valuation.csv').sort_values('symbol')
    dfacc = dfacc.dropna()

    average_gap = []
    for i in df.symbol:
        if i in pd.DataFrame(dfacc.groupby('symbol').count()).reset_index().symbol.values:
            average_gap.append(dfacc[dfacc['symbol'] == i].gap.min())
        else:
            average_gap.append(np.absolute(df[df['symbol'] == i].prediction.values[0] - df[df['symbol'] == i].price.values[0]))

    df['gap'] = average_gap

    # compensate - original predicted price is compensated by multipying coefficient correlation
    #              this also partially compensates rsquared per ticker since corr*corr becomes rsquared
    # again, df['gap'] here is calculated from accmulated data, 'predict', and save the result to 'predict_valuation_today'
    for i in range(len(df['symbol'])):
        if df['symbol'][i] == 'GLD' or df['symbol'][i] == 'USO' or df['symbol'][i] == 'BTC-USD':
            pass
        else:
            if df['prediction'][i] < df['price'][i]:
                df['prediction'][i] = df['prediction'][i]# + (df['gap'][i] * df['compensation_corr'][i])
            else:
                df['prediction'][i] = df['prediction'][i]# - (df['gap'][i] * df['compensation_corr'][i])
            
    # final valuation, and save the result to 'predict_valuation_today'
    df.loc[df['prediction'] - df['price'] > 0, 'valuation'] = 1
    df.loc[df['prediction'] - df['price'] <= 0, 'valuation'] = 0
    df['gap'] = np.absolute(df['prediction'] - df['price'])
    df['error'] = np.absolute(df['prediction'] - df['price']) / df['price']

    today = df.sort_values('error').reset_index(drop=True)
    today.to_csv('predict_valuation_today %s.csv' %currentdate, index = False)

    # update 'predict_valuation_today' into 'predict_valuation'
    df = pd.read_csv('predict_valuation.csv')
    df = pd.concat([df, today])
    df.to_csv('predict_valuation.csv', index = False)

    # plot and data print
    compensation_plot(df)

if __name__ == '__main__':
    print('\n', '\033[1m' + 'The valuation gets ready.' + '\033[0m', '\n')    
    
    
# valuation(pd.read_csv('srank.csv').Symbol[:100].values.tolist())

#  Portfolio

## Portfolio - Design

In [21]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import yfinance as yf
import pandas as pd
import numpy as np
import xlrd
import csv
import jupyter_beeper

# email with attachments
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pytz
import sean

# Tableau
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

snp = pd.read_csv('./snp500/snp500.csv')


def tableau_snp500():
    with open("currentdate.txt", 'r') as f:
        currentdate = int([line.rstrip('\n') for line in f][0])
    currentdate = str(currentdate)[0:4]+'-'+str(currentdate)[4:6]+'-'+str(currentdate)[6:8]

    df = pd.read_csv(f"snp500 {currentdate}.csv")
    # Sean score preparation
    def score(grade):
        if grade == 'A+':
            return np.arange(1,5,0.3)[13]
        elif grade == 'A':
            return np.arange(1,5,0.3)[12]
        elif grade == 'A-':
            return np.arange(1,5,0.3)[11]
        elif grade == 'B+':
            return np.arange(1,5,0.3)[10]
        elif grade == 'B':
            return np.arange(1,5,0.3)[9]
        elif grade == 'B-':
            return np.arange(1,5,0.3)[8]
        elif grade == 'C+':
            return np.arange(1,5,0.3)[7]
        elif grade == 'C':
            return np.arange(1,5,0.3)[6]
        elif grade == 'C-':
            return np.arange(1,5,0.3)[5]
        elif grade == 'D+':
            return np.arange(1,5,0.3)[4]
        elif grade == 'D':
            return np.arange(1,5,0.3)[3]
        elif grade == 'D-':
            return np.arange(1,5,0.3)[2]
        elif grade == 'F':
            return np.arange(1,5,0.3)[1]
        else:
            return np.arange(1,5,0.3)[0]

    values = []
    for i in range(len(df['Valuation Grade'])):
        values.append(score(df['Valuation Grade'][i]))
    df['Valuation Grade_score'] = values 

    values = []
    for i in range(len(df['Growth Grade'])):
        values.append(score(df['Growth Grade'][i]))
    df['Growth Grade_score'] = values 

    values = []
    for i in range(len(df['Profitability Grade'])):
        values.append(score(df['Profitability Grade'][i]))
    df['Profitability Grade_score'] = values    

    values = []
    for i in range(len(df['Momentum Grade'])):
        values.append(score(df['Momentum Grade'][i]))
    df['Momentum Grade_score'] = values 

    values = []
    for i in range(len(df['EPS Revision Grade'])):
        values.append(score(df['EPS Revision Grade'][i]))
    df['EPS Revision Grade_score'] = values 
    
    snp_main = df
    snp_se = pd.read_csv('./snp500/snp500.csv')
    snp = pd.merge(snp_main, snp_se, how = 'inner', on = 'Symbol')

    # select required columns 
    data = snp[['Symbol',
     'Valuation Grade_score',
     'Growth Grade_score',           
     'Profitability Grade_score',           
     'Momentum Grade_score',           
     'EPS Revision Grade_score',
     'Volume',
     'Avg. Vol',
     'Prev Close',
     'Open',
     'Day Low',
     'Day High',
     '52W Low_x',
     '52W High_x',
     'Quant',
     'SA Authors',
     'Wall St.',
     'Shares',
     'Cost',
     'Today`s Gain',
     'Today`s % Gain',
     'Total Change',
     'Total % Change',
     'Value',
     'SA Authors Score',
     'Wall St. Score',
     'Quant Score',
     'Valuation Grade',
     'Growth Grade',
     'Profitability Grade',
     'Momentum Grade',
     'EPS Revision Grade',
     'ETF Momentum',
     'ETF Expenses',
     'ETF Dividends',
     'ETF Risk',
     'ETF Flows',
     'Upcoming Announce Date',
     'Release Time',
     'EPS Estimate',
     'Revenue Estimate',
     'EPS Revisions Grade',
     'Last Quarter Announce Date',
     'EPS Actual',
     'EPS Surprise',
     'Revenue Actual',
     'Revenue Surprise',
     'Safety',
     'Growth',
     'Yield',
     'Consistency',
     'Ex-Div Date',
     'Payout Date',
     'Yield TTM',
     'Yield FWD',
     '4Y Avg Yield',
     'Div Rate TTM',
     'Div Rate FWD',
     'Payout Ratio',
     '4Y Avg Payout',
     'Div Growth 3Y',
     'Div Growth 5Y',
     'Years of Growth',
     'Market Cap',
     'EV',
     'P/E TTM',
     'P/E FWD',
     'PEG TTM',
     'PEG FWD',
     'Price / Sales',
     'EV / Sales',
     'EV / EBITDA',
     'Price / Book',
     'Price / Cash Flow',
     'Revenue YoY',
     'Revenue FWD',
     'Revenue 3Y',
     'Revenue 5Y',
     'EBITDA YoY',
     'EBITDA FWD',
     'EBITDA 3Y',
     'Net Income 3Y',
     'EPS YoY',
     'EPS FWD',
     'EPS 3Y',
     'Tangible Book 3Y',
     'Total Assets 3Y',
     'FCF 3Y',
     '52W Low_y',
     '52W High_y',
     '5D Perf',
     '1M Perf',
     '6M Perf',
     'YTD Perf',
     '1Y Perf',
     '3Y Perf',
     '3Y Total Return',
     '5Y Perf',
     '5Y Total Return',
     '10Y Perf',
     '10Y Total Return',
     '10D SMA',
     'Last Price Vs. 10D SMA',
     '50D SMA',
     'Last Price Vs. 50D SMA',
     '100D SMA',
     'Last Price Vs. 100D SMA',
     '200D SMA',
     'Last Price Vs. 200D SMA',
     'Week Vol / Shares',
     '24M Beta',
     '60M Beta',
     'Revenue TTM',
     'NET Income TTM',
     'Cash from Operations',
     'Profit Margin',
     'EBIT Margin',
     'EBITDA Margin',
     'Net Income Margin',
     'FCF Margin',
     'Return on Equity',
     'Return on Assets',
     'Return on Total Capital',
     'Asset Turnover',
     'Net Income / Employee',
     'Shares Outstanding',
     'Float %',
     'Insider Shares',
     'Insider %',
     'Institutional Shares',
     'Institutional Percent',
     'Total Debt',
     'ST Debt',
     'LT Debt',
     'Total Cash',
     'Debt to FCF',
     'Current Ratio',
     'Quick Ratio',
     'Covered Ratio',
     'Debt to Equity',
     'LT Debt to Total Capital',
     'Price',
     'Change',
     'Change_percent',
     'date',
     'avg_score',
     'GICS Sector',
     'GICS Sub-Industry',
     'Headquarters Location']]

    # select required columns 
    data.reset_index(drop = True, inplace = True)

    # fillna as zero
    data = data.fillna(0)

    # data type change
    array_list = []
    for i in data['Payout Date'].values:
        if i == '0':
            array_list.append('1/01/2000')
        else:
            array_list.append(i)
    data['Payout Date'] = array_list
    data['Payout Date'] = data['Payout Date'].astype('datetime64[ns]').dt.date

    array_list = []
    for i in data['Ex-Div Date'].values:
        if i == '0':
            array_list.append('1/01/2000')
        else:
            array_list.append(i)
    data['Ex-Div Date'] = array_list
    data['Ex-Div Date'] = data['Ex-Div Date'].astype('datetime64[ns]').dt.date

    data['Last Quarter Announce Date'] = data['Last Quarter Announce Date'].astype('datetime64[ns]').dt.date

    array_list = []
    for i in data['Upcoming Announce Date'].values:
        if i == '0':
            array_list.append('1/01/2000')
        else:
            array_list.append(i)
    data['Upcoming Announce Date'] = array_list
    data['Upcoming Announce Date'] = data['Upcoming Announce Date'].astype('datetime64[ns]').dt.date

    data['Last Quarter Announce Date'] = data['Last Quarter Announce Date'].astype('datetime64[ns]').dt.date
    data['date'] = data['date'].astype('datetime64[ns]').dt.date

    # change space as underbar
    for i in data.columns:
        change = i.replace(' ','_')
        data.rename(columns = {i : change}, inplace = True)   

    data.rename(columns = 
    {'GICS_Sub-Industry' : 'GICS_Sub_Industry',
    '52W_Low_x' : '_52W_Low_x',  
    '52W_High_x' : '_52W_High_x',
    '4Y_Avg_Yield' : '_4Y_Avg_Yield',
    '4Y_Avg_Payout' : '_4Y_Avg_Payout',
    '52W_Low_y' : '_52W_Low_y',
    '52W_High_y' : '_52W_High_y',
    '5D_Perf' :'_5D_Perf',
    '1M_Perf' :'_1M_Perf',
    '6M_Perf' : '_6M_Perf',
    '1Y_Perf' : '_1Y_Perf',
    '3Y_Perf' : '_3Y_Perf',
    '3Y_Total_Return' : '_3Y_Total_Return',
    '5Y_Perf' : '_5Y_Perf',
    '5Y_Total_Return' : '_5Y_Total_Return',
    '10Y_Perf' : '_10Y_Perf',
    '10Y_Total_Return' : '_10Y_Total_Return',
    '10D_SMA' : '_10D_SMA',
    '50D_SMA' : '_50D_SMA',
    '100D_SMA' : '_100D_SMA',
    '200D_SMA' : '_200D_SMA',
    '24M_Beta' : '_24M_Beta',
    '60M_Beta' : '_60M_Beta', 
    'Avg._Vol' : 'Avg_Vol', 
    'Wall_St.' : 'Wall_St',
    'Wall_St._Score' : 'Wall_St_Score', 
    'Last_Price_Vs._10D_SMA' : 'Last_Price_Vs_10D_SMA',        
    'Last_Price_Vs._50D_SMA' : 'Last_Price_Vs_50D_SMA',   
    'Last_Price_Vs._100D_SMA' : 'Last_Price_Vs_100D_SMA',   
    'Last_Price_Vs._200D_SMA' : 'Last_Price_Vs_200D_SMA', 
    'Today`s_Gain' : 'Today_Gain', 
    'Today`s_%_Gain' : 'Todays_Percent_Gain',
    'Total_%_Change' : 'Total_Percent_Change', 
    'Ex-Div_Date' : 'Ex_Div_Date', 
    'P/E_TTM' : 'PE_TTM',                   
    'P/E_FWD' : 'PE_FWD',                      
    'PEG_TTM' : 'PEG_TTM',                    
    'PEG_FWD' : 'PEG_FWD',                      
    'Price_/_Sales' : 'Price_per_Sales',             
    'EV_/_Sales' : 'EV_per_Sales',                  
    'EV_/_EBITDA' : 'EV_per_EBITDA',                
    'Price_/_Book' : 'Price_per_Book',                 
    'Price_/_Cash_Flow' : 'Price_per_Cash_Flow', 
    'Week_Vol_/_Shares' : 'Week_Vol_per_Shares',
    'Net_Income_/_Employee' : 'Net_Income_per_Employee',        
    'Float_%' : 'Float_Percent',                      
    'Insider_%' : 'Insider_Percent'
    }, inplace = True)

    # Create Dimensions
    company_dim = data.copy()
    company_dim = company_dim[['GICS_Sector', 'GICS_Sub_Industry']]
    company_dim = company_dim.drop_duplicates(subset = 'GICS_Sector', keep = "first")
    company_dim = company_dim.reset_index(drop = True)
    company_dim.insert(0, "company_id", range(10, 10+len(company_dim)))
    data = data.merge(company_dim[["GICS_Sector", "company_id"]],
                      left_on = "GICS_Sector",
                      right_on = "GICS_Sector",
                      how = "left")

    location_dim = data.copy()
    location_dim = location_dim[['Headquarters_Location']]
    location_dim = location_dim.drop_duplicates(subset = 'Headquarters_Location', keep = "first")
    location_dim = location_dim.reset_index(drop = True)
    location_dim.insert(0, "location_id", range(100, 100+len(location_dim)))
    data = data.merge(location_dim[['Headquarters_Location', 'location_id']],
                      left_on = 'Headquarters_Location',
                      right_on = 'Headquarters_Location',
                      how = "left")

    date_dim = data.copy()
    date_dim = date_dim[['date']]
    date_dim = date_dim.drop_duplicates(subset = 'date', keep = "first")
    date_dim = date_dim.reset_index(drop = True)
    date_dim.insert(0, "date_id", range(10000, 10000+len(date_dim)))
    data = data.merge(date_dim[['date', 'date_id']],
                      left_on = 'date',
                      right_on = 'date',
                      how = "left")

    # Create Fact table
    fact_columns = []
    for i in data.columns:
        fact_columns.append(i)

    fact_columns_filtered = []
    for i in fact_columns:
        if i not in location_dim.columns.tolist() and i not in company_dim.columns.tolist() and i not in date_dim.columns.tolist():
            fact_columns_filtered.append(i)

    fact_columns_filtered.append('date_id')
    fact_columns_filtered.append('company_id')
    fact_columns_filtered.append('location_id')

    fact_snp = data[fact_columns_filtered]

    # send to BigQuery
    # first, create a BigQuery client to connect to BigQuery
    from google.cloud import bigquery
    from google.oauth2 import service_account

    key_path = r'C:\Users\aicpa\My Drive\__QED\cpasean-4d80408d4add.json' # must edit to your credentials json file location
    credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                        scopes=["https://www.googleapis.com/auth/cloud-platform"],)
    client = bigquery.Client(credentials = credentials,
                             project = credentials.project_id)

    dataset_id = 'cpasean.qed_snp500'   # PASTE THIS DATASET ID FROM ABOVE STEPS
    dataset_id = dataset_id.replace(':', '.')

    # build a function to load tables to BigQuery

    def load_table_to_bigquery(df, table_name):

        dataset_id = 'cpasean.qed_snp500'

        dataset_ref = client.dataset(dataset_id)
        job_config = bigquery.LoadJobConfig()
        job_config.autodetect = True
        job_config.write_disposition = "WRITE_TRUNCATE"

        upload_table_name = f"cpasean.qed_snp500.{table_name}"

        load_job = client.load_table_from_dataframe(df,
                                                   upload_table_name,
                                                   job_config = job_config)

        print(f"starting job {load_job}")

    load_table_to_bigquery(df = location_dim, table_name = 'location_dim')
    load_table_to_bigquery(df = date_dim, table_name = "date_dim")
    load_table_to_bigquery(df = company_dim, table_name = "company_dim")
    load_table_to_bigquery(df = fact_snp, table_name = "fact_snp")

    # send to *.csv
    location_dim.to_csv('location_dim.csv', index = False)
    date_dim.to_csv('date_dim.csv', index = False)
    company_dim.to_csv('company_dim.csv', index = False)
    fact_snp.to_csv('fact_snp.csv', index = False)

    print('\n\nData Warehousing has been done.\n\n')
    
    voice_message("""\
        Data Warehousing has been done""")
    
    
# extract pages from each spread sheet
def frame(sheet):
    
    page = {}

    for n in range(sheet.ncols):
        page[sheet.cell_value(0,n)] = {}
        for r in range(1, sheet.nrows):
            page[sheet.cell_value(0,n)][r] = sheet.cell_value(r,n)     
    df1 = pd.DataFrame(page)
    df1.replace('-', 0, inplace = True)
    df1.replace('NM', 0, inplace = True)
    df1.replace(np.NaN, 0, inplace = True)
    pages.append(df1)
        
def nxt():        
    # combine each sheet in the one DataFrame    
    df = pages[0].merge(pages[1], how = 'inner', left_on = 'Symbol', right_on = 'Symbol')
    for i in range(2, 12):
        df = df.merge(pages[i], how = 'inner', left_on = 'Symbol', right_on = 'Symbol')

    # clean duplicated columns    
    df['Price'] = pd.DataFrame(df['Price_x'].values)[0]
    df = df.drop(columns = ['Price_x'], axis = 1)
    df = df.drop(columns = ['Price_y'], axis = 1)

    df['Change'] = pd.DataFrame(df['Change_x'].values)[0]
    df = df.drop(columns = ['Change_x'], axis = 1)
    df = df.drop(columns = ['Change_y'], axis = 1)

    df['Change_percent'] = df['Change %_x']*100
    df = df.drop(columns = ['Change %_x'], axis = 1)
    df = df.drop(columns = ['Change %_y'], axis = 1)

    # add date, avg score
    df['date'] = currentdate
    try:
        df['avg_score'] = (df['SA Author Ratings'] + df['Wall Street Ratings'] + df['Quant Ratings'])/3
    except:
        df['avg_score'] = (df['SA Analysts Score'] + df['Wall Street Ratings'] + df['Quant Ratings'])/3
    df = df.sort_values('avg_score', ascending = False).reset_index(drop=True)
    df.drop(index = df.shape[0]-1, inplace = True)

    # edit the new version of Seeking-Alpha download data into the numeric format
    df = df.fillna(0)
## enable below when data contains $ sign in object type. 
#     df['Revenue Estimate'] = df['Revenue Estimate'].str.replace('$', '')
#     df['Revenue Actual'] = df['Revenue Actual'].str.replace('$', '')
#     df['Revenue Surprise'] = df['Revenue Surprise'].str.replace('$', '')

    def value_to_float(x):
        if type(x) == float or type(x) == int:
            return x
        if 'Years' in x:
            return float(x.strip(' Years'))
        if 'Year' in x:
            return float(x.strip(' Year'))
        if 'A+' in x:
            return np.arange(1,5,0.3)[13]
        if 'A' in x:
            return np.arange(1,5,0.3)[12]
        if 'A-' in x:
            return np.arange(1,5,0.3)[11]
        if 'B+' in x:
            return np.arange(1,5,0.3)[10]
        if 'B' in x:
            return np.arange(1,5,0.3)[9]
        if 'B-' in x:
            return np.arange(1,5,0.3)[8]
        if 'C+' in x:
            return np.arange(1,5,0.3)[7]
        if 'C' in x:
            return np.arange(1,5,0.3)[6]
        if 'C-' in x:
            return np.arange(1,5,0.3)[5]
        if 'D+' in x:
            return np.arange(1,5,0.3)[4]
        if 'D' in x:
            return np.arange(1,5,0.3)[3]
        if 'D-' in x:
            return np.arange(1,5,0.3)[2]
        if 'F' in x:
            return np.arange(1,5,0.3)[1]
        if ',' in x:
            return float(x.replace(',', ''))
        if '$' in x:
            return float(x.replace('$', ''))
        if 'K' in x:
            if len(x) > 1:
                return float(x.replace('K', '')) * 1000
            return 1000.0 # else
        if 'M' in x:
            if len(x) > 1:
                return float(x.replace('M', '')) * 1000000
            return 1000000.0
        if 'B' in x:
            if len(x) > 1:
                return float(x.replace('B', '')) * 1000000000
            return 1000000000.0
        if 'T' in x:
            if len(x) > 1:
                return float(x.replace('T', '')) * 1000000000000
            return 1000000000000.0
        return x

    for column in df.columns[1:22]:
        df[column] = df[column].apply(value_to_float).astype(float)

    for column in df.columns[27:32]:
        df[column] = df[column].apply(value_to_float).astype(float)

    for column in df.columns[34:36]:
        df[column] = df[column].apply(value_to_float).astype(float)

    for column in df.columns[37:42]:
        df[column] = df[column].apply(value_to_float).astype(float)

    for column in df.columns[48:57]:
        df[column] = df[column].apply(value_to_float).astype(float)

    for column in df.columns[58:139]:
        df[column] = df[column].apply(value_to_float).astype(float)

    df = df.rename(columns = {'EPS Revision Grade_x' : 'EPS Revision Grade', 'EPS Revision Grade_y' : 'EPS Revisions Grade'})
    df = df.fillna(0)
    df.to_csv('snp500 %s.csv' %currentdate, index = False)
    
    # combine previous snp500 with current snp500
    df = pd.concat(map(pd.read_csv, ['snp500 %s.csv' %mostrecentdate, 'snp500 %s.csv' %currentdate]), ignore_index=True) #optional
    df = df.fillna(0)
    df = df.drop_duplicates().reset_index(drop = True)
    df.to_csv('snp500 %s.csv' %currentdate, index = False) 
    df.to_csv('./Backup_snp500/snp500 %s.csv' %currentdate, index = False)
    
    # remove previous snp500
    import os
    os.remove('snp500 %s.csv' %mostrecentdate)  
    os.remove('snp500 %s.xls' %mostrecentdate)  
    print(f'snp500 {currentdate}.csv | srank.csv have been created.')
    
#     tableau_snp500()
    
def secondpick(bluechips):
    # read accmulated snp500 data *Remove the historical data if not quite practical!!!
    df = pd.read_csv('snp500 %s.csv' %currentdate)

    # filter out date based on 'min_holding_period'
    df = df[df['date'] >= str(datetime.today() - timedelta(days = min_holding_period))[:10]]
    corr_analysis = df.corr().reset_index()[['index','Change_percent']]
    corr_analysis = corr_analysis.sort_values('Change_percent', ascending = False).reset_index(drop = True)
    corr_analysis = corr_analysis.loc[corr_analysis['index'].isin([
        '1Y Perf', 
        'Price / Cash Flow',
        '10Y Total Return', 
        '10Y Perf', 
        'P/E TTM', 
        '5Y Perf',
        '5Y Total Return',
        'EV / EBITDA', 
        'Price / Sales',
        'Last Price Vs. 10D SMA', 
        'Quick Ratio', 
        '3Y Perf',
        '3Y Total Return', 
        'Week Vol / Shares', 
        'Current Ratio',
        'EV / Sales', 
        'P/E FWD',
        'Revenue 3Y', 
        'Revenue FWD', 
        'EBITDA FWD', 
        'Yield FWD', 
        'Yield TTM', 
        'Revenue YoY', 
        'FCF 3Y', 
        '4Y Avg Yield',
        'EBITDA 3Y', 
        '24M Beta', 
        'Covered Ratio', 
        'Profitability Grade_score', 
        'Return on Assets', 
        'Asset Turnover',
        'Profit Margin',
        'Payout Ratio', 
        'FCF Margin',
        'Div Rate FWD', 
        'EPS 3Y', 
        '60M Beta', 
        'Div Rate TTM', 
        'EPS Revision Grade_score',
        'Valuation Grade_score', 
        'Float %', 
        'Net Income Margin',
        'EBIT Margin', 
        '1M Perf', 
        'Price / Book', 
        'Net Income / Employee',
        'YTD Perf', 
        'EBITDA Margin', 
        'Momentum Grade_score', 
        'Institutional Percent', 
        'EPS FWD', 
        'Wall St. Score', 
        'PEG FWD', 
        '6M Perf', 
        'EPS Estimate',
        'EPS Actual',
        'Debt to FCF', 
        'Div Growth 3Y',
        'LT Debt to Total Capital', 
        'Return on Total Capital',
        'Div Growth 5Y', 
        'PEG TTM',
        'Last Price Vs. 100D SMA', 
        'Last Price Vs. 200D SMA', 
        'Last Price Vs. 50D SMA', 
        'avg_score',
        'Growth Grade_score', 
        'SA Authors Score', 
        'Debt to Equity',
        'Return on Equity', 
        'EPS YoY', 
        'Quant Score', 
        'EBITDA YoY'])].reset_index(drop = True)

    neg_vars = ['Price / Cash Flow', 
                'P/E TTM', 
                'EV / EBITDA', 
                'Price / Sales',
                'EV / Sales', 
                'P/E FWD',
                '24M Beta', 
                '60M Beta', 
                'Price / Book', 
                'PEG FWD', 
                'Debt to FCF', 
                'LT Debt to Total Capital', 
                'PEG TTM',
                'Debt to Equity']

    # corr: the higest corr.
    corr_analysis_topindex = corr_analysis['index'].values.tolist()[0]

    today = df[df['date'] == currentdate]
    df = today.copy() # grading is NOT based on the accmulated file ####################
    df = df.reset_index(drop = True)  
    
    # Sean score preparation
    def score(grade):
        if grade == 'A+':
            return np.arange(1,5,0.3)[13]
        elif grade == 'A':
            return np.arange(1,5,0.3)[12]
        elif grade == 'A-':
            return np.arange(1,5,0.3)[11]
        elif grade == 'B+':
            return np.arange(1,5,0.3)[10]
        elif grade == 'B':
            return np.arange(1,5,0.3)[9]
        elif grade == 'B-':
            return np.arange(1,5,0.3)[8]
        elif grade == 'C+':
            return np.arange(1,5,0.3)[7]
        elif grade == 'C':
            return np.arange(1,5,0.3)[6]
        elif grade == 'C-':
            return np.arange(1,5,0.3)[5]
        elif grade == 'D+':
            return np.arange(1,5,0.3)[4]
        elif grade == 'D':
            return np.arange(1,5,0.3)[3]
        elif grade == 'D-':
            return np.arange(1,5,0.3)[2]
        elif grade == 'F':
            return np.arange(1,5,0.3)[1]
        else:
            return np.arange(1,5,0.3)[0]

    # 03042022: grading is NOT based on the accmulated file, 'snp500 2xxx-xx-xx'
    values = []
    for i in range(len(df['Valuation Grade'])):
        values.append(score(df['Valuation Grade'][i]))
    df['Valuation Grade_score'] = values 

    values = []
    for i in range(len(df['Growth Grade'])):
        values.append(score(df['Growth Grade'][i]))
    df['Growth Grade_score'] = values 

    values = []
    for i in range(len(df['Profitability Grade'])):
        values.append(score(df['Profitability Grade'][i]))
    df['Profitability Grade_score'] = values    

    values = []
    for i in range(len(df['Momentum Grade'])):
        values.append(score(df['Momentum Grade'][i]))
    df['Momentum Grade_score'] = values 

    values = []
    for i in range(len(df['EPS Revision Grade'])):
        values.append(score(df['EPS Revision Grade'][i]))
    df['EPS Revision Grade_score'] = values 
    
    # y-axis is set as 'srank_corr_var'
    import statsmodels.api as sm
    df['intercept'] = 1
    lm = sm.OLS(df[corr_analysis_topindex], df[['intercept', 'Valuation Grade_score', 'Growth Grade_score', 'Profitability Grade_score', 'Momentum Grade_score', 'EPS Revision Grade_score', 'Quant Score', 'SA Authors Score', 'Wall St. Score']])

    results = lm.fit()
#     print(results.summary())
    print('\n')

    import warnings
    warnings.filterwarnings("ignore")
    ###########################################################################################################################################################################################
    #Create Sean Score with 6 grades
    if corr_analysis_topindex in neg_vars:
        df['sean_score'] = -(df['Valuation Grade_score']*results.params[1] + df['Growth Grade_score']*results.params[2] + df['Profitability Grade_score']*results.params[3] + df['Momentum Grade_score']*results.params[4] + df['EPS Revision Grade_score']*results.params[5] + df['Quant Score']*results.params[6] + df['SA Authors Score']*results.params[7] + df['Wall St. Score']*results.params[8])/8 
    else:
        df['sean_score'] = (df['Valuation Grade_score']*results.params[1] + df['Growth Grade_score']*results.params[2] + df['Profitability Grade_score']*results.params[3] + df['Momentum Grade_score']*results.params[4] + df['EPS Revision Grade_score']*results.params[5] + df['Quant Score']*results.params[6] + df['SA Authors Score']*results.params[7] + df['Wall St. Score']*results.params[8])/8 
#     today = df[df['date'] == currentdate]
#     today['sean_score'] = (today['Valuation Grade_score']*results.params[1] + today['Growth Grade_score']*results.params[2] + today['Profitability Grade_score']*results.params[3] + today['Momentum Grade_score']*results.params[4] + today['EPS Revision Grade_score']*results.params[5] + today['Quant Score']*results.params[6] + today['SA Authors Score']*results.params[7] + today['Wall St. Score']*results.params[8])/8
    ###########################################################################################################################################################################################
    
    dfsort = pd.DataFrame(df.groupby('Symbol').mean()).reset_index()
    
    # get earning date
    earningdate = df[['Symbol', 'Upcoming Announce Date']]
    earningdate = earningdate.groupby('Symbol')['Symbol', 'Upcoming Announce Date'].tail(1)

    # get ex-dividend date
    divdate = df[['Symbol', 'Ex-Div Date']]
    divdate = divdate.groupby('Symbol')['Symbol', 'Ex-Div Date'].tail(1)

    dfsort = dfsort.merge(divdate, how = 'outer', left_on = 'Symbol', right_on = 'Symbol')
    dfsort = dfsort.merge(earningdate, how = 'outer', left_on = 'Symbol', right_on = 'Symbol')

    sean_rank = dfsort.copy()
    sean_rank = sean_rank.sort_values('sean_score', ascending = False)
    sean_rank.reset_index(inplace = True)
    
    # set number of tickers 
    # delete 'snp500 ----, --, --' if applying the up-to-dated rating
    srank = sean_rank.merge(snp, how = 'inner', left_on = 'Symbol', right_on = 'Symbol')
    srank = srank.drop(columns = 'index')
    srank.to_csv('srank.csv', index = False)
    srank.to_csv('./Backup_srank/srank %s.csv' %currentdate, index = False)
    slist = srank.head(bluechips).Symbol.values   # setting the number of bluechips
    
    with open('slist.txt', 'w') as f:
        for s in slist:
            f.write(str(s) + '\n')
            
#     print(slist)


def main_design():
    # excute only if the file exists    
    from os.path import exists

    # check if the previous date file exists. If yes, excel spreadsheet work starts
    snp500_exists = exists('snp500 %s.csv' %mostrecentdate)
    if snp500_exists == False:
        print('S&P 500 has already been updated.')
        pass
    else:
        for i in range(12):
            sheet = xlrd.open_workbook('snp500 %s.xls' %currentdate).sheet_by_index(i)
            frame(sheet)    
        nxt() # Excel sheets combining done up to here.
    secondpick(bluechips)    
    
    
if __name__ == '__main__':
    if day_check() != 'Saturday' and day_check() != 'Sunday' and timechecknow() < 390:
        # the program starts off from the main function
        pages = []
        main_design()
    else:
        print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[0m', '\n')
        
# import FinanceDataReader as fdr
# fdr.DataReader('GOOGL').reset_index()

## Portfolio - Creation

In [22]:
def dir_cleaner(mostrecentdate):
    '''
    Delete old file to avoid loading delay
    
    '''
    with open('etf_portfolio.txt', 'r') as f:
        etf_portfolio = [line.rstrip('\n') for line in f]
    
    portfolio_to_remove = pd.read_csv('snp500\snp_sector_index.csv').sort_values('Symbol')['Symbol'].to_list()
    portfolio_to_remove = portfolio_to_remove + etf_portfolio
    
    from os.path import exists
    print('• Delete Previous Files: candle_analysis')
    for ticker in portfolio_to_remove:
        candle_analysis = exists(f'candle_analysis_{ticker}_{mostrecentdate}.csv')
        if candle_analysis == True:
            import os
            os.remove(f'candle_analysis_{ticker}_{mostrecentdate}.csv')
            print(f'candle_analysis_{ticker}_{mostrecentdate}.csv ==> Deleted')
        else:
            pass  
    print('\n==> Completed\n\n')

    print('• Delete Previous Files: predict_aim_sourcedata_monitoring')
    for ticker in portfolio_to_remove:
        predict_aim_sourcedata_monitoring = exists(f'predict_aim_sourcedata_monitoring_{ticker}_{mostrecentdate}.csv')
        if predict_aim_sourcedata_monitoring == True:
            import os
            os.remove(f'predict_aim_sourcedata_monitoring_{ticker}_{mostrecentdate}.csv')
            print(f'predict_aim_sourcedata_monitoring_{ticker}_{mostrecentdate}.csv ==> Deleted')
        else:
            pass  
    print('\n==> Completed\n\n')
    
    print('• Delete Previous Files: candle_token')
    for ticker in portfolio_to_remove:
        candle_token = exists(f'candle_token_{ticker}.txt')
        if candle_token == True:
            import os
            os.remove(f'candle_token_{ticker}.txt')
            print(f'candle_token_{ticker}.txt ==> Deleted')
        else:
            pass  
    print('\n==> Completed\n\n')

    print('• Delete Previous Files: strategy_anaysis')
    for ticker in portfolio_to_remove:
        strategy_anaysis = exists(f'strategy_anaysis_{ticker}_{mostrecentdate}.csv')
        if strategy_anaysis == True:
            import os
            os.remove(f'strategy_anaysis_{ticker}_{mostrecentdate}.csv')
            print(f'strategy_anaysis_{ticker}_{mostrecentdate}.csv ==> Deleted')
        else:
            pass  
    print('\n==> Completed\n\n')    
    
    print('• Delete Previous Files: stg_para_search')
    for ticker in portfolio_to_remove:
        stg_para_search = exists(f'stg_para_search_{ticker}_{mostrecentdate}.csv')
        if stg_para_search == True:
            import os
            os.remove(f'stg_para_search_{ticker}_{mostrecentdate}.csv')
            print(f'stg_para_search_{ticker}_{mostrecentdate}.csv ==> Deleted')
        else:
            pass  
    print('\n==> Completed\n\n')    
    
    
def ticker_eventday_check():    
    srank = pd.read_csv('srank.csv')

    # remove ex-dividend tickers
    srank['Ex-Div Date'] = srank['Ex-Div Date'].replace('0', '1/1/1')
    srank['Ex-Div Date'] = srank['Ex-Div Date'].astype('datetime64[ns]')
    exdivticker = srank.sort_values('Ex-Div Date', ascending = False)

    nextdate_div = str(datetime.today() + timedelta(days = 1))[:10]   
    exdivticker = exdivticker[exdivticker['Ex-Div Date'] == nextdate_div].reset_index(drop = True)
    exdivticker = list(exdivticker['Symbol'].values)
    #     print('\n\n\n'+ 'Ex-div. ticker: \n\n', exdivticker)

    # remove earning-day tickers
    today_earning = str(datetime.today())[:10]   
    today_earning = today_earning[5:7].lstrip('0')+'/'+today_earning[8:10].lstrip('0')+'/'+today_earning[:4]
    plusday_earning = str(datetime.today() + timedelta(days = 1))[:10]   
    plusday_earning = plusday_earning[5:7].lstrip('0')+'/'+plusday_earning[8:10].lstrip('0')+'/'+plusday_earning[:4]

    earningticker_1 = srank[srank['Upcoming Announce Date'] == today_earning].Symbol.to_list()
    earningticker_2 = srank[srank['Upcoming Announce Date'] == plusday_earning].Symbol.to_list()

    event_tickers = exdivticker + earningticker_1 + earningticker_2
    event_tickers = sorted([i for n, i in enumerate(event_tickers) if i not in event_tickers[:n]]) # duplicated
    
    return event_tickers


def super_pass_filtering(lookbackdays = 3):
    # super pass - buy sign within the short term period
    
    snp500 = pd.read_csv('./snp500/snp_sector_index.csv').Symbol.to_list()
    with open('remove_tickers_superpass.txt', 'r') as f:
        remove_tickers_superpass = [line.rstrip('\n') for line in f]

    tickers = snp500
    tickers = [i for i in tickers if i not in remove_tickers_superpass]
    tickers = [i for n, i in enumerate(tickers) if i not in tickers[:n]] #duplicated 

    # get sentiment tickers by inputting filters tickers
    sentiment_tickers = sentiment_portfolio(tickers, 9)
    
    # get top volume tickers by inputting filters tickers
    volume_tickers = top_volume_portfolio(tickers)

    superpass_buysign = []
    print(f"\n\n\nSuper Pass [{lookbackdays} Days Buy Sign Check]:\n\n {tickers}\n")
    c = 0 
    for stock in tickers:
        c+=1
        print(f"> Ticker ({c}/{len(tickers)}): {stock}", end = '                                                                 \r')
        try:
            if buy_sign_day(stock, lookbackdays) == True:
                superpass_buysign.append(stock)
            else:
                pass
        except:
            print('\nException >>> Buy Sign Check: ', stock)
            pass    
        
    superpass_buysign.extend(sentiment_tickers)
    superpass_buysign.extend(volume_tickers)
    portfolio_superpass = superpass_buysign
    portfolio_superpass = sorted([i for n, i in enumerate(portfolio_superpass) if i not in portfolio_superpass[:n]]) 

    ticker_final_list = []
    for ticker in portfolio_superpass:
        print(f'Crossover Check - Ticker: {ticker}', end = '                            \r')
        analysis = analysis_price(ticker)
        if analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False:
            ticker_final_list.append(ticker)
    portfolio_superpass = ticker_final_list    
    
    print(f'\n\n\n\n\nCaution >>> Super Pass Tickers(saved):\n{portfolio_superpass}')
    with open("portfolio_superpass.txt", 'w') as f:
        for s in portfolio_superpass:
            f.write(str(s) + '\n')
    
    return portfolio_superpass


def portfolio_strategy_test(portfolio_after_sector_comparison):
    global portfolio
    
    portfolio_after_strategy_test = []
    strategy_analysis_failed = []
    print(f"\n\n\n\n\nSector Comparison Test Candidates ===> Strategy Test Candidates (>{min_SuccessRate}):\n\n {portfolio_after_sector_comparison}\n")
    c = 0 
    for stock in portfolio_after_sector_comparison:
        c+=1
        print(f"> Ticker ({c}/{len(portfolio_after_sector_comparison)}): {stock}", end = '                                                                 \r')
        try:
            time.sleep(0.5)
            strategy_analysis = strategy_analysis_main(stock, 'off', 'off')[0]
            if strategy_analysis == 1:
                portfolio_after_strategy_test.append(stock)
            elif strategy_analysis == 0 or strategy_analysis == -10:
                strategy_analysis = strategy_analysis_main(stock, 'off', 'off')[0]
                if strategy_analysis >= 0:
                    portfolio_after_strategy_test.append(stock)
                elif strategy_analysis == -10:
                    strategy_analysis_failed.append(stock)
                else:
                    pass
            else:
                pass
        except:
            print(f'\nException >>> Strategy Test Candidates: {stock}\n\n')
            pass
    portfolio = portfolio_after_strategy_test
    print(f"\n\n\n\n\nStrategy Test ===> Final Portfolio\n\n {portfolio}\n")
    
    # check success rate of strategy analysis
    analysis_success_rate = (len(portfolio_after_sector_comparison)-len(strategy_analysis_failed)) / len(portfolio_after_sector_comparison)
    print('\033[1m' + f'\n\n\nStrategy Analysis Success Rate: {np.round(analysis_success_rate*100, 2)} %\n\n\n' + '\033[0m')
  
    return portfolio, analysis_success_rate


def pfo_creation(bluechips):
    global yf_counter
    global min_SuccessRate
    
    # reset the counter checking the number of request to yfinance
    yf_counter = 0
    
    # less than a flexible percentile by snp trend
    snp_yfinance = yfinance_df_setting('^GSPC')    
    snp_long = snp_yfinance[-20:]['Adj Close'].mean()
    snp_short = snp_yfinance[-10:]['Adj Close'].mean()
    snptrend = np.round(((snp_short - snp_long)/snp_long),2)*100
    if snptrend > 5:
        snptrend = 5
    else:
        pass

    print('\n\n'+ '• S&P 500 Moving Average(20 vs 70) Percent change: ', '\033[1m' + str(np.round(snptrend,2)) + '\033[0m', '%')    

    # Do Not purchase a day before the ex-div date, which is two days before the record date.
    # Investors who purchase shares any day before the ex-dividend date will be documented as owners of shares on the record date.
    srank = pd.read_csv('srank.csv')
    
    # Get top 20 Volume x Price list
    srank['Vol_Mul_Price'] = srank['Volume'] * srank['Open'] 
    srank = srank.sort_values('Vol_Mul_Price', ascending = False).reset_index(drop = True)
    Vol_Mul_Price = srank[:int(srank.Symbol.nunique()*volume_ticker_percent)].Symbol.to_list()
 
    # Get another portfolist list from the corr. with 'yesterday's return'
    perf_review = pd.read_csv('perf_review %s.csv' %mostrecentdate).sort_values('date')
    corr_analysis = perf_review.corr().reset_index()[['index','day_change']]
    corr_analysis = corr_analysis.sort_values('day_change', ascending = False).reset_index(drop = True)
    corr_analysis = corr_analysis.loc[corr_analysis['index'].isin([
        'sean_score', 
        '1Y Perf', 
        'Price / Cash Flow',
        '10Y Total Return', 
        '10Y Perf', 
        'P/E TTM', 
        '5Y Perf',
        '5Y Total Return',
        'EV / EBITDA', 
        'Price / Sales',
        'Last Price Vs. 10D SMA', 
        'Quick Ratio', 
        '3Y Perf',
        '3Y Total Return', 
        'Week Vol / Shares', 
        'Current Ratio',
        'EV / Sales', 
        'P/E FWD',
        'Revenue 3Y', 
        'Revenue FWD', 
        'EBITDA FWD', 
        'Yield FWD', 
        'Yield TTM', 
        'Revenue YoY', 
        'FCF 3Y', 
        '4Y Avg Yield',
        'EBITDA 3Y', 
        '24M Beta', 
        'Covered Ratio', 
        'Profitability Grade_score', 
        'Return on Assets', 
        'Asset Turnover',
        'Profit Margin',
        'Payout Ratio', 
        'FCF Margin',
        'Div Rate FWD', 
        'EPS 3Y', 
        '60M Beta', 
        'Div Rate TTM', 
        'EPS Revision Grade_score',
        'Valuation Grade_score', 
        'Float %', 
        'Net Income Margin',
        'EBIT Margin', 
        '1M Perf', 
        'Price / Book', 
        'Net Income / Employee',
        'YTD Perf', 
        'EBITDA Margin', 
        'Momentum Grade_score', 
        'Institutional Percent', 
        'EPS FWD', 
        'Wall St. Score', 
        'PEG FWD', 
        '6M Perf', 
        'EPS Estimate',
        'EPS Actual',
        'Debt to FCF', 
        'Div Growth 3Y',
        'LT Debt to Total Capital', 
        'Return on Total Capital',
        'Div Growth 5Y', 
        'PEG TTM',
        'Last Price Vs. 100D SMA', 
        'Last Price Vs. 200D SMA', 
        'Last Price Vs. 50D SMA', 
        'avg_score',
        'Growth Grade_score', 
        'SA Authors Score', 
        'Debt to Equity',
        'Return on Equity', 
        'EPS YoY', 
        'Quant Score', 
        'EBITDA YoY'])].reset_index(drop = True)

    neg_vars = ['Price / Cash Flow', 
                'P/E TTM', 
                'EV / EBITDA', 
                'Price / Sales',
                'EV / Sales', 
                'P/E FWD',
                '24M Beta', 
                '60M Beta', 
                'Price / Book', 
                'PEG FWD', 
                'Debt to FCF', 
                'LT Debt to Total Capital', 
                'PEG TTM',
                'Debt to Equity']

    corr_analysis_topindex = corr_analysis.head(12)[['index', 'day_change']].values.tolist()
    #     print('\n'+ 'Corr. Top 3 Variables: ')
    #     print('\n', corr_analysis_topindex[0])
    #     print('\n', corr_analysis_topindex[1])
    #     print('\n', corr_analysis_topindex[2])

    def corr_tickers(corr_index_number):
        if corr_analysis_topindex[corr_index_number][0] in neg_vars:
            return list(srank[srank[corr_analysis_topindex[corr_index_number][0]] > np.percentile(srank[corr_analysis_topindex[corr_index_number][0]].values.tolist(), fs_cut+snptrend)]['Symbol'].values)
        else:
            return list(srank[srank[corr_analysis_topindex[corr_index_number][0]] < np.percentile(srank[corr_analysis_topindex[corr_index_number][0]].values.tolist(), (100-fs_cut)-snptrend)]['Symbol'].values)

    # the number of conditions to filter out
    corr_1st = corr_tickers(0)
    corr_2nd = corr_tickers(1)
    corr_3rd = corr_tickers(2)
    corr_4th = corr_tickers(3)
    corr_5th = corr_tickers(4)
    corr_6th = corr_tickers(5)
    corr_7th = corr_tickers(6)
    corr_8th = corr_tickers(7)
    corr_9th = corr_tickers(8)
    corr_10th = corr_tickers(9)
    corr_11th = corr_tickers(10)
    corr_12th = corr_tickers(11)

    #     print('\n\n\n'+ 'Filtered out - Corr. 1st: ', corr_analysis_topindex[0][0],'\n\n', corr_1st)
    #     print('\n'+ 'Filtered out - Corr. 2nd: ', corr_analysis_topindex[1][0],'\n\n', corr_2nd)
    #     print('\n'+ 'Filtered out - Corr. 3rd: ', corr_analysis_topindex[2][0],'\n\n', corr_3rd)


# remove - ratio analysis ###############################################################################################################
    # fs_cut_plus: Weight (-) more filter out / (+) less filter out
    
    # positive indices <<<<<<<<<<<<<<<<
    fs_cut_final = fs_cut
    roe = sorted(srank[srank['Return on Equity'] < np.percentile(srank['Return on Equity'].values.tolist(), (100-fs_cut_final)-snptrend)]['Symbol'].tolist())
    #     print('\n\n\n'+ 'Filtered out - ROE: \n\n', roe)

    fs_cut_final = fs_cut
    quick_ratio = sorted(srank[srank['Quick Ratio'] < np.percentile(srank['Quick Ratio'].values.tolist(), (100-fs_cut_final)-snptrend)]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - Quick Ratio: \n\n', quick_ratio)

    fs_cut_final = fs_cut - fs_cut_plus/2
    ebitda_margin = sorted(srank[srank['EBITDA Margin'] < np.percentile(srank['EBITDA Margin'].values.tolist(), (100-fs_cut_final)-snptrend)]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - EBITDA Margin: \n\n', ebitda_margin)

    
    
    # negative indices <<<<<<<<<<<<<<<<
    fs_cut_final = fs_cut
    debt_fcf = sorted(srank[srank['Debt to FCF'] > np.percentile(srank['Debt to FCF'].values.tolist(), fs_cut_final+snptrend)]['Symbol'].tolist())    
    #     print('\n'+ 'Filtered out - Debt to FCF: \n\n', debt_fcf)      

    fs_cut_final = fs_cut
    debt_equity = sorted(srank[srank['Debt to Equity'] > np.percentile(srank['Debt to Equity'].values.tolist(), fs_cut_final+snptrend)]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - Debt to Equity TTM: \n\n', debt_equity)         
    
    fs_cut_final = fs_cut - fs_cut_plus/2
    ev_ebitda = sorted(srank[srank['EV / EBITDA'] > np.percentile(srank['EV / EBITDA'].values.tolist(), fs_cut_final+snptrend)]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - EV / EBITDA: \n\n', ev_ebitda)   

    fs_cut_final = fs_cut - fs_cut_plus/2
    price_book = sorted(srank[srank['Price / Book'] > np.percentile(srank['Price / Book'].values.tolist(), fs_cut_final+snptrend)]['Symbol'].tolist())    
    #     print('\n'+ 'Filtered out - Price / Book: \n\n', Price / Book)      

    

    # fixed indices <<<<<<<<<<<<<<<<
    ni_ttm = sorted(srank[srank['NET Income TTM'] < 0]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - PEG TTM: \n\n', peg_ttm)   
    
    peg_ttm = sorted(srank[srank['PEG TTM'] > 1]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - PEG TTM: \n\n', peg_ttm)   

    cash_operation = sorted(srank[srank['Cash from Operations'] < 0]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - Operating Cash: \n\n', cash_operation)      
    
    rev_surprise = sorted(srank[srank['Revenue Surprise'] < 0]['Symbol'].tolist())
    #     print('\n'+ 'Filtered out - Revenue Surprise: \n\n', rev_surprise) 
    
#########################################################################################################################################
 
    # remove tickers having (1)earning call (2)ex-dividend day 
    event_tickers = ticker_eventday_check()

    # filter-out ########################################################################################
    # adjust the number of conditions to filter out (e.g. corr_2nd, 3rd, 4th,5th)
    
    remove_tickers = list(event_tickers + buymonitor_blacklist
                          + corr_1st + corr_2nd + corr_3rd + corr_4th + corr_5th + corr_6th + corr_7th + corr_8th + corr_9th + corr_10th
                          + roe + quick_ratio + ebitda_margin
                          + debt_fcf + debt_equity + ev_ebitda + price_book
                          + ni_ttm + peg_ttm + cash_operation + rev_surprise)
    remove_tickers = [i for n, i in enumerate(remove_tickers) if i not in remove_tickers[:n]] #duplicated 
    ##################################################################################################### 
    # adjust the number of conditions to filter out (e.g. corr_2nd, 3rd, 4th,5th)
    
    remove_tickers_superpass = list(event_tickers + buymonitor_blacklist
                                  + corr_1st + corr_2nd + corr_3rd + corr_4th + corr_5th + corr_6th + corr_7th + corr_8th + corr_9th + corr_10th + corr_11th + corr_12th
                                  + roe + quick_ratio + ebitda_margin
                                  + debt_fcf + debt_equity + ev_ebitda + price_book
                                  + ni_ttm + peg_ttm + cash_operation)
    remove_tickers_superpass = [i for n, i in enumerate(remove_tickers_superpass) if i not in remove_tickers_superpass[:n]] #duplicated 
    
    with open("remove_tickers_superpass.txt", 'w') as f:
        for s in remove_tickers_superpass:
            f.write(str(s) + '\n')
    ##################################################################################################### 
    
    # get another portfolio from srank.csv according to the top three R-squared
    # and then proportionally add tickers per the R-squared value
    highest_1st = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[0], ascending = False).head(int(corr_analysis_topindex[0][1]*50))['Symbol'].tolist()
    highest_2nd = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[1], ascending = False).head(int(corr_analysis_topindex[1][1]*50))['Symbol'].tolist()
    highest_3rd = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[2], ascending = False).head(int(corr_analysis_topindex[2][1]*50))['Symbol'].tolist()
    highest_4th = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[3], ascending = False).head(int(corr_analysis_topindex[3][1]*50))['Symbol'].tolist()
    highest_5th = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[4], ascending = False).head(int(corr_analysis_topindex[4][1]*50))['Symbol'].tolist()
    highest_6th = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[5], ascending = False).head(int(corr_analysis_topindex[5][1]*50))['Symbol'].tolist()
    highest_7th = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[6], ascending = False).head(int(corr_analysis_topindex[6][1]*50))['Symbol'].tolist()
    highest_8th = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[7], ascending = False).head(int(corr_analysis_topindex[7][1]*50))['Symbol'].tolist()
    highest_9th = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[8], ascending = False).head(int(corr_analysis_topindex[8][1]*50))['Symbol'].tolist()
    highest_10th = srank.sort_values(corr_analysis.head(bluechips)['index'].values.tolist()[9], ascending = False).head(int(corr_analysis_topindex[9][1]*50))['Symbol'].tolist()
    
    highest_corr_index = highest_1st + highest_2nd + highest_3rd + highest_4th + highest_5th + highest_6th + highest_7th + highest_8th + highest_9th + highest_10th

    # remove duplicated tickers in the list
    highest_corr_index = [i for n, i in enumerate(highest_corr_index) if i not in highest_corr_index[:n]] 

    with open("highest_corr_index.txt", 'w') as f:
        for s in highest_corr_index:
            f.write(str(s) + '\n')     
    with open('ex_highest_corr_index.txt', 'r') as f:
        ex_highest_corr_index = [line.rstrip('\n') for line in f]
    with open('slist.txt', 'r') as f:
        slist = [line.rstrip('\n') for line in f]
    with open('snp_value_tickers.txt', 'r') as f:
        snp_value_tickers = [line.rstrip('\n') for line in f]
    with open('etf_portfolio.txt', 'r') as f:
        etf_portfolio = [line.rstrip('\n') for line in f]
    with open('rsi_portfolio.txt', 'r') as f:
        rsi_portfolio = [line.rstrip('\n') for line in f]
    with open('sentiment_portfolio.txt', 'r') as f:
        sentiment_portfolio = [line.rstrip('\n') for line in f]

    # top 3 quant score
    snp_main = pd.read_csv(f"snp500 {currentdate}.csv")
    quant_score_top = snp_main.groupby(['Symbol']).mean().reset_index().sort_values('Quant Score', ascending = False).head(3).reset_index(drop = True)['Symbol'].tolist()

    # previous day - good value tickers    
    good_value = pd.read_csv('predict_valuation.csv')
    good_value = good_value.groupby(['date', 'symbol']).mean().reset_index()
    good_value = good_value[good_value['date'] == mostrecentdate] 
    good_value = good_value.query('valuation > 0.5 and up_down == 1')
    good_value = good_value['symbol'].values.tolist()
    non_timeout_tickers = ['XLE', 'XLF', 'XLRE', 'XLB', 'XLK', 'XLP', 'XLV', 'XLI', 'XLY', 'XLU', 'XLC',
                           '^GSPC', 'BTC-USD', 'GLD', 'USO', '^TNX', '^VIX', 'GC=F', 'CL=F', 'DX-Y.NYB']
    good_value_tickers = []
    for i in good_value:
        if i not in non_timeout_tickers:
            good_value_tickers.append(i)    
    with open("good_value_tickers.txt", 'w') as f:
        for s in good_value_tickers:
            f.write(str(s) + '\n')  

    #     print('\n\n\nImport Portfolio Candidates______________________________________________________________')
    #     print('\n'+ f"Sean Score - Tickers:        \n\n{slist}")
    #     print('\n'+ f"Ratio analysis - Tickers:    \n\n{snp_value_tickers}")
    #     print('\n'+ f"Perf. Review Corr. - Tickers:\n\n{highest_corr_index}")
    #     print('\n'+ f"Radarplot largest - Tickers: \n\n{ex_highest_corr_index}")  
    #     print('\n'+ f"Previous Good Value - Tickers: \n\n{good_value_tickers}")  
    #     print('\n'+ f"ETF - Tickers: \n\n{etf_portfolio}") 
    #     print('\n'+ f"RSI - Tickers: \n\n{rsi_portfolio}")
    #     print('\n'+ f"Quant Score - Tickers: \n\n{quant_score_top}")   

    with open("tickers_to_sell_big.txt", 'r') as f:
        tickers_to_sell_big = [line.rstrip('\n') for line in f]      

    slist_snpvalue_corr = tickers_to_sell_big + slist + ex_highest_corr_index + quant_score_top + rsi_portfolio
    slist_snpvalue_corr = [i for n, i in enumerate(slist_snpvalue_corr) if i not in slist_snpvalue_corr[:n]] 

    # remove tickers
    portfolio = [i for i in slist_snpvalue_corr if i not in remove_tickers]
    portfolio = sorted(portfolio)
    

# filtering starts from below


#     # buy sign check by volume vs price
#     portfolio_after_buysign = []
#     print(f"\n\n\nSrank Filtering ===> F/S Filtering ===> {buy_sign_lookback_days} Days Buy Sign Check:\n\n {portfolio}\n")
#     c = 0 
#     for stock in portfolio:
#         c+=1
#         print(f"> Ticker ({c}/{len(portfolio)}): {stock}", end = '                                                                 \r')
#         try:
#             if buy_sign_day(stock, buy_sign_lookback_days) == True:
#                 portfolio_after_buysign.append(stock)
#             else:
#                 pass
#         except:
#             print('\nException >>> Buy Sign Check: ', stock)
#             pass
#     portfolio = portfolio_after_buysign 


    # Volatility test 
    initial_vola_cut_rate = vola_cut_rate_value
    if snptrend > 0:
        vola_cut_rate = initial_vola_cut_rate
    else:
        vola_cut_rate = initial_vola_cut_rate + snptrend/10

    portfolio_after_vola = []
    print(f"\n\n\n\n\nSrank Filtering ===> F/S Filtering ===> {vola_cut_rate}% Volatility Test Candidates:\n\n {portfolio}\n")
    c = 0 
    for stock in portfolio:
        c+=1
        print(f"> Ticker ({c}/{len(portfolio)}): {stock}", end = '                                                                 \r')
        try:
            ##############################################################################################
            if volatility_check(stock) < volatility_check('^GSPC')*vola_cut_rate: # filter out > S&P500*x
            ##############################################################################################    
                portfolio_after_vola.append(stock)
            else:
                pass
        except:
            print('\nException >>> Volatility Test Candidates: ', stock)
            pass
    portfolio = portfolio_after_vola
    
    
    # mdd test: get ETF(icld. S&P500) MDD average
    avg_etf_mdd = mdd_cal(etf_ticker) * vola_cut_rate

    portfolio_after_mdd = []
    print(f"\n\n\nVolatility filtering ===> MDD Test Candidates:\n\n {portfolio}\n")
    c = 0 

    for stock in portfolio:
        c+=1
        print(f"> Ticker ({c}/{len(portfolio)}): {stock}", end = '                                                                 \r')
        try:
            if avg_etf_mdd > mdd_cal([stock]):
                portfolio_after_mdd.append(stock)
            else:
                pass
        except:
            print('\nException >>> MDD Test Candidates: ', stock)
            pass
    portfolio = portfolio_after_mdd

    
    # add bypassing ticker list
    portfolio_superpass = super_pass_filtering(superpass_buysign_lookbackdays)    # Super Pass filtering   
    
    ################################################
    # w/o remove filter: Only look at F/S condition
    portfolio.extend(buymonitor_whitelist) 
    portfolio.extend(Vol_Mul_Price) 
    portfolio.extend(good_value_tickers)      
    portfolio.extend(highest_corr_index)
    portfolio.extend(portfolio_superpass)
    portfolio.extend(sentiment_portfolio)
    portfolio = sorted([i for n, i in enumerate(portfolio) if i not in portfolio[:n]]) #duplicated 
    ################################################
    
    
    # Moving Average crossover test
    snp_sector = pd.read_csv('./snp500/snp_sector_index.csv')
    portfolio = [i for i in portfolio if i in snp_sector.Symbol.tolist()] # only include tickers within S&P500.csv
    portfolio_after_crossover = []
    print(f"\n\n\n\n\nMDD Test ===> Bypass list adding ===> Moving Average Crossover Test Candidates:\n\n {portfolio}\n")
    c = 0 
    for stock in portfolio:
        c+=1
        print(f"> Ticker ({c}/{len(portfolio)}): {stock}", end = '                                                                 \r')
        try:
            time.sleep(0.5)
            analysis = analysis_price(stock)           
            if analysis[8] == False:
                portfolio_after_crossover.append(stock)
            else:
                pass
        except:
            print('\nException >>> Moving Average Test Candidates: ', stock)
            pass
    portfolio = portfolio_after_crossover
        
        
    # Sector comparison test
    portfolio_after_sector_comparison = []
    print(f"\n\n\n\n\nMoving Average Crossover Test Candidates ===> Sector Comparison Test Candidates:\n\n {portfolio}\n")
    c = 0 
    for stock in portfolio:
        c+=1
        print(f"> Ticker ({c}/{len(portfolio)}): {stock}", end = '                                                                 \r')
        try:
            time.sleep(0.5)
            analysis = analysis_price(stock)           
            if analysis_price(snp_sector[snp_sector['Symbol'] == stock]['Ticker'].values[0])[5]*portfolio_sector_ticker < analysis[5]:
                portfolio_after_sector_comparison.append(stock)
            else:
                pass
        except:
            print('\nException >>> Sector Comparison Test Candidates: ', stock)
            pass
    portfolio = portfolio_after_sector_comparison    
          
        
    # Strategy test     
    while True:
        portfolio, analysis_success_rate = portfolio_strategy_test(portfolio_after_sector_comparison)

        # Strategy test success rate check
        min_success_rate = 0.3
        if analysis_success_rate >= min_success_rate:
            print('\033[1m' + f'\n\n\nStrategy Analysis Success Rate is higher than {min_success_rate*100} %\n\n\n' + '\033[0m')
            break
        else:
            print('\033[1m' + f'\n\n\nStrategy Analysis Success Rate is less than {min_success_rate*100} %\n\n\n' + '\033[0m')
            min_SuccessRate -= 1
    
    if min_SuccessRate < min_SuccessRate_fixed:
        min_SuccessRate = min_SuccessRate_fixed            
    print('\033[1m' + f'\n\n\nAdjusted min_success_rate: {min_SuccessRate} %\n\n\n' + '\033[0m')

    
    # BBand buy sign check
    BBand_Buy_Sign = BBand_Buy_Sign_Count(portfolio)
    print('\033[1m' + f'\n\n\nBollinger Buy Sign Rate: {BBand_Buy_Sign} %\n\n\n' + '\033[0m')
    
    # add inv tickers
    # w/o remove filter
    portfolio.extend(inv_tickers)
    
    # sort out event-day tickers
    portfolio = [i for i in portfolio if i not in event_tickers]
    portfolio = [i for n, i in enumerate(portfolio) if i not in portfolio[:n]] #duplicated 
    portfolio = sorted(portfolio)

    # delete previous files
    dir_cleaner(mostrecentdate)
    
    # load and store the previous portfolio
    with open("portfolio_original.txt", 'r') as f:
        previous_portfolio = [line.rstrip('\n') for line in f]
    with open("previous_portfolio.txt", 'w') as f:
        for s in previous_portfolio:
            f.write(str(s) + '\n')
    
    # the new tickers in the current portfolio
    new_portfolio = []
    for stock in portfolio:
        if stock not in previous_portfolio:
            new_portfolio.append(stock)
    print(f'\n\nNew tickers in the portfolio:\n{new_portfolio}\n')
    message = f'We have {len(new_portfolio)} new tickers in the new portfolio.'
    print('\n' + '\033[1m' + message + '\033[0m' + '\n\n')
    voice_message(message)
    
    # save list to portfolio.txt
    with open("portfolio.txt", 'w') as f:
        for s in portfolio:
            f.write(str(s) + '\n')

    with open("portfolio_original.txt", 'w') as f:
        for s in portfolio:
            f.write(str(s) + '\n')
            
    # portfolio plot vs sector etf and moving average
    portfolio_plot_list = [i for i in portfolio if i not in inv_tickers]
    for stock_ticker in portfolio_plot_list:
        sector_stock_plot(stock_ticker)
        dmv_plot(stock_ticker)
        rsi_plot(stock_ticker)
        print('\n\n\n\n\n')
            
    # export portfolio srank to email
    srank_portfolio = pd.read_csv('srank.csv')
    srank_portfolio = srank_portfolio[srank_portfolio['Symbol'].isin(portfolio)].reset_index(drop = True)
    srank_portfolio.to_csv('srank_portfolio.csv', index = False)

    # send portfolio to the email list members
    slistemail(portfolio) 

    pd.set_option('display.max_columns', 70)
    pd.set_option('display.max_rows', 70)

    srank_portfolio_df = srank.loc[srank['Symbol'].isin(portfolio)].sort_values('Upcoming Announce Date').reset_index(drop=True)
    srank_portfolio_df.to_csv('portfolio_df.csv', index = False)
    display(srank_portfolio_df['GICS Sector'].value_counts())
    display(srank_portfolio_df)
    
    
def main_creation():
    if str(yfinance_df_setting('^GSPC')[-1:].Date.values[0])[:10] == currentdate:
        print('> The last line == currentdate\n\n\n')
        
#         webbrowser.open("Nuclear Launch Detected.mp3")
#         alarm_open(2)
        message = "Portfolio analysis has been launched"
        voice_message(message)
        print(message)
        creation_start = current_time()
        print(f'\n\n\n\n\n{creation_start}\n')
       
        tickers = pd.read_csv('snp500\snp_sector_index.csv').sort_values('Symbol')['Symbol'].to_list()
        sentiment_portfolio(tickers, 7)
        num_new_portfolio = pfo_creation(bluechips)
        
        with open('portfolio.txt', 'r') as f:
            portfolio = [line.rstrip('\n') for line in f]
        with open("etf_portfolio.txt", 'r') as f:
            etf_portfolio = [line.rstrip('\n') for line in f]
        print(f"\nETF Portfolio: {etf_portfolio}\n\n")
        print('\033[1m' + f"\n\n\n• Total number of monitoring tickers: {len(portfolio+etf_portfolio)} EA\n\n\n" + '\033[0m')
        
        # back test
        portfolio_backtest()
        
        # time cal
        print('\033[1m' + f'\n\n\n\n\n• Creation Start: {creation_start}' + '\033[0m')
        print('\033[1m' + f'• Creation End:   {current_time()}\n\n\n\n\n' + '\033[0m')
        
    else:
        print('> The last line != currentdate\n')
        if timechecknow() < -30:
            print('Stand by Mode', end = '          \r')
            voice_message("""\
                          Opening bell stand by for 30 minutes""")                
            time.sleep(1800)
        elif timechecknow() < -10:
            print('Stand by Mode', end = '          \r')
            voice_message("""\
                          Opening bell stand by for 10 minutes""")                
            time.sleep(600)
        else:
            print('Stand by Mode', end = '          \r')
            voice_message("""\
                          Opening bell stand by for 3 minutes""")      
            
        main_creation()
                
    return num_new_portfolio
     
     
# if __name__ == '__main__':
#     if day_check() != 'Saturday' and day_check() != 'Sunday' and timechecknow() < -20:
#         print(f'\n\n{current_time()}  Portfolio Pre-generation\n\n')
#         main_creation()
#     else:
#         print('\n', '\033[1m' + 'The market is closed or underway.' + '\033[1m', '\n')
        
        
if __name__ == '__main__':
    print('\n', '\033[1m' + 'The portfolio creation gets ready.' + '\033[0m', '\n')    

## Portfolio - Review

In [23]:
def peertickers():
    peers = {}

#####################################################################################  
    
    peers['GOOGL'] = ['GOOGL', 'META', 'TCEHY', 'SNAP', 'BIDU', 'MTCH', '^GSPC']
    peers['QCOM'] = ['QCOM', 'INTC', 'AMD', 'TXN', 'AVGO', 'MU', '^GSPC']
    peers['AAPL'] = ['AAPL', 'DELL', 'HPQ', 'FUJIY', 'CAJPY', 'STX', '^GSPC']
    peers['MSFT'] = ['MSFT', 'ORCL', 'NOW', 'FTNT', 'PANW', 'VMW', '^GSPC']
    peers['NVDA'] = ['NVDA', 'TSM', 'AVGO', 'INTC', 'AMD', 'QCOM', '^GSPC']
    peers['CRM'] = ['CRM', 'ADBE', 'SAP', 'INTU', 'TEAM', 'DASTY', '^GSPC']
    peers['TSLA'] = ['TSLA', 'TM', 'GM', 'MBGAF', 'F', 'LCID', '^GSPC']
    peers['AMZN'] = ['AMZN', 'BABA', 'JD', 'MELI', 'PDD', 'DASH', '^GSPC']
    peers['V'] = ['V', 'MA', 'PYPL', 'ADP', 'SQ', 'FISV', '^GSPC']
    peers['JPM'] = ['JPM', 'BAC', 'WFC', 'RY', 'TD', 'HSBC', '^GSPC']
    peers['JNJ'] = ['JNJ', 'PFE', 'LLY', 'NVO', 'MRK', 'AZN', '^GSPC']
    peers['WMT'] = ['WMT', 'COST', 'WMMVY', 'CRRFY', 'BJ', 'ASAI', '^GSPC']
    peers['XOM'] = ['XOM', 'CVX', 'SHEL', 'BP', 'TTE', 'EQNR', '^GSPC']
    peers['AMT'] = ['AMT', 'CCI', 'EQIX', 'PSA', 'DLR', 'SBAC', '^GSPC']
    peers['DIS'] = ['DIS', 'NFLX', 'SPOT', 'LYV', 'WMG', 'ROKU', '^GSPC']
    peers['GIANT'] = ['GOOGL', 'AAPL', 'MSFT', 'TSLA', 'AMZN', 'META', '^GSPC']
    
#####################################################################################    
    
    
    for key, values in peers.items():
        try:
            print('\033[1m' + key + '____________________________________________________________________________' + '\033[0m', '\n')

            dfpeer = {}

            for i in values:
                tday_yes = yfinance_df_rsi(i).tail(2)
                change_percent = np.round((tday_yes['Adj Close'].values[1]-tday_yes['Adj Close'].values[0])/tday_yes['Adj Close'].values[0]*100, 2)
                dfpeer[i] = change_percent
            df = pd.DataFrame([dfpeer]).T.rename(columns = {0:'change'}).reset_index()
            gain = df[df['change'] > 0].sort_values('change',ascending = False)
            loss = df[df['change'] < 0].sort_values('change',ascending = False)

            plt.figure(figsize = (10,5))
            plt.bar(gain['index'], gain['change'], alpha = 0.95, color = sb.color_palette()[0])
            plt.bar(loss['index'], loss['change'], alpha = 0.95, color = sb.color_palette()[3])
            plt.grid(axis = 'x')
            plt.grid(axis = 'y')
            plt.show()
        except:
            print('Peer valuation error', key, values)
            pass


def buy_blacklistcheck_norm(portfolio_original):
    global purchased # filter out Norm list from monitoring  
    
    print('\n\n\nPortfolio return updating .....\n')
    # Get change% and normal distribution

    dfportfolio = {}
    yes_price = []
    tod_price = []
    average_price = []
    standard_deviation = []
    normal_distribution = []
    c = 1
    for i in portfolio_original:
        time.sleep(0.3)
        print('Analysing: ', c, '/', len(portfolio_original),'- Ticker: ', i, end = '          \r')
        c+=1  
        try:
            tday_yes = yfinance_df(i).tail(2)
            change_percent = np.round((tday_yes['Adj Close'].values[1]-tday_yes['Adj Close'].values[0])/tday_yes['Adj Close'].values[0]*100, 2)
            dfportfolio[i] = change_percent
            yes_price.append(tday_yes['Adj Close'].values[0])
            tod_price.append(tday_yes['Adj Close'].values[1])
    # Normal distribution 
    # range setting: 52 weeks; 263 working days in a year
            average = yfinance_df(i).tail(253)['Adj Close'].mean()
            average_price.append(average)

            std = yfinance_df(i).tail(253)['Adj Close'].std()
            standard_deviation.append(std)    
    # percentile setting: 10% out of 263 working days
            norm = np.percentile(np.random.normal(average, std, 10000000), 10)
            normal_distribution.append(norm)
        except:
            yfinance_df(i).tail(2)
            pass
    df = pd.DataFrame([dfportfolio]).T.rename(columns = {0:'Change'})

    ttoday = date.today()
    df['Date'] = ttoday 
    df['Time'] = timechecknow()
    df['Today'] = tod_price   
    df['Yesterday'] = yes_price  
    df['Average'] = average_price
    df['Standard_deviation'] = standard_deviation
    df['Normal_distribution'] = normal_distribution
    df = df.sort_values('Change', ascending = False).reset_index()
    df = df[['Date', 'Time', 'index', 'Change', 'Today', 'Yesterday', 'Average', 'Standard_deviation', 'Normal_distribution']]

    df_accu = pd.read_csv('daily_summary_target_tickers.csv')
    df_merged = pd.concat([df_accu, df])
    df_merged.to_csv('daily_summary_target_tickers.csv', index = False)
    print('\n\n')
    print(df[['Date', 'index', 'Change']],'\n')

#     with open("portfolio.txt", 'r') as f:
#         portfolio = [line.rstrip('\n') for line in f]

#     for stock in portfolio:
#     # S&P 500: Do not buy monitor and put Blacklist condition 
#         if stock != "GLD" and stock != "USO" and stock != "BTC-USD" and analysis_price(stock)[4] < df[df['index'] == stock]['Normal_distribution'].values[0]:
#             for i in range(1):
#                 try:
#                     purchased.append(stock)        
#                 except:
#                     print('>>> Norm Blacklist has not filtered out. Ticker: ', stock)
#                     alarm()
#                     pass

#             message = """\
# [{}] Ticker has been placed less than P-value.""".format(stock)
#             print('\n')
#             print(message)

#         else:
#             pass          
        

def performance_review():
    # only execute once a day
    from os.path import exists
    asset_review_exists = exists('asset_review %s.csv' %currentdate)
    
    if asset_review_exists == False:
        
    # robinhood_gain&loss
        robin = robin_login()
        robin['date'] = today_to_add

        robin = robin[['date', 'index', 'name', 'price', 'quantity', 'average_buy_price', 'equity',
               'percent_change', 'equity_change']]
        numer = ['price','quantity','average_buy_price','equity','percent_change','equity_change']
        for i in numer:
            robin[i] = pd.to_numeric(robin[i])

        # samsung info
        krw = exchange_check()
        samsung = {}
        samsung['index'] = '005930.KS'
        samsung['name'] = 'SamsungElec'
        samsung['price'] = yfinance_df_min('005930.KS')['Adj Close'].values[0]
        samsung['quantity'] = 5000
        samsung['average_buy_price'] = 78325
        samsung = pd.DataFrame([samsung])
        samsung['equity'] = samsung.price * samsung.quantity / krw
        samsung['percent_change'] = ((samsung.price - samsung.average_buy_price) / samsung.average_buy_price)*100
        samsung['equity_change'] = (samsung.price - samsung.average_buy_price) * samsung.quantity / krw
     
        # merge
        performance = pd.concat([robin, samsung]).reset_index(drop=True)
        performance['date'] = today_to_add

        # target sell price

        with open("realgainrate_big.txt", 'r') as f:
            realgainrate_big = [line.rstrip('\n') for line in f]    
            realgainrate_big = float(realgainrate_big[0])

        with open("realgainrate_mid.txt", 'r') as f:
            realgainrate_mid = [line.rstrip('\n') for line in f]    
            realgainrate_mid = float(realgainrate_mid[0])

        #     target_percentage = 1 + ((realgainrate_big/100)+(realgainrate_mid/100))/2
        target_percentage = 1 + (realgainrate_mid/100)

        ptg = []
        for i in performance.average_buy_price:
            ptg.append(i*target_percentage)
        performance['target_sell_price'] = ptg   

        # Short-term: 10% <= Up to $80,800
        # Long-term: 0% <= Up to $19,900
        tax = []
        for i in performance.price:
            tax.append(i*0.1) # 
        performance['tax_shorterm_share'] = tax 
        tax = []
        for i in performance.equity:
            tax.append(i*0.1) # 
        performance['tax_shorterm'] = tax 

        # portion%
        ptg = []
        for i in performance.equity:
            ptg.append(i/performance.equity.sum()*100)
        performance['percentage'] = ptg   

    # asset_review: save, and read & combine 
        asset_review = performance.copy() # for asset review
        asset_review['date'] = pd.to_datetime(asset_review['date'])
        asset_review.to_csv('asset_review %s.csv' %currentdate, index = False)
        asset_review = pd.concat(map(pd.read_csv, ['asset_review %s.csv' %mostrecentdate, 'asset_review %s.csv' %currentdate]), ignore_index=True)
        asset_review.to_csv('asset_review %s.csv' %currentdate, index = False) 
        asset_review.to_csv('./Backup_asset_review/asset_review %s.csv' %currentdate, index = False) 

        import os
        os.remove('asset_review %s.csv' %mostrecentdate)  

        # add error from predict_valuation.csv
        predict = pd.read_csv('predict_valuation.csv')
        predict_error = pd.DataFrame(predict.groupby('symbol')[['error', 'compensation_corr']].mean()).reset_index()
        performance = performance.merge(predict_error, how = 'inner', left_on = 'index', right_on = 'symbol').sort_values('percent_change', ascending = False)

    # perf_review for 'srank.csv': save, and read & combine 
        seansc = pd.read_csv('srank.csv')[['Symbol','sean_score']]
        seansc.rename(columns = {'Symbol':'index'}, inplace = True)
        perf_review = performance.merge(seansc, how = 'inner', left_on = 'index', right_on = 'index').sort_values('percentage')
        perf_review = perf_review[['date', 'index','percent_change', 'error', 'compensation_corr']].sort_values('percent_change', ascending = False).reset_index(drop=True)

        srank = pd.read_csv('srank.csv')    
        perf_review = perf_review.merge(srank, how = 'inner', left_on = 'index', right_on = 'Symbol').sort_values('index').reset_index(drop = True)
        perf_review.to_csv('perf_review %s.csv' %currentdate, index = False)
        perf_review = pd.concat(map(pd.read_csv, ['perf_review %s.csv' %mostrecentdate, 'perf_review %s.csv' %currentdate]), ignore_index=True)
        perf_review = perf_review.sort_values(['index', 'date']).reset_index(drop = True)

        day_change = []
        for i in range(len(perf_review.index)):
            if i == 0:
                day_change.append(0)
            elif perf_review['index'][i-1] == perf_review['index'][i]:
                daychange = perf_review['percent_change'][i] - perf_review['percent_change'][i-1]
                day_change.append(daychange)
            elif perf_review['index'][i-1] != perf_review['index'][i]:
                day_change.append(0)
            else:
                pass
        perf_review['day_change'] = day_change
        perf_review['date'] = perf_review['date'].astype('datetime64[ns]')

        perf_review.to_csv('perf_review %s.csv' %currentdate, index = False) 
        perf_review.to_csv('./Backup_perf_review/perf_review %s.csv' %currentdate, index = False) 

        import os
        os.remove('perf_review %s.csv' %mostrecentdate)  
        print('\n\nPerformance Review for Today: Done\n\n')
    else:
        print('\n\nPerformance Review for today has already executed.\n\n')
    
    
if __name__ == '__main__':
    print('\n', '\033[1m' + 'The performance review gets ready.' + '\033[0m', '\n')

# Engine

## Engine #1

In [24]:
def macrosnp(): 
    global proba_initial_value
    global predict_proba_value
    
    # check S&P 500
    snp_dropped_percentage = analysis_price('^GSPC')[2]
    snp_dropped_limit = sum(check_snp_t_y)
    snp_surged_limit = -sum(check_snp_t_y)/2
    
    if snp_dropped_percentage > snp_surged_limit and proba_initial_value != proba_initial_value_original:
        # proba value reset by market trend
        proba_initial_value = proba_initial_value_original
        print(f'\n\n\nImportant >>> S&P 500 has surged by {np.round(snp_dropped_percentage, 2)} %, Initial Proba Value resets at {proba_initial_value_original}\n\n\n')
    else:
        pass

    if snp_dropped_percentage < snp_dropped_limit:
        print('\033[1m' + '\nS&P 500 change(%) is less than the limit.\n\n\n' + '\033[0m')
        
        # proba value reset by market trend   
        proba_initial_value = proba_initial_value + 0.02
        print(f'\n\n\nImportant >>> S&P 500 has dropped by {np.round(snp_dropped_percentage, 2)} %, Initial Proba Value increases by 2 %\n\n\n')
        
        print(f'\n\n{current_time()} - Market Sentiment Reflection: Proba Value adjusting\n\n')
        proba_value_reset()
        
        # decrease the monitoring threshold 
        check_snp_t_y.append(-0.3)
        snp_dropped_limit = sum(check_snp_t_y)
        snp_dropped_percentage = analysis_price('^GSPC')[2]
        print(f'\n\n\nS&P 500 is at {np.round(snp_dropped_percentage, 2)} % | New Monitoring Limit: {np.round(snp_dropped_limit, 2)} %')
        
        # email
        message = """\
Subject: [{}] dropped: {}%""".format('S&P 500', '%.2f' %snp_dropped_percentage)
        voice_message(message)
#         emailsend_to_server(message)
        
    else:
        print('\nS&P 500 change(%) is higher than the limit.\n\n\n')     

        # increase the monitoring threshold
        if sum(check_snp_t_y) < sum(check_snp_t_y_original):
            check_snp_t_y.append(0.3)
            snp_dropped_limit = sum(check_snp_t_y)
        
        # limit the max. value when reached the limit
        if snp_dropped_limit > sum(check_snp_t_y_original):
            snp_dropped_limit = sum(check_snp_t_y_original)
        
        print(f'\n\n\nS&P 500 is at {np.round(snp_dropped_percentage, 2)} % | New Monitoring Limit: {np.round(snp_dropped_limit, 2)} %')
      
        pass


def sectorcheck_main(val_times):      
    voice_message("""\
        Sector valuation starts""")
    
# Macro: S&P500 and Sectors    
# Sector SPDR ETF: https://seekingalpha.com/etfs-and-funds/etf-tables/sectors

    # read S&P 500 to get 'GICS Sector' column    
    snp_sector = pd.read_csv('./snp500/snp500.csv')
    sector = snp_sector[['Symbol', 'GICS Sector']]

    # read 'portfolio.txt' to check each sector' from snp    
    with open("portfolio_original.txt", 'r') as f:
        portfolio_original = [line.rstrip('\n') for line in f]
        
    with open("portfolio_superpass.txt", 'r') as f:
        portfolio_superpass = [line.rstrip('\n') for line in f]
    
    portfolio_original = [i for i in portfolio_original if i not in inv_tickers and i not in portfolio_superpass]
    
    
    # Sector SPDR ETF: https://seekingalpha.com/etfs-and-funds/etf-tables/sectors   
    
    df = pd.DataFrame()
    df['Symbol'] = portfolio_original
    df['Date'] = currentdate
    df['Time'] = timechecknow()
    df = pd.merge(df, sector, how = 'inner', on = 'Symbol')
    df.loc[df['GICS Sector'] == 'Energy', 'Ticker'] =                'XLE'
    df.loc[df['GICS Sector'] == 'Financials', 'Ticker'] =            'XLF'
    df.loc[df['GICS Sector'] == 'Real Estate', 'Ticker'] =           'XLRE'
    df.loc[df['GICS Sector'] == 'Materials', 'Ticker'] =             'XLB'
    df.loc[df['GICS Sector'] == 'Information Technology', 'Ticker'] ='XLK'
    df.loc[df['GICS Sector'] == 'Consumer Staples', 'Ticker'] =      'XLP'
    df.loc[df['GICS Sector'] == 'Health Care', 'Ticker'] =           'XLV'
    df.loc[df['GICS Sector'] == 'Industrials', 'Ticker'] =           'XLI'
    df.loc[df['GICS Sector'] == 'Consumer Discretionary', 'Ticker'] ='XLY'
    df.loc[df['GICS Sector'] == 'Utilities', 'Ticker'] =             'XLU'
    df.loc[df['GICS Sector'] == 'Communication Services', 'Ticker'] ='XLC'
    
    df = df.sort_values('Ticker').reset_index(drop=True)
    print('\n\n\n', '\033[1m' + f">>> Sector Price Trend Valuation ....." + '\033[0m', '\n')
    valuation_mean = []
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(df.Ticker))):
        analysis = analysis_price(df.Ticker[i])
        try:
            print('Sector valuating: ', df['GICS Sector'][i], end = '                                                               \r')
            if i == 0:
                valuation_mean.append(analysis[7] or analysis[10] or analysis[11] or analysis[12])
            elif df.Ticker[i] == df.Ticker[i-1]:
                valuation_mean.append(0.3333333)
            else:
                valuation_mean.append(analysis[7] or analysis[10] or analysis[11] or analysis[12])
        except:
            valuation_mean.append(analysis[7] or analysis[10] or analysis[11] or analysis[12])
            print('Re-excuted >>> Sector check - moving average: ', df.Ticker[i])
            pass
            
    df['valuation_mean'] = valuation_mean        
    for i in range(len(df['valuation_mean'])):
        if df['valuation_mean'][i] == 0.3333333:
            df['valuation_mean'][i] = df['valuation_mean'][i-1]  
        else:
            pass
              
    print('\n\n\n', '\033[1m' + f">>> Sector vs S&P 500 - {averageline} days average price comparison ....." + '\033[0m', '\n')
    valuation_snp = []
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(df.Ticker))):
        try:
            print('Sector valuating: ', df['GICS Sector'][i], end = '                                                               \r')
            if i == 0:
                valuation_snp.append(analysis_price('^GSPC')[5] < analysis_price(df.Ticker[i])[5])
            elif df.Ticker[i] == df.Ticker[i-1]:
                valuation_snp.append(0.3333333)
            else:
                valuation_snp.append(analysis_price('^GSPC')[5] < analysis_price(df.Ticker[i])[5]) 
        except:
            valuation_snp.append(analysis_price('^GSPC')[5] < analysis_price(df.Ticker[i])[5])
            print(f">>> Re-excuted: Sector {averageline} vs {averageline-50} days S&P 500 change(%) comparison")
            pass
            
    df['valuation_snp'] = valuation_snp        
    for i in range(len(df['valuation_snp'])):
        if df['valuation_snp'][i] == 0.3333333:
            df['valuation_snp'][i] = df['valuation_snp'][i-1]  
        else:
            pass
    
    vc = pd.read_csv('sector_valuation.csv')
    for valuecheck_count in range(0, val_times):
        time.sleep(1)
        print('\n\n\n', '\033[1m' + f'>>> Sector R-squared valuation ..... {valuecheck_count+1}th' + '\033[0m')        
        valuation_rsquared = []
        from tqdm.notebook import tqdm
        for i in tqdm(range(len(df.Ticker))):
            try:
                time.sleep(1)
                print('Sector valuating: ', df['GICS Sector'][i], end = '                                                               \r')
                filter_mask = (vc['Date'] == currentdate) & (vc['Ticker'] == df.Ticker[i])
                if vc[filter_mask].Time.nunique() >= 3:
                    valuation_rsquared.append(vc[filter_mask].groupby('Time').mean().reset_index().valuation_rsq.max())
                else:
                    if i == 0:
                        valuation_rsquared.append(valuecheck(df.Ticker[i])[1])    
                    elif df.Ticker[i] == df.Ticker[i-1]:
                        valuation_rsquared.append(0.3333333)  #same ticker as above
                    else:
                        valuation_rsquared.append(valuecheck(df.Ticker[i])[1])
            except: 
                valuation_rsquared.append(valuecheck(df.Ticker[i])[1])
                print('Re-excuted >>> Sector check - R-squared: ', df.Ticker[i])
                pass
            
        for i in valuation_rsquared:
            if type(i) != float and type(i) != int:
                i = 0.3333333
            else:
                pass
        df['valuation_rsq'] = valuation_rsquared

        for i in range(len(df['valuation_rsq'])):
            try:
                df = df.fillna(0)      
                if df['valuation_rsq'][i] == 0.3333333:
                    df['valuation_rsq'][i] = df['valuation_rsq'][i-1]  
                else:
                    pass
            except KeyError:
                sectorcheck_main(1)
                pass
            
        df['Time'] = timechecknow()
        df = df.sort_values(['valuation_mean', 'valuation_snp', 'valuation_rsq'], ascending = False)
        df = df[['Date', 'Time', 'Symbol', 'GICS Sector', 'Ticker', 'valuation_mean', 'valuation_snp', 'valuation_rsq']]

        df_accu = pd.read_csv('sector_valuation.csv')
        df_merged = pd.concat([df_accu, df])
        df_merged.to_csv('sector_valuation.csv', index = False)    
    #     df.to_csv('sector_valuation.csv', index = False)  # for initial draft

#         print(df.tail(df['Symbol'].nunique())[['Symbol', 'valuation_mean', 'valuation_snp', 'valuation_rsq']].reset_index(drop=True))
    
    portfolio_original.extend(inv_tickers)

    
def predict(target, ticker):    
    target = None
    
    if ticker == 'GLD':
        target = 'GLD'
    elif ticker == 'USO':
        target = 'USO'       
    elif ticker == 'BTC-USD':
        target = 'BTC-USD'
    else:
        target = '^GSPC'    

    # target to get corr
    target_index = yfinance_df(target)  
    target_index = target_index[['Date', 'Adj Close']]
    # ticker to get corr
    ticker_index = yfinance_df(ticker)   
    # corr
    cor = pd.merge(target_index, ticker_index, how = 'inner', on = 'Date')

    import statsmodels.api as sm
    cor['intercept'] = 1
    lm = sm.OLS(cor['Adj Close_y'], cor[['intercept', 'Adj Close_x']])
    results = lm.fit()
#     compensation_cor = results.rsquared * results.params[1]    
#     compensation_cor = results.params[1] * np.absolute(cor.corr()['Adj Close_x'][1])
    compensation_cor = np.absolute(cor.corr()['Adj Close_x'][1])
    
# check if exist for today
    # if exists, only update the current rows
    # remove past data
    sourcedata_exists = exists(f"predict_aim_sourcedata_monitoring_{ticker}_{mostrecentdate}.csv")
    if sourcedata_exists == True:
        import os
        os.remove(f"predict_aim_sourcedata_monitoring_{ticker}_{mostrecentdate}.csv")
    else:
        pass
    
    sourcedata_exists = exists(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv")
    if target == '^GSPC' and sourcedata_exists == True:
        print(f"File Exists - Target: {target} & Ticker: {ticker} ..... Updating the current values", end = '                       \r')
        df = pd.read_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv")
        realtime_vars = {'Int_rate':'^TNX', 
                         'Stock_price':ticker, 
                         'Cboe':'^VIX', 
                         'Gold':'GC=F', 
                         'Oil':'CL=F', 
                         'Bitcoin':'BTC-USD', 
                         'Dollar':'DX-Y.NYB'}
        for i, v in realtime_vars.items():
            df.loc[df.shape[0]-1, [i]] = yfinance_df_min(v)['Adj Close'].values[0]
            
    elif target == 'GLD' and sourcedata_exists == True:
        print(f"File Exists - Target: {target} & Ticker: {ticker} ..... Updating the current values", end = '                       \r')
        df = pd.read_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv")
        realtime_vars = {'Int_rate':'^TNX', 
                         'Stock_price':ticker, 
                         'Cboe':'^VIX', 
                         'snp':'^GSPC', 
                         'Oil':'CL=F', 
                         'Bitcoin':'BTC-USD', 
                         'Dollar':'DX-Y.NYB'}
        for i, v in realtime_vars.items():
            df.loc[df.shape[0]-1, [i]] = yfinance_df_min(v)['Adj Close'].values[0]
    
    elif target == 'USO' and sourcedata_exists == True:
        print(f"File Exists - Target: {target} & Ticker: {ticker} ..... Updating the current values", end = '                       \r')
        df = pd.read_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv")
        realtime_vars = {'Int_rate':'^TNX', 
                         'Stock_price':ticker, 
                         'Cboe':'^VIX', 
                         'snp':'^GSPC', 
                         'Oil':'CL=F', 
                         'Bitcoin':'BTC-USD', 
                         'Dollar':'DX-Y.NYB'}
        for i, v in realtime_vars.items():
            df.loc[df.shape[0]-1, [i]] = yfinance_df_min(v)['Adj Close'].values[0]
            
    elif target == 'BTC-USD' and sourcedata_exists == True:
        print(f"File Exists - Target: {target} & Ticker: {ticker} ..... Updating the current values", end = '                       \r')
        df = pd.read_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv")
        realtime_vars = {'Int_rate':'^TNX', 
                         'Stock_price':ticker, 
                         'Cboe':'^VIX', 
                         'snp':'^GSPC', 
                         'Oil':'CL=F', 
                         'Gold':'GC=F', 
                         'Dollar':'DX-Y.NYB'}
        for i, v in realtime_vars.items():
            df.loc[df.shape[0]-1, [i]] = yfinance_df_min(v)['Adj Close'].values[0]
            
    else:
        print(f"Initial downloading - Target: {target} & Ticker: {ticker}", end = '                                                        \r')
    # if not exists, gather data from the beginning
    # 10 year treasery bold yield
        rate = yfinance_df('^TNX')  
        irate = rate[['Date', 'Adj Close']]

        # CBOE Volatility Index
        cboe = yfinance_df('^VIX')
        cboe = cboe[['Date', 'Adj Close']]

        if target == '^GSPC':
            # Gold Feb 22 (GC=F)
            gold = yfinance_df('GC=F')  
            gold = gold[['Date', 'Adj Close']]    

        elif target == 'GLD' or target == 'USO' or target == 'BTC-USD':
            # S&P 500
            snp = yfinance_df('^GSPC')
            snp = snp[['Date', 'Adj Close']]
        else:
            pass

        # Crude Oil
        mar = yfinance_df('CL=F')  
        mar = mar[['Date', 'Adj Close']]

        if target == '^GSPC' or target == 'GLD' or target == 'USO':
            # Bitcoin (*last)
            bit = yfinance_df('BTC-USD')   
            bit = bit[['Date', 'Adj Close']]
        elif target == 'BTC-USD':
            # Gold
            gold = yfinance_df('GC=F')   
            gold = gold[['Date', 'Adj Close']]

        else:
            pass

        # Dollar Index
        dollar = yfinance_df('DX-Y.NYB')
        dollar = dollar[['Date', 'Adj Close']]

        # cpi index
        url = 'https://fred.stlouisfed.org/data/CPIAUCSL.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        cpi = webContent[4075:]
        cpi = cpi.split()
        date = []
        number = []
        for i in range(len(cpi)):
            if i%2 == 0:
                date.append(cpi[i])
            else:
                number.append(cpi[i])
        cpidx = pd.DataFrame({'Date': date, 'Cpi': number})
        cpidx['Date'] = cpidx['Date'].astype('datetime64[ns]')
        cpidx['Cpi'] = pd.to_numeric(cpidx['Cpi'])

        # jobless claim index
        url = 'https://fred.stlouisfed.org/data/ICSA.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        joblessclaim = webContent[710:]
        joblessclaim = joblessclaim.split()
        date = []
        number = []
        for i in range(len(joblessclaim)):
            if i%2 == 0:
                date.append(joblessclaim[i])
            else:
                number.append(joblessclaim[i])
        jls = pd.DataFrame({'Date': date, 'Jclaim': number})
        
        # fix error from 'https://fred.stlouisfed.org/data/ICSA.txt'
        jls.Jclaim.replace('.', np.NaN, inplace = True)
        jls.fillna(method = 'ffill', inplace = True)
        jls = jls[jls['Date'] <= currentdate]
        
        jls['Date'] = jls['Date'].astype('datetime64[ns]')
        jls['Jclaim'] = pd.to_numeric(jls['Jclaim'])
        jls = jls[jls['Date'] > '1970-1-1'].reset_index(drop=True)
        cpidx = cpidx[cpidx['Date'] > '1969-12-1'].reset_index(drop=True)

        # inflation index
        url = 'https://fred.stlouisfed.org/data/T10YIE.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        inf = webContent[1222:]
        inf = inf.split()
        date = []
        number = []
        for i in range(len(inf)):
            if i%2 == 0:
                date.append(inf[i])
            else:
                number.append(inf[i])
        inf = pd.DataFrame({'Date': date, 'Inf': number})
        inf.Inf.replace('.', np.NaN, inplace = True)
        inf.fillna(method = 'ffill', inplace = True)
        inf['Date'] = inf['Date'].astype('datetime64[ns]')
        inf['Inf'] = pd.to_numeric(inf['Inf'])

        time.sleep(0.5)

        # combine & cleaning
        df = cpidx.merge(jls, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.merge(irate, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.merge(ticker_index, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close_x':'Int_rate', 'Adj Close_y':'Stock_price'}).reset_index(drop=True)
        df = df.fillna(0)
        df = df[df['Date'] >= '1970-01-01'].reset_index(drop=True) # since Jclaim

        # cleaning - cpi and Jclaim 
        for i in range(len(df.Cpi)):
            if df.Cpi[i] == 0:
                df.Cpi[i] = df.Cpi[i-1]    
        df = df[df['Date'] > '1970-01-02'].reset_index(drop=True)
        for i in range(len(df.Jclaim)):
            if df.Jclaim[i] == 0.0:
                df.Jclaim[i] = df.Jclaim[i-1]
#         drp = df[df['Int_rate'] == 0].index.values
#         df = df.drop(index=drp)
        df['Int_rate'] = df['Int_rate'].replace(to_replace=0, method='ffill')

        # cleaning - inflation rate since 1990
        df = df.merge(inf, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df[df['Date'] > '2003-01-01']
        df = df.fillna(0)
        if df.tail(1).Inf.values[0] == 0:
            df.tail(1).Inf = df.tail(2).Inf.values[0]

        # merge - cboe     
        df = df.merge(cboe, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close':'Cboe'}).reset_index(drop=True)
        df = df.fillna(0)

        if target == '^GSPC':  
            # merge - gold
            df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
            df = df.fillna(0)

        elif target == 'GLD' or target == 'USO' or target == 'BTC-USD':  
            # merge - snp
            df = df.merge(snp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'snp'}).reset_index(drop=True)
            df = df.fillna(0)

        else:
            pass

        # merge - Crude Oil Mar 22 
        df = df.merge(mar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close':'Oil'}).reset_index(drop=True)

        if target == 'BTC-USD':
            # merge - gold
            df = df.merge(gold, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'Gold'}).reset_index(drop=True)
            df = df.fillna(0)
        elif target == '^GSPC' or target == 'GLD' or target == 'USO':
            # merge - bitcoin since 2014 (test rsquared for both w aad w/t this variable)
            df = df.merge(bit, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
            df = df.rename(columns = {'Adj Close':'Bitcoin'}).reset_index(drop=True)
        else:
            pass

        # merge - Dollar Index 
        df = df.merge(dollar, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date')
        df = df.rename(columns = {'Adj Close':'Dollar'}).reset_index(drop=True)

        # dropna and save the source date
        # fill ZERO value with the previous value
        for i in df.all().reset_index().values:
            if i[1] == False:
                df[i[0]] = df[i[0]].replace(to_replace=0, method='ffill')
        df = df.dropna()

        # fillavg = df[df['Inf'] == 0].index.values
        # df['Inf'].loc[fillavg] = df['Inf'].mean()   # Don't fill zero rows as average value

        dfdrop = df[df['Stock_price'] == 0].index.values
        df = df.drop(index = dfdrop)   

        # dfdrop = df[df['Inf'] == 0].index.values
        # df = df.drop(index = dfdrop)

        fillavg = df[df['Inf'] == 0].index.values
        df['Inf'].loc[fillavg] = df['Inf'].tail(5).max()             

        # Fillna Cboe with the highest value for the past 20 working days. 
        cboe_fillna = df[df['Cboe'] == 0].index.values
        df['Cboe'].loc[cboe_fillna] = df.tail(20)['Cboe'].max() 
        
        # Real GDP
        url = 'https://fred.stlouisfed.org/data/GDPC1.txt'
        response = urllib.request.urlopen(url)
        webContent = response.read().decode('UTF-8')
        Rgdp = webContent[1005:]
        Rgdp = Rgdp.split()
        date = []
        number = []
        for i in range(len(Rgdp)):
            if i%2 == 0:
                date.append(Rgdp[i])
            else:
                number.append(Rgdp[i])
        Rgdp = pd.DataFrame({'Date': date, 'Rgdp': number})
        Rgdp['Date'] = Rgdp['Date'].astype('datetime64[ns]')
        Rgdp['Rgdp'] = pd.to_numeric(Rgdp['Rgdp'])

        df = df.merge(Rgdp, how = 'outer', left_on = 'Date', right_on = 'Date').sort_values('Date').reset_index(drop = True)
        df.Rgdp.replace(np.NaN, 0, inplace = True)
        for i in range(len(df.Rgdp)):
            if df.Rgdp[i] == 0:
                df.Rgdp[i] = df.Rgdp[i-1]
        # fill ZERO value with the previous value
        for i in df.all().reset_index().values:
            if i[1] == False:
                df[i[0]] = df[i[0]].replace(to_replace=0, method='ffill')
        df.dropna(inplace = True)       
        
        if target == '^GSPC':
            dfdrop = df[df['Gold'] == 0].index.values
            df = df.drop(index=dfdrop)

        elif target == 'BTC-USD':
            dfdrop = df[df['Gold'] == 0].index.values
            df = df.drop(index=dfdrop)
            dfdrop = df[df['snp'] == 0].index.values
            df = df.drop(index=dfdrop)

        elif target == 'GLD' or target == 'USO':
            dfdrop = df[df['snp'] == 0].index.values
            df = df.drop(index=dfdrop)

        else:
            pass            

        dfdrop = df[df['Int_rate'] == 0].index.values
        df = df.drop(index=dfdrop)
        df = df.drop(columns = ['Close'])
        ### end of new gathering ####

    # data verification: zero values
    if df.all().sum() != len(df.columns):
        print(f'\nZero value checked - Filling with Mean() ===> Ticker: {ticker}\n')

        fill_zero_mean = df.all().reset_index().rename(columns = {0:'zero_var'})
        zero_var_num = fill_zero_mean[fill_zero_mean['zero_var'] == False]['index'].count() 

        for i in range(zero_var_num):
            zero_col = fill_zero_mean[fill_zero_mean['zero_var'] == False]['index'].values[i]
            df[zero_col] = df[zero_col].replace(0, df[zero_col].mean()) # fill zero with average

        if df.all().sum() == len(df.columns):
            print('> Successful: Non-Zero values')
            
            last_date = str(df[-1:].Date.values[0])[:10]
            if last_date == currentdate or last_date == mostrecentdate:
                df.to_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv", index = False)
            else:
                df.to_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv", index = False)
                print('\033[1m'+"\n\n\nThe last line of the dateframe does not match with the currentdate.\n\nPrediction returns >>> Zero <<<\n\n"+ f'Most recent date: {last_date}\n\n\n'+'\033[0m')
            
        else:
            df.to_csv('errorlog_prediction_monitoring_%s.csv' %ticker, index = False)
            print('> Unsuccessful - Check error log file: ', ticker, '\n') 

    else:
        last_date = str(df[-1:].Date.values[0])[:10]
        if last_date == currentdate or last_date == mostrecentdate:
            df.to_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv", index = False)
        else:
            df.to_csv(f"predict_aim_sourcedata_monitoring_{ticker}_{currentdate}.csv", index = False)
            print('\033[1m'+"\n\n\nThe last line of the dateframe does not match with the currentdate.\n\nPrediction returns >>> Zero <<<\n\n"+ f'Most recent date: {last_date}\n\n\n'+'\033[0m')
    ################################################################################### new merge - add above
     
    with open("rday_oil.txt", 'r') as f:
        rday_oil = [line.rstrip('\n') for line in f][0]
    with open("rday_gold.txt", 'r') as f:
        rday_gold = [line.rstrip('\n') for line in f][0]
    with open("rday_bit.txt", 'r') as f:
        rday_bit = [line.rstrip('\n') for line in f][0]
    with open("rday_snp.txt", 'r') as f:
        rday_snp = [line.rstrip('\n') for line in f][0]

    # range selection - COVID19
    if target == '^GSPC':
        dfc = df[df['Date'] > rday_snp]
    elif target == 'GLD':
        dfc = df[df['Date'] > rday_gold]
    elif target == 'USO':
        dfc = df[df['Date'] > rday_oil]
    elif target == 'BTC-USD':
        dfc = df[df['Date'] > rday_bit]
    else:
        pass
    df_classifier = dfc.copy()

    # Regression analysis
    import statsmodels.api as sm
    dfc['intercept'] = 1

    if target == '^GSPC':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'Gold', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']]) 
    elif target == 'GLD':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'snp', 'Oil', 'Bitcoin', 'Dollar', 'Rgdp']])
    elif target == 'BTC-USD':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Oil', 'Dollar', 'Gold', 'Rgdp']])
    elif target == 'USO':
        lm = sm.OLS(dfc['Stock_price'], dfc[['intercept', 'Cpi', 'Jclaim', 'Int_rate', 'Inf', 'Cboe', 'snp', 'Bitcoin', 'Dollar', 'Rgdp']])
    else:
        pass

    results = lm.fit()
#     print(f'\n\n• Value Check: {ticker}\n')
#     print(results.summary())

    # linear regression
    if target == '^GSPC':
        dfl = dfc.drop(columns = ['intercept'])
    elif target == 'GLD':
        dfl = dfc.drop(columns = ['intercept', 'Cboe'])      
    elif target == 'USO':
        dfl = dfc.drop(columns = ['intercept', 'Oil']) 
    elif target == 'BTC-USD':
        dfl = dfc.drop(columns = ['intercept'])
    else:
        pass

    prediction = pred_lstm(dfl) 
    err = (prediction - dfc.tail(1).Stock_price.values[0])/dfc.tail(1).Stock_price.values[0]
    tprice = yfinance_df(ticker).tail(1)['Adj Close'].values[0]

    # classifier
    prediction_classifier = classifer_data_input(df_classifier, moving_avg_value) # [0]: precision / [1]: prediction result > pass df w/t r-day considered

    # check the most recent date if yFinance includes it    
    lastdate = str(df[-1:].Date.values[0])[:10]
    if lastdate == currentdate or lastdate == mostrecentdate:
        prediction_up_down = prediction_classifier[1]
    else:
        print(f'\n\nExcept >>> Last date error - Most recent date: {lastdate}\n\n')
        prediction_up_down = 0    
    
    return [today_to_add, ticker, prediction, tprice, results.rsquared, err, compensation_cor, prediction_up_down]
    
    
def valuecheck(stock):
    print(end = '                                                                                    \r')
    print('\n'+current_time()+'\n')
    print('\033[1m' + "Ticker Valuation\n" + '\033[0m')
    portfolio_sector_true = [stock]
    
    pricepredict = {}
    date = []
    symbol = []
    prediction = []
    price = []
    rsquared = []
    error = []
    compensation = []
    up_down = []
    target = None

    for i in portfolio_sector_true:
        try:
            result = predict(target, i)
            date.append(result[0])
            symbol.append(result[1])
            prediction.append(result[2])
            price.append(result[3])
            rsquared.append(result[4])
            error.append(result[5])
            compensation.append(result[6])
            up_down.append(result[7])            
        except KeyError:                    
            print('Except: Ticker valuation: ', portfolio_sector_true[i])
            alarm()

            # email
            message = """\
Subject: Ticker valuation error: {}""".format(portfolio_sector_true[i])                               
            context = ssl.create_default_context()
            with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
                server.login(sender_email, password)
                server.sendmail(sender_email, receiver_email, message)                 

            # re-try    
            print('Ticker re-valuation: ', portfolio_sector_true[i])
            result = predict(target, portfolio_sector_true[i])
            date.append(result[0])
            symbol.append(result[1])
            prediction.append(result[2])
            price.append(result[3])
            rsquared.append(result[4])
            error.append(result[5])
            compensation.append(result[6])  
            up_down.append(result[7])  
            print('Re-valuation success: ', portfolio_sector_true[i])
            alarm()

            # email
            message = """\
Subject: Ticker re-valuation success: {}""".format(portfolio_sector_true[i])                               
            context = ssl.create_default_context()
            with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
                server.login(sender_email, password)
                server.sendmail(sender_email, receiver_email, message) 
            pass

    pricepredict['date'] = date
    pricepredict['time'] = timechecknow()
    pricepredict['symbol'] = symbol
    pricepredict['prediction'] = prediction
    pricepredict['price'] = price
    pricepredict['rsquared'] = rsquared
    pricepredict['error'] = np.absolute(error)
    pricepredict['compensation_corr'] = compensation
    pricepredict['up_down'] = up_down

    # to send today_predict
    df = pd.DataFrame(pricepredict)  
    df = df.fillna(0)

    # apply accmulated gap to compensate 'gap'
    dfacc = pd.read_csv('predict_valuation.csv').sort_values('symbol')
    dfacc = dfacc.dropna()

    average_gap = []
    for i in df.symbol:
        if i in pd.DataFrame(dfacc.groupby('symbol').count()).reset_index().symbol.values:
            average_gap.append(dfacc[dfacc['symbol'] == i].gap.min())
        else:
            average_gap.append(np.absolute(df[df['symbol'] == i].prediction.values[0] - df[df['symbol'] == i].price.values[0]))

    df['gap'] = average_gap

    # compensate - original predicted price is compensated by multipying coefficient correlation
    #              this also partially compensates rsquared per ticker since corr*corr becomes rsquared
    # again, df['gap'] here is calculated from accmulated data, 'predict', and save the result to 'predict_valuation_today'
    
# Compensation    
    for i in range(len(df['symbol'])):
        if df['symbol'][i] == 'GLD' or df['symbol'][i] == 'USO' or df['symbol'][i] == 'BTC-USD':
            pass
        else:
            if df['prediction'][i] < df['price'][i]:
                df['prediction'][i] = df['prediction'][i]# + (df['gap'][i] * df['compensation_corr'][i])
            else:
                df['prediction'][i] = df['prediction'][i]# - (df['gap'][i] * df['compensation_corr'][i])
# Compensation  

    # final valuation, and save the result to 'predict_valuation_today'
    df.loc[df['prediction'] - df['price'] > 0, 'valuation'] = 1
    df.loc[df['prediction'] - df['price'] <= 0, 'valuation'] = 0
    df['gap'] = np.absolute(df['prediction'] - df['price'])
    df['error'] = np.absolute(df['prediction'] - df['price']) / df['price']

    today = df.sort_values('error').reset_index(drop=True)
    today.to_csv('predict_valuation_today.csv', index = False)

    # update 'predict_valuation_today' into 'predict'
    df = pd.read_csv('predict_valuation.csv')
    df = pd.concat([df, today])
    df.to_csv('predict_valuation.csv', index = False)
    df.to_csv('./Backup_predict_valuation/predict_valuation %s.csv' %currentdate, index = False)
    
    df = pd.read_csv('predict_valuation.csv').sort_values('symbol')
    predict_valuation_today = pd.read_csv('predict_valuation_today.csv').sort_values(['up_down', 'valuation', 'error'], ascending = False)
#     print(predict_valuation_today[['symbol', 'valuation', 'error']].sort_values(['valuation', 'error'], ascending = False))

    # average error check if zero
    if predict_valuation_today.price.min() == 0:
        print('!!! Null error !!!')
        alarm()

        # email
        message = """\
Subject: Average error zero. Check file. today predict!"""
        emailsend_to_server(message)
        
        drp = predict_valuation_today[predict_valuation_today.price ==0].index
        predict_valuation_today.drop(index = drp, inplace = True)   
    else: 
        pass
    
    df = pd.read_csv('predict_valuation.csv')
    df = df[df.date == currentdate]
    
    return [df[df['symbol'] == stock].tail(3)['valuation'].mean(), df[df['symbol'] == stock].tail(3)['up_down'].mean()] 


if __name__ == '__main__':
    print('\n', '\033[1m' + 'The Engine#1 gets ready.' + '\033[0m', '\n')

## Engine #2

In [25]:
def robinExecution_sell(i, q):
    global sellmonitor_freq
    global num_of_purchased
    global day_trading_count
    
    # only execute during the market hours
    if timechecknow() <= 390:
        try:
            # execue sell order
            login = rs.login('aicpasean@gmail.com', sean.robin_api())
            order_sell_market(i, q)
            rs.logout()
            print('\n'+current_time()+' - The selling order has been executed.\n')
            time.sleep(10)

            # sold list update
            trade_list('sold', i)
            # purchased list update
            trade_list('purchased', i)            
            print('\n'+current_time()+' - Trade_List has been updated.\n')

            # update performance
            sss_perf(i)
            print('\n'+current_time()+' - SSS_Perf has been updated - sss_perf.csv\n')
            
            # count the existing number of stocks for holding period count
            onhold_df = robin_login()
            if i in onhold_df['index'].tolist():
                num_of_stocks_hold = int(float(onhold_df[onhold_df['index'] == i]['quantity'].values[0]))
                print(f'• Stock: {i} / {num_of_stocks_hold} shares on hold\n\n\n')
            else:
                num_of_stocks_hold = 0
                print(f'• Stock: {i} / {num_of_stocks_hold} shares on hold\n\n\n')

                # remove tickers having no balance from the holding period DataFrame
                holding_period = pd.read_csv('holding_period.csv').set_index('index')
                if i in holding_period.index.tolist():
                    print('\n\n\nTicker exists in the holding period list: ' + '\033[1m' + f'{i}' + '\033[0m')
                    holding_period = holding_period.drop(index = i)
                    holding_period.to_csv('holding_period.csv')
                    print('Ticker has been removed from the list: ' + '\033[1m' + f'{i}\n\n\n' + '\033[0m')
                    print('• Updated Hoding Period\n\n' + '\033[1m' + f'{holding_period}\n\n\n' + '\033[0m')
                else:
                    print('\n\n\nTicker does NOT exist in the holding period list: ' + '\033[1m' + f'{i}\n\n\n' + '\033[0m')
                    
        except:
            message = 'The selling transaction had one or more than one errors.'
            print('Except >>> ' + message)
            alarm()
            voice_message(message)
    else:
        message = 'The market has been closed. The sell order has not been executed.'
        print('Except >>> ' + message)
        alarm()
        voice_message(message)
  
    # sell monitor cycle update
    num_of_purchased = len(onhold_df['index'])
    sellmonitor_freq = int((sellmonitor_freq_limit) - (num_of_purchased - len(dead_stock))) ### adjust 
    print('\n\nSell Monitor Freq has been updated as every' + '\033[1m' + f' {sellmonitor_freq} ' + '\033[0m' + 'cycles.\n\n')
    
    
def sss_perf(i):
    global day_trading_count
    
    df = pd.read_csv('robin_all.csv')
    percent_change = df[df['index'] == i].percent_change.values[0]
    equity_change = df[df['index'] == i].equity_change.values[0]
    holding_period = pd.read_csv('holding_period.csv')
    try:
        holding_period = int(holding_period[holding_period['index'] == i].days.values[0])
    except:
        day_trading_count+=1
        print('\033[1m' + f'\n\n\n{i} was purchased today. Updated Day-trading count: {day_trading_count}\n\n\n' + '\033[0m')
        holding_period = 0

    sss_perf = {'date' : date.today(),
                'symbol' : i,
                'percent_change': percent_change, 
                'equity_change' : equity_change, 
                'holding_period' : holding_period}
    sss_perf = pd.DataFrame([sss_perf])

    ss_perf = pd.read_csv('sss_perf.csv')

    sss_perf = pd.concat([ss_perf, sss_perf], axis = 0)
    sss_perf['date'] = sss_perf['date'].astype('datetime64[ns]')
    sss_perf.sort_values('date', ascending = False, inplace = True)
    sss_perf.to_csv('sss_perf.csv', index = False)

    real_performance_selma()
    
    
def split_qty_finder(i, q):
    global split_condition_dollar
    split_condition_dollar_original = split_condition_dollar
    
    current_price = analysis_price(i)[4]
    current_value = current_price * q
    
    # reset split_condition_dollar at 1/3 of dollar_limit_order_dollar
    if (dollar_limit_order_dollar/3) > split_condition_dollar:
        split_condition_dollar = (dollar_limit_order_dollar/3)
        print('\033[1m' + f'\n\n\nAdjusted Split Condition Dollar at ${split_condition_dollar}\n\n\n' + '\033[0m')
    else:
        print('\033[1m' + f'\n\n\nSplit Condition Dollar at ${split_condition_dollar}\n\n\n' + '\033[0m')
    
    # get the number of tickers to split-sell 
    if current_value > split_condition_dollar:
        q = math.ceil((current_value/sale_split)/current_price)
        
    split_condition_dollar = split_condition_dollar_original
    return q   


def stg_loss_rate(ticker):
    global min_SuccessRate
    min_SuccessRate_original = min_SuccessRate
    min_SuccessRate = int(min_SuccessRate * 0.8)
    print('\033[1m' + f'\n==========[ Loss Cut Rate Searching by Strategy - Ticker: {ticker} ]==========' + '\033[0m')
    loss_rate = []
    for i in range(5):    
        stg_loss = strategy_analysis_main(ticker)
        if stg_loss[2] != 0:
            loss_rate.append(stg_loss[2])
        min_SuccessRate = int(min_SuccessRate * 1.1)
        if stg_loss[0] == -10:
            break
    if len(loss_rate) != 0:
        stg_loss_rate = max(loss_rate)
    else:
        stg_loss_rate = -5
    min_SuccessRate = min_SuccessRate_original
    return stg_loss_rate *100
    
    
def sellmonitor():
    with open(f'sold_{currentdate}.txt', 'r') as f:
        sold = [line.rstrip('\n') for line in f]
        
    with open(f'purchased_{currentdate}.txt', 'r') as f:
        purchased = [line.rstrip('\n') for line in f]
    
    # import sector valuation record
    sector_valuation_ticker = pd.read_csv('sector_valuation.csv')
    robin = robin_equity_check()
    robin = robin.loc[-robin['index'].isin(sold)] # exclude [Sold]
    robin.reset_index(drop = True, inplace = True)
    current_gl = pd.read_csv('robin_all.csv')
    
# Sell monitoring start
    # Sell threshold - all portfolio  
    print('\n\n' + '\033[1m' + '>>> 52-week high monitoring for all tickers .....' + '\033[0m', '\n')
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(robin['index'].values))):     
        
        if timechecknow() > 390:
            break
            
        try:
            time.sleep(0.3)
            print('52-weeks high checking: ', robin['index'][i], end = '                    \r')
            high_52_price = yfinance_df(robin['index'].values[i]).tail(260)['Adj Close'].max()
            current_price = yfinance_df(robin['index'].values[i]).tail(1)['Adj Close'].max()
            
            if current_price > high_52_price:
                # include into purchased list
                # purchased list update
                print('\n'+current_time()+'\n')
                purchased = trade_list('purchased', robin['index'].values[i])
                
#             # open the lost-cut list and add the ticker into the list
#                 with open("tickers_to_sell_small.txt", 'r') as f:
#                     tickers_to_sell_small = [line.rstrip('\n') for line in f]                       
#                 tickers_to_sell_small.append(robin['index'].values[i])
#             # open the long-term list and exclude the lost-cut list from the long-term list  
#                 with open("tickers_to_sell_big.txt", 'r') as f:
#                     tickers_to_sell_big = [line.rstrip('\n') for line in f]
#                 tickers_to_sell_small = [i for i in tickers_to_sell_small if i not in tickers_to_sell_big]
#             # save the updated loss-cut list        
#                 with open("tickers_to_sell_small.txt", 'w') as f:
#                     for s in tickers_to_sell_small:  
#                         f.write(str(s) + '\n') 
                
                # email only once if already included the purchased list
                # remove the following three lines if including the ticker into the loss-cut list
                with open(f'purchased_{currentdate}.txt', 'r') as f:
                    purchased = [line.rstrip('\n') for line in f]
                if robin['index'].values[i] in purchased == False:    
                    # alarm
                    webbrowser.open("Nuclear Launch Detected.mp3")
                    # email
                    message = """\
Subject: QED [{}] Alarm: ${} <= 52-week high! The ticker has been included into the purchased list.""".format(robin['index'].values[i], '%.2f' %high_52_price)
                    print(message)
                    voice_message(message)
                    emailsend_to_server(message)
            else:
                pass
        except:
            print('Except >>> 52-week high: ', robin['index'].values[i])
            alarm()
            pass
            
    print('\n')      
    
# Volume-speed Sell Alert
    print('\n' + '\033[1m' + '>>> Trend and Volume Drop monitoring for tickers (excl. Big) .....' + '\033[0m', '\n')
  
    from tqdm.notebook import tqdm
    tickers = robin['index'].tolist()
    # open the long-term list and exclude the lost-cut list from the long-term list  
    with open("tickers_to_sell_big.txt", 'r') as f:
        tickers_to_sell_big = [line.rstrip('\n') for line in f]
    tickers = [i for i in tickers if i not in tickers_to_sell_big]
    
    for i in tqdm(range(len(tickers))):    
    
        if timechecknow() > 390:
            break
            
        try:
            time.sleep(0.3)
            print('Candle Analysing Ticker: ', tickers[i], end = '                    \r')
            analysis = analysis_price(tickers[i])
            
            if analysis_price('^GSPC')[5] > analysis[5] and analysis[8] == True and analysis[6] == False:
                candle_result = volume_vs_sign(tickers[i])
                
                if candle_result == -3:
                # open the lost-cut list and add the ticker into the list
                    with open("tickers_to_sell_small.txt", 'r') as f:
                        tickers_to_sell_small = [line.rstrip('\n') for line in f]                       
                    tickers_to_sell_small.append(tickers[i])
                    tickers_to_sell_small = [i for i in tickers_to_sell_small if i not in tickers_to_sell_big]
                # save the updated loss-cut list        
                    with open("tickers_to_sell_small.txt", 'w') as f:
                        for s in tickers_to_sell_small:  
                            f.write(str(s) + '\n') 
                    
                    # alarm
                    webbrowser.open("Nuclear Launch Detected.mp3")
                    # email
                    message = """\
Subject: QED [{}] Sell Alert: Candle Analysis Sell. The ticker has been included into the loss-cut list.""".format(tickers[i])
                    print(message)
                    voice_message(message)
                    emailsend_to_server(message)
                else:
                    pass
            else:
                pass
        except:
            print('Except >>> Volume Drop Sell: ', tickers[i])
            alarm()
            pass
            
    print('\n')    
    
# Strategy analysis if having 'Sell' sign
    print('\n' + '\033[1m' + '>>> Strategy Analysis for all tickers (excl. Big) .....' + '\033[0m', '\n')
  
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(tickers))):  
        
        if timechecknow() > 390:
            break
            
        try:
            time.sleep(0.3)
            print('Strategy Analysis Ticker: ', tickers[i], end = '                    \r')
            if strategy_analysis_main(tickers[i], 'off', 'off')[0] + strategy_analysis_main(tickers[i], 'off', 'off')[0] == -2:
  
            # open the lost-cut list and add the ticker into the list
                with open("tickers_to_sell_small.txt", 'r') as f:
                    tickers_to_sell_small = [line.rstrip('\n') for line in f]                       
                tickers_to_sell_small.append(tickers[i])
                tickers_to_sell_small = [i for i in tickers_to_sell_small if i not in tickers_to_sell_big]
            # save the updated loss-cut list        
                with open("tickers_to_sell_small.txt", 'w') as f:
                    for s in tickers_to_sell_small:  
                        f.write(str(s) + '\n')      
                # alarm
                webbrowser.open("Nuclear Launch Detected.mp3")
                # email
                message = """\
Subject: QED [{}] Sell Alert: Strategy Analysis Sell. The ticker has been included into the loss-cut list.""".format(tickers[i])
                print(message)
                voice_message(message)
                emailsend_to_server(message)
            else:
                pass
        except:
            print('Except >>> Strategy Analysis Sell: ', tickers[i])
            alarm()
            pass
        
    print('\n')  

    
# Sell threshold - Selected portfolios to realize gain ####################################################################################
# Sell_big list update   
    with open("tickers_to_sell_big.txt", 'r') as f:
        tickers_to_sell_big = [line.rstrip('\n') for line in f]   
    with open("realgainrate_big.txt", 'r') as f:
        realgainrate_big = [line.rstrip('\n') for line in f]    
        realgainrate_big = np.round(float(realgainrate_big[0]), 2)

    dfb = robin.loc[robin['index'].isin(tickers_to_sell_big)]
    print('\n\n\n\n' + '\033[1m' + f">>> {realgainrate_big}" + '\033[0m' + '% Max. Sell Monitoring for Long-Term Sell: ' + '\033[1m' + format(dfb['index'].values) + '\033[0m', '\n')

    for i, c, q in zip(dfb['index'], dfb['percent_change'], dfb['quantity']):
        
        if timechecknow() > 390:
            break
            
        print('Analysing Ticker: ', i, end = '                    \r')
        try:
            realgainrate_big_adj = realgainrate_big*(1-(volatility_check(i) - volatility_check(sector_valuation_ticker[sector_valuation_ticker['Symbol'] == i]['Ticker'].values[0])))
         
            # apply event day risk ratio
            if i in ticker_eventday_check():
                realgainrate_big_adj*= event_risk_ratio    
                print('\n\n' + '\033[1m' + f' {i} ' + '\033[0m' + 'Event Risk Raio' + '\033[1m' + f' {event_risk_ratio*100} % ' + '\033[0m' + 'applied.')

            # safe the margin error
            if realgainrate_big_adj < 0:
                realgainrate_big_adj = 0.01
        
            print('\n\n' + '\033[1m' + f"• {i}: " + '\033[0m' + '\033[1m' + f"{'%.2f' %realgainrate_big_adj}% <--- {'%.2f' %current_gl[current_gl['index'] == i]['percent_change'].values[0]}%" + '\033[0m', '\n')

            if c >= realgainrate_big_adj:
                analysis_snp = analysis_price('^GSPC')
                analysis_sector = analysis_price(sector_valuation_ticker[sector_valuation_ticker['Symbol'] == i]['Ticker'].values[0])
                analysis_ticker = analysis_price(i)  
                strategy_analysis = strategy_analysis_main(i, 'off', 'off')[0] + strategy_analysis_main(i, 'off', 'off')[0]
            
                valuecheck(i)
                if strategy_analysis == -2 or analysis_snp[5] > analysis_sector[5] or (valuecheck(i)[0]+valuecheck(i)[1]*3) < valuation_conversion_value_sell:

                    print(f"""\
                    \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                    """)
                    voice_message(f"""\
                    Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                    """)
                    time.sleep(10)
                    robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>

                    tickers_to_sell_big.remove(i)
                    with open("tickers_to_sell_big.txt", 'w') as f:
                        for s in tickers_to_sell_big:  
                            f.write(str(s) + '\n')    

                    # alarm
                    webbrowser.open("soldsound.mp3")

                    # email
                    message = """\
Subject: QED [{}] Sold: {}% realized""".format(i, '%.2f' %c)
                    emailsend_to_server(message)
                    
                else:
                    print(f'\n\n$$$$$ Realize gain test all passed - Big Ticker: {i} $$$$$\n\n')
            else:
                pass
        except:
            print('Except >>> Sell Monitoring for Long-Term Sell: ', i)
            alarm()
            pass

        
# Sell_mid list update    
    with open("tickers_to_sell_mid.txt", 'r') as f:
        tickers_to_sell_mid = [line.rstrip('\n') for line in f] 
    with open("realgainrate_mid.txt", 'r') as f:
        realgainrate_mid = [line.rstrip('\n') for line in f]    
        realgainrate_mid = np.round(float(realgainrate_mid[0]), 2)     
            
    dfm = robin.loc[robin['index'].isin(tickers_to_sell_mid)]
    print('\n\n\n\n' + '\033[1m' + f">>> {realgainrate_mid}" + '\033[0m' + '% Max. Sell Monitoring for Mid-Term Sell: ' + '\033[1m' + format(dfm['index'].values) + '\033[0m', '\n')
 
    for i, c, q in zip(dfm['index'], dfm['percent_change'], dfm['quantity']):
        
        if timechecknow() > 390:
            break
        
        print('Analysing Ticker: ', i, end = '                    \r')
        try:
            
            ## calculated an adjusted mid-gain% 
            # holding period premium
            holding_period['days_left_percent'] = ((min_holding_period/2)-holding_period['days']) / (min_holding_period/2) # set the date in the middle
            if i in holding_period['index'].tolist():
                holding_period_premium_discount = holding_period[holding_period['index'] == i]['days_left_percent'].values[0]
            else:
                holding_period_premium_discount = 0
            
            # net volatility
            if holding_period_premium_discount < 0:
                realgainrate_mid_adj = holding_period_premium_discount*2 + realgainrate_mid*(1-(volatility_check(i) - volatility_check(sector_valuation_ticker[sector_valuation_ticker['Symbol'] == i]['Ticker'].values[0])))
            else:
                realgainrate_mid_adj = holding_period_premium_discount*0.1 + realgainrate_mid*(1-(volatility_check(i) - volatility_check(sector_valuation_ticker[sector_valuation_ticker['Symbol'] == i]['Ticker'].values[0])))
 
            # apply event day risk ratio
            if i in ticker_eventday_check():
                realgainrate_mid_adj*= event_risk_ratio    
                print('\n\n' + '\033[1m' + f' {i} ' + '\033[0m' + 'Event Risk Raio' + '\033[1m' + f' {event_risk_ratio*100} % ' + '\033[0m' + 'applied.\n\n')
        
            # safe the margin error
            if realgainrate_mid_adj < 0:
                realgainrate_mid_adj = 0.01

            print('\n\n' + '\033[1m' + f"• {i}: " + '\033[0m' + '\033[1m' + f"{'%.2f' %realgainrate_mid_adj}% <--- {'%.2f' %current_gl[current_gl['index'] == i]['percent_change'].values[0]}%" + '\033[0m', '\n')
       
            if c >= realgainrate_mid_adj:   
                analysis_snp = analysis_price('^GSPC')
                analysis_sector = analysis_price(sector_valuation_ticker[sector_valuation_ticker['Symbol'] == i]['Ticker'].values[0])
                analysis_ticker = analysis_price(i)
                strategy_analysis = strategy_analysis_main(i, 'off', 'off')[0] + strategy_analysis_main(i, 'off', 'off')[0]
                
                valuecheck(i)
                if strategy_analysis == -2 or analysis_snp[5] > analysis_sector[5] or (valuecheck(i)[0]+valuecheck(i)[1]*3) < valuation_conversion_value_sell:
                    
                    print(f"""\
                    \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                    """)
                    voice_message(f"""\
                    Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                    """)
                    time.sleep(10)
                    robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>
                    
                    tickers_to_sell_mid.remove(i)
                    with open("tickers_to_sell_mid.txt", 'w') as f:
                        for s in tickers_to_sell_mid:  
                            f.write(str(s) + '\n')

                    # alarm
                    webbrowser.open("soldsound.mp3")

                    # email
                    message = """\
Subject: QED [{}] Sold: {}% realized""".format(i, '%.2f' %c)
                    emailsend_to_server(message)
                    
                else:
                    print(f'\n\n$$$$$ Realize gain test all passed - Mid Ticker: {i} $$$$$\n\n')           
        except:
            print('Except >>> Sell Monitoring for Mid-Term Sell: ', i)
            alarm()
            pass

        
# Sell_inv list update    
    with open("tickers_to_sell_inv.txt", 'r') as f:
        tickers_to_sell_inv = [line.rstrip('\n') for line in f] 
    with open("realgainrate_inv.txt", 'r') as f:
        realgainrate_inv = [line.rstrip('\n') for line in f]    
        realgainrate_inv = np.round(float(realgainrate_inv[0]), 2)

    dfi = robin.loc[robin['index'].isin(tickers_to_sell_inv)]
    print('\n\n\n\n' + '\033[1m' + f">>> {realgainrate_inv}" + '\033[0m' + '% Sell Monitoring for Inverse Tickers: ' + '\033[1m' + format(dfi['index'].values) + '\033[0m', '\n')

    for i, c, q in zip(dfi['index'], dfi['percent_change'], dfi['quantity']):
        
        if timechecknow() > 390:
            break
            
        print('Analysing Ticker: ', i, end = '                    \r')
        try:
            realgainrate_inv_adj = realgainrate_inv + realgainrate_inv*(volatility_check('^GSPC') - volatility_check(i))
            print('\n\n' + '\033[1m' + f"• {i}: " + '\033[0m' + '\033[1m' + f"{'%.2f' %realgainrate_inv_adj}% <--- {'%.2f' %current_gl[current_gl['index'] == i]['percent_change'].values[0]}%" + '\033[0m', '\n')

            if c >= realgainrate_inv_adj:
                analysis = analysis_price(i)
                strategy_analysis = strategy_analysis_main(i, 'off', 'off')[0] + strategy_analysis_main(i, 'off', 'off')[0]

                valuecheck(i)
                if strategy_analysis == -2 or (valuecheck(i)[0]+valuecheck(i)[1]*3) < valuation_conversion_value_sell or analysis[3] > analysis[4]:

                    print(f"""\
                    \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                    """)
                    voice_message(f"""\
                    Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                    """)
                    time.sleep(10)
                    robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>> 

                    tickers_to_sell_inv.remove(i)
                    with open("tickers_to_sell_inv.txt", 'w') as f:
                        for s in tickers_to_sell_inv:  
                            f.write(str(s) + '\n')    

                    # alarm
                    webbrowser.open("soldsound.mp3")

                    # email
                    message = """\
Subject: QED [{}] Sold: {}% realized, Inverse list has been updated.""".format(i, '%.2f' %c)
                    emailsend_to_server(message)
                    
                else:
                    print(f'\n\n$$$$$ Realize gain test all passed - INV Ticker: {i} $$$$$\n\n')
                
            else:
                pass
        except:
            print('Except >>> Sell Monitoring for tickers_to_sell_inv: \n', i)   
            alarm()
            pass
        
        
# Sell_ETF list update    
    with open("tickers_to_sell_etf.txt", 'r') as f:
        tickers_to_sell_etf = [line.rstrip('\n') for line in f] 
    with open("realgainrate_etf.txt", 'r') as f:
        realgainrate_etf = [line.rstrip('\n') for line in f]    
        realgainrate_etf = np.round(float(realgainrate_etf[0]), 2)  

    dfe = robin.loc[robin['index'].isin(tickers_to_sell_etf)]
    print('\n\n\n\n' + '\033[1m' + f">>> {realgainrate_etf}" + '\033[0m' + '% Sell Monitoring for ETF Tickers: ' + '\033[1m' + format(dfe['index'].values) + '\033[0m', '\n')

    for i, c, q in zip(dfe['index'], dfe['percent_change'], dfe['quantity']):
        
        if timechecknow() > 390:
            break
        
        print('Analysing Ticker: ', i, end = '                    \r')
        try:
            realgainrate_etf_adj = realgainrate_etf + realgainrate_etf*(volatility_check('^GSPC') - volatility_check(i))
            print('\n\n' + '\033[1m' + f"• {i}: " + '\033[0m' + '\033[1m' + f"{'%.2f' %realgainrate_etf_adj}% <--- {'%.2f' %current_gl[current_gl['index'] == i]['percent_change'].values[0]}%" + '\033[0m', '\n')

            if c >= realgainrate_etf_adj:
                analysis = analysis_price(i)
                strategy_analysis = strategy_analysis_main(i, 'off', 'off')[0] + strategy_analysis_main(i, 'off', 'off')[0]
 
                valuecheck(i)
                if strategy_analysis == -2 or (valuecheck(i)[0]+valuecheck(i)[1]*3) < valuation_conversion_value_sell or analysis[3] > analysis[4]:
                   
                    print(f"""\
                    \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                    """)
                    voice_message(f"""\
                    Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                    """)
                    time.sleep(10)
                    robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>  

                    tickers_to_sell_etf.remove(i)
                    with open("tickers_to_sell_etf.txt", 'w') as f:
                        for s in tickers_to_sell_etf:  
                            f.write(str(s) + '\n')    

                    # alarm
                    webbrowser.open("soldsound.mp3")

                    # email
                    message = """\
Subject: QED [{}] Sold: {}% realized, ETF list has been updated.""".format(i, '%.2f' %c)
                    emailsend_to_server(message)
                    
                else:
                    print(f'\n\n$$$$$ Realize gain test all passed - ETF Ticker: {i} $$$$$\n\n')              
            else:
                pass
        except:
            print('Except >>> Sell Monitoring for tickers_to_sell_etf: \n', i)   
            alarm()
            pass
        
 
# Loss-Cut (ticker to sell small <= Excluding BIG Tickers) ############################################################################################################################################
    with open("tickers_to_sell_small.txt", 'r') as f:
        tickers_to_sell_small = [line.rstrip('\n') for line in f]  
    
    tickers_to_sell_small = [i for i in tickers_to_sell_small if i not in sold]
        
    with open("realgainrate_small.txt", 'r') as f:
        realgainrate_small = [line.rstrip('\n') for line in f]    
        realgainrate_small = np.round(float(realgainrate_small[0]), 2)
   
  # stocks on hold except Big
    dfa = robin.loc[-robin['index'].isin(tickers_to_sell_big)]
    dfa_original = dfa # to plot the candle chart
    
  # stocks on hold except purchased on the same date  
    dfa = robin.loc[-robin['index'].isin(tickers_to_sell_big)]
#     dfa = dfa.loc[-dfa['index'].isin(purchased)]
    
  # stocks on hold exceed the min. holding peried within the lost-cut threshold
    dfs = robin.loc[robin['index'].isin(tickers_to_sell_small)]
#     dfs = dfs.loc[-dfs['index'].isin(purchased)]
    
    print('\n\n\n' + '\033[1m' + '=====================================< Loss-cut Start >====================================' + '\033[0m')
    print('\n\n\n' + '\033[1m' + f">>> {realgainrate_small}" + '\033[0m' + '% Base Loss-Cut calculating (Small tickers: -Realize Gain Mid% x 2) => ' + '\033[1m' + format(dfs['index'].values) + '\033[0m', '\n')
    
# stocks on hold exceed the min. holding peried within the lost-cut threshold  
    # Loss-cut execution by the adj_losscut
    for i, c, q in zip(dfs['index'], dfs['percent_change'], dfs['quantity']):
        
        if timechecknow() > 390:
            break
            
        print('Analysing Ticker: ', i)
        try:
            # loss cut percentage adjusting by the volatility 
            if i == 'USO' or i == 'GLD' or i == 'BTC-USD' or i in etf_ticker:
                adj_losscut = np.round(realgainrate_small * (1-(volatility_check(i) - volatility_check('^GSPC'))),2)
            else:
                adj_losscut = np.round(realgainrate_small * (1-(volatility_check(i) - volatility_check(sector_valuation_ticker[sector_valuation_ticker['Symbol'] == i]['Ticker'].values[0]))),2)
            print('\n\n' + '\033[1m' + format(adj_losscut) + '\033[0m' + '% adjusted loss cut monitoring: ' + '\033[1m' + format(i) + '\033[0m', f' <=== {c}%')
                
            if c < adj_losscut:
                
                q = split_qty_finder(i, q)                
                
                print(f"""\
                \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                """)
                voice_message(f"""\
                Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                """)
                time.sleep(10)                    
                    
                robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>  

                tickers_to_sell_small.remove(i)
                with open("tickers_to_sell_small.txt", 'w') as f:
                    for s in tickers_to_sell_small:  
                        f.write(str(s) + '\n')                    

                # email
                message = """\
Subject: QED [{}] Sold: {}% loss cut executed""".format(i, '%.2f' %c)
                emailsend_to_server(message)
            else:
                pass
            
        except:
            print('Except >>> Sell Monitoring for Loss Cut Sell: ', i)    
            alarm()
            sellmonitor()
        print('\n===> Passed\n\n')
            
     
    # Loss-cut execution by Strategy Analysis
    print('\n\n\n' + '\033[1m' + '>>> Strategy Analysis ' + '\033[0m' + 'Loss-Cut Execution (Small tickers excl. BIG & Sold) => ' + '\033[1m' + format(dfs['index'].values) + '\033[0m', '\n')

    for i, c, q in zip(dfs['index'], dfs['percent_change'], dfs['quantity']):
        
        if timechecknow() > 390:
            break
            
        print('Analysing Ticker: ', i)
        try:
            if strategy_analysis_main(i, 'off', 'off')[0] + strategy_analysis_main(i, 'off', 'off')[0] == -2:
                    
                q = split_qty_finder(i, q)
                    
                print(f"""\
                \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                """)
                voice_message(f"""\
                Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                """)
                time.sleep(10)                    
                    
                robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>
                 
                if i in tickers_to_sell_small:
                    tickers_to_sell_small.remove(i)
                    with open("tickers_to_sell_small.txt", 'w') as f:
                        for s in tickers_to_sell_small:  
                            f.write(str(s) + '\n')                    

                # email
                message = """\
Subject: QED [{}] Sold: {}% Strategy Analysis loss cut executed""".format(i, '%.2f' %c)
                emailsend_to_server(message)
                
            else:
                pass
        except:
            print('Except >>> Strategy Analysis Loss-Cut Execution: ', i)
            pass
        print('\n===> Passed\n\n')
        
        
    # Loss-cut execution by ValueCheck
    print('\n\n\n' + '\033[1m' + '>>> ValueCheck ' + '\033[0m' + 'Loss-Cut Execution (Small tickers excl. BIG & Sold) => ' + '\033[1m' + format(dfs['index'].values) + '\033[0m', '\n')

    for i, c, q in zip(dfs['index'], dfs['percent_change'], dfs['quantity']):
        
        if timechecknow() > 390:
            break
        
        print('Analysing Ticker: ', i)
        try:
            valuecheck(i)
            if (valuecheck(i)[0]+valuecheck(i)[1]*3) < valuation_conversion_value_losscut:

                q = split_qty_finder(i, q)
                
                print(f"""\
                \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                """)
                voice_message(f"""\
                Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                """)
                time.sleep(10)                  
                
                robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>   

                if i in tickers_to_sell_small:                    
                    tickers_to_sell_small.remove(i)
                    with open("tickers_to_sell_small.txt", 'w') as f:
                        for s in tickers_to_sell_small:  
                            f.write(str(s) + '\n')                    

                # email
                message = """\
Subject: QED [{}] Sold: {}% ValueCheck loss cut executed""".format(i, '%.2f' %c)
                emailsend_to_server(message)
                
            else:
                pass
        except:
            print('Except >>> ValueCheck Loss-Cut Execution: ', i)
            pass
        print('\n===> Passed\n\n')
              
        
    # Loss-cut execution by moving average lines for all tickers except for Big tickers
    print('\n\n\n' + '\033[1m' + '>>> Moving Average ' + '\033[0m' + 'Loss-Cut Execution (All tickers excl. BIG & Purchased/Sold) => ' + '\033[1m' + format(dfa['index'].values) + '\033[0m', '\n')

    for i, c, q in zip(dfa['index'], dfa['percent_change'], dfa['quantity']):
        
        if timechecknow() > 390:
            break
        
        print('Analysing Ticker: ', i)
        try:
            dmv_plot(i)
            analysis_ticker = analysis_price(i)
            
    # crossover losscut condition #1
            if analysis_ticker[8] == True and bollinger_sign_gen(i)[1] == 1: 
                
                valuecheck(i)
                if (valuecheck(i)[0]+valuecheck(i)[1]*3) < valuation_conversion_value_losscut_all:
                
                    q = split_qty_finder(i, q)

                    print(f"""\
                    \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                    """)
                    voice_message(f"""\
                    Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                    """)
                    time.sleep(10)

                    robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>   

                    if i in tickers_to_sell_small:                    
                        tickers_to_sell_small.remove(i)
                        with open("tickers_to_sell_small.txt", 'w') as f:
                            for s in tickers_to_sell_small:  
                                f.write(str(s) + '\n')                    

                    # email
                    message = """\
Subject: QED [{}] Sold: {}% Moving Average loss cut executed""".format(i, '%.2f' %c)
                    emailsend_to_server(message)
                else:
                    pass
                
    # crossover losscut condition #2 
            elif analysis_ticker[8] == True and c < stg_loss_rate(i): 
                
                valuecheck(i)
                if (valuecheck(i)[0]+valuecheck(i)[1]*3) < valuation_conversion_value_losscut_all:
                
                    q = split_qty_finder(i, q)

                    print(f"""\
                    \n\n\n$$$$$ Hi Sean. We are selling {q} stocks of {i} at {c} percent return $$$$$\n\n\n
                    """)
                    voice_message(f"""\
                    Hi Sean. We are selling {q} stocks of {i} at {c} percent return
                    """)
                    time.sleep(10)

                    robinExecution_sell(i, q) #<<<<<<< Sell order >>>>>>>   

                    if i in tickers_to_sell_small:                    
                        tickers_to_sell_small.remove(i)
                        with open("tickers_to_sell_small.txt", 'w') as f:
                            for s in tickers_to_sell_small:  
                                f.write(str(s) + '\n')                    

                    # email
                    message = """\
Subject: QED [{}] Sold: {}% Moving Average loss cut executed""".format(i, '%.2f' %c)
                    emailsend_to_server(message)
                else:
                    pass 
                
            else:
                pass
        except:
            print('Except >>> Moving Average Loss-Cut Execution: ', i)
            pass
        print('\n===> Passed\n\n')
              
        
    print('\n\n' + '\033[1m' + '======================================< Loss-cut End >=====================================' + '\033[0m' + '\n\n')
        
    if cyclecount%10 == 0: 
        print('\n\n' + '\033[1m' + '< Candle Check >' + '\033[0m' + '\n\n')
        for i in dfa_original['index'].to_list():
            print(f'• Ticker: ' + '\033[1m' + f'{i}' + '\033[0m')
            try:
                candle_review(i)
            except:
                pass
            print('\n\n')
    
        
if __name__ == '__main__':
    print('\n', '\033[1m' + 'The Engine#2 gets ready.' + '\033[0m', '\n')

## Engine #3

In [26]:
def robinExecution_buy(stock, qty):
    global sellmonitor_freq
    global num_of_purchased
    
    
    # only execute during the market hours
    if timechecknow() <= 390:
        # count the existing number of stocks before buying-out
        onhold_df = robin_login()

        if stock in onhold_df['index'].tolist():
            num_of_stocks_before = int(float(onhold_df[onhold_df['index'] == stock]['quantity'].values[0]))
            print(f'\n\n\n• Stock: {stock} - Existing stock / {num_of_stocks_before} shares')
        else:
            num_of_stocks_before = 0
            print(f'\n\n\n• Stock: {stock} - New stock / {num_of_stocks_before} shares')
     
        try:
            # execue buy order
            login = rs.login('aicpasean@gmail.com', sean.robin_api())
            order_buy_market(stock, qty)
            rs.logout()  
            print('\n'+current_time()+' - The buying order has been executed.\n')
            time.sleep(10)

        except:
            message = 'Except >>> Purchasing was not successful.'
            print(message)
            alarm()
            voice_message(message)

        # count the existing number of stocks after buying-out
        onhold_df = robin_login()
        
        if stock in onhold_df['index'].tolist():
            num_of_stocks_after = int(float(onhold_df[onhold_df['index'] == stock]['quantity'].values[0]))
            print(f'• Stock: {stock} - Existing stock / {num_of_stocks_after} shares\n\n\n')
        else:
            num_of_stocks_after = 0
            print(f'• Stock: {stock} - New stock / {num_of_stocks_after} shares\n\n\n')

        if num_of_stocks_after - num_of_stocks_before >= 1:
            
            # purchased list update
            print('\n'+current_time()+'\n')
            trade_list('purchased', stock)

            # append to the blacklist; only purchase once a day
            with open("buymonitor_blacklist.txt", 'r') as f:
                buymonitor_blacklist = [line.rstrip('\n') for line in f]
            buymonitor_blacklist.append(stock)
            with open("buymonitor_blacklist.txt", 'w') as f:
                for s in buymonitor_blacklist:  
                    f.write(str(s) + '\n') 
            
            # plot gain & loss
            robin_plot()  

            # email
            message = """\
Subject: QED [{}] {} Stock Purchased at ${}""".format(stock, qty, '%.2f' %analysis_price(stock)[4])
            emailsend_to_server(message)
            
            # sell monitor cycle update   
            num_of_purchased = len(onhold_df['index'])
            sellmonitor_freq = int((sellmonitor_freq_limit) - (num_of_purchased - len(dead_stock))) ### adjust       
            print('\n\nSell Monitor Freq has been updated as every' + '\033[1m' + f' {sellmonitor_freq} ' + '\033[0m' + 'cycles.\n\n')

        else:
            message = f'We have cash balance less than {dollar_limit_order_dollar} dollars'
            print('Except >>> ' + message + '. \n\n\n')
            emailsend_to_server(message)
            
            # sell monitor cycle update   
            sellmonitor_freq = 1
            
    else:
        message = f'The market has been closed. The purchasing order has not been executed.'
        print('Except >>> ' + message)
        alarm()
        voice_message(message)


def buymonitor():    
    global cyclecount
    print(f"#{cyclecount}th Cycle")
    
    with open(f'purchased_{currentdate}.txt', 'r') as f:
        purchased = [line.rstrip('\n') for line in f]
        
    with open("portfolio_superpass.txt", 'r') as f:
        portfolio_superpass = [line.rstrip('\n') for line in f]
        
    with open("buymonitor_blacklist.txt", 'r') as f:
        buymonitor_blacklist = [line.rstrip('\n') for line in f]
     
    with open("portfolio.txt", 'r') as f:
        portfolio = [line.rstrip('\n') for line in f]
        
    portfolio = [i for i in portfolio if i not in buymonitor_blacklist and i not in portfolio_superpass and i not in purchased]
    
    sector_valuation_ticker = pd.read_csv('sector_valuation.csv')
    sector_valuation_ticker = sector_valuation_ticker[sector_valuation_ticker['Date'] == currentdate]
    sector_valuation_ticker = sector_valuation_ticker.reset_index(drop = True)
    line_cut = sector_valuation_ticker.Time.unique()[-1:][0] # get the most recent valuation 
    sector_valuation_ticker = sector_valuation_ticker[sector_valuation_ticker['Time'] == line_cut]
    
    # get the target tickers
    sector_valuation_ticker_meancheck = sector_valuation_ticker[sector_valuation_ticker['valuation_mean'] == True].Symbol.unique()
    sector_valuation_ticker_snpcheck = sector_valuation_ticker[sector_valuation_ticker['valuation_snp'] == True].Symbol.unique()

    # the sector true contains 'mean-True' and 'snp-True'
    sector_valuation_ticker_symbol = np.concatenate((sector_valuation_ticker_meancheck, sector_valuation_ticker_snpcheck), axis = 0)
    sector_valuation_ticker_symbol = [i for n, i in enumerate(sector_valuation_ticker_symbol) if i not in sector_valuation_ticker_symbol[:n]] #duplicated
    
    try:    
        if rsi('^GSPC')[2] == False: # rsi[2] == 'Sell alarm'
            
            # if 'sell alarm' is False, 'or' condition applies
            cross_true = []
            for i in sector_valuation_ticker_symbol:
                if i in sector_valuation_ticker_meancheck or i in sector_valuation_ticker_snpcheck:
                    cross_true.append(i)
                else:
                    pass
            sector_valuation_ticker_symbol = cross_true
        else:
            
            # if 'sell alarm' is True, 'and' condition applies
            cross_true = []
            for i in sector_valuation_ticker_symbol:
                if i in sector_valuation_ticker_meancheck and i in sector_valuation_ticker_snpcheck:
                    cross_true.append(i)
                else:
                    pass
            sector_valuation_ticker_symbol = cross_true   
    except:
        print('\nExcept >>> Error triggered from buymonitor() ==> rsi(^GSPC)[2] == False\n')
        alarm()   

    # value check only for Sector is True
    portfolio_sector_true = [i for i in portfolio if i in sector_valuation_ticker_symbol] 
    portfolio_sector_true = sorted(portfolio_sector_true)   
    
    with open("buymonitor_blacklist.txt", 'r') as f:
        buymonitor_blacklist = [line.rstrip('\n') for line in f]    
    portfolio_sector_true = [i for i in portfolio_sector_true if i not in buymonitor_blacklist]
    with open("portfolio_sector_true.txt", 'w') as f:
        for s in portfolio_sector_true:      
            f.write(str(s) + '\n')   

    # cpi rate average line analysis for 'GLD' and 'USO'    
    cpi = pd.read_csv(f"predict_aim_sourcedata_monitoring_^GSPC_{currentdate}.csv")
    cpi = cpi['Cpi'].tail(120).mean() < cpi['Cpi'].tail(60).mean()
    
    # when S&P500 has a wider swing, proportionally wide the buy limit within 5%
    buy_limit_rate = 4+analysis_price('^GSPC')[5]/10
    
    buy_snp_t_y = [np.round(buy_limit_rate/2, 2)] 
    buy_gold_t_y = [np.round(buy_limit_rate/2.2, 2)] 
    buy_oil_t_y = [np.round(buy_limit_rate/2.2, 2)] 
    buy_bit_t_y = [np.round(buy_limit_rate/2.2, 2)] 
    
    
# Monitoring Starts from here

# Super Pass monitoring    
    # import required data
    portfolio_superpass = [i for i in portfolio_superpass if i not in buymonitor_blacklist and i not in purchased]
    
    print('\n\n\n' + '\033[1m' + format(sum(buy_snp_t_y)) + '\033[0m' + '% limit Buy - Super Pass monitoring: \n\n' + '\033[1m' + format(portfolio_superpass) + '\033[0m', '\n')
    for stock in portfolio_superpass:
        if timechecknow() > 400:
            break
        if analysis_price('^GSPC')[2] < -0.5 and daily_news_sentiment_all() < min_market_sentiment:
            break            
        try:
            print('Ticker: ', stock, end = '                                                                                \r')
            analysis = analysis_price(stock)
            
            # check if the candle sign exists when the loop break executed
            token_exist = exists(f"candle_token_{stock}.txt")
            if token_exist == True:
                with open(f'candle_token_{stock}.txt', 'r') as f:
                    candle_token_time = float([line.rstrip('\n') for line in f][0])
                if timechecknow() - candle_token_time < 7:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nFresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 3
                else:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nUnfresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 0
            else:
                token_exist = 0

    # Buy condition: Super Pass
            if analysis[2] > -0.3 and analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False and analysis[2] < sum(buy_snp_t_y) and analysis[6] == True and analysis[9] == True:
            
                print(f'Candle Check - Ticker: {stock}\n')
                candle_result = volume_vs_sign(stock)
                
                if candle_result+token_exist > 1:
                    print(f'\n > Ticker: {stock}\n > Candle Result: {candle_result}\n')
                
                if candle_result+token_exist >= 3 and bollinger_sign_gen(stock)[0] == 1 and sentiment_analysis(stock) > 0 and strategy_analysis_main(stock, 'off', 'off')[0] + strategy_analysis_main(stock, 'off', 'off')[0] >= 1:
                    focus_record(stock)
                    initial_val_result = valuecheck(stock)[0]+valuecheck(stock)[1]*3
                    if initial_val_result >= 0:
         
                        if (valuecheck(stock)[0]+valuecheck(stock)[1]*3) >= valuation_conversion_value_buy:             

                            try:
                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])

                                print(f"""\
                                \n\n\n$$$$$ Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars $$$$$\n\n\n
                                """)
                                voice_message(f"""\
                                Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars
                                """)

                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 

                            except:                            
                                print(f"""\
                                Except >>> Hi Sean. We have an error while buying {stock}. Re-trying
                                """)
                                voice_message(f"""\
                                Hi Sean. We have an error while buying {stock}. Re-trying
                                """)

                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])
                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>>   
                                
                        else:
                            pass
                    else:
                        pass
                else:
                    pass
            else:
                pass
        except:
            print(f"Exception >>> Buy - Super Pass - Ticker: {stock}")
            alarm()
            pass
 

    # import required data
    print('\n\n\n' + '\033[1m' + format(sum(buy_snp_t_y)) + '\033[0m' + '% limit Buy - Sector/True monitoring: \n\n' + '\033[1m' + format(portfolio_sector_true) + '\033[0m', '\n')

    for stock in portfolio_sector_true:
        if timechecknow() > 400:
            break
        if analysis_price('^GSPC')[2] < -0.5 and daily_news_sentiment_all() < min_market_sentiment:
            break
        try:
            print('Ticker: ', stock, end = '                                                                                \r')
            analysis = analysis_price(stock)
    
            # check if the candle sign exists when the loop break executed
            token_exist = exists(f"candle_token_{stock}.txt")
            if token_exist == True:
                with open(f'candle_token_{stock}.txt', 'r') as f:
                    candle_token_time = float([line.rstrip('\n') for line in f][0])
                if timechecknow() - candle_token_time < 7:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nFresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 3
                else:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nUnfresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 0
            else:
                token_exist = 0

    # Buy condition: S&P 500
            if analysis[2] > -0.3 and analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False and analysis[2] < sum(buy_snp_t_y):
            
                print(f'Candle Check - Ticker: {stock}\n')
                candle_result = volume_vs_sign(stock)
                
                if candle_result+token_exist > 1:
                    print(f'\n > Ticker: {stock}\n > Candle Result: {candle_result}\n')

                if candle_result+token_exist >= 3 and bollinger_sign_gen(stock)[0] == 1 and sentiment_analysis(stock) > 0 and strategy_analysis_main(stock, 'off', 'off')[0] + strategy_analysis_main(stock, 'off', 'off')[0] >= 1:
                    focus_record(stock)
                    initial_val_result = valuecheck(stock)[0]+valuecheck(stock)[1]*3
                    if initial_val_result >= 0:

                        if (valuecheck(stock)[0]+valuecheck(stock)[1]*3) >= valuation_conversion_value_buy:   

                            try:
                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])

                                print(f"""\
                                \n\n\n$$$$$ Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars $$$$$\n\n\n
                                """)
                                voice_message(f"""\
                                Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars
                                """)

                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 

                            except:                            
                                print(f"""\
                                Except >>> Hi Sean. We have an error while buying {stock}. Re-trying
                                """)
                                voice_message(f"""\
                                Hi Sean. We have an error while buying {stock}. Re-trying
                                """)

                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])
                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 
                        
                        else:
                            pass
                    else:
                        pass                        
                else:
                    pass
            else:
                pass   
        except:
            print(f"Exception >>> Buy - Sector/True monitoring - Ticker: {stock}")
            alarm()
            pass
        

    # always execute ML_True tickers - separate monitoring
    sector_valuation_ticker_valuecheck = sector_valuation_ticker[sector_valuation_ticker['valuation_rsq'] > sector_valuation_ticker.valuation_rsq.mean()].Symbol.unique()
    portfolio_valuation_true = [i for i in portfolio if i in sector_valuation_ticker_valuecheck and i not in portfolio_sector_true] 
    portfolio_valuation_true = sorted(portfolio_valuation_true)   

    with open("buymonitor_blacklist.txt", 'r') as f:
        buymonitor_blacklist = [line.rstrip('\n') for line in f]
    portfolio_valuation_true = [i for i in portfolio_valuation_true if i not in buymonitor_blacklist]
    with open("portfolio_sector_valuation_true.txt", 'w') as f:
        for s in portfolio_valuation_true:      
            f.write(str(s) + '\n')   

    # import required data
    print('\n\n\n' + '\033[1m' + format(sum(buy_snp_t_y)) + '\033[0m' + '% limit Buy - Valuation/True monitoring(excl.Sector/True): \n\n' + '\033[1m' + format(portfolio_valuation_true) + '\033[0m', '\n')

    for stock in portfolio_valuation_true:
        if timechecknow() > 400:
            break
        if analysis_price('^GSPC')[2] < -0.5 and daily_news_sentiment_all() < min_market_sentiment:
            break
        try:
            print('Ticker: ', stock, end = '                                                                                \r')
            analysis = analysis_price(stock)
            
            # check if the candle sign exists when the loop break executed
            token_exist = exists(f"candle_token_{stock}.txt")
            if token_exist == True:
                with open(f'candle_token_{stock}.txt', 'r') as f:
                    candle_token_time = float([line.rstrip('\n') for line in f][0])
                if timechecknow() - candle_token_time < 7:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nFresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 3
                else:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nUnfresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 0
            else:
                token_exist = 0

    # Buy condition: S&P 500
            if analysis[2] > -0.3 and analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False and analysis[2] < sum(buy_snp_t_y):
            
                print(f'Candle Check - Ticker: {stock}\n')
                candle_result = volume_vs_sign(stock)
                
                if candle_result+token_exist > 1:
                    print(f'\n > Ticker: {stock}\n > Candle Result: {candle_result}\n')

                if candle_result+token_exist >= 3 and bollinger_sign_gen(stock)[0] == 1 and sentiment_analysis(stock) > 0 and strategy_analysis_main(stock, 'off', 'off')[0] + strategy_analysis_main(stock, 'off', 'off')[0] >= 1: 
                    focus_record(stock)
                    initial_val_result = valuecheck(stock)[0]+valuecheck(stock)[1]*3
                    if initial_val_result >= 0:

                        if (valuecheck(stock)[0]+valuecheck(stock)[1]*3) >= valuation_conversion_value_buy:   

                            try:
                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])

                                print(f"""\
                                \n\n\n$$$$$ Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars $$$$$\n\n\n
                                """)
                                voice_message(f"""\
                                Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars
                                """)

                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 

                            except:
                                print(f"""\
                                Except >>> Hi Sean. We have an error while buying {stock}. Re-trying
                                """)
                                voice_message(f"""\
                                Hi Sean. We have an error while buying {stock}. Re-trying
                                """)   

                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])
                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>>   
                                
                        else:
                            pass
                    else:
                        pass
                else:
                    pass
            else:
                pass   
        except:
            print(f"Exception >>> Buy - Valuation/True monitoring - Ticker: {stock}")
            alarm()
            pass


# Inverse tickers        
    with open("inv_tickers.txt", 'r') as f:
        inv_tickers = [line.rstrip('\n') for line in f]
    
    with open("buymonitor_blacklist.txt", 'r') as f:
        buymonitor_blacklist = [line.rstrip('\n') for line in f]

    inv_tickers = [i for i in inv_tickers if i not in buymonitor_blacklist and i not in purchased]

    print('\n\n\n' + 'Inverse Ticker Monitoring', '\n')
    for stock in inv_tickers:
        if timechecknow() > 400:
            break
        if analysis_price('^GSPC')[2] < -0.5 and daily_news_sentiment_all() < min_market_sentiment:
            break
        try:
            print('Ticker: ', stock, end = '                                                                                \r')
            analysis = analysis_price(stock)
            
            # check if the candle sign exists when the loop break executed
            token_exist = exists(f"candle_token_{stock}.txt")
            if token_exist == True:
                with open(f'candle_token_{stock}.txt', 'r') as f:
                    candle_token_time = float([line.rstrip('\n') for line in f][0])
                if timechecknow() - candle_token_time < 7:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nFresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 3
                else:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nUnfresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 0
            else:
                token_exist = 0

    # Buy condition : Gold 
            if stock == "GLD" and analysis[2] > -0.3 and analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False and analysis[2] < sum(buy_gold_t_y) and analysis_price('DX-Y.NYB')[1] == False and cpi == True and analysis[6] == True and analysis[9] == True:
            
                print(f'Candle Check - Ticker: {stock}\n')
                candle_result = volume_vs_sign(stock)
                
                if candle_result+token_exist > 1:
                    print(f'\n > Ticker: {stock}\n > Candle Result: {candle_result}\n')
                
                if candle_result+token_exist >= 3 and bollinger_sign_gen(stock)[0] == 1 and strategy_analysis_main(stock, 'off', 'off')[0] + strategy_analysis_main(stock, 'off', 'off')[0] >= 1:
                    focus_record(stock)
                    initial_val_result = valuecheck(stock)[0]+valuecheck(stock)[1]*3
                    if initial_val_result >= 0:

                        if (valuecheck(stock)[0]+valuecheck(stock)[1]*3) >= valuation_conversion_value_etf_buy:   

                            try:
                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])

                                print(f"""\
                                \n\n\n$$$$$ Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars $$$$$\n\n\n
                                """)
                                voice_message(f"""\
                                Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars
                                """)

                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 

                            except:
                                print(f"""\
                                Except >>> Hi Sean. We have an error while buying {stock} Re-trying
                                """)
                                voice_message(f"""\
                                Hi Sean. We have an error while buying {stock}. Re-trying
                                """)

                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])
                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 
                                
                        else:
                            pass
                    else:
                        pass
                else:
                    pass

    # Buy condition : Oil
            elif stock == "USO" and analysis[2] > -0.3 and analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False and analysis[2] < sum(buy_oil_t_y) and analysis_price('DX-Y.NYB')[1] == False and cpi == True and analysis[6] == True and analysis[9] == True:
     
                print(f'Candle Check - Ticker: {stock}\n')
                candle_result = volume_vs_sign(stock)
                
                if candle_result+token_exist > 1:
                    print(f'\n > Ticker: {stock}\n > Candle Result: {candle_result}\n')
                
                if candle_result+token_exist >= 3 and bollinger_sign_gen(stock)[0] == 1 and strategy_analysis_main(stock, 'off', 'off')[0] + strategy_analysis_main(stock, 'off', 'off')[0] >= 1:
                    focus_record(stock)
                    initial_val_result = valuecheck(stock)[0]+valuecheck(stock)[1]*3
                    if initial_val_result >= 0:
              
                        if (valuecheck(stock)[0]+valuecheck(stock)[1]*3) >= valuation_conversion_value_etf_buy:

                            try:
                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])

                                print(f"""\
                                \n\n\n$$$$$ Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars $$$$$\n\n\n
                                """)
                                voice_message(f"""\
                                Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars
                                """)

                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 

                            except:                            
                                print(f"""\
                                Except >>> Hi Sean. We have an error while buying {stock}. Re-trying
                                """)
                                voice_message(f"""\
                                Hi Sean. We have an error while buying {stock}. Re-trying
                                """)

                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])
                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>>      
                                
                        else:
                            pass
                    else:
                        pass
                else:
                    pass

    # Buy condition : BitCoin
            elif stock == "BTC-USD" and analysis[2] > -0.3 and analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False and analysis[2] < sum(buy_bit_t_y) and analysis_price('DX-Y.NYB')[2] < analysis[2] and analysis_price('DX-Y.NYB')[3] > analysis_price('DX-Y.NYB')[4] and analysis_price('DX-Y.NYB')[1] == False and analysis[6] == True and analysis[9] == True:
          
                print(f'Candle Check - Ticker: {stock}\n')
                candle_result = volume_vs_sign(stock)
                
                if candle_result+token_exist > 1:
                    print(f'\n > Ticker: {stock}\n > Candle Result: {candle_result}\n')
                
                if candle_result+token_exist >= 3 and bollinger_sign_gen(stock)[0] == 1 and strategy_analysis_main(stock, 'off', 'off')[0] + strategy_analysis_main(stock, 'off', 'off')[0] >= 1:
                    focus_record(stock)
                    initial_val_result = valuecheck(stock)[0]+valuecheck(stock)[1]*3
                    if initial_val_result >= 0:
            
                        if (valuecheck(stock)[0]+valuecheck(stock)[1]*3) >= valuation_conversion_value_etf_buy:

                            # email
                            message = """\
Subject: QED [{}] Buy sign at {}% {}""".format(stock, '%.2f' %analysis_price(stock)[2], stock)
                            emailsend_to_server(message)

                        else:
                            pass
                    else:
                        pass
                else:
                    pass
            else:
                pass   
        except:
            print(f"Exception >>> Buy - L/T holding and Whitelist monitoring - Ticker: {stock}")
            alarm()
            pass


# ETF monitoring    
    # import required data
    with open("etf_portfolio.txt", 'r') as f:
        etf_portfolio = [line.rstrip('\n') for line in f]
        
    with open("buymonitor_blacklist.txt", 'r') as f:
        buymonitor_blacklist = [line.rstrip('\n') for line in f]
   
    etf_portfolio = [i for i in etf_portfolio if i not in buymonitor_blacklist and i not in purchased]
    
    print('\n\n\n' + '\033[1m' + format(sum(buy_snp_t_y)) + '\033[0m' + '% limit Buy - ETF monitoring: \n\n' + '\033[1m' + format(etf_portfolio) + '\033[0m', '\n')
    for stock in etf_portfolio:
        if timechecknow() > 400:
            break
        if analysis_price('^GSPC')[2] < -0.5 and daily_news_sentiment_all() < min_market_sentiment:
            break
        try:
            print('Ticker: ', stock, end = '                                                                                \r')
            analysis = analysis_price(stock)
            
            # check if the candle sign exists when the loop break executed
            token_exist = exists(f"candle_token_{stock}.txt")
            if token_exist == True:
                with open(f'candle_token_{stock}.txt', 'r') as f:
                    candle_token_time = float([line.rstrip('\n') for line in f][0])
                if timechecknow() - candle_token_time < 7:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nFresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 3
                else:
                    os.remove(f'candle_token_{stock}.txt')
                    print ('\n\nUnfresh Candle - File removed: ' + '\033[1m' + f'candle_token_{stock}.txt' + '\033[0m' + ' | Checking Buy Condition...\n\n')
                    token_exist = 0
            else:
                token_exist = 0

    # Buy condition: ETF
            if analysis[2] > -0.3 and analysis[8] == False and analysis[7]+analysis[10]+analysis[11]+analysis[12] != False and analysis[2] < sum(buy_snp_t_y) and analysis_price('^GSPC')[5] < analysis[5]:
      
                print(f'Candle Check - Ticker: {stock}\n')
                candle_result = volume_vs_sign(stock)
                
                if candle_result+token_exist > 1:
                    print(f'\n > Ticker: {stock}\n > Candle Result: {candle_result}\n')
                
                if candle_result+token_exist >= 3 and bollinger_sign_gen(stock)[0] == 1 and strategy_analysis_main(stock, 'off', 'off')[0] + strategy_analysis_main(stock, 'off', 'off')[0] >= 1:
                    focus_record(stock)
                    initial_val_result = valuecheck(stock)[0]+valuecheck(stock)[1]*3
                    if initial_val_result >= 0:
         
                        if (valuecheck(stock)[0]+valuecheck(stock)[1]*3) >= valuation_conversion_value_etf_buy:          

                            try:
                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])

                                print(f"""\
                                \n\n\n$$$$$ Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars $$$$$\n\n\n
                                """)
                                voice_message(f"""\
                                Hi Sean. We are buying {order_qty} stocks of {stock} at {int(analysis_price(stock)[4])} dollars
                                """)

                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>> 

                            except:                            
                                print(f"""\
                                Except >>> Hi Sean. We have an error while buying {stock}. Re-trying
                                """)
                                voice_message(f"""\
                                Hi Sean. We have an error while buying {stock}. Re-trying
                                """)

                                order_qty = int(1 + dollar_limit_order_dollar / analysis_price(stock)[4])
                                robinExecution_buy(stock, order_qty) #<<<<<<< buy order >>>>>>>   
                                
                        else:
                            pass
                    else:
                        pass
                else:
                    pass
            else:
                pass
        except:
            print(f"Exception >>> Buy - ETF - Ticker: {stock}")
            alarm()
            pass

    
if __name__ == '__main__':
    print('\n', '\033[1m' + 'The Engine#3 gets ready.' + '\033[0m', '\n')

## Engine #4

In [27]:
today_to_add = date.today()
pd.set_option('display.max_column', None)
magic = pd.read_csv('magic.csv')
   
    
def robin_equity_check(): 
    global num_of_purchased
    global performance_updated_df
    global stock_return_percent
    global predict_proba_value
    
    try:
        robin = robin_login()
        stock_return_percent = perf_cal(robin)
        performance_updated_df = robin[['index', 'equity', 'equity_change', 'percent_change']]
        performance_updated_df.sort_values('percent_change', ascending = False, inplace = True)
        performance_updated_df = performance_updated_df.reset_index(drop = True)
        equity_change = np.round(performance_updated_df['equity_change'].astype(float).sum(), 2)

        performance_updated_df.to_csv('performance_updated_df.csv', index = False)

        # filter out stocks within the min. holding period
        current_index = sorted(robin['index'].tolist())
        holding_period = pd.read_csv('holding_period.csv')
        holding_period_index = sorted(holding_period[holding_period['days'] >= min_holding_period]['index'].tolist())
        holding_period_index = [i for i in holding_period_index if i in current_index]

        print('\n\n===[ Portfolio Summary ]===================================================================\n')
        print(performance_updated_df) 
        print("\n• Equity change: " + '\033[1m' + f"             $ {equity_change}" + '\033[0m')
        print("\n• Performance average: " + '\033[1m' + f"         {stock_return_percent} %" + '\033[0m')
        print("\n• Number of stocks on hand: " + '\033[1m' + f"    {int(num_of_purchased)} stocks" + '\033[0m')
        print("\n• Loss-cut Min. Holding Period: " + '\033[1m' + f"{min_holding_period} days" + '\033[0m')
        print("\n• Stocks on hold: " + '\033[1m' + f"       {current_index}" + '\033[0m')
        print("\n• Stocks aged the limit: " + '\033[1m' + f"{holding_period_index}" + '\033[0m')
        print('\n===================================================================[ Portfolio Summary ]===\n')

        vars = ['price', 'quantity', 'average_buy_price', 'equity', 'percent_change', 
                'intraday_percent_change', 'equity_change', 'pe_ratio', 'percentage']
        for var in vars:
            robin[var] = pd.to_numeric(robin[var])
        robin['quantity'] = robin['quantity'].astype(int)
        robin.to_csv('robin_all.csv', index = False)
        portfolio_list = robin['index'].values.tolist()
        losscut = robin[['index','percent_change']]

# Mid Gain Setting ###########################################################################################################
        def mid_gain_cal(ticker):
            ticker_gain = yfinance_df_rsi(ticker)
            ticker_gain = (ticker_gain[-1:]['Adj Close'].mean() - ticker_gain[-(min_holding_period+1):-min_holding_period]['Adj Close'].mean()) / ticker_gain[-(min_holding_period+1):-min_holding_period]['Adj Close'].mean()
            return ticker_gain

        active_tickers = [ticker for ticker in robin['index'].tolist() if ticker not in dead_stock]
        if len(active_tickers) == 0:
            mid_gain_avg = realize_gain_max/100    
        else:
            mid_gain_list = [mid_gain_cal(ticker) for ticker in active_tickers]
            mid_gain_avg = np.median(mid_gain_list)

        portfolio_sector = pd.read_csv('./snp500/snp_sector_index.csv')
        portfolio_sector = portfolio_sector.loc[portfolio_sector.Symbol.isin(robin['index'].tolist())]
        portfolio_sector = portfolio_sector['GICS Sector'].tolist()

        etf_index = etf_index_dic.loc[etf_index_dic.sector.isin(portfolio_sector)].ticker.tolist()
        etf_gain_list = [mid_gain_cal(ticker) for ticker in etf_index]
        etf_gain_avg = np.average(etf_gain_list)

        realgainrate_mid = np.round((mid_gain_avg*mid_gain_adjust_percent+etf_gain_avg*mid_gain_market_reflect_percent_by_etf)*100, 2) # reflect market trend 20%
  
        def strategy_analysis_mid_gain(tickers):
            global min_SuccessRate
            print('\n\n' + 'Mid-Gain Percentage Calculating' + '\n')
            min_SuccessRate_original = min_SuccessRate
            min_SuccessRate = int(min_SuccessRate * 0.8)
            
            strategy_analysis_gain = []
            for ticker in tickers:
                strategy_analysis_result = strategy_analysis_main(ticker, 'off', 'off')[1]
                if strategy_analysis_result != -10:
                    strategy_analysis_gain.append(strategy_analysis_result)
            min_SuccessRate = min_SuccessRate_original
            return np.average(strategy_analysis_gain)*100*(2/3) #<--------- adjust
        
        realgainrate_mid = np.round(realgainrate_mid + strategy_analysis_mid_gain(robin['index'].values.tolist()), 2)
        print(f'\n\n• Mid-Gain Percentage Before Adjusting : {realgainrate_mid}\n')
       
        print(f'• Mid-Gain Adjust Percentage - Portfolio:   {mid_gain_adjust_percent}\n\n• Mid-Gain Adjust Percentage - Market(ETF): {mid_gain_market_reflect_percent_by_etf}\n\n• Initial Realize Gain Mid: {realgainrate_mid}\n\n• Realize Gain Max: {realize_gain_max}\n\n• Realize Gain Min: {realize_gain_min}')
        print('\n===========================================================================================\n')

        # threshold - max. gain
        if realgainrate_mid > realize_gain_max:
            realgainrate_mid = realize_gain_max
            message = f'\n\n\nImportant >>> Mid Realize Gain is higher than the limit, reset at {realize_gain_max} %\n\n\n'
            print(message)

        # proba value reset by market trend before adjusting    
        if realgainrate_mid < -0.5:
            predict_proba_value = predict_proba_value + 0.025
            print(f'\n\n\nImportant >>> Mid Realize Gain is less than the limit, Predict Proba Value increase by 2.5 %\n\n\n')
            print(f'\n\n===> New Probe Value: {np.round(predict_proba_value, 2)} <===\n\n')
        elif realgainrate_mid < -1:
            predict_proba_value = predict_proba_value + 0.05
            print(f'\n\n\nImportant >>> Mid Realize Gain is less than the limit, Predict Proba Value increase by 5 %\n\n\n')
            print(f'\n\n===> New Probe Value: {np.round(predict_proba_value, 2)} <===\n\n')
        elif realgainrate_mid < -1.5:
            predict_proba_value = predict_proba_value + 0.1
            print(f'\n\n\nImportant >>> Mid Realize Gain is less than the limit, Predict Proba Value increase by 10 %\n\n\n')
            print(f'\n\n===> New Probe Value: {np.round(predict_proba_value, 2)} <===\n\n')
        else:
            pass    
        
        # threshold - min. gain
        if realgainrate_mid < realize_gain_min:
            realgainrate_mid = realize_gain_min
            message = f'\n\n\nImportant >>> Mid Realize Gain is smaller than the limit, reset at {realize_gain_min} %\n\n\n'
            print(message)
            
#             emailsend_to_server(message)
    
        # if event day triggering a huge volatility, cut the gain
        if upcoming_remain == 0:
            analysis_market = analysis_price('^GSPC')
            if analysis_market[1]+analysis_market[7]+analysis_market[10]+analysis_market[11]+analysis_market[12] == 0:
                realgainrate_mid = realgainrate_mid * event_risk_ratio
            else:
                print('\n\n\nAn event is scheduled today, but the S&P 500 index is not in a downturn. No adjusting Mid-Term Margin.\n\n\n')
        else:
            pass
##############################################################################################################################
        
        # get the loss-cut percentage
        losscutpercentage = -np.round(realgainrate_mid, 2) #<<<<<<<<<<<<<<<<<< Important <<<<<<<<<<<<<<<<<<<<<<<<
        with open("losscutpercentage.txt", 'w') as f:
            f.write(str(losscutpercentage))

        losscut_index = losscut[losscut['percent_change'] <= losscutpercentage].sort_values('percent_change').reset_index(drop = True)['index'].tolist()
        losscut_index = [i for i in losscut_index if i in holding_period_index]
        
        # ETF tickers
        etf = ['XLE', 'XLF', 'XLRE', 'XLB', 'XLK', 'XLP', 'XLV', 'XLI', 'XLY', 'XLU', 'XLC', 'QQQ', 'SPY', 'SPYD', 'SPYG', 'SPYV']  

        # Big Sell (excl. ETF)
        snp_sector = pd.read_csv('./snp500/snp_sector_index.csv')
        try:
            on_hand_non_etf = robin['index'].values
            on_hand_non_etf = [i for i in on_hand_non_etf if i not in etf and i not in inv_tickers and i not in dead_stock]
            
            tickers_to_sell_big = []
            for i in sorted(on_hand_non_etf):
                print(f'Big Ticker Analyzing: {i}', end = '           \r')
                analysis = analysis_price(i)
                if rsi(i)[0] == True and analysis_price(snp_sector[snp_sector['Symbol'] == i]['Ticker'].values[0])[5] < analysis[5] and volatility_check(i) < volatility_check('^GSPC') and analysis[1]+analysis[7]+analysis[10]+analysis[11]+analysis[12] >= 3:
                    tickers_to_sell_big.append(i)
            print(end = '                                                              \r')
        except:
            print('Except >>> tickers_to_sell_big - RSI Check Error: ', i)
            alarm()
            pass
        tickers_to_sell_big.extend(dead_stock) #temp #####################################################
        tickers_to_sell_big = [i for n, i in enumerate(tickers_to_sell_big) if i not in tickers_to_sell_big[:n]] # duplicated
        
        with open("tickers_to_sell_big.txt", 'w') as f:
            for s in tickers_to_sell_big:  
                f.write(str(s) + '\n')     
        realgainrate_big = np.round(realgainrate_mid*1.1, 2) #L/T gain%
        with open("realgainrate_big.txt", 'w') as f:
            f.write(str(realgainrate_big))

        # Inverse Sell 
        tickers_to_sell_inv = [i for i in robin['index'].values.tolist() if i in inv_tickers]
        with open("tickers_to_sell_inv.txt", 'w') as f:
            for s in tickers_to_sell_inv:  
                f.write(str(s) + '\n')   
        realgainrate_inv = np.round(realgainrate_mid*0.9, 2) #Inv gain%
        with open("realgainrate_inv.txt", 'w') as f:
            f.write(str(realgainrate_inv))

        # Loss-cut tickers
        tickers_to_sell_small = []
        for s in losscut_index:
            if s not in tickers_to_sell_big:
                tickers_to_sell_small.append(s)

        # Exclude 'Sold' list from tickers_to_sell_small
        with open(f'sold_{currentdate}.txt', 'r') as f:
            sold = [line.rstrip('\n') for line in f]
        for i in sold:
            if i in tickers_to_sell_small:
                tickers_to_sell_small.remove(i)

        with open("tickers_to_sell_small.txt", 'w') as f:
            for s in tickers_to_sell_small:  
                f.write(str(s) + '\n')    
                
        realgainrate_small = np.round(losscutpercentage*2, 2) #loss cut%  #<<<<<<<<<<<<<<<<<<<<<<<<< Important >>>>>>>>>>>>>>>>
        with open("realgainrate_small.txt", 'w') as f:
            f.write(str(realgainrate_small))
            
        print('\n===[ Realize Gain or Loss ]================================================================')
        print("\n • Loss-cut Min. Holding Period: " + '\033[1m' + f"{min_holding_period} days" + '\033[0m')
        print('\n', f"• Loss Cut Execution : " + '\033[1m' + f"{realgainrate_small} % (Max. Loss Cut) <---x2---  {losscutpercentage} % (Monitoring Start)" + '\033[0m')
        print('___________________________________________________________________________________________')
        for i in tickers_to_sell_small:
            adj_losscut = np.round(realgainrate_small * (1-(volatility_check(i) - volatility_check('^GSPC'))), 2)
            print('\n', "  >>> Tickers to monitor Loss Cut: " + '\033[1m' + f" {i} " + '\033[0m' + " at " + '\033[1m' + f"{adj_losscut} % <===  {robin[robin['index'] == i]['percent_change'].values[0]} %" + '\033[0m')
            print('___________________________________________________________________________________________')         

        # Mid Sell
        tickers_to_sell_mid = []
        for s in portfolio_list:
            if s not in tickers_to_sell_big and s not in tickers_to_sell_small and s not in tickers_to_sell_inv and s not in etf:
                tickers_to_sell_mid.append(s)
        with open("tickers_to_sell_mid.txt", 'w') as f:
            for s in tickers_to_sell_mid:  
                f.write(str(s) + '\n')     
        # gain% is already set above
        with open("realgainrate_mid.txt", 'w') as f:
            f.write(str(realgainrate_mid))

        # ETF Sell 
        tickers_to_sell_etf = [i for i in robin['index'].values.tolist() if i in etf and i not in tickers_to_sell_small]
        with open("tickers_to_sell_etf.txt", 'w') as f:
            for s in tickers_to_sell_etf:  
                f.write(str(s) + '\n')   
        realgainrate_etf = np.round(realgainrate_mid*0.9, 2) #ETF gain%
        with open("realgainrate_etf.txt", 'w') as f:
            f.write(str(realgainrate_etf))

        print('\n',f"• Tickers to sell BIG: {tickers_to_sell_big} at " + '\033[1m' + f"Max. {np.round(realgainrate_big, 2)} %" + '\033[0m')
        print('\n',f"• Tickers to sell MID: {tickers_to_sell_mid} at " + '\033[1m' + f"Max. {np.round(realgainrate_mid, 2)} %" + '\033[0m')
        print('\n',f"• Tickers to sell INV: {tickers_to_sell_inv} at " + '\033[1m' + f"Max. {np.round(realgainrate_inv, 2)} %" + '\033[0m')
        print('\n',f"• Tickers to sell ETF: {tickers_to_sell_etf} at " + '\033[1m' + f"Max. {np.round(realgainrate_etf, 2)} %" + '\033[0m')
        print('\n',f"• Tickers to Loss-Cut: {tickers_to_sell_small} at " + '\033[1m' + f"Max. {np.round(realgainrate_small, 2)} %" + '\033[0m')
        print('\n================================================================[ Realize Gain or Loss ]===\n')
        print('\n\n\n')
    except:
        print('Except >>> Robin Equity Check', end = '                                                                \r')
        robin_equity_check()

    return robin


def robin_plot(): # equity check
    try:
        
        def gain_loss_plot(df):
            gain = df[df['change'] >= 0].sort_values('change',ascending = False).reset_index(drop = True)
            loss = df[df['change'] < 0].sort_values('change',ascending = False).reset_index(drop = True)

            plt.figure(figsize = (15,5))
            plt.bar(gain['index'], gain['change']*100, alpha = 0.95, color = sb.color_palette()[0])
            plt.bar(loss['index'], loss['change']*100, alpha = 0.95, color = sb.color_palette()[3])
            plt.title('Robinhood - Total Return(%)')
            plt.grid(axis = 'x')
            plt.grid(axis = 'y')
            plt.show()

        # SEC Gain & Loss
        krw = exchange_check()
        currentprice = yfinance_df_min('005930.KS')['Adj Close'].values[0]
        cost = 78325
        shares = 5000
        sec_gain_loss = ((currentprice - cost) * shares) / krw
        sec_values = (currentprice * shares) / krw
        print('\n')
        print('\033[1m' + 'Portfolio' + '\033[0m', '\n')
        print('- SEC Gain & Loss: ' + '\033[1m' + str(int(sec_gain_loss)) + '\033[0m', 'USD', '\n')

        login = rs.login('aicpasean@gmail.com', sean.robin_api())
        my_stocks = rs.build_holdings()
        rs.logout()
        robin = pd.DataFrame(my_stocks)
        robin = robin.T.reset_index()

        mkv = []
        for i in range(len(robin.equity.values)):
            value = float(robin.equity.values[i])
            mkv.append(value)  
        percentage = []
        for i in range(len(robin.percent_change.values)):
            value = float(robin.percent_change.values[i])
            percentage.append(value)  
        percentage = np.array(percentage)*0.01
        gain_loss = sum(mkv) - sum(mkv / (percentage + 1)) 
        print('- Robinhood Gain & Loss: ' + '\033[1m' + str(int(gain_loss)) + '\033[0m', 'USD')
        print('___________________________________\n')
        print('> Total Gain & Loss: ' + '\033[1m' + str(int(sec_gain_loss + gain_loss)) + '\033[0m', 'USD', '\n')
        print('> Total Asset: ' + '\033[1m' + str(int(sec_values + sum(mkv))) + '\033[0m', 'USD', '\n')

        # plot - Robinhood
        portfolio_symbols = robin.reset_index()['index'].values
        df = pd.DataFrame({'index': portfolio_symbols, 'change' : percentage})

        df.to_csv('robinhood_gain_loss.csv', index = False)
        gain_loss_plot(df)

        # plot only alive tickers
        df_plot = []
        for ticker, change in df.values:
            if ticker not in dead_stock:
                df_plot.append([ticker, change])       
                
        if len(df_plot) != 0:
            df = pd.DataFrame(np.array(df_plot), columns = ['index', 'change'])
            df.change = df.change.astype(float)
            print('\n\n', '\033[1m' + 'Active Tickers' + '\033[0m', '\n')
            gain_loss_plot(df)
    
    except:
        print('\nExcept >>> robin_plot()\n\n')
        alarm()
        robin_plot()
    

def volume_speed(ticker):
    with open("averageline.txt", 'r') as f:
        averageline = [line.rstrip('\n') for line in f]
        averageline = int(averageline[0])

    # Market hour - 09:30 ~ 16:00, 6.5 hours or 390 minutes
    minutepassed = timechecknow()

    df = yfinance_df(ticker)
    drp = df.loc[~(df!=0).all(axis=1)].index.values
    df = df.drop(drp)  

    # volume speed check    
    ticker_volume_mean = df.tail(averageline)['Volume'].mean()   
    ticker_volume_current = df.tail(1)['Volume'].mean()
    current_speed_volume = ticker_volume_current / minutepassed
    mean_speed_volume = ticker_volume_mean / 390
    volumeSpeed_vs_mean = current_speed_volume / mean_speed_volume 

    return current_speed_volume, volumeSpeed_vs_mean

        
def volume_vs_sign(stock):
    
    time.sleep(0.3)
    # remove candle_analysis.csv for the previous date
    candel_exists = exists('candle_analysis_{}_%s.csv'.format(stock) %mostrecentdate)
    if candel_exists == True:
        print('The old version of the candle will be removed from candle_gathering(). Ticker: ', stock)
        import os
        os.remove('candle_analysis_{}_%s.csv'.format(stock) %mostrecentdate)
    else:
        pass
    candel_exists = exists('candle_analysis_{}_%s.csv'.format(stock) %currentdate)
    if candel_exists == True:
        pass
    else:
        # create initial data for today
        # Market hour - 09:30 ~ 16:00, 6.5 hours or 390 minutes
        minutepassed = timechecknow()
        candle = yfinance_df(stock).tail(1)[['Date', 'Adj Close','Volume']]
    
        candle['Minutepassed'] = minutepassed
        candle['VolumeSpeed'] = volume_speed(stock)[0]
        candle['VolumeSpeed_vs_mean'] = volume_speed(stock)[1]
        candle.to_csv('candle_analysis_{}_%s.csv'.format(stock) %currentdate)
        
    # Market hour - 09:30 ~ 16:00, 6.5 hours or 390 minutes
    minutepassed = timechecknow()

    # recent
    candle = yfinance_df(stock).tail(1)[['Date', 'Adj Close','Volume']]
    candle['Minutepassed'] = minutepassed
    candle['VolumeSpeed'] = volume_speed(stock)[0]
    candle['VolumeSpeed_vs_mean'] = volume_speed(stock)[1]

    # accumulated
    candle_accu = pd.read_csv('candle_analysis_{}_%s.csv'.format(stock) %currentdate, index_col = 0)
    candle_accu['Date'] = pd.to_datetime(candle_accu['Date']) #type change to datetime

    # merge
    candle = pd.concat([candle_accu, candle]).reset_index(drop = True)
    candle = candle[['Date', 'Minutepassed', 'Adj Close', 'Volume', 'VolumeSpeed', 'VolumeSpeed_vs_mean']]
    
    # Volume change
    volume_change = []
    for i in range(len(candle.Volume)):
        if i-1 < 0:
            volume_change.append(0)
        else:
            volumeChange = ((candle.Volume[i] - candle.Volume[i-1]) / candle.Volume[i-1]) * 100
            volume_change.append(volumeChange)
    candle['Volume_change'] = volume_change
    
    # Price change
    price_change = []
    for i in range(len(candle['Adj Close'])):
        if i-1 < 0:
            price_change.append(0)
        else:
            priceChange = ((candle['Adj Close'][i] - candle['Adj Close'][i-1]) / candle['Adj Close'][i-1]) * 100
            price_change.append(priceChange)
    candle['Price_change'] = price_change
    
    # Volume speed acceleration
    volume_speed_change = []
    for i in range(len(candle.VolumeSpeed)):
        if i-1 < 0:
            volume_speed_change.append(0)
        else:
            with np.errstate(divide='ignore'):
                volumeSpeedChange = (candle.VolumeSpeed[i] - candle.VolumeSpeed[i-1]) / (candle.Minutepassed[i] - candle.Minutepassed[i-1])
            volume_speed_change.append(volumeSpeedChange)
    candle['Volume_speed_acceleration'] = volume_speed_change   
    # drop non numeric rows
    candle = candle[pd.to_numeric(candle['Volume_speed_acceleration'], errors='coerce').notnull()].reset_index(drop = True)
    
    # Fill 0 as 1 for the change
    candle['Volume_change'] = candle['Volume_change'].replace(0, 0.000001)
    candle['Price_change'] = candle['Price_change'].replace(0, 0.000001)

    # Volume Change per Price Change
    candle['VolumeC_per_PriceC'] = np.absolute(candle['Volume_change'] / candle['Price_change'])

    # save
    candle.Minutepassed.drop_duplicates(inplace = True)
    candle.to_csv('candle_analysis_{}_%s.csv'.format(stock) %currentdate)

    return candle_sign_gen(candle, stock)

########################################################################################################################################################################
def candle_sign_gen(candle, stock):
    with open("averageline.txt", 'r') as f:
        averageline = [line.rstrip('\n') for line in f]
        averageline = int(averageline[0])
    
    actual_price_change = analysis_price(stock)[2]
    averageline_min_price = yfinance_df(stock).tail(averageline)['Adj Close'].min()

# buy-sign 
    # Volume(+++) + Price(+)
    min_passed = candle[-1:]['Minutepassed'].mean()
    VC_per_PC = candle[-1:]['VolumeC_per_PriceC'].mean()
    PC = candle[-1:]['Price_change'].mean()
    VS_vs_MEAN = candle[-1:]['VolumeSpeed_vs_mean'].mean()
    VS_Accel = np.absolute(candle[-1:]['Volume_speed_acceleration'].mean())
    AVG_Price = candle[-1:]['Adj Close'].mean()

    if VC_per_PC >= VC_per_PC_limit_V and PC >= PC_limit_V and VS_vs_MEAN >= VS_vs_MEAN_limit and VS_Accel >= VS_Accel_limit and AVG_Price <= averageline_min_price*2:

        print(f'\n\n===[Volume Buy Sign: {stock}]=========================================================\n')  
        print(current_time()+'\n')
        print(f' > Minute Passed: {min_passed}\n')
        print(f' > Actual Price Change: {actual_price_change}\n')
        print(f' • Volume Change vs Price Change: {VC_per_PC} >= Limit: {VC_per_PC_limit_V}\n')
        print(f' • Price Change: {PC} >= Limit: {PC_limit_V}\n')
        print(f' • Volume Speed vs Mean: {VS_vs_MEAN} >= Limit: {VS_vs_MEAN_limit}\n')
        print(f' • Volume Speed Acceleration: {VS_Accel} >= Limit: {VS_Accel_limit}\n')
        print(f' • Candle Price vs Period Min Price x2: {AVG_Price} <= {averageline_min_price*2}\n')
        print('==================================================================================\n\n\n\n\n')        

        return 3    

# realize gain
    elif PC >= PC_limit_V and VS_vs_MEAN >= VS_vs_MEAN_limit and VS_Accel >= VS_Accel_limit:
        
#         if timechecknow() < 400:            
#             print(f'\n\n===[Realize Gain Sign: {stock}]===================================================\n')  
#             print(current_time()+'\n')
#             print(f' > Actual Price Change: {actual_price_change}\n')
#             print(f' x Volume Change vs Price Change: {VC_per_PC} >= Limit: {VC_per_PC_limit_V}\n')
#             print(f' • Price Change: {PC} >= Limit: {PC_limit_V}\n')
#             print(f' • Volume Speed vs Mean: {VS_vs_MEAN} >= Limit: {VS_vs_MEAN_limit}\n')
#             print(f' • Volume Speed Acceleration: {VS_Accel} >= Limit: {VS_Accel_limit}\n')
#             print(f' x Candle Price vs Period Min Price x2: {AVG_Price} <= {averageline_min_price*2}\n')
#             print('==================================================================================\n')   
           
        return 2    

# Volatile
    #2 Volume(+++) + Price(-)
    elif VC_per_PC <= VC_per_PC_limit_V and VS_vs_MEAN <= VS_vs_MEAN_limit:

#         print(f'\n\n===[Volatile Sign: {stock}]===========================================================\n')    
#         print(current_time()+'\n')
#         print(f' > Actual Price Change: {actual_price_change}\n')
#         print(f' • Volume Change vs Price Change: {VC_per_PC} <= Limit: {VC_per_PC_limit_V}\n')
#         print(f' • Volume Speed vs Mean: {VS_vs_MEAN} <= Limit: {VS_vs_MEAN_limit}\n')
#         print('==================================================================================\n')    

        return 1


# sell-sign
    #3 Price(-, --) + Volume(++) + VolumeSpeedAccel(+) 
    elif VC_per_PC > np.sqrt(VC_per_PC_limit_V) and PC < -0.3 and VS_vs_MEAN < VS_vs_MEAN_limit and VS_Accel >= VS_Accel_limit:

        message = f'Candle Alert: {stock} is short selling at -30%.'
        webbrowser.open("Nuclear Launch Detected.mp3")
        time.sleep(2.5)
        print('\n\n\n (-)(-)(-)', message, '(-)(-)(-)\n\n\n')
        voice_message(message)

        return -3

    else:
        return 0

    
if __name__ == '__main__':
    print('\n', '\033[1m' + 'The Engine#4 gets ready.' + '\033[0m', '\n\n\n')
#     robin_equity_check()

___

# Start

In [28]:
# monitoring time count
import time
tstart = time.time()

# portfolio monitoring start within schedules():
def schedules():
    global cyclecount
    global bluechips
    global corr
    global proba_initial_value
    global yf_counter
    
    # reset the counter checking the number of request to yfinance
    yf_counter = 0

    # cycle work starts from here
    tend = time.time()
    tpss = tend - tstart

    cyclecount+=1

    print('\n\n\n\n\n\n\n\n\n\n')
    print(np.round(tpss/60), ' minutes elapsed >> {}th << cycle start'.format(cyclecount))
    print('_____________________________________________________________________________')
    
    # scheduled monitoring starts
    # time passed
    print('\n'+current_time()+'\n')
    
    with open(f'purchased_{currentdate}.txt', 'r') as f:
        purchased = [line.rstrip('\n') for line in f]

    with open(f'sold_{currentdate}.txt', 'r') as f:
        sold = [line.rstrip('\n') for line in f]
        
    with open("portfolio_original.txt", 'r') as f:
        portfolio_original = [line.rstrip('\n') for line in f]

    with open("etf_portfolio.txt", 'r') as f:
        etf_portfolio = [line.rstrip('\n') for line in f]
        
    portfolio_original+=etf_portfolio
        
        
    # sell monitor cycle check    
    if sellmonitor_freq == 1:
        print('\033[1m' + '\n\n\nThe number of holding stocks has reached the limit. No buy monitor executes.\n\n' + '\033[0m')
    elif sellmonitor_freq < 1:
        message = 'Sell Monitor frequency is less than 1. Code cell breaks.'
        print('\n\n\n' + '\033[1m' + 'Except >>> ' + message + '\n\n' + '\033[0m')
        alarm()    
        # code cell break ###############
        class StopExecution(Exception):
            def _render_traceback_(self):
                pass

        raise StopExecution
        # code cell break ###############
    else:
        pass
        
    # S&P 500 Percentage Change Check
    snp_now = yfinance_df_setting('^GSPC').tail(2)
    snp_now = (snp_now['Adj Close'].values[1] - snp_now['Adj Close'].values[0])/snp_now['Adj Close'].values[0]*100

    # display list
    print('\033[1m' + f"\n\n• S&P 500 Today: {np.round(snp_now,2)}%\n"+ '\033[0m')    
    print(f'• Purchased List: {purchased}\n')
    print(f'• Sold List:      {sold}\n')  
    print("• Sell monitoring cycle: " + '\033[1m' + f"{sellmonitor_freq} cycles\n" + '\033[0m')
    print(f'• Proba Initial Value: {np.round(proba_initial_value, 2)}\n')
    print(f'• Min. Proba Value:    {np.round(predict_proba_value, 2)}\n\n\n')
    
    # magic quote
    print(magic.quote.sample(np.random.randint(1,magic.shape[0],1)[0]).values[0].strip(' '))
    print('\n\n\n')
    
    # Market hour - 09:30 ~ 16:00, 6.5 hours or 390 minutes
    minutepassed = timechecknow()
    if minutepassed > 390:  
        
        robin_plot() 
        
        # after market MUST #######################
        performance_review()        
        buy_blacklistcheck_norm(portfolio_original)
        ###########################################
        
        from os.path import exists
        perf_review_exists = exists('perf_review %s.csv' %currentdate)
        df = pd.read_csv('daily_summary_target_tickers.csv')
        print(f'\n> Date Verifying: perf_review ==================> {perf_review_exists}')
        print(f'> Date Verifying: daily_summary_target_tickers => {df[-1:].Date.values[0] == currentdate}')
        
        if df[-1:].Date.values[0] == currentdate and perf_review_exists == True:
            print('\n\nThe performance review data gets ready.\n\n\n\n\n')
        
            message = """\
Subject: The market has been closed. The Performance review data gets ready."""
            
        else:
            print('\n\nExcept >>> The performance review data has not been updated.')
            
            message = """\
Subject: Except>> The market has been closed. The performance review data has not been updated."""
        
        peertickers()    
        
        real_performance_selma()
        
        print(f'\n\n{current_time()}  The market has been closed. After market works start\n\n')
        time.sleep(5)
        
        
# after market works
        rsi_portfolio()
        
        with open(f"qed_monitoring_done_{currentdate}.txt", 'w') as f:
            f.write(str(timechecknow()))  
        
#         print('\n', '\033[1m' + '>>> Value Ticker Sorting .....' + '\033[1m', '\n')
#         voice_message("""\
#             Value ticker sorting starts""")
#         value_snp_pick()
#         print(f'\n\n{current_time()}  After market works have been completed. End of QED.\n\n')
#         voice_message("""\
#             After market works have been completed.""")

#         with open(f"qed_monitoring_done_{currentdate}.txt", 'w') as f:
#             f.write(str(timechecknow()))  
            
        ##################################### End of QED Monitoring ###########################################

        
    elif minutepassed < -60:
        print('Standby Mode', end = '          \r')
        time.sleep(3600)           
        
    elif minutepassed < -30:
        print('Standby Mode', end = '          \r')
        time.sleep(300)        
        
    elif minutepassed < 0:
        print('Standby Mode', end = '          \r')
        time.sleep(60)          
                
    else:
#         if minutepassed > 220 and minutepassed < 225:
#             webbrowser.open("baron1.m4a")
#             sean_index_plot()
#             real_performance_selma()
#             sectorcheck_main(1)
    
        if cyclecount%30 == 0:  
#             robin_plot()
            exrate()  
            super_pass_list = super_pass_filtering(superpass_buysign_lookbackdays)
#             for stock_ticker in super_pass_list:
#                 sector_stock_plot(stock_ticker)  
            
        elif cyclecount%sellmonitor_freq == 0:  # edit this when loss-cut excuted for big-stocks
            daily_news_sentiment_all(display = 'on')
            macrosnp() # snp500 drop % check and adjust Proba value
            # moving average plot for holding tickers
            for ticker in sorted(robin_login()['index'].tolist()):
#                 try:
#                     sector_stock_plot(ticker)
#                 except:
#                     pass
                dmv_plot(ticker)
#                 rsi_plot(ticker)               
            sellmonitor() #icld. robin_equity_check() to update: losscut rate, realize gain rate
            
        else:
            portfolio_backtest()
            buymonitor()
            candle_gathering()
               
            with open("_QED Time Checker.txt", 'w') as f:
                f.write(str(current_time()))  

                
def main():
    print('The engine has been initialized.\n\n\n')
    
    ## initial modules
#     webbrowser.open("baron.m4a")

    if timechecknow() < 60: ###### Adjust time here when delayed start
        if str(yfinance_df_setting('^GSPC')[-1:].Date.values[0])[:10] == currentdate:
            
            voice_message("""\
                Market starts""")
            
#             # realize gain or loss
#             robin_plot()
            # moving average plot for holding tickers
            for ticker in sorted(robin_login()['index'].tolist()):
                try:
                    sector_stock_plot(ticker)
                except:
                    pass
                dmv_plot(ticker)
                rsi_plot(ticker)          
                
#             sellmonitor() #icld. robin_equity_check() to update: losscut rate, realize gain rate

            # portfolio creation
            main_creation()   # start when the current date == yFinance Date
        
            # proba value reset - L/T
            proba_value_reset()
            
            # proba value reset - S/T
            macrosnp()
        
            try:
                snp_predict = valuecheck('^GSPC')
            except:
                snp_predict = valuecheck('^GSPC')
            sean_index_plot()
            
            if snp_predict[1] == 1:
                voice_message("""\
                    S&P 500 will increase""")
            elif snp_predict[1] > 0.3:
                voice_message("""\
                    S&P 500 will volatile""")        
            else:
                voice_message("""\
                    S&P 500 will decrease""")
            time.sleep(10)
            
            # parameter setting by tf
            main_param()
            
        else:
            print('> The last line != currentdate\n')
            if timechecknow() < -30:
                print('Stand by Mode', end = '          \r')
                voice_message("""\
                              Opening bell stand by for 30 minutes""")                
                time.sleep(1800)
            elif timechecknow() < -10:
                print('Stand by Mode', end = '          \r')
                voice_message("""\
                              Opening bell stand by for 10 minutes""")                
                time.sleep(600)
            elif timechecknow() < -5:
                print('Stand by Mode', end = '          \r')
                voice_message("""\
                              Opening bell stand by for 5 minutes""")                
                time.sleep(300)
            else:
                print('Stand by Mode', end = '          \r')
                time.sleep(60)   
            main()
    else:
        pass
        
    # initial secter valuation
    sector_valuation_ticker = pd.read_csv('sector_valuation.csv')
    if sector_valuation_ticker[-1:].Date.values[0] != currentdate:
        sectorcheck_main(1)
        
        # initial fbeta score check with buymonitor_whitelist  
        with open("portfolio.txt", 'r') as f:
            portfolio = [line.rstrip('\n') for line in f]
        prevaluation_tickers = []
        if len(portfolio) >= 5:
            for i in range(5):
                prevaluation_tickers.append(random.choice(portfolio))
        valuation(prevaluation_tickers)
        
        # delete past files
        from os.path import exists
        predict_valuation_today_exists = exists('predict_valuation_today %s.csv' %mostrecentdate)
        if predict_valuation_today_exists == True:
            import os
            os.remove('predict_valuation_today %s.csv' %mostrecentdate)
        else:
            pass    
        
        peertickers()  
        exrate()
    else:
        pass
    
    #     alarm_open(3)    
    
    if upcoming_remain == 0:
        voice_message(f"""\
        We have an event scheduled today. Be careful Sean
        """)
        time.sleep(5)
    else:
        pass
    
    voice_message("""\
        Engine starts""")
    
#     # realize gain or loss
#     robin_plot()
    daily_news_sentiment_all(display = 'on')
    
    while True:
        # check if monitoring done
        from os.path import exists
        qed_monitoring_done = exists(f"qed_monitoring_done_{currentdate}.txt")
        
        if qed_monitoring_done == True:
            break
        else:
            schedules()

            
# count
cyclecount = 0
day_trading_count = 0
    
if __name__ == '__main__':
    if day_check() != 'Saturday' and day_check() != 'Sunday' and timechecknow() < 420:
        
#         webbrowser.open("Nuclear Launch Detected.mp3")
#         alarm_open(1)

        voice_message("""\
            Engine gets ready""")
    
        exrate()
    
        # trade list update
        blank = 'blank'
        trade_list('purchased', blank)
        trade_list('sold', blank)

        with open(f'purchased_{currentdate}.txt', 'r') as f:
            purchased = [line.rstrip('\n') for line in f]

        with open(f'sold_{currentdate}.txt', 'r') as f:
            sold = [line.rstrip('\n') for line in f]
        
        print(f'\n\n• Purchased List: {purchased}\n')
        print(f'• Sold List:      {sold}\n\n')  
      
        # proba value reset - L/T
        proba_value_reset()
    
        # execute when delayed start
#         main_creation()     # <<<<<<<<<<<<<<<<<   
#         sectorcheck_main(1) # <<<<<<<<<<<<<<<<<

        main()
        
    else:
        print('\n', '\033[1m' + 'The market is closed today.' + '\033[0m', '\n')
        
# import FinanceDataReader as fdr
# fdr.DataReader('GOOGL').reset_index()

___

In [30]:
# with open("portfolio_original.txt", 'r') as f:
#     portfolio_original = [line.rstrip('\n') for line in f] 
# performance_review()        
# buy_blacklistcheck_norm(portfolio_original)

___

After Market Modules

In [31]:
if day_check() != 'Saturday' and day_check() != 'Sunday' and timechecknow() > 390:
    # plt.style.use('classic')
    yf_day_plus = datetime.today() + timedelta(days = 1)
    yf_day_plus = str(yf_day_plus)[:10]
    
else:
    print('Market is not closed. Code Cell Break')
    alarm()
    
    # code cell break ###############
    class StopExecution(Exception):
        def _render_traceback_(self):
            pass

    raise StopExecution
    # code cell break ###############
    


# Original #####################################################################################################################

# def stock_data(ticker):
#     try:
#         time.sleep(0.1)
#         start_date = str(datetime.today() - timedelta(days = 300))[:10]
#         return data.DataReader(ticker, data_source, start_date, retry_count=10).reset_index()
#     except:
#         return yf.download(ticker, end = yf_day_plus, prepost = True, progress=False, show_errors=False).reset_index()
#         pass
    
################################################################################################################################
    
# Sub ##########################################################################################################################
    
# def yfinance_clean(df):
#     find = df.all().reset_index().rename(columns = {0:'result'}).set_index('result').reset_index()
#     find = find[find['result'] == False]['index'].values.tolist()
#     for i in find:
#         df[i] = df[i].replace(0, df[i].tail(20).mean())
#     return df
    
def stock_data(ticker):

    try:
        start_date = str(datetime.today() - timedelta(days = 300))[:10]   
        df = yf.download(ticker, start = start_date, end = yf_day_plus, prepost = True, progress=False).reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        df['Date'] = df.Date.astype('datetime64[ns]')        
#         return yfinance_clean(df)  
        return df

    except:
        return stock_data(ticker)
   
################################################################################################################################
  


# code cell warpping
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Modules Imported"></form>
''')

# Review

In [32]:
# Original #####################################################################################################################

# def yfinance_df_setting(ticker):
#     return data.DataReader(ticker, data_source, start_date).reset_index()

################################################################################################################################

# Sub ##########################################################################################################################
    
def yfinance_df_setting(ticker):
    try:
        # start date is since 1970, which is default value
        start_date = '1970-01-01'
        df = yf.download(ticker, start = start_date, end = yf_day_plus, prepost = True, progress=False, show_errors=False).reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        df['Date'] = df.Date.astype('datetime64[ns]')
        return df
#         return yfinance_clean(df)  # when Date type is datetime
    except ValueError:
        return data.DataReader(ticker, data_source, start_date, retry_count=100, pause=0.5).reset_index()
        pass

################################################################################################################################

def robin_equity_check():
    login = rs.login('aicpasean@gmail.com', sean.robin_api())
    my_stocks = rs.build_holdings()
    rs.logout()
    robin = pd.DataFrame(my_stocks)
    robin = robin.T.reset_index()
    vars = ['price', 'quantity', 'average_buy_price', 'equity', 'percent_change', 
            'intraday_percent_change', 'equity_change', 'pe_ratio', 'percentage']
    for var in vars:
        robin[var] = pd.to_numeric(robin[var])
    robin['quantity'] = robin['quantity'].astype(int)
    robin.to_csv('robin_all.csv', index = False)
    return robin
     

if __name__ == '__main__':
    
    with open("mostrecentdate.txt", 'r') as f:
        mostrecentdate = int([line.rstrip('\n') for line in f][0])
    with open("currentdate.txt", 'r') as f:
        currentdate = int([line.rstrip('\n') for line in f][0])
    with open("averageline.txt", 'r') as f:
        averageline = [line.rstrip('\n') for line in f]
        averageline = int(averageline[0])
    with open("portfolio_original.txt", 'r') as f:
        portfolio = [line.rstrip('\n') for line in f]
    with open("buymonitor_blacklist_original.txt", 'r') as f:
        buymonitor_blacklist = [line.rstrip('\n') for line in f]   

    mostrecentdate = str(mostrecentdate)[0:4]+'-'+str(mostrecentdate)[4:6]+'-'+str(mostrecentdate)[6:8]
    currentdate = str(currentdate)[0:4]+'-'+str(currentdate)[4:6]+'-'+str(currentdate)[6:8]

    from os.path import exists
    perf_review_exists = exists('perf_review %s.csv' %currentdate)
    df = pd.read_csv('daily_summary_target_tickers.csv')
    
    print(f'> Date Verifying: perf_review ==================> {perf_review_exists}')
    print(f'> Date Verifying: daily_summary_target_tickers => {df[-1:].Date.values[0] == currentdate}')
    
    print('\n\n • mostrecentdate: ', '\033[1m' + f"{mostrecentdate}" + '\033[0m')
    print('\n • currentdate:    ', '\033[1m' + f"{currentdate}" + '\033[0m')
    
    if df[-1:].Date.values[0] == currentdate and perf_review_exists == True:
        df = df[df['Date'] == currentdate]
        df = df.tail(df['index'].nunique())[['index', 'Change', 'Today', 'Yesterday', 'Average']].sort_values('Change', ascending = False).reset_index(drop=True)
        srank = pd.read_csv('srank.csv')
        srank = srank[['Symbol']].reset_index()
        srank = srank.rename(columns = {'index' : 'Initial_Rank'})
        result = pd.merge(df, srank, how = 'inner', left_on = 'index', right_on = 'Symbol').reset_index()
        result = result.rename(columns = {'level_0' : 'Final_Rank'})
        result['Rank_Gap'] = result['Initial_Rank'] - result['Final_Rank'] 
        result_review = result[['Symbol', 'Change', 'Final_Rank', 'Initial_Rank', 'Rank_Gap', 'Today', 'Yesterday', 'Average']]
        
    else:
        print('Except >>> Performance data has not been updated.')
        print('Code cell break')
              
        # code cell break ###############
        class StopExecution(Exception):
            def _render_traceback_(self):
                pass

        raise StopExecution
        # code cell break ###############

## Percentage Change

In [33]:
snp_today = yfinance_df_setting('^GSPC').tail(2)
snp_today = (snp_today['Adj Close'].values[1] - snp_today['Adj Close'].values[0])/snp_today['Adj Close'].values[0]*100
print('\033[1m' + f"\nS&P 500 Today: {np.round(snp_today,2)}%"+ '\033[0m')

plt.figure(figsize = (16,18))
gain = df[df['Change'] > 0].sort_values('Change')
loss = df[df['Change'] < 0].sort_values('Change')

plt.barh(loss['index'], loss['Change'], alpha = 0.95, color = sb.color_palette()[3])
plt.barh(gain['index'], gain['Change'], alpha = 0.95, color = sb.color_palette()[0])

plt.axvline(x = snp_today, color = 'black', linewidth = 2)

plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.show();

### S&P 500 Plot

In [34]:
i = '^GSPC'
rsi_plot(i)
print('\n\n\n\n\n')

___

### Top Tickers

In [35]:
top_perf_tickers = df[df['Change'] > 0]['index'].tolist()
top_perf_tickers = [i for i in top_perf_tickers if i not in inv_tickers]
top_perf_tickers = top_perf_tickers[:3]
bottom_perf_tickers = df[df['Change'] < 0]['index'].tolist()
bottom_perf_tickers = [i for i in bottom_perf_tickers if i not in inv_tickers]
bottom_perf_tickers = bottom_perf_tickers[-3:]

if len(top_perf_tickers) != 0:
    for ticker in top_perf_tickers:
        try:
            rsi_plot(ticker)

            try:
                sector_stock_plot(ticker)
            except:
                print('\033[1m' + '\nExcept >>> Sector does not exist' + '\033[0m')
                pass

            volume_price_period_plot(ticker)

            try:
                candle_review(ticker)
            except:
                print('\033[1m' + 'Except >>> Candle does not exist' + '\033[0m')
                pass
        except:
            pass

___

### Bottom Tickers

In [36]:
if len(bottom_perf_tickers) != 0:
    for ticker in bottom_perf_tickers:
        try:
            rsi_plot(ticker)
            try:
                sector_stock_plot(ticker)
            except:
                print('\033[1m' + '\nExcept >>> Sector does not exist' + '\033[0m')
                pass

            volume_price_period_plot(ticker)

            try:
                candle_review(ticker)
            except:
                print('\033[1m' + 'Except >>> Candle does not exist' + '\033[0m')
                pass    
        except:
            pass

### Inverse Tickers

In [37]:
for ticker in inv_tickers:
    try:
        rsi_plot(ticker)
        try:
            sector_stock_plot(ticker)
        except:
            print('\033[1m' + '\nExcept >>> Sector does not exist' + '\033[0m')  
            pass

        volume_price_period_plot(ticker)

        try:
            candle_review(ticker)
        except:
            print('\033[1m' + 'Except >>> Candle does not exist' + '\033[0m')
            pass    
    except:
        pass

___

## Sean Rank Gap

In [38]:
print('\033[1m' + 'Portfolio Score Gap Review (excl. ETF)' + '\033[0m')
result_gap = result_review.sort_values('Rank_Gap', ascending = False)
df = result_gap

plt.figure(figsize = (20,9))
plt.bar(df['Symbol'], df['Rank_Gap'], alpha = 0.95, color = sb.color_palette()[0])
plt.xticks(rotation = 90)
plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.show();

___

## **Portfolio Return**

In [39]:
df = pd.read_csv('daily_summary_target_tickers.csv')
df = df.groupby(['Date']).mean().reset_index()[['Date', 'Change']]
df['Date'] = df['Date'].astype('datetime64[ns]')

snp = yfinance_df_setting('^GSPC')[['Date', 'Adj Close']]

# delete rows during off-days: i.g, vacation # 
# snp = snp.drop(index = snp.query('Date > "2022-07-13" & Date < "2022-08-19"').index).reset_index(drop = True)
#############################################################################

snp_change = []
for i in range(snp.shape[0]):
    if i == 0:
        snp_change.append(0)
    else:
        snp_change.append((snp['Adj Close'][i] - snp['Adj Close'][i-1])/snp['Adj Close'][i-1]*100)
snp['snp_change'] = snp_change

portfolio_performance = pd.merge(df, snp, how = 'inner', on = 'Date')
portfolio_performance.to_csv('portfolio_performance.csv', index = False)

portfolio_performance = portfolio_performance[-averageline:]

vola_port = np.std(portfolio_performance['Change']) * np.sqrt(portfolio_performance.shape[0]) / 100
vola_snp = np.std(portfolio_performance['snp_change']) * np.sqrt(portfolio_performance.shape[0]) / 100
vola_ratio = vola_port / vola_snp

portfolio_performance['Date'] = portfolio_performance['Date'].astype(str)
plt.figure(figsize = (16,18))
plt.subplot(2,1,2)
plt.errorbar(x = portfolio_performance['Date'], y = portfolio_performance['Change'], label = 'Portfolio +/-')
plt.errorbar(x = portfolio_performance['Date'], y = portfolio_performance['snp_change'], label = 'S&P 500 +/-')
plt.title('Portfolio vs S&P 500')
plt.xticks(rotation = 90)
plt.legend()
plt.grid(axis = 'x')
plt.grid(axis = 'y')

print(f"\nTotol Period: {portfolio_performance.shape[0]} days")
print("______________________________________")
print(f"\n• Volatility Portfolio:        {'%.5f' %vola_port}\n")
print(f"• Volatility S&P 500:          {'%.5f' %vola_snp}")
print(f"\n• Volatility Portfolio to S&P: " + '\033[1m' + f"{'%.3f' %vola_ratio}" + '\033[0m')
print("______________________________________\n")

> Adjust S&P(snp_change) rows by dropping indexes during off-days

___

## **Fbeta Result**

In [40]:
from datetime import date, datetime, timedelta
day_plus = datetime.today() - timedelta(days = 10)
day_plus = str(day_plus)[:10]

prt = pd.read_csv('predict_valuation.csv')
prt = prt[prt['date'] >= day_plus]

prt = prt.groupby(['date', 'symbol']).mean().reset_index()
df = pd.read_csv('daily_summary_target_tickers.csv')
df = df.groupby(['Date', 'index']).mean().reset_index()

prt_result = pd.merge(df, prt, how = 'inner', left_on = 'index', right_on = 'symbol')[['index', 'Date', 'Change', 'valuation', 'up_down']].sort_values('Date').reset_index(drop = True)
prt_result['value_score'] = prt_result['up_down']

# H0: Bearish (negative)
# H1: Bullish (positive)

accuracy = []
for i in prt_result['index'].values.tolist():
    if prt_result[prt_result['index'] == i].Change.values[0] >= 0 and prt_result[prt_result['index'] == i].value_score.values[0] == 1:
        accuracy.append('True_Positive')
        
    elif prt_result[prt_result['index'] == i].Change.values[0] < 0 and prt_result[prt_result['index'] == i].value_score.values[0] == 1:
        accuracy.append('False_Positive') # Type I Error (Alpha) More Serious Error, Required Higher Precision than Recall
        
    elif prt_result[prt_result['index'] == i].Change.values[0] < 0 and prt_result[prt_result['index'] == i].value_score.values[0] < 1:
        accuracy.append('True_Negative')
        
    elif prt_result[prt_result['index'] == i].Change.values[0] >= 0 and prt_result[prt_result['index'] == i].value_score.values[0] < 1:
        accuracy.append('False_Negative') # Type II Error (Beta) 
    
    else:
        accuracy.append('NA')
prt_result['accuracy'] = accuracy

confusion_matrix = prt_result.accuracy.value_counts()

FN = confusion_matrix[0]
TN = confusion_matrix[1]
TP = confusion_matrix[2]
FP = confusion_matrix[3]

Accuracy = (TP + TN) / confusion_matrix.sum() * 100
Precision = TP / (TP + FP) * 100
Specificity = TN / (FP + TN) * 100
Error_Rate = (FP + FN) / confusion_matrix.sum() * 100
Recall = TP / (TP + FN) * 100
Fbeta = (1+(0.5*0.5))*(Precision*Recall)/((0.5*0.5*Precision)+Recall)

lb = pd.read_csv('predict_valuation.csv').up_down.mean()*100
print(f"predict_valuation - up_down Mean() *Classifier: {'%.2f'%lb}%\n")

print("_____________________")
print(f"\n• Accuracy:    " + '\033[1m' + f"{'%.2f' %Accuracy}%" + '\033[0m')
print(f"• Error Rate:  " + '\033[1m' + f"{'%.2f' %Error_Rate}%" + '\033[0m')

print(f"\n• Precision*:  " + '\033[1m' + f"{'%.2f' %Precision}%" + '\033[0m')
print(f"• Specificity: " + '\033[1m' + f"{'%.2f' %Specificity}%" + '\033[0m')

print(f"• Recall:      " + '\033[1m' + f"{'%.2f' %Recall}%" + '\033[0m')
print(f"\n• Fbeta:       " + '\033[1m' + f"{'%.2f' %Fbeta}%" + '\033[0m')
print("_____________________")

pd.DataFrame(confusion_matrix)

In [41]:
prt_score_today = {}

prt_score_today['Date'] = currentdate
prt_score_today['Accuracy'] = Accuracy
prt_score_today['Precision'] = Precision
prt_score_today['Specificity'] = Specificity
prt_score_today['Error_Rate'] = Error_Rate
prt_score_today['Recall'] = Recall
prt_score_today['Fbeta'] = Fbeta

prt_score_today = pd.DataFrame(prt_score_today, index = ['Score']).reset_index(drop = True)
prt_score_today['Date'] = prt_score_today['Date'].astype('datetime64[ns]')
prt_score_today.to_csv(f"fbeta_score_today_{currentdate}.csv", index = False)

# combine previous with current 
from os.path import exists
sourcedata_exists = exists(f"fbeta_score_{mostrecentdate}.csv")
if sourcedata_exists == True:
    prt_score_mostrecent = pd.read_csv(f"fbeta_score_{mostrecentdate}.csv")
    prt_score_current = pd.read_csv(f"fbeta_score_today_{currentdate}.csv")
    prt_score = pd.concat([prt_score_mostrecent, prt_score_current], ignore_index=True)
    prt_score = prt_score.drop_duplicates().sort_values('Date').reset_index(drop = True)
    # export
    prt_score.to_csv(f"fbeta_score_{currentdate}.csv", index = False)

    import os
    os.remove(f"fbeta_score_{mostrecentdate}.csv")
    os.remove(f"fbeta_score_today_{currentdate}.csv")
else:
    pass

# import 
prt_score = pd.read_csv(f"fbeta_score_{currentdate}.csv")
prt_score['Date'] = prt_score['Date'].astype(str)
prt_score = prt_score.sort_values('Date').reset_index(drop = True)

# plot
prt_score = prt_score[-averageline:]

plt.figure(figsize = (16,30))
plt.subplot(3,1,1)
plt.errorbar(x = prt_score['Date'], y = prt_score['Accuracy'])
plt.xticks(rotation = 90)
plt.title('Accuracy Score (%)')
plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.subplot(3,1,2)
plt.errorbar(x = prt_score['Date'], y = prt_score['Precision'])
plt.xticks(rotation = 90)
plt.title('Precision (%)')
plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.subplot(3,1,3)
plt.errorbar(x = prt_score['Date'], y = prt_score['Fbeta'])
plt.xticks(rotation = 90)
plt.title('Fbeta (%)')
plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.show()

# import webbrowser
# webbrowser.open("assist_end.mp3")

___

## Asset Review

### Data Importing

In [42]:
asset_review = pd.read_csv('asset_review %s.csv' %currentdate)

# re-excute 'Day_Change' for the possible miss-calculation
perf_review = pd.read_csv('perf_review %s.csv' %currentdate)
perf_review = perf_review.sort_values(['index', 'date']).reset_index(drop = True)

day_change = []
for i in range(len(perf_review.index)):
    if i == 0:
        day_change.append(0)
    elif perf_review['index'][i-1] == perf_review['index'][i]:
        daychange = perf_review['percent_change'][i] - perf_review['percent_change'][i-1]
        day_change.append(daychange)
    elif perf_review['index'][i-1] != perf_review['index'][i]:
        day_change.append(0)
    else:
        pass
perf_review['day_change'] = day_change
perf_review['date'] = perf_review['date'].astype('datetime64[ns]')

perf_review.to_csv('perf_review %s.csv' %currentdate, index = False) 
perf_review.to_csv('./Backup_perf_review/perf_review %s.csv' %currentdate, index = False) 

print('Data has been imported.')

### Total Equity Change

In [43]:
total_gain_loss = asset_review.groupby('date').sum()[['equity', 'equity_change', 'tax_shorterm']].reset_index()
total_gain_loss = total_gain_loss.tail(20)

print('\nTotal Equity (USD):        '+'\033[1m' + str('%.2f'%total_gain_loss.tail(1).equity.values[0]) + '\033[0m', '\n')
print('Total Net Equity (USD):    '+'\033[1m' + str('%.2f'%(total_gain_loss.tail(1).equity.values[0]*0.9)) + '\033[0m', '\n')
print('Total Equity Change (USD): '+'\033[1m' + str('%.2f'%total_gain_loss.tail(1).equity_change.values[0]) + '\033[0m', '\n')
plt.figure(figsize = (16,9))

plt.errorbar(x = total_gain_loss['date'], y = total_gain_loss['equity'])
plt.bar(total_gain_loss['date'], total_gain_loss['equity'], alpha = 0.1, label = 'Total Amount')

plt.errorbar(x = total_gain_loss['date'], y = total_gain_loss['equity_change'])
plt.bar(total_gain_loss['date'], total_gain_loss['equity_change'], alpha = 0.1, label = 'Gain & Loss', color = sb.color_palette()[1])

plt.bar(total_gain_loss['date'], total_gain_loss['tax_shorterm'], alpha = 0.1, label = 'Short-Term Tax', color = sb.color_palette()[3])

plt.xticks(rotation = 90)
plt.legend()
# plt.yscale('log')
plt.title('Total Equity Change (USD)')

# for i in range(total_gain_loss['equity'].shape[0]):
#     count = total_gain_loss['equity'][i]
#     plt.text(i, count + 10000, int(count), ha = 'center', va = 'top');  
    
# for i in range(total_gain_loss['equity_change'].shape[0]):
#     count = total_gain_loss['equity_change'][i]
#     plt.text(i, count + 10000, int(count), ha = 'center', va = 'top');  
    
# for i in range(total_gain_loss['tax_shorterm'].shape[0]):
#     count = total_gain_loss['tax_shorterm'][i]
#     plt.text(i, count + 10000, int(count), ha = 'center', va = 'top');      

plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.show()

### Total Equity Breakdown

In [44]:
ticker_gain_loss = asset_review[asset_review['date'] == currentdate].sort_values('equity', ascending = False).reset_index(drop = True)

plt.figure(figsize = (16,9))
plt.bar(ticker_gain_loss['index'], ticker_gain_loss['equity'], alpha = 0.85, label = 'Equity')
plt.bar(ticker_gain_loss['index'], ticker_gain_loss['tax_shorterm'], alpha = 0.35, label = 'Short-Term Tax')

plt.title('Total Equity Breakdown (USD)')

for i in range(ticker_gain_loss['index'].shape[0]):
    count = ticker_gain_loss['equity'][i]
    plt.text(i, count + 7000, int(count), ha = 'center', va = 'top');  

for i in range(ticker_gain_loss['index'].shape[0]):
    count = ticker_gain_loss['tax_shorterm'][i]
    plt.text(i, count, int(count), ha = 'center', va = 'top');      
    
plt.legend()
plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.show()

### Equity Breardown Gain & Loss

In [45]:
plt.figure(figsize = (16,9))
ticker_gain_loss = ticker_gain_loss.sort_values('percent_change',ascending = False).reset_index(drop = True)

plt.bar(ticker_gain_loss['index'], ticker_gain_loss['percent_change'], alpha = 0.95, color = sb.color_palette()[0])
plt.title('Total Return(%)')

plt.grid(axis = 'x')
plt.grid(axis = 'y')

for i in range(ticker_gain_loss['index'].shape[0]):
    count = ticker_gain_loss['percent_change'][i]
    pct_string = '{:0.1f}%'.format(count)
    plt.text(i, count, pct_string, ha = 'center', va = 'top'); 

### Holding Period (days)

In [46]:
if day_check() != 'Saturday' and day_check() != 'Sunday' and timechecknow() > 390:
    # plus 1 days
    df = pd.read_csv('holding_period.csv')
    df['days'] = df['days'] + 1 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# remove non-existing tickers
holding_tickers = robin_equity_check()['index'].values.tolist()
df = df.loc[df['index'].isin(holding_tickers)].reset_index(drop = True)

# add new tickers
for i in holding_tickers:
    if i not in df['index'].values.tolist():
        df.loc[df.shape[0], ['index', 'days']] = i, 1

df.to_csv('holding_period.csv', index = False)
df.to_csv('./Backup_holding_period/holding_period %s.csv' %currentdate, index = False)

# plot
df = df.loc[-df['index'].isin(dead_stock)]

if len(df) != 0:
    plt.figure(figsize = (16,9))
    sb.barplot(x = df['days'], y = df['index']);
    plt.axvline(x = min_holding_period, color = 'black', linewidth = 7)
    plt.grid()
else:
    print('No Active Stocks')

### Equity Change History

In [47]:
import shutil
columns = shutil.get_terminal_size().columns

tickers_as_of = pd.read_csv('robin_all.csv')['index'].values
asset_review = asset_review.loc[asset_review['index'].isin(tickers_as_of)].reset_index(drop = True)
asset_review['date'] = asset_review['date'].astype('datetime64[ns]')

i = 1
if len(asset_review['index'].unique()) >= 8:
    plt.figure(figsize = [21,60])
if len(asset_review['index'].unique()) >= 4 and len(asset_review['index'].unique()) < 8:
    plt.figure(figsize = [21,40])
if len(asset_review['index'].unique()) < 4:
    plt.figure(figsize = [21,20])
    
print('\033[1m'+ 'Equity-Breakdown Amount Change (USD)'.center(columns) +'\033[0m')
for ticker in asset_review['index'].unique():
    plt.subplot(asset_review['index'].nunique(), 2, i)
    plt.errorbar(x = asset_review[asset_review['index'] == ticker]['date'], y = asset_review[asset_review['index'] == ticker]['equity'])
    plt.title(ticker)
    plt.xticks(rotation = 45)
#     plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    i+=1

### Equity Realize Strategy

In [48]:
import shutil
columns = shutil.get_terminal_size().columns

i = 1
if len(asset_review['index'].unique()) >= 8:
    plt.figure(figsize = [21,60])
if len(asset_review['index'].unique()) >= 4 and len(asset_review['index'].unique()) < 8:
    plt.figure(figsize = [21,40])
if len(asset_review['index'].unique()) < 4:
    plt.figure(figsize = [21,20])

print('\033[1m'+ 'Equity-Breakdown Price Change (USD)'.center(columns) +'\033[0m')
for ticker in asset_review['index'].unique():
    plt.subplot(asset_review['index'].nunique(), 2, i)
    plt.errorbar(x = asset_review[asset_review['index'] == ticker]['date'], y = asset_review[asset_review['index'] == ticker]['price'], label = 'Current Price')
    plt.errorbar(x = asset_review[asset_review['index'] == ticker]['date'], y = asset_review[asset_review['index'] == ticker]['average_buy_price'], label = 'Buy Price')
    plt.errorbar(x = asset_review[asset_review['index'] == ticker]['date'], y = asset_review[asset_review['index'] == ticker]['target_sell_price'], label = 'Sell Price')
    plt.legend(bbox_to_anchor = (1.02, 0.9), loc = 'center', borderaxespad = 0)
    plt.title(ticker)
    plt.xticks(rotation = 45)
#     plt.grid(axis = 'x')
    plt.grid(axis = 'y')
    i+=1

## Performance Review

### **Return vs S&P 500**

`excluded dead stocks from asset_performance.csv`

In [49]:
def asset_return_plot(asset_performance, day_cut):
    asset_performance = asset_performance[day_cut:]
        
    vola_port = np.std(asset_performance['Asset Change%']) * np.sqrt(asset_performance.shape[0]) / 100
    vola_snp = np.std(asset_performance['S&P Change% today']) * np.sqrt(asset_performance.shape[0]) / 100

    return_port = np.round(asset_performance['Asset Change%'].mean(), 2)
    return_snp = np.round(asset_performance['S&P Change% today'].mean(), 2)

    print("\nTotol Period: " + '\033[1m' + f"{asset_performance.shape[0]} days" + '\033[0m')
    print("______________________________________")
    print(f"\n• Volatility Asset:          {'%.3f' %vola_port}\n")
    print(f"• Volatility S&P 500:        {'%.3f' %vola_snp}")
    print("______________________________________\n")
    print("______________________________________")
    print('\033[1m' + f'\n• Average Asset Change:     {return_port} %\n' + '\033[0m')
    print(f"• S&P 500 Performance:      {return_snp} %")
    print("______________________________________\n\n\n")
    
    plt.figure(figsize = (16,9))
    asset_performance['Date'] = asset_performance['Date'].astype(str)
    plt.errorbar(x = asset_performance['Date'], y = asset_performance['Asset Change%'], label = 'Asset +/-')
    plt.errorbar(x = asset_performance['Date'], y = asset_performance['S&P Change% today'], label = 'S&P 500 +/-')
    plt.title(f'Agg. Asset Change%     vs     S&P Change% Today - {np.absolute(i)} days')
    plt.xticks(rotation = 90)
    plt.legend()
    plt.grid(axis = 'x')
    plt.grid(axis = 'y')

df = pd.read_csv(f"asset_review {currentdate}.csv")
dead_stock.append('005930.KS')
df = df[df['index'].isin(dead_stock) == False] #<<<<<<<<<<<<<<<<<<<<<<< exclude dead stocks from asset_performance.csv

percent_change_df = df.groupby(['date']).mean().reset_index()[['date', 'percent_change']]
equity_change_df = df.groupby(['date']).sum().reset_index()[['date', 'equity_change']]
df = pd.merge(percent_change_df, equity_change_df, how = 'inner', on = 'date')

df['date'] = df['date'].astype('datetime64[ns]')
asset_performance = pd.merge(df, snp, how = 'inner', left_on = 'date', right_on = 'Date')
asset_performance.rename(columns = {'percent_change' : 'Agg. Change%'}, inplace = True)
asset_performance['Asset Change%'] = asset_performance['Agg. Change%'].diff()
asset_performance.rename(columns = {'snp_change' : 'S&P Change% today'}, inplace = True)
asset_performance.to_csv('asset_performance.csv', index = False)

dmv_list = [-120, -60, -30, -20, -10, -5]
for i in dmv_list:
    asset_return_plot(asset_performance, i)

display(asset_performance[dmv_list[-1]:])
print('\n\n\n')

### Corr: Today's Return

- perf_review cuts by 'averageline' - **70 days** for below cells

In [50]:
perf_review = perf_review.sort_values('date').reset_index(drop = True).tail(averageline)

corr_analysis = perf_review.corr().reset_index()[['index','day_change']]
corr_analysis['day_change'] = corr_analysis['day_change']
corr_analysis = corr_analysis.sort_values('day_change', ascending = False).dropna().reset_index(drop = True)
corr_analysis = corr_analysis.loc[corr_analysis['index'].isin([
    'sean_score', 
    '1Y Perf', 
    'Price / Cash Flow',
    '10Y Total Return', 
    '10Y Perf', 
    'P/E TTM', 
    '5Y Perf',
    '5Y Total Return',
    'EV / EBITDA', 
    'Price / Sales',
    'Last Price Vs. 10D SMA', 
    'Quick Ratio', 
    '3Y Perf',
    '3Y Total Return', 
    'Week Vol / Shares', 
    'Current Ratio',
    'EV / Sales', 
    'P/E FWD',
    'Revenue 3Y', 
    'Revenue FWD', 
    'EBITDA FWD', 
    'Yield FWD', 
    'Yield TTM', 
    'Revenue YoY', 
    'FCF 3Y', 
    '4Y Avg Yield',
    'EBITDA 3Y', 
    '24M Beta', 
    'Covered Ratio', 
    'Profitability Grade_score', 
    'Return on Assets', 
    'Asset Turnover',
    'Profit Margin',
    'Payout Ratio', 
    'FCF Margin',
    'Div Rate FWD', 
    'EPS 3Y', 
    '60M Beta', 
    'Div Rate TTM', 
    'EPS Revision Grade_score',
    'Valuation Grade_score', 
    'Float %', 
    'Net Income Margin',
    'EBIT Margin', 
    '1M Perf', 
    'Price / Book', 
    'Net Income / Employee',
    'YTD Perf', 
    'EBITDA Margin', 
    'Momentum Grade_score', 
    'Institutional Percent', 
    'EPS FWD', 
    'Wall St. Score', 
    'PEG FWD', 
    '6M Perf', 
    'EPS Estimate',
    'EPS Actual',
    'Debt to FCF', 
    'Div Growth 3Y',
    'LT Debt to Total Capital', 
    'Return on Total Capital',
    'Div Growth 5Y', 
    'PEG TTM',
    'Last Price Vs. 100D SMA', 
    'Last Price Vs. 200D SMA', 
    'Last Price Vs. 50D SMA', 
    'avg_score',
    'Growth Grade_score', 
    'SA Authors Score', 
    'Debt to Equity',
    'Return on Equity', 
    'EPS YoY', 
    'Quant Score', 
    'EBITDA YoY'])].reset_index(drop = True)

plt.figure(figsize = (16,30))
plt.bar(corr_analysis['day_change'], corr_analysis['index'], alpha = 0.85)

plt.title('corr_analysis')
# plt.xticks(rotation = 90)

plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.show()

### Sean Score by ticker

- perf_review **ISIN** by the current portfolio

In [51]:
perf_review = perf_review.loc[perf_review['Symbol'].isin(robin_equity_check()['index'].values)]

g = sb.FacetGrid(data = perf_review, hue = 'Symbol', height = 10, aspect = 1.5)
g.map(sb.regplot, 'Symbol', 'sean_score', fit_reg = False);
g.add_legend();
plt.title('Stock Symbol vs Sean Score\n')
plt.xlabel('Stock symbol')
plt.ylabel('Sean Score')
# plt.xscale('log')
# plt.yscale('log')

plt.grid(axis = 'x')
plt.grid(axis = 'y')

plt.show()

___

### Linear Regression

- Top Corr. Variables

In [52]:
corr_analysis_topindex = corr_analysis.head(5)['index'].values.tolist()
print(corr_analysis_topindex)

#### Day return vs  1st variable

In [53]:
import statsmodels.api as sm
perf_review['intercept'] = 1
lm = sm.OLS(perf_review['day_change'], perf_review[['intercept', corr_analysis_topindex[0]]])
results = lm.fit()
results.summary()

In [54]:
plt.figure(figsize = (16,9));
sb.regplot(data = perf_review , x = corr_analysis_topindex[0], y = 'day_change', scatter_kws = {'alpha': 1/2}, ci = 95);
plt.title(f"Day Return vs {corr_analysis_topindex[0]} (R² = {'%.2f' %results.rsquared})")
plt.xlabel(corr_analysis_topindex[0])
plt.ylabel('Day Return (%)')
# plt.ylim(-np.percentile(perf_review['day_change'], 95), np.percentile(perf_review[corr_analysis_topindex[0]], 95))

plt.grid(axis='x')
plt.grid(axis='y')
plt.show()

#### Day return vs  2nd variable

In [55]:
import statsmodels.api as sm
perf_review['intercept'] = 1
lm = sm.OLS(perf_review['day_change'], perf_review[['intercept', corr_analysis_topindex[1]]])
results = lm.fit()
results.summary()

In [56]:
plt.figure(figsize = (16,9));
sb.regplot(data = perf_review , x = corr_analysis_topindex[1], y = 'day_change', scatter_kws = {'alpha': 1/2}, ci = 95);
plt.title(f"Day Return vs {corr_analysis_topindex[1]} (R² = {'%.2f' %results.rsquared})")
plt.xlabel(corr_analysis_topindex[1])
plt.ylabel('Day Return (%)')
# plt.ylim(-np.percentile(perf_review['day_change'], 95), np.percentile(perf_review[corr_analysis_topindex[0]], 95))

plt.grid(axis='x')
plt.grid(axis='y')
plt.show()

#### Day return vs  3rd variable

In [57]:
import statsmodels.api as sm
perf_review['intercept'] = 1
lm = sm.OLS(perf_review['day_change'], perf_review[['intercept', corr_analysis_topindex[2]]])
results = lm.fit()
results.summary()

In [58]:
plt.figure(figsize = (16,9));
sb.regplot(data = perf_review , x = corr_analysis_topindex[2], y = 'day_change', scatter_kws = {'alpha': 1/2}, ci = 95);
plt.title(f"Day Return vs {corr_analysis_topindex[2]} (R² = {'%.2f' %results.rsquared})")
plt.xlabel(corr_analysis_topindex[2])
plt.ylabel('Day Return (%)')
# plt.ylim(-np.percentile(perf_review['day_change'], 95), np.percentile(perf_review[corr_analysis_topindex[0]], 95))

plt.grid(axis='x')
plt.grid(axis='y')
plt.show()

___

#### Score Plot

In [59]:
score_review = perf_review[['day_change', corr_analysis_topindex[0], corr_analysis_topindex[1], corr_analysis_topindex[2], corr_analysis_topindex[3], corr_analysis_topindex[4]]].reset_index(drop=True)
score_review['intercept'] = 1
lm = sm.OLS(score_review['day_change'], score_review[['intercept', corr_analysis_topindex[0], corr_analysis_topindex[1], corr_analysis_topindex[2], corr_analysis_topindex[3], corr_analysis_topindex[4]]])
result = lm.fit()
result.summary()

In [60]:
score_plot = perf_review.groupby('Symbol').mean()[[corr_analysis_topindex[0], corr_analysis_topindex[1], corr_analysis_topindex[2], corr_analysis_topindex[3], corr_analysis_topindex[4]]].reset_index()
symbols = score_plot.Symbol.values.tolist()
standard = score_plot.drop(columns = ['Symbol']).values

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
standard_var = sc.fit_transform(standard)
standard_var = standard_var + np.max(standard_var)

score_plot = pd.DataFrame(standard_var, columns = [corr_analysis_topindex[0], corr_analysis_topindex[1], corr_analysis_topindex[2], corr_analysis_topindex[3], corr_analysis_topindex[4]])
score_plot['Symbol'] = symbols

import warnings
warnings.simplefilter('always', category=UserWarning)
warnings.filterwarnings("ignore")
from math import pi

def radarplot(ticker):
    print('\033[1m'+ ticker +'\033[0m')

    categories = list(score_plot)[0:5]
    values = score_plot[score_plot['Symbol'] == ticker].mean().values.flatten().tolist()
    values += values[:1] # repeat the first value to close the circular graph
    angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
    angles += angles[:1]

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5),
                           subplot_kw=dict(polar=True))

    plt.xticks(angles[:-1], categories, color='grey', size=12)
    plt.yticks(np.arange(1, 6), ['1', '2', '3', '4', '5'],
               color='grey', size=12)
    plt.ylim(0, 5)
    ax.set_rlabel_position(30)

    ax.plot(angles, values, linewidth=1, linestyle='solid')
    ax.fill(angles, values, 'skyblue', alpha=0.4)

    plt.show()

for i in score_plot.Symbol:
    radarplot(i)

In [61]:
score_plot['Average'] = (score_plot[score_plot.columns.tolist()[0]] + score_plot[score_plot.columns.tolist()[1]] + score_plot[score_plot.columns.tolist()[2]] + score_plot[score_plot.columns.tolist()[3]] + score_plot[score_plot.columns.tolist()[4]])/5
score_plot = score_plot.sort_values('Average', ascending = False)
ex_highest_corr_index = score_plot.head(1)['Symbol'].tolist()
with open("ex_highest_corr_index.txt", 'w') as f:
    for s in ex_highest_corr_index:
        f.write(str(s) + '\n')  

plt.figure(figsize = (16,9));
plt.bar(score_plot['Symbol'], score_plot['Average'])
plt.grid(axis='x')
plt.grid(axis='y')
plt.show()

## **ETF Tickers by Return**

### Return: ETF vs Asset (`Add New ETF`)

`icld. Dead Stocks`

In [62]:
from datetime import date, datetime, timedelta
day_plus = datetime.today() - timedelta(days = 7)
day_plus = str(day_plus)[:10]

portfolios = pd.read_csv('daily_summary_target_tickers.csv')
portfolios = portfolios[portfolios['Date'] >= day_plus]

portfolios = portfolios.groupby(['Date']).mean().reset_index()[['Date', 'Change']]
portfolios['Date'] = portfolios['Date'].astype('datetime64[ns]')

def price_change(ticker, symbol):
    price_record = yfinance_df_setting(ticker)[['Date', 'Adj Close']]
    price_change = []
    for i in range(price_record.shape[0]):
        if i == 0:
            price_change.append(0)
        else:
            price_change.append((price_record['Adj Close'][i] - price_record['Adj Close'][i-1])/price_record['Adj Close'][i-1]*100)
    price_record[f"{symbol}"] = price_change
    return price_record[['Date', symbol]]


#### Add new ETF from below ########################################################################

snp =                    price_change('^GSPC', 'SnP')
Energy =                 price_change('XLE', 'Energy')
Financials =             price_change('XLF', 'Financials')
Real_Estate =            price_change('XLRE', 'Real Estate')
Materials =              price_change('XLB', 'Materials')
Information_Technology = price_change('XLK', 'Information Technology')
Consumer_Staples =       price_change('XLP', 'Consumer Staples')
Health_Care =            price_change('XLV', 'Health Care')
Industrials =            price_change('XLI', 'Industrials')
Consumer_Discretionary = price_change('XLY', 'Consumer Discretionary')
Utilities =              price_change('XLU', 'Utilities')
Communication_Services = price_change('XLC', 'Communication Services')
QQQ                    = price_change('QQQ', 'Invesco QQQ')
SPY                    = price_change('SPY', 'SPDR S&P 500')
SPYD                   = price_change('SPYD', 'SPDR S&P 500 Dividend')
SPYG                   = price_change('SPYG', 'SPDR S&P 500 Growth')
SPYV                   = price_change('SPYV', 'SPDR S&P 500 Value')
ROBO                   = price_change('ROBO', 'Robo Global Robotics and Automation Index ETF')
XT                     = price_change('XT', 'iShares Exponential Technologies ETF')
QTUM                   = price_change('QTUM', 'Defiance Quantum ETF')
VOO                    = price_change('VOO', 'Vanguard 500 Index Fund')
IVV                    = price_change('IVV', 'iShares Core S&P 500 ETF')
SPLG                   = price_change('SPLG', 'SPDR Portfolio S&P 500 ETF')

compare_index = [Energy,                 
                 Financials,             
                 Real_Estate,             
                 Materials,              
                 Information_Technology,
                 Consumer_Staples,       
                 Health_Care,         
                 Industrials,        
                 Consumer_Discretionary,
                 Utilities,           
                 Communication_Services, 
                 QQQ, 
                 SPY, 
                 SPYD, 
                 SPYG,
                 SPYV,
                 ROBO,
                 XT,
                 QTUM, 
                 VOO, 
                 IVV,
                 SPLG]

#### Add new ETF from above ########################################################################


portfolio_performance = pd.merge(portfolios, snp, how = 'inner', on = 'Date')
for index in compare_index:
    portfolio_performance = pd.merge(portfolio_performance, index, how = 'inner', on = 'Date')
portfolio_performance = portfolio_performance.sort_values('Date').reset_index(drop = True)

# verify the last date on the portfolio_performance dataframe
last_date_verfy = portfolio_performance[-1:].Date.astype(str).values[0]
if last_date_verfy != currentdate:
    print('\nExcept >>> portfolio_performance has the last date != currentdate ==> ' + '\033[1m' + f'{last_date_verfy}\n' + '\033[0m')
    alarm()    
    
portfolio_performance.rename(columns = {'Change':'**Portfolio**'}, inplace = True)
portfolio_performance.rename(columns = {'SnP':'*S&P 500*'}, inplace = True)

asset_performance = pd.read_csv('asset_performance.csv')
asset_performance['date'] = asset_performance['date'].astype('datetime64[ns]')
asset_performance = asset_performance[['date', 'Asset Change%']]
asset_performance.rename(columns = {'date':'Date', 'Asset Change%':'>>>>>>>> Asset Change <<<<<<<<'}, inplace = True)

performance = pd.merge(asset_performance, portfolio_performance, how = 'inner', on = 'Date')
performance.to_csv('performance_sourcedata.csv', index = False)
performance.to_csv(f"./Backup_performance_sourcedata/performance_sourcedata_{currentdate}.csv", index = False)

performance_stats = performance.describe().T.reset_index()
performance_stats['Volatility'] = performance_stats['std'] * np.sqrt(performance_stats['count'][0])

performance_summary = performance_stats[['index', 'mean', 'Volatility']].sort_values('mean', ascending = False).reset_index(drop = True)
performance_summary.rename(columns = {'index': f"{int(performance_stats['count'][0])} Days", 'mean':'Return(avg)'}, inplace = True)
performance_summary['Return to Vola'] = performance_summary['Return(avg)'] / performance_summary['Volatility']
performance_summary.to_csv('performance_summary.csv', index = False)
performance_summary.to_csv(f"./Backup_performance_summary/performance_summary_{currentdate}.csv", index = False)
display(performance_summary)

### Volatility

In [63]:
plt.figure(figsize = (16,9))

performance_summary = performance_summary.sort_values('Volatility', ascending = True).reset_index(drop = True)
plt.bar(x = performance_summary[f"{int(performance_stats['count'][0])} Days"], height = performance_summary['Volatility'], color = 'green')
plt.xticks(rotation = 90)
plt.ylabel('Average Return')
plt.grid(axis = 'x')
plt.grid(axis = 'y')

### Return

In [64]:
plt.figure(figsize = (16,9))

performance_summary = performance_summary.sort_values('Return(avg)', ascending = False).reset_index(drop = True)
plt.bar(x = performance_summary[f"{int(performance_stats['count'][0])} Days"], height = performance_summary['Return(avg)'])
plt.xticks(rotation = 90)
plt.ylabel('Average Return')
plt.grid(axis = 'x')
plt.grid(axis = 'y')

### ETF Portfolio (`Add New ETF`)

In [65]:
symbol = f"{int(performance_stats['count'][0])} Days"
asset = performance_summary[performance_summary[symbol] == '>>>>>>>> Asset Change <<<<<<<<']
etf = performance_summary[performance_summary['Return(avg)'] > asset['Return(avg)'].values[0]][symbol].tolist()
blacklist = ['**Portfolio**', '*S&P 500*']
etf = [i for i in etf if i not in blacklist]

etf_index = {}
etf_index['*S&P 500*'] = '^GSPC'
etf_index['Energy'] = 'XLE'
etf_index['Financials'] = 'XLF'
etf_index['Real Estate'] = 'XLRE'
etf_index['Materials'] = 'XLB'
etf_index['Information Technology'] = 'XLK'
etf_index['Consumer Staples'] = 'XLP'
etf_index['Health Care'] = 'XLV'
etf_index['Industrials'] = 'XLI'
etf_index['Consumer Discretionary'] = 'XLY'
etf_index['Utilities'] = 'XLU'
etf_index['Communication Services'] = 'XLC'
etf_index['Invesco QQQ'] = 'QQQ'
etf_index['SPDR S&P 500'] = 'SPY'
etf_index['SPDR S&P 500 Dividend'] = 'SPYD'
etf_index['SPDR S&P 500 Growth'] = 'SPYG'
etf_index['SPDR S&P 500 Value'] = 'SPYV'
etf_index['Robo Global Robotics and Automation Index ETF'] = 'ROBO'
etf_index['iShares Exponential Technologies ETF'] = 'XT'
etf_index['Defiance Quantum ETF'] = 'QTUM'
etf_index['Vanguard 500 Index Fund'] = 'VOO'
etf_index['iShares Core S&P 500 ETF'] = 'IVV'
etf_index['SPDR Portfolio S&P 500 ETF'] = 'SPLG'

etf_index_df = pd.DataFrame([etf_index])
etf_index_df.to_csv('etf_index_df.csv', index = False)

min_SuccessRate = int(min_SuccessRate * 0.9)
etf_portfolio = []
for i in etf:
    strategy_analysis = strategy_analysis_main(etf_index[i], 'off', 'off')[0] + strategy_analysis_main(etf_index[i], 'off', 'off')[0]
    if strategy_analysis >= 1 or strategy_analysis == -9:
        print(i,':', etf_index[i])
        etf_portfolio.append(etf_index[i])

if day_check() != 'Saturday' and day_check() != 'Sunday' and timechecknow() > 390:
    with open("etf_portfolio.txt", 'w') as f:
        for s in etf_portfolio:
            f.write(str(s) + '\n')  

print(f'\nTest Passed:\n\n{etf_portfolio}')

Save File

In [ ]:
# file w/ the output
!jupyter nbconvert --to ipynb SSS.QED.V5.0.ipynb 


# fils w/o the output
from nbformat import read, write

file_name = 'SSS.QED.V5.0'
original_file_name = f'{file_name}.ipynb'
new_file_name = f'{file_name}.backup.ipynb'

def strip_output(nb):
    for cell in nb.cells:
        if hasattr(cell, "outputs"):
            cell.outputs = []
        if hasattr(cell, "prompt_number"):
            del cell["prompt_number"]
            
nb = read(open(original_file_name, encoding = 'utf8'), 4)
strip_output(nb)
write(nb, open(new_file_name, "w", encoding = 'utf8'), 4)


# notification
message = "Subject: The performance review gets ready. Backup File Saved"
print(message)
emailsend_to_server(message)  
time.sleep(5)
voice_message('The performance review gets ready')

____